# Deep Learning Analyses - P300 Decoding from 2D EEG Data Formulation

## Impostazione **Weight & Biases DL Training** con **Rappresentazione Time-Frequency Signal (2D)** 

## Optimization Weight and Biases - EEG Spectrograms - Frequency x Time

#### **Weight & Biases Procedure FINAL SEQUENCE OF STEPS - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

#### **Utils Functions - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''
QUI DENTRO HO CONFIGURATO 
LE FUNZIONI DI CONTROLLO DELLE STRINGHE 
PER IL SALVATAGGIO DELLE PERFORMANCE DEL MODELLO
NELLE RELATIVE SUBFOLDERS

(I.E., get_subfolder_from_key, get_subfolder_from_key_hyper)

IN MODO CHE SI LEGHINO ALLA CHIAVE 'STANDARDIZATION' DELL'OGGETTO SWEEP_CONFIG

'''


import pickle
import numpy as np


def load_data_hyper(data_type, category, wavelet_level=None, condition= "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, seleziona la finestra temporale (50°-300° punto, ossia 0-1000 mms).

    Parameters:
    - data_type: str, "1_20", "1_45" o "wavelet"
    - category: str, "familiar" o "unfamiliar"
    - wavelet_level: str, "theta", "delta", ecc. (solo per dati wavelet)
    - condition: str, condizione sperimentale da selezionare

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto)
    - y: Etichette corrispondenti
    """
    # Definizione dei percorsi base
    base_paths = {
        "1_20": {
            "familiar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_20/hyper_dataset_EEG_preprocessed_1_20_familiar_{condition}.pkl",
            "unfamiliar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_20/hyper_dataset_EEG_preprocessed_1_20_unfamiliar_{condition}.pkl"
        },
        "1_45": {
            "familiar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_45/hyper_dataset_EEG_preprocessed_1_45_familiar_{condition}.pkl",
            "unfamiliar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_45/hyper_dataset_EEG_preprocessed_1_45_unfamiliar_{condition}.pkl"
        },
        "wavelet": {
            "familiar": "/home/stefano/Interrogait/all_datas/Hyper_Datasets_Wavelet_Reconstructions/hyper_dataset_wavelet_familiar.pkl",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Hyper_Datasets_Wavelet_Reconstructions/hyper_dataset_wavelet_unfamiliar.pkl"
        }
    }

    # Seleziona il path corretto
    filepath = base_paths[data_type][category]
    
    # Caricamento del file
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    # Selezione della finestra temporale e delle etichette
    X = data[wavelet_level][condition]["data"][:, :, 125:200] if data_type == "wavelet" else data["data"][:, :, 50:300]
    y = data[wavelet_level][condition]["labels"] if data_type == "wavelet" else data["labels"]
        
    return X, y



def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (50°-300° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "familiar" o "unfamiliar"
    - subject_type: str, "th" (terapisti) o "pt" (pazienti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Familiar_Spectrograms/",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test



'''ATTENZIONE MODIFICATA FUNZIONE DI STANDARDIZZAZIONE'''
# Funzione per standardizzare i dati
# Con questa modifica eviti che std==0 produca NaN e i tuoi loss torneranno numeri sensati.
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    #aggiungo eps per evitare divisione per zero
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test


# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels=3, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    '''OCCHIO QUI CAMBIATO'''
    #if model_type == "CNN2D":
    
    if model_type == "CNN2D_LSTM_TF":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor


'''
QUESTE DUE FUNZIONI PRESE DA 

EEG Motor Movement - Imagery Dataset (EEGMMIDB) - TASK 1 - 2D GRID - ALL FREQS + 3D CONV CONV SEP.ipynb

DA SEZIONE

## Impostazione **Weight & Biases DL Training** con **Rappresentazione Tempo-Frequenza dei miei dati EEG** 
a seconda del Dataset del Task scelto
'''


# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
     # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
        
'''

QUESTE FUNZIONI TROVATE COME ERANO NEL NOTEBOOK
# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, sweep_config):
    
    
    #Mi richiamo la chiave 'standardization' che ho impostato nella configurazione dell'oggetto weight and biases
    #(i.e., sweep_config['standardization']) e eseguo una procedura condizionale 
    
    #ossia che, se risulta o True o False, lui cambi le condizioni di gestione 
    #della costruzione delle path di salvataggio 
    
    
    # Controlla se i dati sono standardizzati
    if sweep_config['standardization']:
    
        #PER I DATI SCALED
        if '_familiar_th' in key:
            return 'TH_FAM'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM'
        elif '_familiar_pt' in key:
            return 'PT_FAM'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM'
        else:
            return None
    else: 
        #PER I DATI UNSCALED

        if '_familiar_th' in key:
            return 'TH_FAM_UNSCALED'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM_UNSCALED'
        elif '_familiar_pt' in key:
            return 'PT_FAM_UNSCALED'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM_UNSCALED'
        else:
            return None
    
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "wavelet" in key:
        data_type_str = "wavelet_delta"
    elif "1_20" in key:
        data_type_str = "1_20"
    elif "1_45" in key:
        data_type_str = "1_45"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
        
        
# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key_hyper(key, sweep_config):
    
    
    #Mi richiamo la chiave 'standardization' che ho impostato nella configurazione dell'oggetto weight and biases
    #(i.e., sweep_config['standardization']) e eseguo una procedura condizionale 
    
    #ossia che, se risulta o True o False, lui cambi le condizioni di gestione 
    #della costruzione delle path di salvataggio 
    
    
    
    
    # Controlla se i dati sono standardizzati
    if sweep_config['standardization']:
        
        #PER I DATI SCALED
            
        if '_familiar' in key:
            return 'HYPER_FAM'
        elif '_unfamiliar' in key:
            return 'HYPER_UNFAM'
        else:
            return None
    else:
        
        #PER I DATI UNSCALED
        if '_familiar' in key:
            return 'HYPER_FAM_UNSCALED'
        elif '_unfamiliar' in key:
            return 'HYPER_UNFAM_UNSCALED'
        else:
            return None
    
    
# Funzione per salvare i risultati
def save_performance_results_hyper(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition= "th_resp_vs_pt_resp"):
    
    #Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key_hyper(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "wavelet" in key:
        data_type_str = "wavelet_delta"
    elif "1_20" in key:
        data_type_str = "1_20"
    elif "1_45" in key:
        data_type_str = "1_45"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
'''



#### **DL Models - EEG Spectrograms - Time x Frequencies (OLD VERSIONS BEFORE SEPTEMBER 2025!)**

##### **DEFINIZIONE DEI MODELLI NUOVI PER P300 FROM 2D TIME-FREQUENCY SIGNAL  - DA LUGLIO A SETTEMBRE 2025** 

PS: 

La **vecchia CNN2D** (creata a **LUGLIO 2025**) con LE FUNZIONI DI ATTIVAZIONI DINAMICHE (LAYER CONVOLUTIVI + FC1) E DINAMISMO DEI KERNEL SIZE DI CONV e POOL LAYER e STRIDE è stata **SOSTITUITA DA QUELLA CNN2D_LSTM_TF**, USATA PER **BRAIN DECODING DEL MOTOR TASK, PER LA RAPPRESENTAZIONE TEMPO x FREQUENZA!**

(**SALTA QUESTA PRIMA CELLA DI CODICE QUI SOTTO**, DOVE CI SAREBBERO **LA VECCHIE RETI CNN2D, BILSTM e TRANSFORMER DI LUGLIO 2025** CON O**LE FUNZIONI DI ATTIVAZIONI DINAMICHE (LAYER CONVOLUTIVI + FC1) E DINAMISMO DEI KERNEL SIZE DI CONV e POOL LAYER e STRIDE**!

#### **DL Models - EEG Spectrograms - Time x Frequencies NEW VERSIONS (SAME OF MOTOR TASKS!)**

In [ ]:
'''
DEFINIZIONE DEI MODELLI NUOVI PER P300 FROM 2D TIME-FREQUENCY SIGNAL  - AGGIORNATI A SETTEMBRE 2025 COME QUELLI DEL TASK MOTORIO!


                                                                ***CNN2D_LSTM_TF*** 


Uso la stessa rete neurale usata per Brain Decoding Task Motorio


'''

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN2D_LSTM_TF(nn.Module):

    def __init__(self, input_channels=61, num_classes=2, dropout=0.2):
        super().__init__()
        # --- Block 1 ---
        self.bn1   = nn.BatchNorm2d(input_channels)    # normalizza 64 canali
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)

        # --- Block 2 (residual) ---
        # Proiezione 1×1 per riallineare i canali di skip (32→64)
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn   = nn.BatchNorm2d(64)

        self.bn2a   = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b   = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        # --- Head: Dropout + LSTM + FC finale ---
        self.dropout     = nn.Dropout(dropout)
        self.hidden_size = 64
        
        # dopo 3 pool: freq da 81→10, time da 9→1 → feature per timestep = 128×1
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.classifier = nn.Linear(self.hidden_size, num_classes)

    def forward(self, x):
        # x: (B,64,81,9)

        # --- Block 1 ---
        x = self.bn1(x)                   # → (B,64,81,9)
        x = F.relu(self.conv1(x))         # → (B,32,81,9)
        x = self.pool(x)                  # → (B,32,40,4)

        # --- Block 2 (residuo) ---
        res = x                           # skip: (B,32,40,4)
        res = self.res_conv(res)          # progetto: → (B,64,40,4)
        res = self.res_bn(res)            # → (B,64,40,4)

        # main path
        x = self.bn2a(x)                  # → (B,32,40,4)
        x = F.relu(self.conv2a(x))        # → (B,64,40,4)
        x = self.bn2b(x)                  # → (B,64,40,4)
        x = self.conv2b(x)                # → (B,64,40,4)

        x = x + res                       # somma residua valida → (B,64,40,4)
        x = F.relu(x)                     
        x = self.pool(x)                  # → (B,64,20,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))  # → (B,128,20,2)
        x = self.pool(x)                     # → (B,128,10,1)

        # --- Prepara per LSTM ---
        x = x.permute(0, 2, 1, 3)         # → (B,10,128,1)
        b, seq, ch, tw = x.size()        
        x = x.reshape(b, seq, ch * tw)    # → (B,10,128)

        # --- LSTM + classificazione ---
        out, _ = self.lstm(self.dropout(x))  # → out: (B,10,64)
        last = out[:, -1, :]                 # prendo l’ultima uscita → (B,64)
        logits = self.classifier(last)       # → (B,2)
        return logits
    

'''
Gli LSTM si aspettano un input in forma (batch, lunghezza_sequenza, dimensione_feature). 
Dovrai quindi decidere qual è la dimensione sequenziale.

Opzione comune: usare il tempo come sequenza
Step 1: Trasponi i dati in modo da avere il tempo come dimensione sequenziale.

Dalla forma (batch, canali, frequenze, tempo) puoi fare:


x = x.permute(0, 3, 1, 2)  # Diventa (batch, tempo, canali, frequenze)

Step 2: Unisci le dimensioni dei canali e dei bin di frequenza in un’unica dimensione di feature:


batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # Ora: (batch, tempo, canali*frequenze)

Nel tuo caso, per 3 canali e 38 bin di frequenza: input_size = 3 * 38 = 114 e lunghezza sequenza = 6.

Nota: Se invece preferisci usare i bin di frequenza come sequenza, potresti fare:

x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
x = x.reshape(batch, frequenze, canali * tempo)  # Sequence length = 38, feature size = 3*6 = 18
La scelta dipende dal tipo di informazione temporale o spettrale che vuoi evidenziare.

'''

class ReadMEndYou(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.5, bidirectional=False):
        """
        input_size: dimensione delle feature per time-step (dovrà essere canali * frequenze)
        hidden_sizes: lista con le dimensioni degli hidden state, es. [24, 48, 62]
        output_size: numero di classi
        
        """
    
        super(ReadMEndYou, self).__init__()
        
        self.bidirectional = bidirectional # Impostazione della bidirezionalità    
        
        # Adattiamo hidden_size in base alla bidirezionalità
        self.hidden_sizes = [
            hidden_sizes[0] * 2 if bidirectional else hidden_sizes[0],
            hidden_sizes[1] * 2 if bidirectional else hidden_sizes[1],
            hidden_sizes[2] * 2 if bidirectional else hidden_sizes[2]
        ]
        
        self.lstm1 = nn.LSTM(input_size=input_size, 
                             hidden_size=self.hidden_sizes[0], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0, 
                             bidirectional=bidirectional)
        self.lstm2 = nn.LSTM(input_size=self.hidden_sizes[0] * 2 if bidirectional else self.hidden_sizes[0],
                             hidden_size=self.hidden_sizes[1], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        self.lstm3 = nn.LSTM(input_size=self.hidden_sizes[1] * 2 if bidirectional else self.hidden_sizes[1],
                             hidden_size=self.hidden_sizes[2],
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_sizes[2] * 2 if bidirectional else self.hidden_sizes[2], output_size)
    
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Trasponi per avere il tempo come dimensione sequenziale:
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        # Ora input_size deve essere channels * freqs (es. 64 * 81 = 7471)
        
        # LSTM 1
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        
        # LSTM 2
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        
        # LSTM 3
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        
        # Estraiamo l'output dell'ultimo time-step
        out = out[:, -1, :]
        
        # Dropout prima del layer fully connected    
        out = self.dropout(out)
        
        # Passaggio attraverso il layer finale per la previsione
        out = self.fc(out)
        return out
        


'''
Il modulo Transformer in PyTorch lavora tipicamente su input di forma (seq_length, batch, embedding_dim).

Nel codice attuale, si parte da una forma simile a (batch, canali, seq_length), ma dovrai adattarla alla nuova struttura.

Possibili approcci:

1) Approccio A: usare il tempo come sequenza

Se consideri il tempo (6 time windows) come la sequenza, puoi procedere come segue:

A) Unisci canali e frequenze in un’unica dimensione di feature:

# Dati originali: (batch, canali, frequenze, tempo)
x = x.permute(0, 3, 1, 2)  # (batch, tempo, canali, frequenze)
batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # (batch, tempo, 3*38 = 114)

B) Modifica il layer di embedding:

Nel codice attuale, l'embedding è definito come:

self.embedding = nn.Linear(seq_length, d_model)
Dovrai cambiarlo in modo che mappi le dimensioni delle feature (in questo caso 114) a uno spazio latente:

self.embedding = nn.Linear(canali * frequenze, d_model)

C) Permuta per il Transformer:

Dopo l'embedding, passa l'input alla forma (seq_length, batch, d_model):

x = x.permute(1, 0, 2)  # Ora: (tempo, batch, d_model)


2) Approccio B: usare i bin di frequenza come sequenza
In alternativa, se reputi più rilevante la risoluzione spettrale, puoi considerare i 38 bin come sequenza e combinare canali e tempo:


x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
batch, frequenze, canali, tempo = x.shape
x = x.reshape(batch, frequenze, canali * tempo)  # (batch, frequenze, 3*6 = 18)

E poi procedere con un embedding layer che mappa da 18 a d_model e permutare in (frequenze, batch, d_model).

Scelta dell'approccio:
Se l'aspetto temporale è più critico, probabilmente è meglio usare l’Approccio A (sequenza di lunghezza 6).
Se invece vuoi dare maggior rilievo alla struttura spettrale, l’Approccio B potrebbe essere più indicato.

Ricorda che la scelta dipende dalla natura del tuo problema e dalla rilevanza delle informazioni temporali rispetto a quelle spettrali.
'''

import torch
import torch.nn as nn

#Scelta: In questa implementazione abbiamo deciso di usare il tempo come sequenza.
#In alternativa, potresti scegliere i bin di frequenza come sequenza, ma ciò richiederebbe una diversa riorganizzazione delle dimensioni 
#(ad esempio, un permute diverso).



class ReadMYMind(nn.Module):

    def __init__(self, d_model, num_heads, num_layers, num_classes, channels=61, freqs=26):
        
        super(ReadMYMind, self).__init__()

        # Il layer di embedding mapperà la feature dimension (channels * freqs) a d_model
        self.embedding = nn.Linear(channels * freqs, d_model)
        
        # Transformer per l'attenzione spaziale (qui si applica direttamente alla sequenza temporale)
        self.spatial_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Transformer per l'attenzione temporale (si potrebbe considerare un'iterazione successiva)
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Cross-attention per combinare le rappresentazioni
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads)
        
        # Fusione e classificazione finale
        self.fc_fusion = nn.Linear(d_model, d_model)
        self.fc_classify = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        # x: (batch, canali, frequenze, tempo)
        
        # Utilizziamo il tempo come sequenza
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        
        # Embedding: (batch, tempo, d_model)
        x = self.embedding(x)
        
        # Transformer richiede input di forma (seq_length, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # -> (tempo, batch, d_model)
        
        # Applichiamo il Transformer per l'attenzione spaziale
        x_spatial = self.spatial_transformer(x)
        # Applichiamo il Transformer per l'attenzione temporale
        x_temporal = self.temporal_transformer(x_spatial)
        
        # Cross-attention: (tempo, batch, d_model)
        x_cross, _ = self.cross_attention(x_spatial, x_temporal, x_temporal)
        
        # Fusione: per esempio, facciamo una media sul tempo (dimensione 0)
        x_fused = self.fc_fusion((x_spatial + x_temporal).mean(dim=0))  # -> (batch, d_model)
        
        # Classificazione finale
        output = self.fc_classify(x_fused)  # -> (batch, num_classes)
        
        return output
    

In [ ]:
'''
Ecco un codice che fornisce dati di input fittizi a ciascuna rete neurale, 
stampa le dimensioni a ogni passaggio e verifica che gli output abbiano le forme attese.

Ho mantenuto le forme coerenti con i tuoi parametri:


Batch size: 8
Numero di canali EEG: 3
Numero di frequenze: 38
Numero di timepoints (campioni temporali): 100
Numero di classi: 2

'''


import torch
import torch.nn as nn
import torch.nn.functional as F

# Parametri
batch_size = 45
input_channels = 61   # Canali EEG
num_freqs = 26       # Numero di frequenze
num_timepoints = 11  # Numero di campioni temporali

num_classes = 2       # Numero di classi

dropout = 0.2

# Creazione di dati fittizi per il test
x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
print(f"Input iniziale: {x.shape}\n")

# ---- CNN2D ----
#cnn_model = CNN2D(input_channels=input_channels, num_classes=num_classes)

cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
cnn_output = cnn_model(x)
print(f"Output CNN2D_LSTM_TF: {cnn_output.shape}\n")  # Atteso: (batch_size, num_classes)


# ---- ReadMEndYou (LSTM) ----
hidden_sizes = [24, 48, 62]
lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
lstm_output = lstm_model(x)
print(f"Output ReadMEndYou (LSTM): {lstm_output.shape}\n")  # Atteso: (batch_size, num_classes)

# ---- ReadMYMind (Transformer) ----
d_model = 64   # Dimensione embedding
num_heads = 8   # Numero di teste di attenzione
num_layers = 3  # Numero di strati Transformer

transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)
transformer_output = transformer_model(x)
print(f"Output ReadMYMind (Transformer): {transformer_output.shape}\n")  # Atteso: (batch_size, num_classes)


#### **Early Stopping - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''
DEFINIZIONE EARLY STOPPING
'''

import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle
import numpy as np


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
        
        """
        :param patience: Numero di epoche da attendere prima di interrompere il training se non c'è miglioramento
        
        Esempio: il training si interromperà se non si osserva un miglioramento per (N = 5) epoche consecutive.
        
        :param min_delta: Variazione minima richiesta per considerare un miglioramento
        
        definisce il miglioramento minimo richiesto per essere considerato significativo. 
        Se il miglioramento è inferiore a min_delta, non viene considerato un vero miglioramento.
        
        Il parametro min_delta in una configurazione di early stopping indica 
        la minima variazione del valore di una metrica 
        (ad esempio, la perdita o l'accuratezza) 
        che deve verificarsi tra un'epoca e la successiva 
        per continuare l'allenamento. 
        
        In genere, il valore di min_delta dipende dal tipo di modello e dai dati specifici, 
        ma di solito si trova in un intervallo tra 0.001 e 0.01.
    
            - Se stai cercando di evitare che l'allenamento si fermi troppo presto,
            puoi impostare un valore più basso per min_delta (come 0.001), 
            - Se vuoi essere più conservativo e permettere fluttuazioni nei valori della metrica,
            un valore più alto (come 0.01) potrebbe essere appropriato.

        Un buon punto di partenza potrebbe essere 0.001, e poi fare dei test per capire quale valore funziona meglio
        nel tuo caso specifico!
        
        :param mode: 'min' per monitorare la loss (minimizzazione), 'max' per l'accuracy (massimizzazione)
        
        'max' → ottimizza metriche da massimizzare (es. accuracy, F1-score, AUC).
        'min' → ottimizza metriche da minimizzare (es. loss).
        
        """
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True


#### **Weight & Biases Login - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''

WEIGHT AND BIASES LOGIN

Il messaggio che stai ricevendo indica 
che sei già connesso al tuo account Weights & Biases (wandb).

Se vuoi forzare il login, puoi usare il comando suggerito:

wandb login --relogin

Questo comando ti permetterà di reinserire le credenziali e riconnetterti al tuo account.
Se non hai bisogno di disconnetterti o di cambiare l'account,
puoi semplicemente continuare a usare wandb senza ulteriori passaggi. 
Hai bisogno di ulteriore assistenza con wandb o con il tuo progetto?
'''


import wandb
wandb.login()
print("✅ Weights & Biases login effettuato con successo!")

In [ ]:
'''
Per modificare il percorso in cui W&B salva i dati localmente,
puoi configurare la variabile di ambiente WANDB_DIR.

Questo ti permette di specificare una directory personalizzata in cui W&B salva tutti i file associati al tuo run, inclusi i dati e i log.
'''

import os

# Imposta la directory per i dati W&B:
# Questo cambierà la cartella in cui W&B salva i dati per quella sessione di esecuzione






'''ATTENZIONE CHE QUI HO AGGIUNTO --> "_time_frequency_" alla WB_dir!'''

# Definisci la cartella principale
WB_dir = "/home/stefano/Interrogait/WB_spectrograms_time_frequency_analyses"
os.makedirs(WB_dir, exist_ok=True)


os.environ["WANDB_DIR"] = WB_dir


In [ ]:
import pickle

# Apri il file in modalità lettura binaria ('rb')

#path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/'
path = '/home/stefano/Interrogait/all_datas/Familiar_Spectrograms/'

with open(f"{path}new_all_th_concat_spectrograms_coupled_exp.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
# Itera sulle chiavi del dizionario principale
for condition, values in data.items():
    if isinstance(values, dict) and "data" in values and "labels" in values:
        X_shape = values["data"].shape
        y_length = len(values["labels"])
        print(f"🔹 Condizione: {condition}")
        print(f"   ➡ Shape dati: {X_shape}")
        print(f"   ➡ Lunghezza labels: {y_length}\n")


#### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''

Sì, è perfettamente normale: in W&B un progetto è semplicemente un contenitore di run e di sweep, 
e rimane visibile (anche se vuoto) fintanto che non lo archivi o lo cancelli esplicitamente. 

Cancellare tutte le run e tutti i sweep in un progetto non elimina il progetto stesso — lascia solo un progetto “vuoto” con zero run/sweep.


Come rimuovere (o archiviare) anche i progetti


1) Dal web UI

Vai nella pagina del progetto che vuoi rimuovere.

Clicca sui tre puntini (⋯) accanto al nome del progetto.
Scegli “Archive project” (per nasconderlo) o, se disponibile, “Delete project” per eliminarlo definitivamente.


2) Da CLI
Se hai una versione recente del wandb CLI, puoi usare (da terminale):


wandb project delete <ENTITY>/<PROJECT_NAME>

Esempio:

wandb project delete stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp_1_20_familiar_th




Programmaticamente
Attualmente W&B non espone un’API Python ufficiale per cancellare progetti; l’opzione più rapida resta il CLI o la UI.

In sintesi
Cancellare run/sweep → lascia il progetto “vuoto” (0 run, 0 sweep).

Progetto visibile → finché non lo archivi o elimini esplicitamente, lo vedrai nella lista.

Per toglierlo di mezzo → archivia o cancella il progetto tramite i tre puntini in UI, oppure con wandb project delete da CLI.



IO:
ok ma quindi se ricreo il progetto con lo stesso nome semplicemente ripopola quel progetto ma con sweeps e runs nuovi giusto?


ChatGPT:

Esatto. In W&B il progetto è identificato dal solo nome (più l’entity) e non ha uno “stato” permanente di esistenza: se tu:

Elimini o archivi totalmente il progetto (tramite UI o wandb project delete),

E poi lanci di nuovo uno wandb.init(..., project="quel_nome"),

W&B creerà un nuovo progetto con lo stesso nome, vuoto all’inizio, in cui andranno a confluire i tuoi nuovi sweep e run.

Quindi:

Se archivi il vecchio progetto, è solo nascosto ma rimane là;

Se lo cancelli definitivamente, sparisce e puoi ricrearlo identico per nome, e in quel momento sarà “pulito” (0 sweep, 0 run)
pronto a riempirsi con i tuoi esperimenti successivi.


'''


import sys
import subprocess
from itertools import product
import wandb

# 1) Parti delle stringhe da combinare
prefixes = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]
middles = [
    "spectrograms_time_frequency",
]

suffixes = [
    "familiar_th",
    "familiar_pt",
    "unfamiliar_th",
    "unfamiliar_pt",
]

# 2) Genera tutti i nomi di progetto
projects = [
    f"{p}_{m}_{s}"
    for p, m, s in product(prefixes, middles, suffixes)
]

# 3) Configura l’API e l’entity
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# 4) Itera su ogni progetto: svuota le run e poi cancella gli sweep
for proj in projects:
    project_path = f"{entity}/{proj}"
    print(f"\n→ Progetto: {project_path}")

    # 4.1 Cancella tutte le run via Python API
    try:
        runs = api.runs(project_path)
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                except Exception as e:
                    print(f"     – Errore cancellando run {run.id}: {e}")
                else:
                    print(f"     – run {run.id} eliminata")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Impossibile caricare le run: {e}")

    # 4.2 Cancella tutti gli sweep via CLI Python module
    #    Evitiamo di chiamare un eseguibile esterno, usiamo `python -m wandb`
    #    Lo stesso interprete che esegue questo script è in sys.executable
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", project_path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    # Ogni riga di res.stdout ha uno sweep_id come primo token
    for line in res.stdout.splitlines():
        sweep_id = line.split()[0]
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{project_path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    print(f"  ✅ Run e sweep eliminati per {project_path}")


#### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e ANCHE SWEEPS!) - EEG Spectrograms - Time x Frequencies V2**

In [ ]:
'''
Perfetto, hai due script:

uno funzionale e robusto che elimina run e sweep, ma usa pattern statici,

uno che usa prefisso, medio e suffisso per generare i nomi dei progetti, ma è meno dettagliato.

Ti creo una versione unificata che:

Usa prefissi, medii e suffissi per generare i nomi dei progetti (come nel secondo script).

Per ogni progetto, cancella tutte le run (come nel primo script).

Cancella tutti gli sweep usando wandb sweep --delete.

Verifica che gli sweep siano stati eliminati correttamente.


✅ Script finale combinato:
    
✅ Cosa fa questo script:

Genera i progetti combinando prefix, middle, suffix.

Cancella le run usando l’API Python (run.delete()).

Cancella gli sweep usando wandb sweep --delete, richiamato tramite subprocess.

Verifica se gli sweep sono stati realmente eliminati, usando proj.sweeps().

📝 Dipendenze e prerequisiti:

wandb dev'essere installato e autenticato (wandb login)

Lo script va eseguito in un ambiente con accesso alla CLI di wandb (es. terminale Python)


'''


import sys
import subprocess
from itertools import product
import wandb

# --- Configurazione ---
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# --- Parti del nome progetto ---
prefixes = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]
middles = [
    "spectrograms_time_frequency",
]
suffixes = [
    "familiar_th",
    "familiar_pt",
    "unfamiliar_th",
    "unfamiliar_pt",
]

# --- Genera i nomi dei progetti ---
project_names = [
    f"{p}_{m}_{s}"
    for p, m, s in product(prefixes, middles, suffixes)
]

# --- Itera su ogni progetto ---
for proj_name in project_names:
    path = f"{entity}/{proj_name}"
    print(f"\n→ Progetto: {path}")

    # --- 1. Elimina tutte le run ---
    try:
        runs = api.runs(path, per_page=None)
        runs = list(runs)
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                    print(f"     – Run {run.id} eliminata")
                except Exception as e:
                    print(f"     – Errore eliminando run {run.id}: {e}")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Errore caricando le run: {e}")
        continue  # salta alla prossima

    # --- 2. Ottieni e cancella gli sweep tramite CLI ---
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    sweep_ids = []
    for line in res.stdout.strip().splitlines():
        sweep_id = line.split()[0]
        sweep_ids.append(sweep_id)
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    # --- 3. Verifica cancellazione sweep ---
    print("   • Verifica sweep attivi dopo la cancellazione...")
    try:
        project_obj = next(p for p in api.projects(entity=entity) if p.name == proj_name)
        remaining_sweeps = project_obj.sweeps()
        if not remaining_sweeps:
            print("   ✅ Nessuno sweep attivo trovato.")
        else:
            print(f"   ⚠️ Sweep ancora attivi: {remaining_sweeps}")
    except Exception as e:
        print(f"   ⚠️ Errore nella verifica sweep: {e}")

    print(f"  ✅ Run e sweep eliminati per {path}")
    

#### **Datasets Loading - EEG Spectrograms - Time x Frequencies**

In [ ]:
##################### CODICE UFFICIALE DEL 04/03/2025 ORE 9:30 #####################
                                 ##################### SENZA DETTAGLI SCRITTI V°3 #####################
        
'''ATTENZIONE: 

HO SOSTITUITO LE VARIABILI DI 

    1) DATASET_TRAIN_LOADER -->  TRAIN_LOADER
    2) DATASET_VAL_LOADER -->  VAL_LOADER

    VEDI FUNZIONE 'PREPARE_DATA_FOR_MODEL --> NOMI DELLE VARIABILI DEI TORCH TENSOR DATASET LOADER SON  'TRAIN_LOADER' E VAL_LOADER!!!  

'''
import os
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy as cp
import numpy as np

import wandb
import random
import copy as cp


# Definisci le lista delle coppie di condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

# Inizializza il dizionario per caricare i dati
data_dict = {}





'''ATTENZIONE CHE QUI HO AGGIUNTO --> "_time_frequency_" alla base_dir!'''

# Definisci la cartella principale
base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_time_frequency"
os.makedirs(base_dir, exist_ok=True)



'''LOOP DI CARICAMENTO DATI'''

for condition in experimental_conditions:
    # Crea la cartella per la condizione sperimentale
    condition_dir = os.path.join(base_dir, condition)
    os.makedirs(condition_dir, exist_ok=True)
    
    # Aggiungi un livello di annidamento per ogni condizione
    data_dict[condition] = {}
    
    for data_type in ["spectrograms"]:
        
        # Crea la cartella per il tipo di dato
        data_dir = os.path.join(condition_dir, data_type)
        os.makedirs(data_dir, exist_ok=True)
        
        for category in ["familiar", "unfamiliar"]:
            # Crea la cartella per la categoria
            #category_dir = os.path.join(data_dir, category)
            #os.makedirs(category_dir, exist_ok=True)
            
            for subject_type in ["th", "pt"]:
                # Caricamento e suddivisione dei dati
                
                #if data_type == "spectrograms":
                    
                print(f"Caricamento dati per: {condition} - {data_type} - {category}_{subject_type}")
                X, y = load_data(data_type, category, subject_type, condition=condition)
                
                
                # Creazione della chiave per il dizionario annidato
                data_dict[condition][data_type] = data_dict[condition].get(data_type, {})
                data_dict[condition][data_type][f"{category}_{subject_type}"] = (X, y)
                
                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{condition}\033[0m_\033[1m{data_type}\033[0m_\033[1m{category}_{subject_type}\033[0m - Shape X: \033[1m{X.shape}\033[0m, Shape y: \033[1m{len(y)}\033[0m\n")

In [ ]:
data_dict['th_resp_vs_pt_resp'].keys()

#### **Sweep Configuration - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            if category_subject not in sweep_ids[condition][data_type]:
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['th_resp_vs_pt_resp'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp'].keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys())
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

In [ ]:
'''OGNI IPER-PARAMETRO DI OGNI RETE


ALLO STESSO LIVELLO DI PARAMETERS!


                                                                POST 14 LUGLIO 2025
                                                                
                                                                
                                                                
                                                                ***CNN2D NEW*** 

1) All'interno di ogni layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

a) il numero di output channels (ossia 16 impostato di default qui sotto, ma che potrebbe variare da 16 a 32 con step di 4 
come grandezza della feature map sostanzialmente

b) la grandezza del kernel size (tra 2 e 8 con step di 2)
c) la grandezza dello stride (metti solo valori tra 1 e 2) 


2) Per il layer di batch normalisation del relativo layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#batchnorm1d

deve avere il valore del numero di features di quel layer di batch normalisation
(che deve corrispondere come valore a quello dell'output channels del layer convolutivo che lo precede sostanzialmente) 


3) Al layer di pooling del relativo strato della della CNN1D, far variare la scelta tra

a) max pooling ed average pooling 

b) Il valore del kernel_size del layer di max od average pooling (a seconda di quello che viene scelto tra i due), 
che può variare tra 1 e 2 

4) Al solo primo layer fully connected della CNN1D, far variare la scelta del suo valore 
(che nella mia rete sarebbe "self.fc1 = nn.LazyLinear(8)") in questo set di valori, ossia tra i valori 8,10,12,14,16

5) Il valore del dropout layer (con valori tra  0.0 e 0.5) 


6) Il valore della possibile funzione di attivazione tra 3 (relu, selu ed elu)

 a) per gli strati convolutivi (3) +
 b) per il primo fully connected layer (FC1) (prendendone una a caso tra quelle 3 possibili



TABELLA FINALE RIASSUNTIVA - CNN1D 


| Iper-parametro                     | Descrizione                                             | Valori possibili                 |
| ---------------------------------- | ------------------------------------------------------- | -------------------------------- |
| `conv_out_channels`                | Numero di feature-map di base                           | `[16, 20, 24, 28, 32]`           |
| `conv_k1`, `conv_k2`, `conv_k3`    | Kernel size rispettivamente per i 3 blocchi convolutivi | `[2, 4, 6, 8]`                   |
| `conv_s1`, `conv_s2`, `conv_s3`    | Stride rispettivamente per i 3 blocchi convolutivi      | `[1, 2]`                         |
| `pool_type`                        | Tipo di pooling                                         | `["max","avg"]`                  |
| `pool_p1`, `pool_p2`, `pool_p3`    | Kernel size rispettivamente per i 3 blocchi di pooling  | `[1, 2]`                         |
| `fc1_units`                        | Numero di unità nel primo fully-connected               | `[8, 10, 12, 14, 16]`            |
| `cnn_act1`, `cnn_act2`, `cnn_act3` | Funzione di attivazione per ciascun blocco (layer1,2,3) | `["relu","selu","elu"]`          |
| **+ comune**                       | `dropout`                                               | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |






                                                                ***BILSTM NEW*** 


Per la rete BiLSTM, bisogna configurare hidden_size e dropout come indicato nel sweep_config, 
insieme alla possibilità di scegliere se utilizzare o meno la bidirezionalità.


1) il valore di hidden_sizes ossia dello spazio di embedding multidimensionale dei miei punti temporali 
del dato EEG (tutti i valori tra 16 e 32 con step di 2, ossia 16, 18, 20.. e così via)

2) la scelta sulla bidirezionalità o meno (True o False)

3) il valore di dropout (tra 0.0 e 0.5)


TABELLA FINALE RIASSUNTIVA - BILSTM


| Iper-parametro  | Descrizione                                       | Valori possibili                 |
| --------------- | ------------------------------------------------- | -------------------------------- |
| `hidden_size`   | Dimensione dello stato nascosto per layer LSTM    | `[16, 18, …, 32]` (passo 2)      |
| `bidirectional` | Se usare LSTM bidirezionale (0 → False, 1 → True) | `[0, 1]`                         |
| **+ comune**    | `dropout`                                         | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |




                                                                ***TRANSFORMER NEW***
                                                                
Per il Transformer varieremo                                                        

1) il valore dell'embedding, ossia "d_model" (con valori tra 8 e 64 con step di 8)
2) il valore di head attenzionali, ossia "num_heads" (con valori tra 2 e 12 con step di 2) 
3) il valore di fully connected layers (con valori tra 1 e 3) dell

4) il valore del feed_forward multiplier: descrive esattamente il suo ruolo, ossia è un moltiplicatore (mult) applicato
alla dimensione del modello (d_model)per fissare l’ampiezza dell’FFN
Ossia, il fattore con cui moltiplichi il tuo d_model per ottenere la dimensione interna del blocco feed-forward nel Transformer!

In pratica, lo sweep esplora solo due moltiplicatori [2,4] invece di decine di valori hard-coded.
Il modello transformer calcola internamente ogni run il corretto dim_feedforward = ff_mult * d_model.
        
5) il valore (stringa) della funzione di attivazione del layer fully connected (tra relu e gelu)


      
        
        
TABELLA FINALE RIASSUNTIVA - TRANSFORMER


| Iper-parametro            | Descrizione                                                     | Valori possibili                 |
| ------------------------- | --------------------------------------------------------------- | -------------------------------- |
| `d_model`                 | Dimensione dell’embedding (modello)                             | `[8, 16, 24, …, 64]` (step 8)    |
| `num_heads`               | Numero di teste di attenzione                                   | `[2, 4, 6, 8, 10, 12]`           |
| `num_layers`              | Numero di blocchi encoder                                       | `[1, 2, 3]`                      |
| `ff_mult`                 | Moltiplicatore per la dimensione interna del feed-forward (FFN) | `[2, 4]`                         |
| `transformer_activations` | Funzione di attivazione nel layer FFN                           | `["relu", "gelu"]`               |
| **+ comune**              | `dropout`                                                       | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |





TABELLA FINALE RIASSUNTIVA - IPER-PARAMETRI COMUNI PER IL LEARNING 


| Iper-parametro                                | Descrizione                                               | Valori                                 |
| --------------------------------------------- | --------------------------------------------------------- | -------------------------------------- |
| `lr`, `weight_decay`, `beta1`, `beta2`, `eps` | Ottimizzatore Adam (learning rate, decay, betas, epsilon) | come da sweep: varie decadi            |
| `n_epochs`                                    | Epoche di training                                        | `100` (fisso)                          |
| `patience`                                    | Pazienza per early-stopping                               | `12` (fisso)                           |
| `batch_size`                                  | Dimensione del batch                                      | `[16, 24, 32, 48, 52, 64, 72, 84, 96]` |
| `standardization`                             | Se standardizzare i dati prima del training               | `[True, False]`                        |
| `dropout`                                     | Tasso di dropout                                          | `[0.0 … 0.5]`                          |
| `model_name`                                  | Scelta del modello in sweep                               | `["CNN1D", "BiLSTM", "Transformer"]`   |



'''


sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        # Ottimizzatore
        "lr":            {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay":  {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "beta1":         {"values": [0.8, 0.85, 0.9, 0.95]},
        "beta2":         {"values": [0.98, 0.99, 0.995, 0.999]},
        "eps":           {"values": [1e-8, 1e-7, 1e-6, 1e-5]},

        # Training
        "n_epochs":      {"value": 100},
        "patience":      {"value": 12},

        # Scelta del modello
        "model_name":    {"values": ["CNN2D", "BiLSTM", "Transformer"]},

        # Dati e regolarizzazione generale
        "batch_size":    {"values": [16, 24, 32, 48, 52, 64, 72, 84, 96]},
        "standardization":{"values":[True, False]},
        
        # --- CNN1D solo quando model_name=="CNN2D" ---
        "conv_out_channels":{"values":[16,20,24,28,32]},

        "conv_k1_h":{"values":[3,5,7,9]},
        "conv_k1_w":{"values":[3,5,7,9]},
        
        "conv_k2_h":{"values":[3,5,7,9]},
        "conv_k2_w":{"values":[3,5,7,9]},
        
        "conv_k3_h":{"values":[3,5,7,9]},
        "conv_k3_w":{"values":[3,5,7,9]},

        "conv_s1_h":{"values":[1,2]},
        "conv_s1_w": {"values":[1,2]},
        
        "conv_s2_h":{"values":[1,2]},
        "conv_s2_w": {"values":[1,2]},
        
        "conv_s3_h":{"values":[1,2]},
        "conv_s3_w": {"values":[1,2]},
        
        "pool_p1_h":{"values":[1,2]},
        "pool_p1_w":{"values":[1,2]},
        
        "pool_p2_h":{"values":[1,2]},
        "pool_p2_w":{"values":[1,2]},
        
        #"pool_p3_h":{"values":[1,2]},
        #"pool_p3_w":{"values":[1,2]},
        
        "pool_p3_h":{"values":[1]},
        "pool_p3_w":{"values":[1]},

        "pool_type":{"values":["max","avg"]},
        "fc1_units":{"values":[8,10,12,14,16]},

        "cnn_act1":{"values":["relu","selu","elu"]},
        "cnn_act2":{"values":["relu","selu","elu"]},
        "cnn_act3":{"values":["relu","selu","elu"]},
        
        
        # --- BiLSTM solo quando model_name=="BiLSTM" ---
        "hidden_size":{"values":list(range(16,34,2))},
        "bidirectional":{"values":[0,1]},

        # --- Transformer solo quando model_name=="Transformer" ---
        "d_model":{"values":list(range(8,65,8))},
        #"num_heads":{"values":[2,4,6,8,10,12]},
        
        "num_heads":{"values":[2,4,8]}, # solo divisori di tutti i d_model
        
        "num_layers":{"values":[1,2,3]},
        "ff_mult":{"values":[2,4]},
        "transformer_activations":{"values":["relu","gelu"]},

        # comune
        "dropout":{"values":[0.0,0.1,0.2,0.3,0.4,0.5]}
    }
}

    
'''SWEEP_IDS_FOR_MODELS'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

for condition in sweep_ids_for_models:
    for data_type in sweep_ids_for_models[condition]:
        for category_subject in sweep_ids_for_models[condition][data_type]:
            for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                sweep_ids[condition][data_type][category_subject] = []
                    
print(f"\nAggiornato \033[1msweep_ids\033[0m")

In [ ]:
import pprint
pprint.pprint(sweep_config)

In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE: A DIFFERENZA DI PRIMA, DOVE GLI SWEEPS ERANO CREATI SOLO PER OGNI CONDIZIONE SPERIMENTALE,
ADESSO INVECE VENGONO CREATI PER OGNI COMBINAZIONI DI FATTORI, CHE INCLUDONO:

1) DATI DI COPPIE DI CONDIZIONI SPERIMENTALI
2) PROVEVIENZA DEI DATI (IN QUESTO SPETTOGRAMMI TIME-FREQUENCY
3) PROVENIENZA DEI DATI STESSI (FAMILIAR VS UNFAMILIAR; THERAPIST VS PATIENT)

'''

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_{data_type}_time_frequency_{category_subject}")

                    '''QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                     CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA '''
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['th_resp_vs_pt_resp'].keys()

#### **Sweep Configuration - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''



from sklearn.model_selection import train_test_split

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Modelli che useremo nei sweep
MODEL_LIST = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"] #ReadMEndYou and ReadMYMind



# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            # 1. Prepara spazio nei dizionari: sotto category_subject, un dict per ogni modello
            
            data_dict_preprocessed[condition][data_type][category_subject] = None
            
            if category_subject not in sweep_ids[condition][data_type]:
                
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }

                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids_for_models[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test Set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['th_resp_vs_pt_resp'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp'].keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys())
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
''' 

                                                                    AGGIORNATA AL 19 LUGLIO
                                                                    
                                                                    
#"learning rate : {"value"[1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]"}
#"n_epochs": {"value": 100},
# "patience": {"value": 12},
#"batch_size": {"values": [16, 24, 32, 48, 64, 72, 84, 96]}
#"standardization": {"values": [True, False]}, 
# "beta1": {"values": [0.8, 0.85, 0.9, 0.95]},
#  "beta2": {"values": [0.98, 0.99, 0.995, 0.999]},
#  "eps": {"value": [1e-8, 1e-7, 1e-6, 1e-5]}                                                                                                                            



sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]}, # fissato al valore di default del paper

        "weight_decay":  {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        
        
        "model_name":{"values": ['CNN3D_LSTM_FC']},

        "batch_size": {"values": [32, 48, 64, 96]},

        "standardization":{"values": [True, False]},

        "beta1": {"values": [0.9, 0.95]},

        "beta2": {"values": [0.99, 0.995]},
        
        "eps": {"values": [1e-8, 1e-7]},
        
        #In questo modo:
        
        "use_lstm":      {"values":[True, False]},
        "lstm_hidden":   {"values":[32]},
        "dropout":       {"values":[0.5]},
        
    }
}


'''


#Tutti gli sweep saranno organizzati sotto lo stesso progetto,
#che corrisponde alla coppia di condizioni sperimentali corrente (i.e., exp_cond).

#Questo significa che tutte le runs che verranno lanciate con quello sweep, 
#saranno associate a quella specifica coppia di condizioni sperimentali corrente.

#Dato che sto iterando su ogni coppia di condizioni sperimentali, 
#ogni sweep verrà automaticamente salvato all'interno del progetto corrispondente 
#della specifica condizione sperimentale (exp_cond).

#In pratica, se hai più condizioni sperimentali 
#(ad esempio, "Condizione_A", "Condizione_B", ecc.),
#WandB creerà automaticamente sweep separati all'interno dei rispettivi progetti


#Creo la configurazione dello sweep e la eseguo:

#uno per il modello CNN3D_LSTM_FC, originariamente era così


#sweep_config_cnn3d= {
#    "method": "random",
#    "metric": {"name": "val_accuracy", "goal": "maximize"},
#    "parameters": {
#        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
#        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
#        "n_epochs": {"value": 100},
#       "patience": {"value": 12},
#        "model_name": {"values": ["CNN2D_LSTM_TF"]},
#        "batch_size": {"values": [32, 48, 64, 96]},
        
#        "standardization": {"values": [True,False]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
#        "beta1": {"values": [0.9, 0.95]},
#        "beta2": {"values": [0.99, 0.995]},
#        "eps": {"values": [1e-8, 1e-7]},
#        "use_lstm": {"values": [True, False]},
#        "lstm_hidden": {"values": [32]},
#        "dropout": {"values": [0.5]},
#    }
#}


#sweep_config_cnn_sep = {
#    "method": "random",
#    "metric": {"name": "val_accuracy", "goal": "maximize"},
#    "parameters": {
#        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
#        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
#        "n_epochs": {"value": 100},
#        "patience": {"value": 12},
#        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
#        "batch_size": {"values": [32, 48, 64, 96]},
#        "standardization": {"values": [True,False]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
#        "beta1": {"values": [0.9, 0.95]},
#        "beta2": {"values": [0.99, 0.995]},
#        "eps": {"values": [1e-8, 1e-7]},
#        "use_lstm": {"values": [True, False]},
#        "lstm_hidden": {"values": [32]},
#        "dropout": {"values": [0.5]},
#    }
#}


'''

# Comodo mapper per il tuo loop
sweep_config_dict_stft = {
    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
    "ReadMEndYou": sweep_config_bilstm,
    "ReadMYMind": sweep_config_transformer,
}
Nota pratica (per l’integrazione nel training)
CNN2D_LSTM_TF: passa dropout=config.dropout.

ReadMEndYou: costruisci con
ReadMEndYou(input_size=channels*freqs, hidden_sizes=[config.hidden1, config.hidden2, config.hidden3],
output_size=2, dropout=config.dropout, bidirectional=config.bidirectional).

ReadMYMind: costruisci con
ReadMYMind(d_model=config.d_model, num_heads=config.num_heads, num_layers=config.num_layers, num_classes=2, channels=config.channels, freqs=config.freqs).

'''
# 2.1 – Sweep config per ciascun modello

#CNN2D_LSTM_TF
sweep_config_cnn2d_lstm_tf = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
    
        "dropout": {"values": [0.5]},
    }
}


sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "dropout": {"values": [0.5]},
        "bidirectional": {"values": [False, True]},
        
        #Soluzione 1 per mettere valori agli hidden sizes
        #"hidden1": {"values": [24, 32, 48, 64]},
        #"hidden2": {"values": [48, 64, 96, 128]},
        #"hidden3": {"values": [62, 96, 128, 160]}
        # in build del modello: hidden_sizes=[hidden1, hidden2, hidden3]
        
        #Soluzione 2 per mettere valori agli hidden sizes
        
        #hidden_sizes = [24, 48, 62]
        #lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    }
}


sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "d_model": {"values": [32]},
        "num_heads": {"values": [2]},
        "num_layers": {"values": [2]},
    }
}





    
'''SWEEP_IDS_FOR_MODELS

# 2) Popolo sweep_ids_for_models in base a MODEL_LIST (già inizializzato nella prima cella)
'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

#for condition in sweep_ids_for_models:
    #for data_type in sweep_ids_for_models[condition]:
        #for category_subject in sweep_ids_for_models[condition][data_type]:
            #for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                #if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    #sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


'''BEST_MODELS

# 3) Creo best_models da sweep_ids_for_models
'''

#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


#'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
#for condition in sweep_ids:
    #for data_type in sweep_ids[condition]:
        #for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            #if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                #sweep_ids[condition][data_type][category_subject] = []
                    
#print(f"\nAggiornato \033[1msweep_ids\033[0m")




In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
print(best_models)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE CHE A QUESTO PUNTO


1) sweep_ids[cond][dtype][cat][model_name] contiene le tuple (sweep_id, combo_key) per ciascun modello, che ancora non esistono perché devo esser create durante la creazione degli sweeps, ma ho solo una lista

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': []}}, 
'rest_vs_right_fist': {'spectrograms': {'familiar_th': []}}, 
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': []}}}


2) sweep_ids_for_models e best_models sono paralleli a sweep_ids con lo stesso livello model_name

ossia 

sweep_ids_for_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}},
'rest_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}},
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}}}

best_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'rest_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},


'''


In [ ]:
'''
Popolamento di sweep_ids e lancio degli agenti:

Obiettivo: 

Per ogni combinazione (condition, data_type, category_subject, model_name), 
Se la lista è vuota, crei uno sweep usando wandb.sweep(sweep_config, project=condition) e lo inserisci nella lista. 
In seguito, iteri su quella lista (che ora contiene IL TUO SPECIFICO sweep_id) e lanci wandb.agent() per eseguire il training.



Nota importante:
L'ID restituito da wandb.sweep() è una STRINGA UNIVOCA generata automaticamente da WandB.
Non puoi assegnargli direttamente una stringa personalizzata, ma puoi comunque usarlo per mappare nel tuo dizionario la combinazione di fattori! 

In questo ciclo, il fatto che la lista parta vuota è normale: il codice la popola se necessario e poi lancia l'agente per ogni sweep_id presente.


****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******
INOLTRE, BISOGNA CONTROLLARE CHE SI STIA ITERANDO CORRETTAMENTE SOLO SULLA COMBINAZIONE CORRENTE DI 

                CONDITION, DATA_TYPE, CATEGORY_SUBJECT E MODEL_NAME
                
QUESTO PERCHÉ SE UN CICLO SI RIPETE PER UNA CONDIZIONE IN PIÙ UNA COMBINAZIONE, POTREBBE GENERARE PIÙ  SWEEP IDS DI QUELLI CHE TI ASPETTI!
****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******



SOLUZIONE:

Un buon approccio per evitare la creazione ripetuta di Sweep ID 
per la stessa combinazione di condition, data_type, category_subject e model_name 
è quello di utilizzare un SET per tenere traccia delle combinazioni già processate.
Se una combinazione è già presente nel set, non dovresti creare un nuovo Sweep ID, ma semplicemente saltare quella parte del codice


Inoltre, ho avuto una idea ad un certo punto! 


****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************


Quando creo ogni sweep singolarmente, si genera una stringa univoca di quello sweep, che si riferisce ad un dataset che è il prodotto di diversi fattori:

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Di conseguenza, iterando su ogni sweep_ids (che ho fatto in modo avesse la STESSA struttura dei miei dati già splittati i.e, data_dict_preprocessed
io posso, 

1) da un lato eseguire la creazione della stringa univoca associata a quello sweep,
2) crearmi una 'combination_key', che sarebbe l'insieme delle stringhe che descrivono quel dataset specifico di data_dict_preprocessed

che sarà costituito da

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Poiché quindi so già la corrispondenza tra ogni Sweep ID e la sua combinazione di fattori (condition, data_type, category_subject), 
posso creare un MAPPING, che associ, ad certo Sweep ID e la stringa che descrive i suoi fattori associati!


In questo modo, forse, si riesce a risolvere il PROBLEMA 2 NELLA CELLA DI CREAZIONE DELLA FUNZIONE DI TRAINING (VEDI SOTTO!)



                                                        ******IMPORTANTE MODIFICA*****
                                                        
Ora lo sweep_ids non si deve sdoppiare ora, perché sostanzialmente, 
per ogni modello si creano gli sweeps ids corrispondenti e salvati come valore
dentro la chiave del modello corrispondente, sotto forma di tupla...

cioè non più così

"sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))"

ma una cosa del genere

"sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))




COME ERA PRIMA

#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
'''


'''
ADESSO


Cosa fa questo snippet

Cicla su ogni (condition, data_type, category_subject) una volta sola grazie a created_combinations.

All’interno, fa un sottoloop su MODEL_LIST (i tuoi due modelli).

In base a model_name, sceglie sweep_config_cnn3d o sweep_config_cnn_sep.

Chiama wandb.sweep(...) con il config giusto e salva il risultato in


sweep_ids[condition][data_type][category_subject][model_name]
anziché nella lista “piatta” che avevi prima.


In questo modo:

sweep_ids[cond][dtype][cat] resta un dict con due chiavi ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")

Ognuna di quelle chiavi punta a una propria lista di tuple (sweep_id, combo_key)

Non serve sdoppiare l’intero sweep_ids, perché tiene già separati gli sweep di ciascun modello

Più tardi, quando lancerai gli agent, ti basterà:


for model_name, sweeps in sweep_ids[cond][dtype][cat].items():
    for sweep_id, combo_key in sweeps:
        # qui scegli il train_fn in base a model_name
        wandb.agent(sweep_id, function=train_fn_map[model_name], count=200)
e ogni modello girerà solo i suoi sweep.



Alla fine, sweep_ids avrà la forma:

{
  'rest_vs_left_fist': {
    'spectrograms': {
      'familiar_th': {
         'CNN3D_LSTM_FC':       [(sweep_id_1, 'rest_vs_left_fist_spectrograms_familiar_th')],
         'SeparableCNN2D_LSTM_FC': [(sweep_id_2, 'rest_vs_left_fist_spectrograms_familiar_th')]
      }
    }
  },
  …
}
'''


#Ecco come puoi riscrivere solo la TERZA CELLA (quella in cui crei effettivamente gli sweep) 
#mantenendo la tua struttura “a celle” e usando per ognuno il sweep_config giusto in base al model_name.

#Creazione degli sweep (Terza cella)
#Ecco il solo snippet che devi usare per creare gli sweep ripartiti per modello, usando i due sweep_config_*:


'''
Per mantenere la stessa logica di prima ma tenendo conto che ora stai lavorando con modelli separati, 
dovresti modificare il controllo in modo che verifichi se una combinazione di condition, data_type, category_subject
è già stata processata per ciascun modello.

Quindi, il controllo dovrebbe essere fatto separatamente per ogni modello dentro il loop che itera sui modelli (MODEL_LIST).
Di seguito ti mostro la versione modificata che tiene conto di questo:



#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
                
                
'''

                    
'''

Cosa è stato cambiato rispetto alla versione precedente?
Controllo della combinazione di modello:
La logica del controllo della combinazione (combination_key, model_name) nel set created_combinations è corretta, 
perché vogliamo evitare di creare più volte lo stesso sweep per una combinazione di condition, data_type, category_subject, e model_name.

Controllo e creazione dello sweep:
Il codice controlla prima se la combinazione con il modello non è stata già processata 
con il controllo if (combination_key, model_name) not in created_combinations. 

Se non è stata processata, procede a creare lo sweep corrispondente. 
Se la combinazione esiste già, salta la creazione dello sweep per quel modello.

Aggiunta del nuovo sweep ID:
Una volta creato il nuovo sweep per il modello, viene aggiunto correttamente 
alla lista del modello specifico sotto sweep_ids[condition][data_type][category_subject][model_name].

Aggiunta al set delle combinazioni:
Dopo aver creato lo sweep, aggiungiamo (combination_key, model_name) al set created_combinations
per tenere traccia delle combinazioni già elaborate.

Verifica della logica:
La combinazione (combination_key, model_name) deve essere unica per ciascun modello, 
e quindi il controllo che evita duplicazioni nel set è corretto.

La creazione dello sweep per ciascun modello separato è mantenuta, 
e viene applicata solo quando la combinazione specifica non è già stata elaborata per quel modello.

In questo modo, la logica funziona come nel codice precedente, ma ora si tiene conto anche dei modelli separati, 
creando un sweep per ciascuno di essi e mantenendo la traccia delle combinazioni in modo appropriato.

'''
created_combinations = set()



#CNN2D_LSTM_TF
#sweep_config_cnn2d_lstm_tf

#sweep_config_bilstm
#sweep_config_transformer

# Per semplicità, tieni MODEL_LIST a portata di mano
MODEL_LIST = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"
            
            # per ciascun modello, creo uno sweep separato
            for model_name in MODEL_LIST:

                # Controlla se la combinazione di condition, data_type, category_subject + modello è già stata elaborata
                if (combination_key, model_name) not in created_combinations:

                    # Scegli il config in base al model_name
                    if model_name == "CNN2D_LSTM_TF":
                        sweep_conf = sweep_config_cnn2d_lstm_tf
                        
                    elif model_name == "BiLSTM":  # ReadMEndYou
                        sweep_conf = sweep_config_bilstm
                        
                    elif model_name == "Transformer":  # ReadMYMind
                        sweep_conf = sweep_config_transformer
                    else:
                        raise ValueError(f"Modello non riconosciuto: {model_name}")
                    
                    # Controllo se la lista per il modello specifico è vuota
                    if not sweep_ids[condition][data_type][category_subject][model_name]:

                        # Crea lo sweep e lo appendo nella lista dedicata a quel modello
                        
                        '''
                        QUESTO COME ERA IMPOSTATO PRIMA NELLA VERSIONE SENZA SWEEP CONFIG MODEL-SPECIFIC
                        
                        #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_new")
                        '''
                        
                        #new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")
                        
                        '''
                        QUESTO SAREBBE COME ORA LO IMPOSTO PER LA VERSIONE CON SWEEP CONFIG MODEL-SPECIFIC
                        
                        '''
                        
                        #new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_time_freqs_new_3d_grid_multiband")
                        
                        new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_{data_type}_time_frequency_{category_subject}")
                        
                        
                        #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                        #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                        
                        sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))

                    print(f"▶ Sweep \033[1m{new_sweep_id}\033[0m creato per \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
                    
                    # Aggiungi la combinazione al set per evitare duplicazioni
                    created_combinations.add((combination_key, model_name))  # Aggiungi la combinazione con il modello
                else:
                    # Se la combinazione è già stata creata, salta
                    print(f"⚠️ {combination_key} già processato per il modello {model_name}, skip.")
                    continue


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['th_resp_vs_pt_resp'].keys()

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA)**

##### **Training Function Edits - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

In [ ]:
'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

# Test
combination_key = "pt_resp_vs_shared_resp_spectrograms_familiar_th"
condition_experiment, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Soggetto:", subject_key)


In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    
    '''OLD VERSION'''
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''NEW VERSION
    
    Questo assicura la coerenza tra la creazione degli sweep e le run che li eseguono,
    e permette di tracciare meglio ogni combinazione anche su W&B.
    '''
    wandb.init(project = f"{condition}_{data_type}_time_frequency_{category_subject}", name = run_name, tags = tags)
    
    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo

        
    # Inizializza il modello in base al valore scelto in config.model_name
    
    '''OLD VERSION'''
    #if config.model_name == "CNN2D":
        #model = CNN2D(input_channels=61, num_classes=2)
    
    #Caricamento dati per: th_resp_vs_pt_resp - spectrograms - familiar_th
    #Dataset caricato: th_resp_vs_pt_resp_spectrograms_familiar_th - Shape X: (1586, 3, 26, 11), Shape y: 1586
 
    '''PRENDO LA SHAPE DEI DATI PER FORNIRE VALORI GIUSTI PER OGNI INPUt DI CIASCUNA RETE'''
    
    # Appena caricato X_train, X_val, X_test, etc.
    # X_train.shape == (N, channels, freq_bins, time_steps)
    
    _, channels, freq_bins, time_steps = X_train.shape
        

    #if config.model_name == "CNN2D":
        
        # Canali EEG  
        #input_channels = 3
        
        #input_channels = channels * freq_bins
        
        #Classi fissi
        #num_classes = 2
        
        # conv_kernel_size_2d è una tupla di 3 coppie: [ ((h1,w1),(h2,w2),(h3,w3)) ]
        #k1, k2, k3 = config.conv_kernel_size_2d
        
        # conv_stride_2d è una tupla di 3 coppie: [ ((s1,s1),(s2,s2),(s3,s3)) ]
        #s1, s2, s3 = config.conv_stride_2d
        
        # pool_kernel_size_2d è una tupla di 3 coppie: [ ((p1,p1),(p2,p2),(p3,p3)) ]
        #p1, p2, p3 = config.pool_kernel_size_2d
        
        #model = CNN2D(
            #input_channels   = input_channels,
            #num_classes      = num_classes,
            #conv_out_channels= config.conv_out_channels,
            
            #conv_kernel_size = (k1, k2, k3),
            #conv_stride      = (s1, s2, s3),
            
            #pool_type        = config.pool_type,
            
            #pool_kernel_size = (p1, p2, p3),
            
            #fc1_units        = config.fc1_units,
            #dropout          = config.dropout,
            
            #activations      = tuple(config.activations)
        #)
        
    '''NEW VERSION'''
    
    if config.model_name =="CNN2D":
        
        # Canali EEG  
        #input_channels = 3
        
        input_channels = channels 
        
        #Classi da riconoscere
        num_classes = 2
        
        model = CNN2D(
            input_channels   = channels,
            num_classes      = num_classes,
            conv_out_channels= config.conv_out_channels,

            conv_k1_h = config.conv_k1_h, 
            conv_k1_w = config.conv_k1_w,
            
            conv_k2_h = config.conv_k2_h, 
            conv_k2_w = config.conv_k2_w,
            
            conv_k3_h = config.conv_k3_h,
            conv_k3_w = config.conv_k3_w,

            conv_s1_h = config.conv_s1_h, 
            conv_s1_w = config.conv_s1_w,
            
            conv_s2_h = config.conv_s2_h,
            conv_s2_w = config.conv_s2_w,
            
            conv_s3_h = config.conv_s3_h,
            conv_s3_w = config.conv_s3_w,

            pool_p1_h = config.pool_p1_h,
            pool_p1_w = config.pool_p1_w,
            
            pool_p2_h = config.pool_p2_h,
            pool_p2_w = config.pool_p2_w,
            
            pool_p3_h = config.pool_p3_h,
            pool_p3_w = config.pool_p3_w,
            
            pool_type = config.pool_type,

            fc1_units = config.fc1_units,
            dropout   = config.dropout,

            cnn_act1  = config.cnn_act1,
            cnn_act2  = config.cnn_act2,
            cnn_act3  = config.cnn_act3,
        )
    
        
        print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
    
    
    #'''NEW VERSION'''
    elif config.model_name == "BiLSTM":
        
        # Input Size = channels * freq_bins = 3 * 26 = 78
        input_size = channels * freq_bins
        
        # Classi 
        num_classes = 2 
        
        model = ReadMEndYou(
            input_size   = input_size,
            hidden_size  = config.hidden_size,
            output_size  = num_classes,
            num_layers   = 3,
            dropout      = config.dropout,
            bidirectional= config.bidirectional
        )
        print(f"\nInizializzazione Modello \033[1mBiLSTM\033[0m")
    
    
    
    #'''OLD VERSION'''
    #elif config.model_name == "Transformer":
        # Per il Transformer, passiamo anche i parametri channels e freqs per adattare l'embedding
        #model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)

    #else:  # Transformer
        #num_classes = 2 
        #num_channels = channels
        
        #model = ReadMYMind(
            #num_channels= num_channels,
            #freq_bins   = freq_bins,
            #d_model     = config.d_model,
            #num_heads   = config.num_heads,
            #num_layers  = 3,
            #num_classes = num_classes
        #)
    
    
    #''NEW VERSION'''
    elif config.model_name == "Transformer":
        
        num_classes = 2      # o il numero di classi che hai
        
        model = ReadMYMind(
            num_channels            = channels,
            num_freqs               = freq_bins,
            
            seq_length              = time_steps,
            
            d_model                 = config.d_model,
            num_heads               = config.num_heads,
            
            num_layers              = config.num_layers,
            num_classes             = num_classes,
            
            ff_mult                 = config.ff_mult,
            dropout                 = config.dropout,
            transformer_activations = config.transformer_activations,
        )

        print(f"\nInizializzazione Modello \033[1mTransformer\033[0m")

        
    #ORIGINAL VERSION OF TIME SERIES EEG DATA REPRESENTATION  
    #def initialize_models():
        #model_CNN = CNN1D(input_channels=3, num_classes=2)
        #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
        
        #return model_CNN, model_LSTM, model_Transformer
        
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    # 1) Optimizer con betas, eps, weight_decay
    optimizer = optim.Adam(
        model.parameters(),
        lr=config.lr,
        betas=(config.beta1, config.beta2),
        eps=config.eps,
        weight_decay=config.weight_decay
    )
    
    
    criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    '''OLD VERSION'''
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')

        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)

        wandb.log({
            "epoch": epoch,
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            "val_loss": loss_val,
            "val_accuracy": accuracy_val
        })

        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break
        
        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break
            
    
        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #model_file = f"{model_path}/{best_model_name}.pth"
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #torch.save(best_model.state_dict(), model_file)
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                #model_file = f"{model_path}/{best_model_name}.pth"

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    
                    '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                    #torch.save(best_model.state_dict(), model_file)

                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
        
    return best_models


#### **Weight & Biases Procedure Final Edits - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config,
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        
                
# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                sweep_id, combination_key = sweep_tuple
                
                # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione
                
                
                # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                if num_gpus > 1:
                    
                    # Assegna la GPU in modo rotazionale
                    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                    
                    '''OLD VERSION'''
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{condition}", count=100)
                    
                    '''NEW VERSION'''
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{combination_key}", count=200)
                    
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project =f"{condition}_{data_type}_time_frequency_{category_subject}", count=200)
                    
                    
                    # Passa alla prossima GPU per il prossimo sweep
                    gpu_counter = (gpu_counter + 1) % num_gpus

                else:
                    # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                    
                    '''OLD VERSION'''
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{condition}", count=100)
                    
                    '''NEW VERSION'''
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{combination_key}", count=200)
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project =f"{condition}_{data_type}_time_frequency_{category_subject}", count=200)
                    
                # Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
                #def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
                    #def train_wrapper():
                        
                        # Qui chiamiamo la funzione di training con i parametri appropriati
                        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
                        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
                        #training_sweep(
                            #data_dict_preprocessed, 
                            #sweep_config,
                            #sweep_ids,
                            #sweep_id,
                            #sweep_tuple,
                            #best_models  # Best models viene aggiornato all'interno della funzione
                        #)
                    #return train_wrapper
                
                # Crea la funzione wrapper per l'agent
                '''COMMENTATO'''
                #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                
                
                # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                   ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''
                
                print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")
                
                '''COMMENTATO'''
                #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)
                
                print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")
                
                torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

In [ ]:
print('Finito Training su W&B !')

In [ ]:
# Stampa il numero totale di sweeps
#print(f"Numero totale di sweeps che verranno eseguiti: {total_sweeps}")

In [ ]:
#sweep_ids.keys()

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA)**

##### **Training Function Edits - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|th_resp_vs_shared_resp|pt_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")
    

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    
    '''OCCHIO DA CAMBIARE CAMBIATO'''
        
    #wandb.init(project=f"{exp_cond}_spectrograms_channels_freqs_new_3d_grid_multiband", name=run_name, tags=tags)
    
    #PER TASK 1/3
    wandb.init(project=f"{exp_cond}_{data_type}_time_frequency_{category_subject}", name=run_name, tags=tags)
    
    #PER TASK 2/4
    #wandb.init(project=f"{exp_cond}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", name=run_name, tags=tags)

    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo

        
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
    #    model = CNN2D(input_channels=64, num_classes=2)
    #    print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
    
    '''# ====== MODELLI TIME×FREQ ======
    
    cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
    hidden_sizes = [24, 48, 62]
    lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)
    
    # Creazione di dati fittizi per il test
    x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
    
    '''
    
    # ricavo channels e freqs dai dati (shape attesa: N, C, F, T)
    channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])
    
    if config.model_name == "CNN2D_LSTM_TF":
        
        '''OCCHIO QUI ADESSO SAREBBE TEMPO x FREQUENZA'''
    
        model = CNN2D_LSTM_TF(
            input_channels = channels, # qui 61
            num_classes = 2,
            dropout=config.dropout,
        )

        print(f"\nInizializzazione Modello \033[1mCNN2D_LSTM_TF\033[0m")
    
    
    elif config.model_name == "BiLSTM":
        hidden_sizes = [24, 48, 62]
        model = ReadMEndYou(
            input_size = channels * freqs, #  qui 61*26
            hidden_sizes = hidden_sizes,
            output_size = 2,
            dropout = config.dropout,
            bidirectional = config.bidirectional
        )
        print(f"\nInizializzazione Modello \033[1mReadMEndYou (BiLSTM)\033[0m")
    
    
    elif config.model_name == "Transformer":
        model = ReadMYMind(
            d_model=config.d_model,
            num_heads=config.num_heads,
            num_layers=config.num_layers,
            num_classes=2,
            channels = channels,
            freqs = freqs 
            
        )
        
        print(f"\nInizializzazione Modello \033[1mReadMYMind (Transformer)\033[0m")
    else:
        raise ValueError(f"Modello sconosciuto: {config.model_name}")
        
    #elif config.model_name == "BiLSTM":
        # Qui, input_size = canali * frequenze = 3 * 38 = 78
        #model = ReadMEndYou(input_size= 64 * 45, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #print(f"\nInizializzazione Modello \033[1mBiLSTM\033[0m")
        
    #elif config.model_name == "Transformer":
        # Per il Transformer, passiamo anche i parametri channels e freqs per adattare l'embedding
        #model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=45)
        #print(f"\nInizializzazione Modello \033[1mTransformer\033[0m")

        
    #ORIGINAL VERSION OF TIME SERIES EEG DATA REPRESENTATION  
    #def initialize_models():
        #model_CNN = CNN1D(input_channels=3, num_classes=2)
        #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
        
        #return model_CNN, model_LSTM, model_Transformer

    '''
    Cosa è cambiato rispetto alla tua versione
    Optimizer Adam ora prende betas=(0.9,0.999) e eps=1e-8.

    ReduceLROnPlateau posizionato subito dopo l’optimizer, chiamato su val_loss ogni epoca.

    EarlyStopping con patience=12, mode='min' su val_loss.

    Loop sulle epoche fino a config.n_epochs (100), senza limitare a 60.

    Tutti i parametri di sweep_config (lr, weight_decay, n_epochs, patience, batch_size, standardization…) rimangono esposti e loggati.

    In questo modo riproduci fedelmente il training descritto nel paper, senza stravolgere la tua pipeline di sweep.
    '''
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    optimizer = optim.Adam(
        model.parameters(),
        lr = config.lr,               # da sweep: es. [0.01,0.001,...]
        betas = (config.beta1, config.beta2),          # paper
        eps = config.eps,                    # paper
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    #'''AGGIORNAMENTI FINALI'''
    #from sklearn.metrics import roc_auc_score

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        # ---------------------- TRAIN ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.train()  
        
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
            
            #'''AGGIORNAMENTI FINALI'''
            
            # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
            # OPZIONE A: puoi usare la Softmax per avere le probabilità,
            # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
            
            # Opzione A: usare le probabilità (softmax) 
            
            #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
            
            #probs_train = torch.softmax(y_pred, dim=1)
            #y_score_train_list.extend(probs_train[:, 1].detach().cpu().numpy())
            
            # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
            
            #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
            
            # y_score_train_list.extend(y_pred[:, 1].detach().cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        '''come dovrebbe essere calcolato se non si dovesse passare al load_best_run_results'''
        #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list)
        
        '''come è stato calcolato se si dovesse passare al load_best_run_results'''
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        
        
        #'''AGGIORNAMENTI FINALI'''
        #try:
            #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel train set c'è una sola classe.")
            #auc_val = np.nan
        
        # ---------------------- VALIDATION ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.eval()
        
        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []
        
                
        #'''AGGIORNAMENTI FINALI'''
        #y_score_val_list = []  # per AUC valida

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())
                
                #'''AGGIORNAMENTI FINALI'''
                
                # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
                
                # OPZIONE A: puoi usare la Softmax per avere le probabilità,
                # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
                
                # Opzione A: usare le probabilità (softmax) 
                
                #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
                
                #probs_val = torch.softmax(y_pred, dim=1)
                #y_score_val_list.extend(probs_val[:, 1].detach().cpu().numpy())
                
                # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
                
                #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
                # y_score_val_list.extend(y_pred[:, 1].detach().cpu().numpy())
                

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)
        
        #'''AGGIORNAMENTI FINALI'''
        #precision_val = precision_score(y_true_val_list, y_pred_val_list, average='weighted')
        #recall_val    = recall_score(y_true_val_list, y_pred_val_list, average='weighted')
        #f1_val        = f1_score(y_true_val_list, y_pred_val_list, average='weighted')
        
        #try:
            # ATTENZIONE: qui usiamo gli score continui, NON le etichette
            #auc_val = roc_auc_score(y_true_val_list, y_score_val_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel validation set c'è una sola classe.")
            #auc_val = np.nan

        wandb.log({
            "epoch": epoch,
            
            # TRAIN
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            
            # VALIDATION
            
            "val_loss": loss_val,
            "val_accuracy": accuracy_val,
            
            # se vuoi loggare anche queste (consigliato):
            
            #"val_precision": precision_val,
            #"val_recall": recall_val,
            #"val_f1": f1_val,
            #"val_auc": auc_val,
        })
        
        #Nota: questa patch qua sopra (correzione su train e validation) rende corretto anche train_auc per le run future, 
        #quindi non avrai più bisogno della “correzione a posteriori” in load_best_run_results 
        #per i nuovi esperimenti (ma la puoi lasciare per compatibilità coi vecchi run).

        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break

        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #model_file = f"{model_path}/{best_model_name}.pth"
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #torch.save(best_model.state_dict(), model_file)
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                #model_file = f"{model_path}/{best_model_name}.pth"

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    
                    '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                    #torch.save(best_model.state_dict(), model_file)

                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
    
    return best_models


#### **Weight & Biases Procedure Final Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**
#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
print('ciao')

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''



'''

Per modificare il loop in modo che accetti i sweeps per ogni modello e gestisca correttamente
l'esecuzione del training per ciascun modello con il relativo sweep, dobbiamo fare alcune modifiche.


Modifiche principali:

1) Funzione make_train_wrapper:
La funzione dovrà essere adattata per passare correttamente la configurazione di sweep per ogni modello, 
invece di passare un'unica configurazione generica (sweep_config).

2) Identificazione corretta del modello: 
Nel loop, per ogni combinazione (condition, data_type, category_subject)
e per ogni modello (ad esempio, CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC), 

dobbiamo passare al wandb.agent il relativo sweep ID per il modello e la sua configurazione.

3) Modifica della funzione make_train_wrapper per gestire ogni modello separatamente: 
Ogni modello avrà il proprio sweep e la propria configurazione.


Spiegazione delle modifiche:

1) Funzione make_train_wrapper:

Adesso prende anche model_name per passare il relativo sweep_config dal dizionario sweep_config_dict.
Passa il sweep_config corretto per ogni modello, a seconda del model_name passato nel ciclo.

2) Dizionario sweep_config_dict:

Ho creato un dizionario sweep_config_dict che associa ciascun modello ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")
alla sua configurazione di sweep (sweep_config_cnn3d e sweep_config_cnn_sep).
Questo permette di usare la corretta configurazione per ogni modello.

3) Modifica nel ciclo:

Il ciclo ora scorre su model_name (i.e., i modelli CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC) 
per ogni combinazione di condition, data_type, category_subject.

Per ogni modello, il relativo sweep viene creato ed eseguito.


Risultato:
Ora, per ogni combinazione di condition, data_type, e category_subject, 
il codice creerà e gestirà separatamente gli sweeps per ciascun modello,
e li eseguirà utilizzando la funzione training_sweep con la relativa configurazione specifica per ogni modello.

Questa modifica ti consente di avere il corretto flusso di lavoro per eseguire
il training separato per ogni modello con la sua configurazione.


'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili

'''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper'''

def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config_dict[model_name], # Prendi la configurazione per il modello specifico
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        

# Dizionari di configurazione per ogni modello

# Comodo mapper per il tuo loop

#sweep_config_dict = {
#    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
#    "BiLSTM": sweep_config_bilstm,
#    "Transformer": sweep_config_transformer,
#}


'''AL PRIMO GIRO ABILITA SOLO QUESTO'''
#sweep_config_dict = {
#    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
#    "BiLSTM": sweep_config_bilstm
#}

'''AL SECONDO GIRO ABILITA QUESTO'''
sweep_config_dict = {
    "Transformer": sweep_config_transformer
}

'''AL SECONDO GIRO ABILITA QUESTO'''
enabled_models = set(sweep_config_dict.keys())  # {'Transformer'}


# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for model_name in sweep_ids[condition][data_type][category_subject]:  # Aggiunto loop per il modello
                
                '''AL SECONDO GIRO ABILITA QUESTO'''
                # ⬇️ SKIPPA tutti i modelli non abilitati (CNN2D_LSTM_TF, BiLSTM, ...)
                if model_name not in enabled_models:
                    print(f"Skip {model_name}: non abilitato in questo giro")
                    continue
                #for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                for sweep_tuple in sweep_ids[condition][data_type][category_subject][model_name]:  # Itera sugli sweep per ciascun modello

                    # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                    sweep_id, combination_key = sweep_tuple
                    
                    
                    combination_key = f"{condition}_{data_type}_{category_subject}"
                    
                    # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                    # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                    # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione


                    # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                    if num_gpus > 1:

                        '''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper''' 
                        
                        # Assegna la GPU in modo rotazionale
                        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                    
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 1/3
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_frequency_{category_subject}", count=200)
                        
                        #PER TASK 2/4
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", count=200)
                        
                        # Passa alla prossima GPU per il prossimo sweep
                        gpu_counter = (gpu_counter + 1) % num_gpus

                    else:
                        
                        # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                        
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 1/3
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_frequency_{category_subject}", count=200)
                        
                        #PER TASK 2/4
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", count=200)


                    # Crea la funzione wrapper per l'agent
                    '''COMMENTATO'''
                    #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)


                    # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                    '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                       ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''

                    print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")

                    '''COMMENTATO'''
                    #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)

                    print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")

                    torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")
                

## Impostazione **Recupero DL Optimized Models** - EEG Spectrograms - **Frequency x Time (2D)**

### IMPLEMENTAZIONE DEI BEST MODELS DOPO W&B - EEG SPECTROGRAMS **+ GRADCAM FREQUENCY x TIME (ALL SUBJECTS)**! 

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

##### **NUOVE MODIFICHE SPECIFICHE PER I DATI NON HYPER POST W&B CON GRADCAM**

Allora le modifche che ho ultimato quindi sono:

- **1)Creazione della classe GradCAM**


    **GRADCAM CLASS**

        import torch
        import torch.nn.functional as F
        import cv2
        import numpy as np
        import matplotlib.pyplot as plt

        class GradCAM:
            def __init__(self, model, target_layer):
                self.model = model
                self.target_layer = target_layer
                self.activations = None
                self.gradients = None
                # Registra hook per catturare attivazioni e gradienti
                self.target_layer.register_forward_hook(self.save_activation)
                self.target_layer.register_backward_hook(self.save_gradient)

            def save_activation(self, module, input, output):
                self.activations = output.detach()

            def save_gradient(self, module, grad_input, grad_output):
                self.gradients = grad_output[0].detach()


- **2)** Creazione della funzione per generare delle immagini associate alla GradCAM compution**

    
    **FUNCTION FOR CREATING GRAD-CAM MAPS & FIGURES ASSOCIATED TO GRADCAM COMPUTATION**

        import cv2
        import numpy as np
        import matplotlib.pyplot as plt
        import io

        def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):

            """
            Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
            calcola la GradCAM e costruisce una figura con:
              - Riga 1: Heatmap per classe 0 e classe 1.
              - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
            I titoli della figura vengono personalizzati con exp_cond, data_type, category_subject.
            """

            # Assumiamo che il modello sia CNN2D e che il layer target sia model.conv3
            target_layer = model.conv3
            gradcam = GradCAM(model, target_layer)

            # Dizionari per salvare il campione per ogni classe
            samples = {}      # Salveremo il sample input per ogni classe
            labels_found = {} # Per tenere traccia delle etichette già trovate

            # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                for i, label in enumerate(labels):
                    label_int = int(label.item())
                    if label_int not in labels_found:
                        samples[label_int] = inputs[i].unsqueeze(0)  # salva come tensore 4D
                        labels_found[label_int] = True
                    if 0 in labels_found and 1 in labels_found:
                        break
                if 0 in labels_found and 1 in labels_found:
                    break

            # Se non troviamo entrambi gli esempi, esci con un messaggio
            if 0 not in samples or 1 not in samples:
                print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
                return None

            # Per ciascun campione, calcola GradCAM
            cams = {}
            overlays = {}
            for cls in [0, 1]:
                sample_input = samples[cls]
                sample_input.requires_grad = True  # Abilita gradiente per il campione
                cam = gradcam.generate_cam(sample_input)
                cams[cls] = cam

                # Converti il sample in immagine numpy per la visualizzazione
                img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
                # Normalizza l'immagine in scala 0-255
                img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))
                # Applica la heatmap
                heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
                heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
                # Sovrapponi la heatmap all'immagine originale
                overlay = cv2.addWeighted(img_norm, 0.6, heatmap, 0.4, 0)
                overlays[cls] = overlay

            # Crea la figura con due righe e due colonne
            fig, axs = plt.subplots(2, 2, figsize=(12, 10))

            # Titolo per la prima riga
            title_row1 = f"Grad-CAM mapping of experimental condition {exp_cond}, EEG {data_type}, Subject {category_subject}"
            # Titolo per la seconda riga
            title_row2 = f"Grad-CAM mapping superimposition over EEG Spectrogram of experimental condition {exp_cond}, Subject {category_subject}"

            # Prima riga: solo le heatmap
            for j, cls in enumerate([0, 1]):
                axs[0, j].imshow(cv2.cvtColor(cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB))
                axs[0, j].set_title(f"Class {cls} Heatmap")
                axs[0, j].axis('off')
            axs[0, 0].set_ylabel(title_row1, fontsize=10)

            # Seconda riga: overlay della heatmap sullo spettrogramma originale
            for j, cls in enumerate([0, 1]):
                axs[1, j].imshow(overlays[cls])
                axs[1, j].set_title(f"Class {cls} Overlay")
                axs[1, j].axis('off')
            axs[1, 0].set_ylabel(title_row2, fontsize=10)

            # Ottimizza la disposizione della figura
            plt.tight_layout()

            # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            fig_image = buf.getvalue()
            buf.close()
            plt.close(fig)

            return fig_image


- **3) Modifica delle funzioni per il salvataggio delle immagini create tramite la GradCAM compution**

    **FUNCTIONS FOR GRADCAM COMPUTATION & SAVING**
    
    Questa modifica consente di creare ed adattare le path di salvataggio ANCHE delle immagini calcolate dalla classe customizzata di GradCAM, 
    delle mappe di attivazione prodotte dalle feature maps e della sovrapposizione delle stesse aree decisionali
    rilevanti per la migliore classificazione dei dati di esempio di una certa classe,
    a partire da un certo dataset composto da una certa combinazione di fattori
    (i.e., exp_cond, data_type, category_subject)


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

    **Funzione per determinare a quale subfolder appartiene la chiave**
    def get_subfolder_from_key(key, model_standardization):

        #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
        if '_familiar_th' in key:
            return 'th_fam'
        elif '_unfamiliar_th' in key:
            return 'th_unfam'
        elif '_familiar_pt' in key:
            return 'pt_fam'
        elif '_unfamiliar_pt' in key:
            return 'pt_unfam'
        else:
            return None


    from PIL import Image
    import io
    import pickle
    import os

    **Funzione per salvare i risultati**
    def save_performance_results(model_name, 
                                 my_train_results,
                                 my_test_results, 
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder,
                                 gradcam_image = None):
        """
        Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
        Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
        """

        # Identificazione del subfolder in base alla chiave
        subfolder = get_subfolder_from_key(key, model_standardization)

        # Debug: controllo sulla subfolder
        print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")

        if subfolder is None:
            print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
            return

        # Determinazione del tipo di dato direttamente dalla chiave
        if "spectrograms" in key:
            data_type_str = "spectrograms"
        else:
            print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
            return

        # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
        if model_standardization:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        else:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)

        # Verifica se la cartella di destinazione esiste, altrimenti creala
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        file_path = os.path.join(folder_path, file_name)

        # Creazione del dizionario con i risultati
        results_dict = {
            'my_train_results': my_train_results,
            'my_test_results': my_test_results
        }

        # Salvataggio del dizionario con i risultati
        try:
            with open(file_path, 'wb') as f:
                pickle.dump(results_dict, f)
            print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
        except Exception as e:
            print(f"❌Errore durante il salvataggio dei risultati: {e}")

        # Se è stata fornita l'immagine GradCAM, salvala come file PNG
        if gradcam_image is not None:
            if model_standardization:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
            else:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"

            gradcam_file_path = os.path.join(folder_path, gradcam_file_name)

            #try:
            #    with open(gradcam_file_path, "wb") as f_img:
            #        f_img.write(gradcam_image)
            #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")

            try:

                '''
                Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
                Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
                Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
                👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()

                Per maggior info leggi cella successiva!
                '''

                # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
                if isinstance(gradcam_image, io.BytesIO):
                    gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                    gradcam_image = Image.open(gradcam_image)

                print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
                # 🔄 Salvare l'immagine nel percorso specificato
                gradcam_image.save(gradcam_file_path, format = "PNG")

            except Exception as e:
                print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")


- **4) Integrazione nel loop di training e test dei punti 1), 2) e 3)**

    **INTEGRATION OF GRADCAM COMPUTATION IN THE TRAINING E FOR LOOP**

    Nel loop che esegue il training ed il testing, integrazione della parte di inizializzazione della classe custom di GradCAM, con cui si esegue 

    il calcolo delle mappe di attivazione e della sovrapposizione delle mappe di attivazione stesse sullo spettogramma originale, 
    riportate poi in due immagini distinte create nella stessa figura che vengono salvate correttamente nella stessa directory path. 

    Le due immagini dovrebbero rappresentare l'heatmap activation e la sovrapposizione della mappa di attivazione sullo spettogramma originale,
    relativo ad un esempio rappresentativo per ciascuna delle due classi possibili presenti nello stesso dataset correntemente iterato.

    Il loro contributo è di descrivere se la CNN2D abbia identificato delle (possibili) differenti aree decisionali delle feature maps 
    (e dunque dello spettrogramma) maggiormente utili ai fini della discriminazione delle due condizioni sperimentali inserite all'interno del dataset correntemente iterato.


        ** Dizionario per tracciare la standardizzazione usata per ogni combinazione di dati**
        ** Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)**

        models_info = {}

        ** Set per tenere traccia dei dataset già elaborati**
        processed_datasets = set()

        ** Set per tenere traccia delle combinazioni già elaborate**
        processed_models = set()

        ** Path delle performance dei modelli ottimizzati con weight and biases**
        ** Path per trovare le best performances di ogni modello per ogni combinazione dei dati**
        base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results"

        ** Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder**
        save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


        ** --- LOOP PRINCIPALE (con minime modifiche) ---**
        for key, (X_data, y_data) in data_dict.items():

            print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")

            if key in processed_datasets:
                print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
                continue

            processed_datasets.add(key)

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")

            for model_name in ["CNN2D", "BiLSTM", "Transformer"]:

                model_key = f"{model_name}_{key}"
                if model_key in processed_models:
                    print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
                    continue
                processed_models.add(model_key)

                print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")

                # Prova a caricare la configurazione e i pesi ottimali dal file .pkl

                '''
                load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
                parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate

                exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
                pesi del modello e i suoi iper-parametri

                Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
                '''

                config, best_weights = load_config_if_available(key, model_name, base_folder)

                if config is None:
                    raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")

                '''
                Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
                MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
                '''

                # Parsifica la chiave una volta sola e memorizza i valori
                exp_cond, data_type, category_subject = parse_combination_key(key)

                '''
                Dpodiché, 

                1) si carica i vari valori degli iper-parametri,
                2) si esegue la standardizzazione se servisse,
                3) prepara il modello per la divisione in train_loader etc.,
                4) si carica la configurazione dei pesi del modello, 
                5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 

                6) esegue il training e il test e poi

                7) si salva il tutto nella path corrispondente...

                '''

                '''
                PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
                DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''

                model_batch_size = config["batch_size"]
                model_n_epochs = config["n_epochs"]
                model_patience = config["patience"]
                model_lr = config["lr"]
                model_weight_decay = config["weight_decay"]
                model_standardization = config["standardization"]

                print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")

                # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
                models_info[model_key] = {"standardization": model_standardization}


                '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
                IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
                '''

                if model_standardization:
                    X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
                    print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
                else:
                    print(f"\033[1mNO Standardizzazione Dati!\033[0m")

                # Sposta il modello sulla GPU (se disponibile)
                device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


                # Preparazione dei dataloaders
                train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
                    X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)

                # Inizializzazione del modello
                if model_name == "CNN2D":
                    model = CNN2D(input_channels=3, num_classes=2)
                elif model_name == "BiLSTM":
                    model = ReadMEndYou(input_size= 3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
                elif model_name == "Transformer":
                    model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
                else:
                    raise ValueError(f"Modello {model_name} non riconosciuto.")

                # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
                if best_weights is not None:
                    try:
                        model.load_state_dict(best_weights)
                        print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
                    except Exception as e:
                        print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                        continue


                # Definizione del criterio di perdita
                criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)

                # Definizione dell'ottimizzatore con i parametri aggiornati
                optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)

                print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)

                print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_test_results = testing(my_train_results, test_loader, criterion)

                '''
                GRADCAM COMPUTATION PER IL MODELLO CNN2D

                La funzione compute_gradcam_figure estrae due campioni (uno per ogni classe) e crea una figura con le due righe richieste.

                Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
                'save_performance_results', in modo da essere salvato nella path corretta. 

                La funzione 'save_performance_results' è stata modificata 
                per gestire ANCHE questo nuovo input dell'immagine 

                (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
                seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:

                - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
                - tipologia di dato EEG prelevato (i.e., spectrograms) 
                - provenienza del dato stesso (i.e., familiar_th)
                )

                '''

                # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
                gradcam_image = None

                if model_name == "CNN2D":
                    gradcam_image = compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device)
                    if gradcam_image is not None:
                        print(f"GradCAM image computed successfully for {model_name}.")

                print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                save_performance_results(model_name,
                                         my_train_results,
                                         my_test_results,
                                         key,
                                         exp_cond,
                                         model_standardization,
                                         base_folder = save_path_folder,
                                         gradcam_image = gradcam_image)

##### **UTILS DATI NON HYPER**

In [ ]:
import pickle
import numpy as np


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (50°-300° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "familiar" o "unfamiliar"
    - subject_type: str, "th" (terapisti) o "pt" (pazienti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Familiar_Spectrograms/",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test




'''ATTENZIONE CAMBIATA'''
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    #aggiungo eps per evitare divisione per zero
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test


# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels=3, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    if model_type == "CNN2D_LSTM_TF":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor




'''
OLD VERSIONS BEFORE GRADCAM COMPUTATION ON CNN2D

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
    
     
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, exp_cond, model_standardization, base_folder):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
'''


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None


from PIL import Image
import io
import pickle
import os
     
# Funzione per salvare i risultati
def save_performance_results(model_name, 
                             my_train_results,
                             my_test_results, 
                             key,
                             exp_cond,
                             model_standardization,
                             base_folder,
                             gradcam_image = None):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
    
    # Se è stata fornita l'immagine GradCAM, salvala come file PNG
    if gradcam_image is not None:
        if model_standardization:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
        else:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"
        
        gradcam_file_path = os.path.join(folder_path, gradcam_file_name)
        
        #try:
        #    with open(gradcam_file_path, "wb") as f_img:
        #        f_img.write(gradcam_image)
        #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
        
        try:
            
            '''
            Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
            Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
            Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
            👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()
            
            Per maggior info leggi cella successiva!
            '''
            
            # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
            if isinstance(gradcam_image, io.BytesIO):
                gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                gradcam_image = Image.open(gradcam_image)
            
            '''
            Il messaggio di errore indica che il tuo oggetto gradcam_image è di tipo bytes e non ha il metodo save(), 
            che è tipico di un oggetto PIL. 
            
            Per risolvere questo, devi convertire i byte in un'immagine PIL. 
            Per farlo, controlla se gradcam_image sia un oggetto di tipo bytes e,
            in tal caso, usa io.BytesIO per creare un buffer da passare a Image.open(). 
            
            Inserisci questa conversione all'interno del blocco che salva l'immagine, così da assicurarti che,
            indipendentemente dal tipo, gradcam_image diventi un oggetto PIL e possa chiamare il metodo save().
            '''
            
            if isinstance(gradcam_image, bytes):
                gradcam_image = io.BytesIO(gradcam_image)
                gradcam_image.seek(0)
                gradcam_image = Image.open(gradcam_image)
            
            
            print(f"\n📸Immagine \033[1mGradCAM salvata\033[0m con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
            # 🔄 Salvare l'immagine nel percorso specificato
            gradcam_image.save(gradcam_file_path, format = "PNG")
            
        except Exception as e:
            print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")

##### **NUOVE UTILS DATI NON HYPER POST W&B**

###### **SUGGERIMENTI DI MODIFICA CHATGPT DELLE UTILS DATI NON HYPER POST W&B**

###### **IMPLEMENTAZIONE ADOTTATA**

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''
import re 

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$", 
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
combination_key = "pt_resp_vs_shared_resp_spectrograms_familiar_th"
condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Data Type:", data_type)
print("Soggetto:", subject_key)

In [ ]:
'''
Verifica del file .pkl:
La funzione load_config_if_available cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. 

Se il file esiste, allora viene passata poi a load_model_config_and_weights, 
che carica il dizionario di partenza 
e da questo estrae i 2 sotto-dizionari 'config' e 'state_dict'.
'''

def load_config_if_available(dataset_key, model_name, base_path):
    """
    Data una chiave (es. "th_resp_vs_pt_resp_wavelet_familiar_th") e il nome del modello,
    cerca il file .pkl corrispondente e ritorna (config, state_dict).
    Se non esiste, restituisce (None, None).
    """
    try:
        exp_cond, data_type, category_subject = parse_combination_key(dataset_key)
        config, state_dict = load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path)
        print(f"✅ File .pkl trovato per \033[1m{model_name}\033[0m su \033[1m{dataset_key}\033[0m")
        
        return config, state_dict
    except Exception as e:
        print(f"⚠️ Nessun file .pkl per {model_name} su {dataset_key} - uso parametri di default. ({e})")
        return None, None

In [ ]:
'''
Caricamento del file .pkl:
La funzione load_model_config_and_weights cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. Se il file esiste, vengono restituiti config e state_dict.
'''

# Funzione per caricare il file .pkl con la configurazione e i pesi ottimali
def load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path):
    """
    Costruisce il path usando:
        base_path / exp_cond / data_type / category_subject
    e il nome del file:
        {model_name}_{exp_cond}_{data_type}_{category_subject}.pkl
    Se il file esiste, lo carica e restituisce (config, state_dict).
    """
    
    file_name = f"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
    file_path = os.path.join(base_path, exp_cond, data_type, category_subject, file_name)
    
    if os.path.exists(file_path):
        print(f"🕵️‍♂️🔍Caricamento file .pkl: \033[1m{file_path}\033[0m")
        
        # Il file .pkl è stato salvato con torch.save() e contiene un dizionario con chiavi al suo interno che sono: "config" e "state_dict"
        with open(file_path, "rb") as f:
            data = torch.load(f)
        return data["config"], data["state_dict"]
    else:
        raise FileNotFoundError(f"File {file_path} non trovato.")

##### **Early Stopping - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''
DEFINIZIONE EARLY STOPPING
'''

import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle
import numpy as np


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
        
        """
        :param patience: Numero di epoche da attendere prima di interrompere il training se non c'è miglioramento
        
        Esempio: il training si interromperà se non si osserva un miglioramento per (N = 5) epoche consecutive.
        
        :param min_delta: Variazione minima richiesta per considerare un miglioramento
        
        definisce il miglioramento minimo richiesto per essere considerato significativo. 
        Se il miglioramento è inferiore a min_delta, non viene considerato un vero miglioramento.
        
        Il parametro min_delta in una configurazione di early stopping indica 
        la minima variazione del valore di una metrica 
        (ad esempio, la perdita o l'accuratezza) 
        che deve verificarsi tra un'epoca e la successiva 
        per continuare l'allenamento. 
        
        In genere, il valore di min_delta dipende dal tipo di modello e dai dati specifici, 
        ma di solito si trova in un intervallo tra 0.001 e 0.01.
    
            - Se stai cercando di evitare che l'allenamento si fermi troppo presto,
            puoi impostare un valore più basso per min_delta (come 0.001), 
            - Se vuoi essere più conservativo e permettere fluttuazioni nei valori della metrica,
            un valore più alto (come 0.01) potrebbe essere appropriato.

        Un buon punto di partenza potrebbe essere 0.001, e poi fare dei test per capire quale valore funziona meglio
        nel tuo caso specifico!
        
        :param mode: 'min' per monitorare la loss (minimizzazione), 'max' per l'accuracy (massimizzazione)
        
        'max' → ottimizza metriche da massimizzare (es. accuracy, F1-score, AUC).
        'min' → ottimizza metriche da minimizzare (es. loss).
        
        """
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True


##### **TRAINING (NON DEVI ESEGUIRLA VAI DIRETTAMENTE AL TESTING!)**

###### **VERSIONE PRE- WEIGHT AND BIASES (W&B)**

###### **VERSIONE POST- WEIGHT AND BIASES (W&B)**

In [ ]:
'''UFFICIALE - VERSIONE POST- WEIGHT AND BIASES SENZA COMMENTI'''


import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True
                

def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    # Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    #plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    '''
    
    # Salvare il plot in un buffer di memoria
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()

    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data



def training(model, dataset_train_loader, dataset_val_loader, optimizer, criterion, n_epochs = 100, patience = 10):
    
    # Sposta il modello sulla GPU (se disponibile)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    model.to(device)
    
    #Setta il modello in fase di training
    model.train()
    
    # Storico delle metriche per ogni epoca
    loss_train_history = []  # History of Training loss
    loss_val_history = []    # History of Validation loss
    accuracy_train_history = []  # History of Training Accuracy
    accuracy_val_history = []    # History of Validation Accuracy
    
    early_stopping = EarlyStopping(patience=patience, mode='max')
    
    # Liste per le metriche di valutazione (precision, recall, F1, AUC)
    precision_train_history = []
    recall_train_history = []
    f1_train_history = []
    auc_train_history = []
    
    #Questa sarebbe la migliore accuratezza ottenuta sul validation set
    #in base alla quale viene preso il modello migliore!
    
    max_val_acc = 0
    best_model = None
    
    best_epoch = 0  # Epoca con la migliore validazione
    
    best_metrics = {} # Dizionario con le metriche del migliore modello nel set di validazione
    
    # Variabili per memorizzare le etichette vere e predette per l'intero training
    y_true_train_list = []
    y_pred_train_list = []
    
    
    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        #Create a list for temporary monitoring of train loss and accuracy at each epoch
        train_loss_tmp = [] 
        correct_train = 0 
        
        
        #'''STARTING OF THE TRAINING PHASE'''
        
        #Iterating for every batch inside dataset_train_loader
        for x, y in dataset_train_loader:
            
            x, y = x.to(device), y.to(device)
            
            #Run forward pass through my network and get a prediction
            y_pred = model(x)
            
            train_loss = criterion(y_pred, y.view(-1))
            optimizer.zero_grad() #so essentially finding where gradients is 0
                                  #we're looking for minimum's there

            train_loss.backward() #performing the backprop step
            optimizer.step() #update the model's hyperparameters based off of the step
        
            train_loss_tmp.append(train_loss.item()) #append the loss at each epoch in the temporary train loss list inside each epoch
            
            # Calculate the Accuracy Score during the Training Phase
                
            #qui il "_,"
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            
            # Aggiungere le etichette vere e quelle predette alla lista
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
        
        # Save the results of training set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        loss_train_history.append(np.mean(train_loss_tmp))
        accuracy_train = correct_train / len(dataset_train_loader.dataset)
        accuracy_train_history.append(accuracy_train)
        
        # Calcolare precision, recall, F1-score e AUC durante il training
        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        precision_train_history.append(precision_train)
        recall_train_history.append(recall_train)
        f1_train_history.append(f1_train)
        auc_train_history.append(auc_train)
        
        # '''STARTING OF THE VALIDATION PHASE'''
        
        #Setta il modello in fase di validation
        #model.eval() 
        
        loss_tmp_val = []  #create a list for temporary val list at each epoch
        correct_val = 0
        
        y_true_list = []
        y_pred_list = []
        
        #Here we disable gradient computation for the validation phase!
        with torch.no_grad():
            
            for x, y in dataset_val_loader:
                
                x, y = x.to(device), y.to(device)
                
                #Run forward pass through my network and get a prediction
                y_pred = model(x)

                #Calculate Validation Loss

                #remember: since we use CrossEntropyLoss we DO NOT need
                #to do any ONE HOT ENCODING between y_pred and y_train 
                
                #loss = criterion(y_pred.to(device), y.view(-1).to(device))
                
                val_loss = criterion(y_pred, y.view(-1))

                #Perform Backpropagation

                #HOW TO ADJUST THE VALUES (weights and biases)?
                #well, at every step the gradients will accumulate with every backprop,
                #so to prevent 'compounding', we need to reset the stored gradient for each new epoch!

                loss_tmp_val.append(val_loss.item()) #append the loss at each epoch in the temporary val loss list inside each epoch 
                
                # Calculate the Accuracy Score during the Validation Phase
                _, predicted_val = torch.max(y_pred, 1)
                correct_val += (predicted_val == y).sum().item()
                
                # Aggiungi le etichette e le predizioni per la confusion matrix
                y_true_list.extend(y.cpu().numpy())
                y_pred_list.extend(predicted_val.cpu().numpy())

                
        # Save the results of validation set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        
        loss_val_history.append(np.mean(loss_tmp_val)) 
        accuracy_val = correct_val / len(dataset_val_loader.dataset)
        accuracy_val_history.append(accuracy_val)
        
        #L'early stopping deve essere basato sulla val accuracy,
        #ma quando il training si interrompe, 
        #dobbiamo salvare le migliori performance ottenute sul training in corrispondenza dell'epoca in cui
        #la val accuracy era massima
        
        # Controllo della miglior validazione
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            
            best_metrics = {
                "train_loss": [round(loss_train_history[best_epoch], 4)],
                "train_accuracy": [round(accuracy_train_history[best_epoch], 4)],
                "train_precision": [round(precision_train, 4)],
                "train_recall": [round(recall_train, 4)],
                "train_f1_score": [round(f1_train, 4)],
                "train_auc": [round(auc_train, 4)]
            }
            best_model = cp.deepcopy(model)  # Salvo il miglior modello

        # Controllo Early Stopping
        early_stopping(accuracy_val)
        if early_stopping.early_stop:
            print(f"⚠️ Early stopping attivato all'epoca \033[1m{epoch}\033[0m, recupero il modello dell'epoca \033[1m{best_epoch}\033[0m")
            break

        # Update of the progress bar
        pbar.set_description(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {loss_train_history[-1]:.4f}, Val Loss: {loss_val_history[-1]:.4f}, Train Acc: {accuracy_train:.4f}, Val Acc: {accuracy_val:.4f}")

        # Calculate the confusion matrix and the classification report after all epochs in the Validation Phase
        conf_matrix = confusion_matrix(y_true_list, y_pred_list)
        class_report = classification_report(y_true_list, y_pred_list)

    # Salvataggio della configurazione del modello e iper-parametri
    model_config = {
        "model_architecture": str(model),
        "batch_size_train": train_loader.batch_size,
        "batch_size_val": val_loader.batch_size,
        "batch_size_test": test_loader.batch_size,
        "n_epochs": n_epochs
    }

    # Dizionario degli iper-parametri
    hyperparams = {
    "optimizer": str(optimizer),
    "loss_function": str(criterion),
    "learning_rate": optimizer.param_groups[0]['lr'],
   }

    
    # Plot dei risultati
    #plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history, exp_cond_1, exp_cond_2)
    training_plot = plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history)

    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        "best_model": best_model,
        "confusion_matrix_val": conf_matrix,
        "classification_report": class_report,
        "model_configuration": model_config,
        "hyperparameters": hyperparams,
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }

    return train_results


##### **TESTING**

In [ ]:
'''
TESTING FUNCTION: CORRETTA ANCHE PER IL GRAD-CAM

SUCCESSIVAMENTE, DENTRO AL FOR LOOP DEL TRAINING E TESTING, 
SI RICHIAMA LA FUNZIONE DIRETTAMENTE DI 

1) compute_gradcam_figure, LA QUALE AL SUO INTERNO PRESENTA GIÀ 
TUTTO QUELLO CHE SERVE PER CALCOLARE IL GRADCAM, DI MODO CHE VADA A 

Selezionare esempi rappresentativi per ciascuna classe.
Calcolare le mappe GradCAM e gli overlay.
Creare una figura con le heatmap e le sovrapposizioni, completa di titoli esplicativi.
Restituire un'immagine (buffer) pronta per essere salvata

SUCCESSIVAMENTE, QUINDI, IL PROCEDIMENTO DIVENTA COME SEGUE:

1) Si esegue il TESTING, per ottenere le metriche e salvare i risultati (senza GradCAM)

2) Nel loop principale di TRAINING & TESTING, se il modello è CNN2D, allora 

 - richiama la funzione 'compute_gradcam_figure', la quale va a
    - calcolare le mappe di attivazione e successivamente creo le immagini che gli ho chiesto
    - passa l'immagine ottenuta da GradCAM alla funzione 'save_performance_results', la quale va a 
        - salvare i risultati di test ottenuti dalla funzione di 'testing'
        - salvare l'immagine risultatante del GradCAM e la sovrapposizione del GradCAM sullo spettrogramma originale della classe risultante
        
        
Questo approccio garantisce chiarezza e separa la parte di performance (testing) dalla parte di explainability (GradCAM).


'''

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import io
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


def testing(results, test_loader, criterion):
    
    # Recupera il miglior modello ottenuto durante la validazione
    model = results['best_model']
    model.to(device)
    
    model.eval()  # Imposta il modello in modalità valutazione

    y_true_list = []  # Lista per salvare le etichette reali
    y_pred_list = []  # Lista per salvare le previsioni del modello
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
    y_score_list = []   # <— Lista per salvare gli score per le probabilità della classe positiva (per auc-roc!)
    
    total_loss = 0
    correct = 0
    
    test_performances = {
        "test_loss": [],
        "test_accuracy": [],
        "test_precision": [],
        "test_recall": [],
        "test_f1_score": [],
        "test_auc": []
    }
    

    with torch.no_grad():
        
        pbar = tqdm(test_loader, desc="Testing")
        
        for inputs, labels in pbar:
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Ottenere le predizioni del modello
            outputs = model(inputs)
            
            '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
            # aggiungi queste due righe
            probs = torch.softmax(outputs, dim=1)
            y_score_list.extend(probs[:,1].cpu().numpy())

            # Calcolare la loss
            test_loss = criterion(outputs, labels)
            total_loss += test_loss.item()

            # Memorizzare predizioni ed etichette vere
            _, predicted = torch.max(outputs, 1)
            y_pred_list.extend(predicted.cpu().numpy())
            y_true_list.extend(labels.cpu().numpy())

            # Aggiornare il numero di predizioni corrette
            correct += (predicted == labels).sum().item()

            pbar.set_description(f"Loss: {test_loss.item():.4f}")

    # Calcolare l'accuratezza complessiva
    accuracy = correct / len(test_loader.dataset)
    
    
    # Calcolare precision, recall, F1-score, AUC durante il testing
    precision_test = precision_score(y_true_list, y_pred_list, average='weighted')
    recall_test = recall_score(y_true_list, y_pred_list, average='weighted')
    f1_test = f1_score(y_true_list, y_pred_list, average='weighted')
    
    '''OLD VERSION'''
    #auc_test = roc_auc_score(y_true_list, y_pred_list, average='weighted')  # Assicurati che il problema sia binario o multi-class
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC
    
    In questo modo l’roc_auc_score calcola l’area sotto tutta la curva ROC (tutte le soglie), invece di valutare un solo punto corrispondente alla soglia 0.5
    '''
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    auc_test = roc_auc_score(y_true_list, y_score_list)


    # Aggiungere questi valori nel dizionario delle performance (arrotondando a 4 decimali)
    test_performances["test_loss"].append(round(total_loss / len(test_loader), 4))  # Media della loss
    test_performances["test_accuracy"].append(round(accuracy, 4))
    test_performances["test_precision"].append(round(precision_test, 4))
    test_performances["test_recall"].append(round(recall_test, 4))
    test_performances["test_f1_score"].append(round(f1_test, 4))
    test_performances["test_auc"].append(round(auc_test, 4))
    
    # Creare la confusion matrix
    conf_matrix = confusion_matrix(y_true_list, y_pred_list)
    
    # Stampare classification report
    class_report = classification_report(y_true_list, y_pred_list)

    print(f"\nTest Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", class_report)

    # Visualizzare la confusion matrix
    #plt.figure(figsize=(8, 6))
    #sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    #plt.title("Confusion Matrix")
    #plt.xlabel("Predicted")
    #plt.ylabel("True")
    #plt.show()
    
    # Salviamo l'immagine della confusion matrix in un buffer
    #buf = io.BytesIO()
    #plt.savefig(buf, format='png')
    #buf.seek(0)
    #conf_matrix_image_data = buf.getvalue()
    #buf.close()
    
    
    # Salviamo l'immagine della confusion matrix in un buffer
    buf = io.BytesIO()
    plt.figure(figsize=(8, 6))  # Nuova figura per evitare sovrapposizioni
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(buf, format='png')  # Salva l'immagine nel buffer
    buf.seek(0)  # Torna all'inizio del buffer
    conf_matrix_image_data = buf.getvalue()  # Ottieni l'immagine in formato binario
    buf.close()  # Chiudi il buffer

    # Mostra la confusion matrix (opzionale)
    #plt.show()
    
    # Salvataggio della configurazione del modello e iper-parametri
    '''COMMENTATO'''
    #model_config = {
        #"model_architecture": str(model),
        #"batch_size_test": test_loader.batch_size,
    #}
    
    '''COMMENTATO'''
    # Dizionario degli iper-parametri
    #hyperparams = {
        #"optimizer": str(optimizer),
        #"loss_function": str(criterion),
        #"learning_rate": optimizer.param_groups[0]['lr'],
    #}

    
    '''COMMENTATO'''
    # Restituisci i risultati come dizionario
    #test_results = {
        #"test_performances": test_performances,  # Aggiungi il dizionario delle performance
        #"confusion_matrix": conf_matrix,
        #"classification_report": class_report,
        #"model_configuration": model_config,
        #"hyperparameters": hyperparams,  # Aggiunti i due nuovi dizionari
        #"confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    #}
    
    
    # Restituisci i risultati come dizionario
    test_results = {
        "test_performances": test_performances,  # Aggiungi il dizionario delle performance
        "confusion_matrix": conf_matrix,
        "classification_report": class_report,
        "confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    }
    
        
    return test_results


##### **CREAZIONE CLASSE GRADCAM**

In [ ]:
##### **CREAZIONE CLASSE GRADCAM**

'''
Creazione della classe GradCAM

-----1. Costruttore (init)-----

Cosa fa:

Salva il modello e il layer target (ad esempio, l'ultimo strato convoluzionale) su cui calcolare le mappe di attivazione.

A) Inizializza due variabili, 

1) self.activations e 2) self.gradients, che verranno usate per memorizzare rispettivamente 
1) le attivazioni (feature maps) e 2) i gradienti di quel layer

B) Registra due hook sul target_layer:

1) Forward Hook: Quando il modello effettua la forward pass, viene eseguito save_activation per salvare le attivazioni
2) Backward Hook: Durante la backward pass, save_gradient viene chiamato per salvare i gradienti


-----2. Hook per Salvare Attivazioni e Gradienti-----

B) Save Activation

def save_activation(self, module, input, output):
    self.activations = output.detach()

Cosa fa:

Quando viene eseguita la forward pass sul target_layer, questo hook cattura l'output (le attivazioni) del layer.
Usa detach() per ottenere una copia dei dati senza il tracking dei gradienti, in modo da non interferire con la retropropagazione.

C) Save Gradient

def save_gradient(self, module, grad_input, grad_output):
    self.gradients = grad_output[0].detach()


Cosa fa:

Durante la backward pass, questo hook cattura i gradienti che fluiscono attraverso il target_layer.
grad_output è una tupla; solitamente il primo elemento contiene i gradienti utili. 

Anche qui si usa detach() per isolare i dati dai grafi di calcolo.


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

##### **DRAFT IMPLEMENTATIONS OF GRADCAM COMPUTATION**

In [ ]:
### INITIAL IMPLEMENTATIONS OF GRADCAM

'''
import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
        
        
1) Funzione generate_cam (interna alla classe GradCAM)

    La differenza chiave tra i due approcci è proprio la selezione dell'input su cui viene calcolata la Grad-CAM. Ti riassumo le due opzioni:

    1️⃣ Approccio attuale (generate_cam)
    Viene passato un singolo input_tensor, e il Grad-CAM viene calcolato su di esso.
    Se target_class non è specificata, viene selezionata la classe predetta dal modello per quell'input.
    Il calcolo del Grad-CAM si basa su una backward pass del gradiente rispetto alla classe target.
    
    def generate_cam(self, input_tensor, target_class=None):
        # Effettua la forward pass
        output = self.model(input_tensor)
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        # Azzeramento dei gradienti
        self.model.zero_grad()
        # Calcola il gradiente per la classe target
        target = output[0, target_class]
        target.backward()

        # Calcola i pesi come media dei gradienti su width e height
        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        # Somma pesata delle attivazioni
        cam = torch.sum(weights * self.activations, dim=1)
        cam = F.relu(cam)

        # Normalizza la mappa
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)

        # Upsample alla dimensione dell'immagine di input
        cam = F.interpolate(cam.unsqueeze(1), size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = cam.squeeze().cpu().numpy()
        return cam
    
    
2) Funzione compute_gradcam_figure (esterna alla classe GradCAM)   
    
2️⃣ Alternativa proposta (compute_gradcam_figure)
Seleziona esplicitamente un esempio per ciascuna classe (0 e 1) iterando sul test_loader.
Questo garantisce che il Grad-CAM sia calcolato su esempi rappresentativi di entrambe le classi.
La visualizzazione finale confronta le heatmap delle due classi, sovrapponendole agli spettrogrammi.


import cv2
import numpy as np
import matplotlib.pyplot as plt
import io

def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):
    
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
      - Riga 1: Heatmap per classe 0 e classe 1.
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
    I titoli della figura vengono personalizzati con exp_cond, data_type, category_subject.
    """
    
    # Assumiamo che il modello sia CNN2D e che il layer target sia model.conv3
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)

    # Dizionari per salvare il campione per ogni classe
    samples = {}      # Salveremo il sample input per ogni classe
    labels_found = {} # Per tenere traccia delle etichette già trovate

    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int not in labels_found:
                samples[label_int] = inputs[i].unsqueeze(0)  # salva come tensore 4D
                labels_found[label_int] = True
            if 0 in labels_found and 1 in labels_found:
                break
        if 0 in labels_found and 1 in labels_found:
            break

    # Se non troviamo entrambi gli esempi, esci con un messaggio
    if 0 not in samples or 1 not in samples:
        print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
        return None

    # Per ciascun campione, calcola GradCAM
    cams = {}
    overlays = {}
    for cls in [0, 1]:
        sample_input = samples[cls]
        sample_input.requires_grad = True  # Abilita gradiente per il campione
        cam = gradcam.generate_cam(sample_input)
        cams[cls] = cam

        # Converti il sample in immagine numpy per la visualizzazione
        img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
        # Normalizza l'immagine in scala 0-255
        img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))
        # Applica la heatmap
        heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        # Sovrapponi la heatmap all'immagine originale
        overlay = cv2.addWeighted(img_norm, 0.6, heatmap, 0.4, 0)
        overlays[cls] = overlay

    # Crea la figura con due righe e due colonne
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Titolo per la prima riga
    title_row1 = f"Grad-CAM mapping of experimental condition {exp_cond}, EEG {data_type}, Subject {category_subject}"
    # Titolo per la seconda riga
    title_row2 = f"Grad-CAM mapping superimposition over EEG Spectrogram of experimental condition {exp_cond}, Subject {category_subject}"
    
    # Prima riga: solo le heatmap
    for j, cls in enumerate([0, 1]):
        axs[0, j].imshow(cv2.cvtColor(cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB))
        axs[0, j].set_title(f"Class {cls} Heatmap")
        axs[0, j].axis('off')
    axs[0, 0].set_ylabel(title_row1, fontsize=10)
    
    # Seconda riga: overlay della heatmap sullo spettrogramma originale
    for j, cls in enumerate([0, 1]):
        axs[1, j].imshow(overlays[cls])
        axs[1, j].set_title(f"Class {cls} Overlay")
        axs[1, j].axis('off')
    axs[1, 0].set_ylabel(title_row2, fontsize=10)
    
    # Ottimizza la disposizione della figura
    plt.tight_layout()
    
    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

''' 

In [ ]:
'''SOLUZIONE ? INTEGRARE LA 2) IN 1)


🛠️ Cosa conviene fare?
Se il tuo obiettivo è sempre confrontare le attivazioni per entrambe le classi, 
allora conviene integrare compute_gradcam_figure dentro la classe GradCAM e rimuovere generate_cam come metodo separato.

📌 Quindi suggerirei di fare così:

Rendere compute_gradcam_figure un metodo della classe GradCAM.
Rimuovere generate_cam, perché il calcolo della CAM viene già eseguito all'interno del loop che seleziona i campioni.
Mantenere la logica che seleziona i campioni da entrambe le classi, perché è più robusta rispetto a calcolare la CAM su un singolo input arbitrario.
⚠️ Attenzione a una cosa però!
Il metodo generate_cam fa un passaggio importante che non è presente in compute_gradcam_figure:

python
Copia
Modifica
weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
cam = torch.sum(weights * self.activations, dim=1)
cam = F.relu(cam)
🔹 Bisogna assicurarsi che questa logica venga mantenuta nel nuovo metodo!
Attualmente compute_gradcam_figure chiama gradcam.generate_cam(sample_input), quindi se generate_cam viene eliminato, questa parte va spostata nel nuovo metodo.

📌 In sintesi, cosa farei
✅ Modificare la classe GradCAM e aggiungere direttamente compute_gradcam_figure.
✅ Eliminare generate_cam, ma mantenere la sua logica di calcolo della CAM.
✅ Garantire che il calcolo dei pesi e della CAM sia integrato nel nuovo metodo.
✅ Mantenere la selezione di un campione per ciascuna classe, per una migliore interpretabilità.



Ha senso integrare compute_gradcam_figure direttamente come metodo della classe GradCAM ed eliminare generate_cam, perché:

Selezione più rappresentativa dei campioni

Il metodo compute_gradcam_figure assicura che vengano selezionati esempi di entrambe le classi (0 e 1), cosa che generate_cam non fa.
Questo approccio fornisce una migliore interpretabilità della Grad-CAM confrontando diverse classi.
Chiarezza e modularità

generate_cam è attualmente chiamato da compute_gradcam_figure, ma possiamo integrare direttamente la logica dentro GradCAM.
Questo evita la duplicazione del codice e rende più chiaro il flusso.
Ottimizzazione del calcolo

La pipeline di compute_gradcam_figure gestisce direttamente la forward pass e il calcolo del gradiente per entrambi i campioni in un'unica operazione, evitando di dover chiamare generate_cam separatamente.
Prossimi passi:
Spostiamo compute_gradcam_figure dentro GradCAM come metodo della classe.
Eliminiamo generate_cam e integriamo direttamente la logica di forward pass e backward pass dentro compute_gradcam_figure.

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION**

In [ ]:
'''
Creazione della funzione per generare le immagini associate alla GradCAM compution

FINAL VERSION WITH ULTIMATED EDITING PHASES


Spiegazione:

1) Selezione dei Campioni:
La funzione itera sul test_loader e salva il primo campione trovato per ciascuna delle due classi (0 e 1).

2) Calcolo GradCAM per ciascun campione:

Per ogni campione, si abilita il gradiente e si esegue la forward pass.
Viene scelto il target (se non specificato, quello predetto) e si esegue la backward pass per calcolare i gradienti.

- I pesi vengono calcolati come la media dei gradienti lungo le dimensioni spaziali (dim=(2,3)) e usati per eseguire una somma pesata sulle attivazioni.
- La mappa risultante viene passata attraverso una ReLU, normalizzata e upsampled per avere la stessa dimensione dell’input.

Creazione degli Overlay:
Viene normalizzata l’immagine originale e viene applicata una heatmap (usando OpenCV), quindi l’overlay viene ottenuto con cv2.addWeighted.

Costruzione della Figura:
Viene creata una figura con due righe e due colonne:

- La prima riga mostra le heatmap per ciascuna classe.
- La seconda riga mostra le sovrapposizioni (overlay) tra heatmap e spettrogramma originale.

I titoli sono personalizzati in base a exp_cond, data_type e category_subject.

Questa struttura mantiene tutta la logica necessaria (incluso il calcolo dei pesi) e la rende simile alla versione precedente,
con la differenza che il calcolo della CAM viene eseguito per campioni rappresentativi di entrambe le classi. 



Ciao ho bisogno che in questo calcolo del gradcam, tu qui nella funzione di calcolo del gradcam...
tu mi presenti poi le immagini (sia gradcam che per l'overlay) non solo un esempio, ma il valore medio di attivazione per la relativa condizione sperimentale...
ossia, anziché prendere un solo esempio e presentare il gradcam e l'overlay del solo esempio della classe 0, dovresti mostrarmi il valore medio, 
e allo stesso modo della classe 1.

quindi sia nel gradcam che nell'overlay dovrei avere una rappresentazione media sia delle regioni delle feature maps per la relativa condizione sperimentale,
sia dell'overlay risultante....

Ti è chiaro che cosa voglio?



Esatto, ho capito. Invece di estrarre un singolo campione per ciascuna classe, la funzione dovrebbe:
Accumulare le mappe GradCAM e gli overlay per tutti i campioni appartenenti a ciascuna classe
Calcolare la media delle mappe GradCAM ottenute per ciascuna condizione sperimentale.
Calcolare la media degli overlay corrispondenti, in modo da ottenere una rappresentazione aggregata della risposta del modello.
Quindi, anziché mostrare l'attivazione per un solo esempio, la figura finale presenterà:
Una heatmap media della GradCAM per la classe 0 e per la classe 1.
Un overlay medio (CAM + spettrogramma) per la classe 0 e per la classe 1.



Puoi implementare la media delle mappe GradCAM e degli overlay per ogni classe modificando la funzione nel modo seguente. 
In sostanza, invece di fermarti al primo campione trovato per ciascuna classe, raccogli tutti (o un sottoinsieme rappresentativo) e poi calcola la media.
Ecco i passaggi principali con qualche suggerimento su dove inserire il nuovo codice:


1. Raccogliere le mappe e gli overlay per ogni classe
Dove modificare:

Dopo il passaggio 3 (calcolo della GradCAM per ciascun campione), invece di elaborare un solo esempio per ciascuna classe, 
crea due liste (una per la classe 0 e una per la classe 1) in cui accumulare le mappe CAM e gli overlay per ogni campione processato.

Cosa fare:

Invece di usare un dizionario samples che salva solo il primo esempio trovato, modifica il ciclo sul test_loader per salvare tutti gli esempi 
(o un numero sufficiente di campioni) per ciascuna classe.

Per ogni campione, calcola la GradCAM e l'overlay e aggiungili alle rispettive liste, per esempio:

    cams_list[cls].append(cam)
    overlays_list[cls].append(overlay)


2. Calcolare la media delle mappe
Dove modificare:

Dopo aver processato tutti i campioni (o quelli desiderati) per ciascuna classe, 
aggiungi un nuovo passaggio per calcolare la media lungo l'asse delle mappe accumulate.

Cosa fare:

Per ogni classe, converti la lista in un array NumPy e calcola la media. Ad esempio:


    mean_cam = np.mean(np.array(cams_list[cls]), axis=0)
    mean_overlay = np.mean(np.array(overlays_list[cls]), axis=0).astype(np.uint8)
    
Salva questi valori medi in due nuovi dizionari (ad es. mean_cams e mean_overlays).


3. Utilizzare le mappe medie per la visualizzazione
Dove modificare:

Nel passaggio 7, durante la creazione della figura finale, utilizza mean_cams e mean_overlays anziché le mappe di un singolo esempio.

Cosa fare:

Nel ciclo che disegna le immagini, usa ad esempio:

    cam_img = cv2.applyColorMap(np.uint8(255 * mean_cams[cls]), cv2.COLORMAP_INFERNO)

E per l'overlay:

    overlay_img = np.flipud(mean_overlays[cls])

In questo modo la figura finale rappresenterà la media delle attivazioni per ogni condizione sperimentale.

Considerazioni finali
Selezione del campione: Se il dataset è molto grande, potresti voler limitare il numero di campioni per evitare calcoli troppo onerosi.
Ad esempio, puoi considerare i primi N campioni per ogni classe oppure usare un campionamento casuale.

Prestazioni: Calcolare la media su molti campioni potrebbe essere intensivo, quindi valuta se sia il caso di eseguire questo calcolo
su un sottoinsieme rappresentativo.

Visualizzazione: Assicurati che la normalizzazione sia coerente per la media.
                A volte, potrebbe essere utile normalizzare ogni mappa individualmente prima di calcolare la media 
                oppure normalizzare l'immagine media risultante.

Questi passaggi ti permetteranno di ottenere una rappresentazione aggregata (media) sia della GradCAM che dell'overlay, 
fornendo una visione più robusta della risposta del modello per ogni condizione sperimentale.

---- ---- ---- ---- ---- ----  ---- ---- ----  ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- 
Riepilogo delle parti modificate
Selezione dei campioni:

Sostituisci la logica che si ferma al primo campione per ogni classe con la raccolta di tutti i campioni (o un campione rappresentativo) 
per ciascuna classe, salvandoli in un dizionario di liste.

Calcolo della Grad-CAM e Overlay:

Per ogni campione in ciascuna classe, calcola la mappa CAM e l'overlay e aggiungili a liste (cams_list e overlays_list).

Calcolo della media:

Dopo il ciclo, calcola la media per ogni classe usando np.mean.

Visualizzazione:

Nel passaggio di creazione della figura, utilizza le mappe medie (mean_cams e mean_overlays) al posto dei singoli campioni.

Queste modifiche ti permetteranno di ottenere, per ciascuna classe (0 e 1), una rappresentazione media sia della mappa Grad-CAM che dell'overlay, come richiesto.
---- ---- ---- ---- ---- ----  ---- ---- ----  ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- 


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io


#La funzione compute_gradcam_figure serve a calcolare e visualizzare 
#le mappe di attivazione Grad-CAM per un modello CNN2D, applicandole a spettrogrammi EEG. 

#In particolare, seleziona un campione per ciascuna classe (0 e 1), calcola la Grad-CAM e costruisce una figura con:

#Prima riga → Heatmap della Grad-CAM per entrambe le classi.
#Seconda riga → Heatmap sovrapposta allo spettrogramma originale.
#Questa visualizzazione aiuta a interpretare su quali parti dell'immagine il modello si sta concentrando per prendere decisioni.



#Questa funzione aiuta a visualizzare le regioni attivate dalla rete CNN su immagini di spettrogrammi EEG,
#evidenziando le aree più importanti per la classificazione.

#🔹 Esempio finale:
#La figura risultante avrà due righe:

#Heatmap puro della Grad-CAM.
#Heatmap sovrapposta allo spettrogramma EEG originale.

def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
    
      - Riga 1: Heatmap per classe 0 e classe 1.
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
      
    I titoli e le etichette degli assi sono personalizzati:
    
    - L'asse x rappresenta il tempo (ms) e l'asse y le frequenze (Hz) (solo per la riga overlay)    
    - I titoli dei subplot usano i nomi delle condizioni estratte automaticamente da 'exp_cond'
        (assumendo che exp_cond sia del tipo "th_resp_vs_pt_resp"), data_type e category_subject
    
    Il calcolo della CAM include il passaggio:
       weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
       cam = torch.sum(weights * activations, dim=1)
       cam = F.relu(cam)
    """
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    # Imposta il layer target (ad esempio conv3) e crea un'istanza di GradCAM
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)
    
    # Estrai i nomi delle condizioni separando exp_cond (es: "th_resp_vs_pt_resp")
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Dizionari per salvare un campione per ciascuna classe
    #samples = {}      # Qui salveremo il sample input per ogni classe 
    #labels_found = {} # Per tracciare se abbiamo già trovato un esempio per ciascuna classe di etichette
    
    '''CON MEDIA'''
    
    #Ora che ogni classe ha una sua chiave nel dizionario samples, non c'è più bisogno di usare labels_found 
    #per verificare la presenza di entrambe le classi.
    #In precedenza, stavi iterando nel test_loader e verificando la presenza di almeno un esempio per entrambe le classi (0 e 1),
    #ma ora i dati vengono direttamente organizzati nel dizionario in base alla loro classe. Quindi, se la classe non esiste nel dataset,
    #semplicemente non avrà una chiave nel dizionario samples.
    #Il controllo finale if 0 not in samples or 1 not in samples: è ancora necessario per assicurarsi che entrambe le classi siano presenti.
    #Se manca una classe, possiamo ancora uscire con un messaggio di errore.
    
    # Dizionari per salvare tutti i campioni per ciascuna classe
    samples = {0: [], 1: []}

    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            
            # Aggiungi il campione alla lista della classe corrispondente
            if label_int in samples:  # Assumendo solo classi 0 e 1
                samples[label_int].append(inputs[i].unsqueeze(0))
                
            '''SOLO UN ESEMPIO'''
            #if label_int not in labels_found:
            #    samples[label_int] = inputs[i].unsqueeze(0)  # Salva come tensore 4D
                
                
                #labels_found[label_int] = True
            #if 0 in labels_found and 1 in labels_found:
            #    break
        #if 0 in labels_found and 1 in labels_found:
        #    break

    # Se non troviamo entrambi gli esempi, esci con un messaggio
    #if 0 not in samples or 1 not in samples:
    #    print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
    #    return None
    
    
    
    
    
    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Per ciascun campione, calcola la GradCAM
    #cams = {} # Qui salveremo la mappa CAM per ogni classe
    #overlays = {} # Qui salveremo l'overlay (CAM + spettrogramma)
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''
    
    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}
    
    for cls in [0, 1]:
        for sample_input in samples[cls]:
        
        #sample_input = samples[cls]
        
            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)

            #Passaggio 5: Normalizzazione e upsampling

            #La mappa CAM viene normalizzata tra 0 e 1.
            #Viene ridimensionata (upsampling) per adattarsi alla dimensione originale dell'immagine

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            # -------------------------------
            # Passaggio 5: Normalizzazione e upsampling della CAM
            # ---------------------------

            # Normalizza la mappa
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            # Upsample alla dimensione dell'immagine di input
            cam = F.interpolate(cam.unsqueeze(1), size=sample_input.shape[2:], mode='bilinear', align_corners=False)
            cam = cam.squeeze().cpu().numpy()

            '''SOLO UN ESEMPIO'''
            #cams[cls] = cam


            '''CON MEDIA'''
            # Aggiungi la mappa alla lista per la classe
            cams_list[cls].append(cam)


            #Passaggio 6: Creazione dell’overlay Grad-CAM

            #L'immagine originale viene convertita in un array numpy.
            #La mappa CAM viene colorata con COLORMAP_JET.
            #Si sovrappone l'heatmap all'immagine originale.

            #🔹 Esempio pratico:
            #Se il CAM ha valori alti in alcune regioni, il colormap evidenzierà in rosso le aree più attivate.

            # -------------------------------
            # Passaggio 6: Creazione dell'Overlay
            # -------------------------------

            # Converte l'immagine originale in numpy; considerando che l'input è (batch, canali, frequenze, tempo)
            # dopo squeeze si ottiene (canali, frequenze, tempo). Per visualizzare come immagine color, trasformiamo in (frequenze, tempo, canali).

            # Prepara l'immagine originale per la visualizzazione
            img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)

            # Normalizza l'immagine in scala 0-255
            img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))

            # Applica la heatmap usando OpenCV
            #Per l'Overlay possiamo scegliere un colormap alternativo,
            # ad esempio COLORMAP_HOT o COLORMAP_INFERNO, per contrastare lo spettrogramma originale

            '''
            Il processo è lo stesso di quello descritto per le cam:

            I valori del CAM (normalizzati) vengono scalati a 255 e convertiti in un'immagine in scala di grigi.
            Il colormap INFERNO viene applicato per ottenere una rappresentazione colorata (dove i valori elevati diventano in genere rossi/gialli).
            La conversione BGR→RGB assicura una visualizzazione corretta
            '''

            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_VIRIDIS)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

            # Sovrapponi la heatmap all'immagine originale
            # Crea l'overlay: scegliendo pesi diversi per ottenere un contrasto chiaro


            '''
            Overlay troppo sfocato e colori discordanti
            Il problema che descrivi (overlay con toni azzurri/turchesi anziché il rosso della heatmap) può derivare da:

            Differenza di colormap e blending:
            L'overlay viene creato con una combinazione di due immagini: 
                1)lo spettrogramma originale (che potrebbe avere un proprio mapping di colori) e
                2) la heatmap

            Se il bilanciamento (i pesi) è 0.5-0.5, l'influenza dello spettrogramma può "modificare" i colori della heatmap.

            Suggerimenti:

            a) Modifica i pesi in cv2.addWeighted:
            Ad esempio, prova con 0.3 per l'immagine originale e 0.7 per la heatmap, in modo che il colore della heatmap (ad es. il rosso) prevalga.

            b) Uniforma il formato dell'immagine originale:
            Se lo spettrogramma originale è in scala di grigi o usa un colormap diverso,
            considera di convertirlo in un'immagine in scala di grigi a 8 bit prima di creare l'overlay.

            c) Usa lo stesso colormap: 
            Se vuoi che l'overlay abbia colori simili a quelli della heatmap, 
            usa lo stesso colormap (qui COLORMAP_INFERNO) per entrambe e regola il blending.

            '''
            overlay = cv2.addWeighted(img_norm, 0.4, heatmap, 0.6, 0)
            #overlay = cv2.addWeighted(img_norm, 0.5, heatmap, 0.5, 0)

            '''SOLO UN ESEMPIO'''
            #overlays[cls] = overlay

            '''CON MEDIA'''
            # Aggiungi l'overlay alla lista per la classe
            overlays_list[cls].append(overlay)
    
    
    mean_cams = {}
    mean_overlays = {}
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
        mean_overlays[cls] = np.mean(np.array(overlays_list[cls]), axis=0).astype(np.uint8)
        
    #Passaggio 7: Creazione della figura finale
    
    #La prima riga mostra solo le heatmap Grad-CAM.
    #La seconda riga mostra le heatmap sovrapposte agli spettrogrammi.

    # Crea la figura con due righe e due colonne

    # -------------------------------
    # Passaggio 7: Creazione della figura finale
    # -------------------------------
    # Creiamo una figura con 2 righe e 2 colonne:
    # - Prima riga: le heatmap CAM (da 0 a 1) per ciascuna condizione.
    # - Seconda riga: l'overlay (CAM + spettrogramma) per ciascuna condizione, con etichette per gli assi.
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Imposta un titolo generale per la figura
    
    #plt.suptitle(f"Grad-CAM Mapping - Experimental Condition: {exp_cond} - Subject: {category_subject}", fontsize=12)
    
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG trial Spectrogram\nExperimental Condition: {exp_cond} - Subject: {category_subject}",
    #fontsize=10,
    #y=0.95  # Puoi regolare la posizione verticale se necessario
    #)
    
    plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    
    # Prima riga: Visualizza solo le heatmap (CAM)
    for j, cls in enumerate([0, 1]):
        
        # Qui usiamo il colormap INFERNO per la CAM, ma puoi modificare se preferisci
        
        '''
        np.uint8(255 * cams[cls]):
        La mappa CAM (calcolata e normalizzata) ha valori compresi tra 0 e 1.
        Moltiplicando per 255 e convertendo in uint8, ottieni un'immagine in scala di grigi a 8 bit (0-255).
        
        cv2.applyColorMap(..., cv2.COLORMAP_INFERNO):
        Applica il colormap INFERNO che trasforma la scala di grigi in un'immagine a colori, 
        dove i valori bassi saranno scuri e quelli alti appariranno in toni caldi (ad es. giallo/rosso).
        
        cv2.cvtColor(..., cv2.COLOR_BGR2RGB):
        OpenCV usa il formato BGR per impostazione predefinita. 
        Convertire in RGB assicura che l'immagine venga visualizzata correttamente (matplotlib si aspetta RGB).
        
        '''
        
        '''SOLO UN ESEMPIO'''
        #cam_img = cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_INFERNO)
        
        '''CON MEDIA'''
        cam_img = cv2.applyColorMap(np.uint8(255 * mean_cams[cls]), cv2.COLORMAP_INFERNO)
        
        cam_img = cv2.cvtColor(cam_img, cv2.COLOR_BGR2RGB)
        
        '''QUI AGGIUNGIAMO L'INVERSIONE DEGLI ASSI'''
        # Se necessario, inverti gli assi per ottenere la visualizzazione desiderata
        cam_img = np.flipud(cam_img)  # Inverte verticalmente
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[0, j].imshow(cam_img)
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        axs[0, j].imshow(cam_img, extent=[0, 1000, 0, 26], aspect='auto')
        
        axs[0, j].set_title(f"Grad-CAM Mean Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[0, j].axis('off')
    
    # Seconda riga: Visualizza gli overlay con etichette degli assi
    for j, cls in enumerate([0, 1]):
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[1, j].imshow(overlays[cls])
        
        # Qui, se vuoi che l'asse y (frequenze) venga ordinato in modo crescente,
        # puoi anche invertire l'immagine verticalmente, se non è già corretto.
        
        '''SOLO UN ESEMPIO'''
        #overlay_img = np.flipud(overlays[cls])
        
        '''CON MEDIA'''
        overlay_img = np.flipud(mean_overlays[cls])
        
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        axs[1, j].imshow(overlay_img, extent=[0, 1000, 0, 26], aspect='auto')
        
        axs[1, j].set_title(f"Overlay of Grad-CAM Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[1, j].set_xlabel("Time (mms)", fontsize=10)
        axs[1, j].set_ylabel("Frequency (Hz)", fontsize=10)
        axs[1, j].axis('on')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION PER EEG STATS**

In [ ]:
'''GRADCAM ALGORITHM PER RAPPRESENTAZIONE EEG TEMPO x FREQUENZA NUOVA CON 

1) gestione del cudrnn per modelli con layer LSTM per far sì che il contesto imposta train() per sbloccare CuDNN-RNN, 
congela BN/Dropout in eval(), abilita i gradienti e ripristina tutto alla fine.

2) integri la gestione del test loader in formato raw per i plot sullo spettogramma




                                                VERSIONE NUOVA PER RAPPRESENTAZIONE TEMPO x FREQUENZA
                                                
                                                            (POST 27/06 ULTIMA VERSIONE DATATA)
                                                                        
                                                                        17/09/2025
                                                                    


ATTENZIONE CHE QUI LE SHAPE DEI DATI SONO DIVERSE OSSIA

1000 mms e 61 CANALI (PROGETTO INTERROGAIT!)

quindi il parametro "extent" passa in riga 2 e 3 da

extent=[0, 4000, 0, 81] a --> extent=[0, 1000, 0, 26]


'''




import torch.nn as nn

def model_has_cudnn_rnn(model):
    """Ritorna True se il modello usa LSTM/GRU/RNN supportati da CuDNN."""
    return any(isinstance(m, (nn.LSTM, nn.GRU, nn.RNN)) for m in model.modules())
    

'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''
def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device):
    
    
    '''SOLO PER I MODELLI OTTIMIZZATI CON ANCHE LA LSTM'''
    
    #Solo i modelli con LSTM entrano in questo giro; gli altri non cambiano di stato.
    #Con questa sequenza:
    #non ottieni più l’errore “cudnn RNN backward…”;
    #la rete “si comporta” come in eval (Dropout off, BN congelato) mentre calcoli le CAM;
    #l’ambiente di chiamata (il tuo loop di testing) riceve il modello esattamente nello stato in cui l’aveva passato alla funzione compute_gradcam_figure
    

    ### Perché serve model.train() anche se la CAM è presa prima della LSTM
    
    #Il backward, per arrivare dal loss (o dal logit scelto) fino al tuo layer conv3, deve comunque attraversare l’LSTM che sta più avanti nella rete.
    #Le implementazioni CuDNN degli RNN (LSTM/GRU) alzano un’eccezione se provi a chiamare tensor.backward() mentre il modulo è in modalità eval().
    #RuntimeError: cudnn RNN backward can only be called in training mode
    #Quindi, anche se la CAM è calcolata su conv3, devi mettere l’intero modello in train() per il tempo del backward.
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    ### Che cos’è model.training
    
    #model.training è un semplice flag booleano (impostato da nn.Module.train() / nn.Module.eval()), ereditato da tutti i sotto‑moduli.
    #Con was_training = model.training ricordi in che stato era il modello (quasi sempre False, cioè eval, nel tuo flusso)
    #per poterlo ripristinare dopo.
    
    #Facendo così
    
    #for m in model.modules():
    #if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                      #nn.Dropout, nn.Dropout2d, nn.Dropout3d)):
        #if m.training:         # cioè erano in train
            #m.eval()
            #frozen_layers.append(m)
    
    #Li sposti in eval uno per uno, senza toccare il resto della rete che deve restare in train() per far funzionare CuDNN‑RNN.
    
    
    ### Perché, a fine blocco, servono due ripristini
    
    #1) Riattivo i BatchNorm / Dropout che avevo forzato in eval:
    
    #for m in frozen_layers:
        #m.train()              # torna come prima
    
    #2) Riporto l’intero modello nello stato in cui si trovava prima del Grad‑CAM:
    
    #model.train(was_training)  # se era eval() torna eval, altrimenti resta train
    
    #Se non facessi il punto 1, lasceresti quei moduli permanentemente in eval anche quando, più tardi, 
    #rientri in training (per esempio in un fine‑tuning).
    #Se non facessi il punto 2, lasceresti tutto il modello in train → dropout attivo, BN che accumula statistiche, ecc.

    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❶ — se serve, abilito temporaneamente la modalità train per il modello ottimizzato che aveva ANCHE la LSTM... 
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    needs_train_mode = model_has_cudnn_rnn(model)
    
    if needs_train_mode:
        was_training = model.training      # salvo lo stato
        model.train()                      # abilito backward su CuDNN‑RNN
        
        # ➊ salvo lo stato di OGNI BN/Dropout
        
        saved = [(m, m.training) for m in model.modules()
             if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                               nn.Dropout, nn.Dropout2d, nn.Dropout3d))]
        
        model.train()                              # abilita backward su CuDNN‑RNN
        
        # ➋ congelo in ogni layer della rete gli strati di BatchNorm e Dropout
        for m, _ in saved:
            m.eval()
    
    # ------------------------------------------------------------
    # ❷ — QUI sotto metti tutto il tuo codice Grad‑CAM
    #      (forward, backward, costruzione delle mappe, plot, …)
    # ------------------------------------------------------------

    # … il tuo lunghissimo corpo della funzione rimane invariato …
    # → al momento di fare backward NON avrà più l’eccezione
    #   “cudnn RNN backward can only be called in training mode”
    
    
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)
    
    # Determina il target layer in base al tipo di modello
    #if isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.dw_conv1  # Per il modello separabile 2D
    #else:
        #target_layer = model.conv3  # Per il modello CNN3D
    
    '''OLD APPROACH'''
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    '''NEW APPROACH'''
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    # -------------------------------
    # Mapping etichette condizioni per i TITOLI dei plot
    # -------------------------------
    
    label_map = {
        "th_resp": "obs_resp",
        "pt_resp": "rec_resp",
    }
    
    def remap_condition_label(s: str) -> str:
        for old, new in label_map.items():
            s = s.replace(old, new)
        return s
    
    # Rimappa i nomi delle condizioni usati nei titoli dei subplot
    condition_names = [remap_condition_label(x) for x in condition_names]
    
    # Rimappa anche la stringa mostrata nel titolo principale (suptitle)
    exp_cond_display = remap_condition_label(exp_cond)
    
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    
    # ✅ Raccogli TUTTI i campioni per ciascuna classe
    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    
    '''DATI ORIGINALI DEL TEST LOADER'''
    samples = {0: [], 1: []}
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                
                samples[label_int].append(inputs[i].unsqueeze(0))
                
    
    '''TEST_LOADER RAW (DATI NON STANDARDIZZATI)'''
    samples_raw = {0: [], 1: []}
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
                
    
    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''
    
    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}
    
    for cls in [0, 1]:
        for sample_input in samples[cls]:
        
        #sample_input = samples[cls]
        
            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)
            
            
            '''
            
            TUTTO IL PASSAGGIO DELLO STEP 5 
            
            OSSIA NORMALIZZAZIONE i.e.,  NEL SENSO DI RISCALATURA NEL RANGE 0-1 + UPSAMPLING 
            
            (CHE SERVIVA PER UNIFORMARE I VALORI E ADATTARSI ALLA DIMENSIONE DELLA IMMAGINE ORIGINALE 
            PER VEDERE UN SOLO ESEMPIO DELLA CLASSE RISPETTO ALLA MAPPA DI ATTIVAZIONE E ALL'OVERLAY
            DEL GRADCAM RISPETTO ALLA IMMAGINE ORIGINALE)

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            
            NON SERVE PIU', AD ECCEZIONE DI QUESTE RIGHE CHE ORA TI RIMETTO QUI SOTTO!'''
            
            '''
            
            
            ✅ Cosa fa correttamente questo codice:
            
            Estrae i campioni da test_loader separandoli in samples[0] e samples[1].
            
            Per ogni campione di ogni classe:
            
            Calcola la Grad-CAM raw (senza riscaling),
            La interpola per adattarla alla dimensione originale (n_freq, n_time)
            Applica ReLU per tenere solo le attivazioni positive (come da standard Grad-CAM)
            La converte in NumPy e la salva in cams_list[cls].
            
            Alla fine, fa la media delle CAM raw per ciascuna classe:
            
            mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
            
            🔍 Stato attuale del dato
            
            cams_list[cls] → lista di array cam 2D non normalizzati, uno per ogni trial.
            mean_cams[cls] → array 2D (frequenza × tempo), media dei trial per ciascuna classe.
            
            La normalizzazione Z-score congiunta la farai dopo, sulla base di mean_cams.
            
            '''
            
            target_size = (sample_input.shape[2:]) # -> (n_freq, n_time)
            
            cam = F.interpolate(cam.unsqueeze(1), size = target_size, mode='bilinear', align_corners=False)
            
            # squeeze
            cam = cam.squeeze()                 # tensor 2D
            
            # Infine sposti su CPU e passi a numpy
            cam = cam.cpu().numpy()


            '''CON MEDIA'''
            # Aggiungi la mappa del singolo esempio alla lista per la classe (per poi dopo farci la media dentro mean_cams!)
            cams_list[cls].append(cam)
            
    
    # ============================================================
    # Calcolo dello heatmap media dei valori (raw) per ciascuna classe
    # ============================================================
    
    mean_cams = {}
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
    
    
    # ============================================================
    # Calcolo dello spettrogramma medio (raw) per ciascuna classe
    # ============================================================
    
    mean_raw_spectrograms = {}
    for cls in [0, 1]:
        if len(samples_raw[cls]) > 0:
            
            mean_raw_spectrograms[cls] = torch.cat(samples_raw[cls], dim=0).mean(dim =(0, 1)).detach().cpu().numpy()
        else:
            mean_raw_spectrograms[cls] = None
            
            #mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=0).squeeze().cpu().numpy()
            '''
            🔍 Perché dovrebbe funzionare?
            .detach(): Disattiva il tracciamento del gradiente (rende il tensore statico, senza dipendenze dalla computational graph di PyTorch).
            .cpu(): Porta il tensore sulla CPU (necessario per numpy()).
            .numpy(): Converte il tensore in un array NumPy.
            '''
            
            
            '''
            
            1) Calcolo della media degli spettrogrammi (rimozione dei canali)
            L'errore "Invalid shape (3, 26, 11)" con questa riga commentata sopra ☝️
            
            #mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=0).squeeze().detach().cpu().numpy()
             
            indica che l'array finale ha 3 canali in più (la prima dimensione) che non ti aspetti. 
            I tuoi dati originali hanno la forma:

            (trials, canali, frequenze, tempo)

            Se vuoi ottenere una rappresentazione media dello spettrogramma per tutti i trial di una classe, mediando anche sui canali, 
            allora devi calcolare la media lungo la dimensione dei trial e quella dei canali.
            
            Dovresti fare:
            
            mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=(0, 1)).detach().cpu().numpy()
            
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************
            SPIEGAZIONE:

            torch.cat(samples[cls], dim=0)
            => Concatena tutti i trial per quella classe lungo la dimensione 0, ottenendo un tensore con forma:
            (num_trials, canali, frequenze, tempo).

            .mean(dim=(0, 1))
            => Calcola la media prima lungo la dimensione dei trial (dim=0) e poi lungo quella dei canali (dim=1) in un'unica operazione, 
            ottenendo un tensore di forma (frequenze, tempo).

            .detach().cpu().numpy()
            => Rimuove il tracking del gradiente, sposta il tensore sulla CPU e lo converte in un array NumPy, pronto per imshow.

            Questo ti darà l'array 2D (frequenze × tempo) che imshow si aspetta.
            
            
            CHAGPT:
            
            Nel contesto del tuo esempio:

            La forma iniziale dei dati EEG in un formato tempo-frequenza era (batch, canali, frequenze, tempo), che è una matrice 4D. 
            
            Qui, hai un batch di dati, dove ogni dato ha la dimensione dei canali, frequenze, e tempo.

            Usando il codice:

                mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=(0,1)).detach().cpu().numpy()
                
            Stai concatenando lungo la dimensione del batch (dim=0), quindi ottieni una matrice che somma tutte le informazioni sul batch. 
            Successivamente, con .mean(dim=(0,1)) stai calcolando la media lungo le dimensioni dei canali (0) e del batch (1), 
            riducendo il risultato a una matrice 2D con la forma (frequenze, tempo), che è quella che desideri, ovvero 
            
            --> la media delle frequenze e del tempo su tutto il batch e i canali.

            Quindi sì, la forma risultante di mean_raw_spectrograms[cls] sarà una matrice 2D che rappresenta 
                1) le frequenze sulle righe e 
                2) il tempo sulle colonne
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************

            '''
    
    
    '''
    # =======================================================
    # Passaggio Finale: Creazione della figura finale
    # Ora la figura ha 3 righe:
    
    #  - Riga 1: Istogramma della distribuzione dei valori della heatmap media per ciascuna classe 
    #            normalizzata rispetto alla distribuzione congiunta!
    
    #  - Riga 2: GradCAM medio della distribuzione dei valori della heatmap media per ogni classe, 
    #            a seguito della normalizzazione rispetto alla distribuzione congiunta!
    
    #  - Riga 3: Spettrogramma medio (raw) rispetto ai Trial della Stessa Classe, su range logaritmico 
    # =======================================================
    
    
    Quando devo plottare l'istogramma dei valori di ogni heatmap media solamente (riga 3), 
    devo plottarli in base alla normalizzazione rispetto alla distribuzione congiunta.
    
    Quindi, devo plottarli in base al range minimo e massimo della intera distribuzione congiunta, quando è stata normalizzata!
    Di conseguenza devo fare
    
    1) Prendere la Media delle CAM per ogni classe (già fatto)
    2) Costruzione distribuzione congiunta raw
    3) Calcolo Media e Deviazione Standard della Distribuzione Congiunta
    4) Normalizzazione Z-score della Distribuzione Congiunta
    
    5) Prendo il range minimo e massimo della Distribuzione Congiunta Normalizzata
    
    Ossia, il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta,
    dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione.
    
    Quindi, dovrei ricreare un'altra variabile che contiene i valori normalizzati di entrambe le distribuzioni assieme,
    ossia una cosa del tipo
    
    normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    e da questa prendere il minimo ed il massimo!
    
    
    '''
   

    # Creiamo una figura con 4 righe e 2 colonne
    #fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    #plt.suptitle(f"Grad-CAM Mapping and Overlay over EEG Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    #fig, axs = plt.subplots(4, 2, figsize=(12, 20))
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    # Creiamo una figura con 3 righe e 2 colonne
    fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    
    #plt.suptitle(f"Grad-CAM Mapping over EEG Trials\nExperimental Conditions: {exp_cond}", fontsize=15)
    plt.suptitle(f"Grad-CAM Mapping over EEG Trials\nExperimental Conditions: {exp_cond_display}", fontsize=15)
    
    
    plt.tight_layout()  # Regola automaticamente la spaziatura globale
    plt.subplots_adjust(hspace = 0.5, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    

    
        
    # PLOT RIGA 1: Visualizzazione degli istogrammi della distribuzione dei valori delle heatmap medie RAW
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA
        
    '''
    Questi valori rappresentano la distribuzione delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta!

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Istogramma dei valori raw medi di ogni classe (su distribuzione congiunta!): Stai visualizzando un istogramma di questi valori medi, 
                                                           sulla distribuzione congiunta, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                          
                                                           - calcolo minimo e massimo a seguito della normalizzazione (?) e non prima
                                                           - faccio i plot di entrambe delle heatmap medie raw,
                                                             ma rispetto a distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello RAW!
                                                        
    
    N.B. PER IL NOME DEL TITOLO DEL PLOT
    
    Perché "Grad-CAM value" può creare confusione:
    Il termine "Grad-CAM value" potrebbe sembrare che faccia riferimento direttamente ai valori generati dalla mappa Grad-CAM finale. 
    Ma in realtà, i valori che stai trattando sono le attivazioni mediate e clippate, che formano la heatmap. 
    L'istogramma che stai tracciando rappresenta la distribuzione delle attivazioni prima della normalizzazione.

    Riepilogo
    Quindi, questi valori sono attivazioni pesate per ciascun pixel della mappa Grad-CAM, e mediati per classe. 
    Il processo di normalizzazione che segue (basato sui percentili) serve a enfatizzare il contrasto in modo da focalizzarsi sulle aree più significative 
    per la previsione.

    Per rispondere alla tua domanda: sì, è corretto dire che stai visualizzando la distribuzione delle attivazioni pesate prima della normalizzazione
    per migliorare il contrasto, ma è meglio riferirsi a questi valori come valori di attivazione della mappa Grad-CAM o valori della heatmap Grad-CAM, 
    piuttosto che "Grad-CAM value" che potrebbe risultare ambiguo.

    Se vuoi, puoi anche aggiungere una nota nella visualizzazione dell'istogramma che chiarisca il processo:
    
    axs[2, j].set_title(f"Histogram of Heatmap Activation Values (Raw, before Normalization) Class {condition_names[cls]}", fontsize=12)
    oppure
    axs[2, j].set_title(f"Histogram of Mean Heatmap Activation Values - Class {condition_names[cls]}", fontsize=12)
    
    è molto chiara e corretta!
    Indica perfettamente che stai visualizzando l'istogramma dei valori di attivazione medi della heatmap, 
    senza fare confusione sul fatto che si tratti di valori medi per ciascuna classe.
    
    In sintesi, questa frase comunica in modo preciso che stai mostrando la distribuzione delle attivazioni mediate dalla mappa Grad-CAM
    per una specifica classe. Quindi sì, va benissimo!
    
    '''
    
    #PER PLOT RIGA 1 
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    # Il range minimo e massimo su cui plottare entrambe le heatmap medie raw in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta raw
    
    vmin_raw = all_vals_raw.min()
    vmax_raw = all_vals_raw.max()
    
    
    # Prima riga: Visualizza l'istogramma della heatmap media rispetto alla distribuzione congiunta!
    for j, cls in enumerate([0, 1]):
        
        # Calcola l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
        axs[0, j].hist(mean_cams[cls].flatten(), bins= 'auto', color='blue', edgecolor='black')
        #axs[0, j].set_title(f"Histogram of Mean Grad-CAM values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_title(f"Histogram of Mean Heatmap Activation Values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_xlabel("Grad-CAM value", fontsize=10)
        axs[0, j].set_ylabel("Frequency", fontsize=10)
        
        
    
    # PLOT RIGA 2: Visualizzazione dei valori delle heatmap medie delle due classi
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    
    '''
    Questi valori rappresentano le heatmap medie delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta NORMALIZZATA

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Calcolo la distribuzione congiunta dei valori raw medi di ogni classe (su distribuzione congiunta!): 
        Stai visualizzando un istogramma di questi valori medi, sulla DISTRIBUZIONE CONGIUNTA, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                           
                                                           - calcolo media e deviazione standard delle distribuzione congiunta
                                                           - faccio la normalizzazione della distribuzione congiunta
                                                           
                                                           - calcolo minimo e massimo a seguito della normalizzazione e non prima
                                                             della distribuzione congiunta normalizzata
                                                           
                                                           - faccio i plot di entrambe delle heatmap medie normalizzate,
                                                             ma rispetto alla distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello NORMALIZZATO!
                                                        
    '''
    
    '''SOPRA ABBIAMO CREATO --> all_vals_raw'''
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    #all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    #Calcolo media e deviazione standard della distribuzione congiunta dei valori (raw) delle heatmap medie di entrambe le classi 
    #joint_mean = np.mean(all_vals_raw)
    #joint_std = np.std(all_vals_raw)
    
    # Normalizzazione Z-score della distribuzione congiunta
    #normalized_mean_cams = {}
    
    #for cls in [0, 1]:
        #normalized_mean_cams[cls] = (mean_cams[cls] - joint_mean) / joint_std

    # Il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione
    
    #normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    #vmin_normalized = normalized_all_vals.min()
    #vmax_normalized = normalized_all_vals.max()
    
    vmin_normalized = all_vals_raw.min()
    vmax_normalized = all_vals_raw.max()

    
    '''
    # Opzione: normalizzazione robusta con percentili
    vmin_normalized, vmax_normalized = np.percentile(all_vals_raw, [5, 95])
    '''
    
    # Seconda riga: Mean heatmap di ogni classe normalizzata a partire dalla distribuzione congiunta ( = di entrambe le classi)
    for j, cls in enumerate([0, 1]):
        
        
        im = axs[1, j].imshow(
            mean_cams[cls],
            #normalized_mean_cams[cls], #QUI LA RENDO IN 2D, NON IN 1D COME PRIMA
            #cmap='seismic',
            cmap='RdYlBu_r',
            vmin= vmin_normalized, vmax= vmax_normalized,
            #extent=[0, 4000, 0, 81],
            extent=[0, 1000, 0, 26],
            aspect='auto',
            origin='lower'
        )
        
        
        # → calcola 6 tick equi-spaziati
        ticks = np.linspace(vmin_normalized, vmax_normalized, 6)  
        
        cbar = fig.colorbar(
            im,
            ax=axs[1, j],
            orientation='horizontal',
            pad=0.12,
            ticks=ticks)
        
        cbar.ax.set_xticklabels([f"{t:.4f}" for t in ticks])

        
        axs[1, j].set_title(f"Mean Grad-CAM Heatmap (Raw) - Class {condition_names[cls]}", fontsize=12)
        
        '''QUESTA NON CONSENTE DEFINIZIONE ASSI!'''
        #axs[1, j].axis('off')
        
        axs[1, j].axis('on') 
        axs[1,j].set_xlabel("Time (mms)")
        axs[1,j].set_ylabel("Frequency (Hz)")
        
        #fig.colorbar(im, ax=axs[3, j], orientation='horizontal', pad=0.05)
    
    print(f"\033[1mRange heatmap raw globale (vmin_raw, vmax_raw): {vmin_normalized}, {vmax_normalized}\033[0m")
    
    # PLOT RIGA 3: Spettrogramma medio (raw) per ciascuna classe log-scaled
    
    '''
    Spiegazione delle modifiche aggiunte:

    1) Calcolo dello spettrogramma medio raw:

    Dopo aver raccolto i campioni nel dizionario samples, viene creato il dizionario mean_raw_spectrograms.
    Per ogni classe, i tensori vengono concatenati lungo la dimensione batch e si calcola la media sul batch (dim=0).
    
    Poi, però, ogni spettogramma medio deve congiunto in una distribuzione in modo da plottare poi il valore dello spettrogramma  
    rispetto al minimo ed al massimo della distribuzione congiunta dello spettrogramma medio di entrambe le classi! 
    
    Il risultato viene convertito in un array NumPy per il plotting.
    '''
    
    # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
    #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
    
    '''SE VOLESSI RESTRINGERE TRA 5° e 95° PERCENTILE'''
    #low_raw, high_raw = np.percentile(all_vals_raw, [5, 95])
    #half_width_raw = max(abs(low_raw), abs(high_raw))   
    #vmin_raw, vmax_raw = -half_width_raw, +half_width_raw
    
    '''ALTRIMENTI, TENGO TUTTO IL RANGE, DAL MINIMO AL MASSIMO'''
    
    #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
    #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
    '''
    
    1) Qual è la differenza tra prima e ora?
    
    Prima calcolavo, dentro il for cls in [0,1], un nuovo vmin_raw_samples e vmax_raw_samples separatamente per ciascuna classe.
    Di conseguenza ogni subplot sulla riga 3 aveva la sua scala di colori, rendendo impossibile un confronto diretto visivo 
    fra le due condizioni.
    
    Ora invece calcolerai una sola volta il log-power medio di entrambe le classi, ne ricavi un unico array congiunto,
    quindi ne estrai un solo vmin e vmax. Questo ti garantisce che entrambi i subplot della riga 3 useranno la stessa scala di colori.


    Per far sì che tutte e due le condizioni usino lo stesso minimo e massimo, sposto la raccolta dei limiti fuori dal ciclo,
    usando la distribuzione congiunta dei log-power di entrambe le classi
    
    vmin_raw_samples e vmax_raw_samples li calcoli una volta sola, su tutti i valori logaritmici concatenati.
    Entrambe le mappe usano esattamente lo stesso range, così le barre dei colori saranno allineate.
    
    Con questa modifica:

    log_mean_power contiene già i valori in scala logaritmica.
    vmin_raw_samples e vmax_raw_samples sono condivisi fra entrambe le colonne.
    Ogni subplot userà la stessa “barretta” di colore, quindi potrai confrontare direttamente “deep blues” e “reds” delle due condizioni.
    '''
    
    # 1. Calcola i log-power medi per ciascuna classe
    log_mean_power = {
        cls: np.log1p(mean_raw_spectrograms[cls])
        for cls in [0,1]
    }

    # 2. Raccogli TUTTI i valori in un unico array
    all_log_vals = np.concatenate([
        log_mean_power[0].flatten(),
        log_mean_power[1].flatten()
    ])

    # 3. Estrai un unico vmin/vmax condiviso
    vmin_raw_samples = all_log_vals.min()
    vmax_raw_samples = all_log_vals.max()

        
    # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
    for j, cls in enumerate([0, 1]):
        
        #if mean_raw_spectrograms[cls] is not None:
        if log_mean_power[cls] is not None:    
            
            #Trasformo in scala logaritmica i miei dati EEG sulla spettro medio di ogni classe
            #mean_raw_spectrograms[cls] = np.log1p(mean_raw_spectrograms[cls])
        
            
            # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
            #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
            
            #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
            #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
            im = axs[2, j].imshow(log_mean_power[cls], 
                                  #mean_raw_spectrograms[cls], 
                                  #extent=[0, 4000, 0, 81],
                                  extent=[0, 1000, 0, 26],
                                  aspect='auto', 
                                  cmap='jet', 
                                  vmin = vmin_raw_samples, vmax = vmax_raw_samples,
                                  origin='lower')
            
            axs[2, j].set_title(f"Log-Scaled Mean Raw Spectrogram - Class {condition_names[cls]}", fontsize=12)
            axs[2, j].set_xlabel("Time (mms)", fontsize=10)
            axs[2, j].set_ylabel("Frequency (Hz)", fontsize=10)
            
        
            
            '''
            
            ATTENZIONE QUI CHE C'ERA UN GRAVE ERRORE
            
            --> fig.colorbar(im, ax=axs[3, j]) 
            
            #Qui la Color Bar Verticale sarebbe 
            #scala dello spettrogramma raw, finita per sbaglio sul Δ-GradCAM perché hai scritto ax=axs[3,j] invece di ax=axs[4,j].
            
            
            La barra VERTICALE (CHE DOVEVA STAR NELLA 5° RIGA!!!!) della color bar accanto alla heatmap ti sta mostrando
            
            i VALORI ASSOLUTI della Grad-CAM (nel tuo caso non normalizzati, quindi scala di milioni --> variabile hist_data
            ossia l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
            '''
            fig.colorbar(im, ax=axs[2, j])
            
            axs[2, j].axis('on')
        else:
            axs[2, j].axis("off")
            
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❸ — Ripristino allo stato precedente il modello ottimizzato trovato migliore, che aveva incluso anche layer LSTM
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    if needs_train_mode:
        # ➌ ripristino layer singoli (i.e., riporto BN/Dropout dove stavano in eval mode)
        for m, old_flag in saved:
            m.train(old_flag)
        # ➍ ripristino lo stato globale del modello (di nuovo ad .eval())
        # i.e.,  come era stato passato in input alla funzione compute_gradcam_figure a partire 'load_best_run_results'!
        
        #Così simuli l’eval (Dropout off, BN congelato) pur essendo in train() per soddisfare CuDNN‑RNN.
        model.train(was_training)
        
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

##### **MODELLI CNN2D, BiLSTM e Transformer**

In [ ]:
'''
DEFINIZIONE DEI MODELLI NUOVI PER P300 FROM 2D TIME-FREQUENCY SIGNAL  - AGGIORNATI A SETTEMBRE 2025 COME QUELLI DEL TASK MOTORIO!


                                                                ***CNN2D_LSTM_TF*** 


Uso la stessa rete neurale usata per Brain Decoding Task Motorio


'''

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN2D_LSTM_TF(nn.Module):

    def __init__(self, input_channels=61, num_classes=2, dropout=0.2):
        super().__init__()
        # --- Block 1 ---
        self.bn1   = nn.BatchNorm2d(input_channels)    # normalizza 64 canali
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)

        # --- Block 2 (residual) ---
        # Proiezione 1×1 per riallineare i canali di skip (32→64)
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn   = nn.BatchNorm2d(64)

        self.bn2a   = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b   = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        # --- Head: Dropout + LSTM + FC finale ---
        self.dropout     = nn.Dropout(dropout)
        self.hidden_size = 64
        
        # dopo 3 pool: freq da 81→10, time da 9→1 → feature per timestep = 128×1
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.classifier = nn.Linear(self.hidden_size, num_classes)

    def forward(self, x):
        # x: (B,64,81,9)

        # --- Block 1 ---
        x = self.bn1(x)                   # → (B,64,81,9)
        x = F.relu(self.conv1(x))         # → (B,32,81,9)
        x = self.pool(x)                  # → (B,32,40,4)

        # --- Block 2 (residuo) ---
        res = x                           # skip: (B,32,40,4)
        res = self.res_conv(res)          # progetto: → (B,64,40,4)
        res = self.res_bn(res)            # → (B,64,40,4)

        # main path
        x = self.bn2a(x)                  # → (B,32,40,4)
        x = F.relu(self.conv2a(x))        # → (B,64,40,4)
        x = self.bn2b(x)                  # → (B,64,40,4)
        x = self.conv2b(x)                # → (B,64,40,4)

        x = x + res                       # somma residua valida → (B,64,40,4)
        x = F.relu(x)                     
        x = self.pool(x)                  # → (B,64,20,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))  # → (B,128,20,2)
        x = self.pool(x)                     # → (B,128,10,1)

        # --- Prepara per LSTM ---
        x = x.permute(0, 2, 1, 3)         # → (B,10,128,1)
        b, seq, ch, tw = x.size()        
        x = x.reshape(b, seq, ch * tw)    # → (B,10,128)

        # --- LSTM + classificazione ---
        out, _ = self.lstm(self.dropout(x))  # → out: (B,10,64)
        last = out[:, -1, :]                 # prendo l’ultima uscita → (B,64)
        logits = self.classifier(last)       # → (B,2)
        return logits
    

'''
Gli LSTM si aspettano un input in forma (batch, lunghezza_sequenza, dimensione_feature). 
Dovrai quindi decidere qual è la dimensione sequenziale.

Opzione comune: usare il tempo come sequenza
Step 1: Trasponi i dati in modo da avere il tempo come dimensione sequenziale.

Dalla forma (batch, canali, frequenze, tempo) puoi fare:


x = x.permute(0, 3, 1, 2)  # Diventa (batch, tempo, canali, frequenze)

Step 2: Unisci le dimensioni dei canali e dei bin di frequenza in un’unica dimensione di feature:


batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # Ora: (batch, tempo, canali*frequenze)

Nel tuo caso, per 3 canali e 38 bin di frequenza: input_size = 3 * 38 = 114 e lunghezza sequenza = 6.

Nota: Se invece preferisci usare i bin di frequenza come sequenza, potresti fare:

x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
x = x.reshape(batch, frequenze, canali * tempo)  # Sequence length = 38, feature size = 3*6 = 18
La scelta dipende dal tipo di informazione temporale o spettrale che vuoi evidenziare.

'''

class ReadMEndYou(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.5, bidirectional=False):
        """
        input_size: dimensione delle feature per time-step (dovrà essere canali * frequenze)
        hidden_sizes: lista con le dimensioni degli hidden state, es. [24, 48, 62]
        output_size: numero di classi
        
        """
    
        super(ReadMEndYou, self).__init__()
        
        self.bidirectional = bidirectional # Impostazione della bidirezionalità    
        
        # Adattiamo hidden_size in base alla bidirezionalità
        self.hidden_sizes = [
            hidden_sizes[0] * 2 if bidirectional else hidden_sizes[0],
            hidden_sizes[1] * 2 if bidirectional else hidden_sizes[1],
            hidden_sizes[2] * 2 if bidirectional else hidden_sizes[2]
        ]
        
        self.lstm1 = nn.LSTM(input_size=input_size, 
                             hidden_size=self.hidden_sizes[0], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0, 
                             bidirectional=bidirectional)
        self.lstm2 = nn.LSTM(input_size=self.hidden_sizes[0] * 2 if bidirectional else self.hidden_sizes[0],
                             hidden_size=self.hidden_sizes[1], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        self.lstm3 = nn.LSTM(input_size=self.hidden_sizes[1] * 2 if bidirectional else self.hidden_sizes[1],
                             hidden_size=self.hidden_sizes[2],
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_sizes[2] * 2 if bidirectional else self.hidden_sizes[2], output_size)
    
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Trasponi per avere il tempo come dimensione sequenziale:
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        # Ora input_size deve essere channels * freqs (es. 64 * 81 = 7471)
        
        # LSTM 1
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        
        # LSTM 2
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        
        # LSTM 3
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        
        # Estraiamo l'output dell'ultimo time-step
        out = out[:, -1, :]
        
        # Dropout prima del layer fully connected    
        out = self.dropout(out)
        
        # Passaggio attraverso il layer finale per la previsione
        out = self.fc(out)
        return out
        


'''
Il modulo Transformer in PyTorch lavora tipicamente su input di forma (seq_length, batch, embedding_dim).

Nel codice attuale, si parte da una forma simile a (batch, canali, seq_length), ma dovrai adattarla alla nuova struttura.

Possibili approcci:

1) Approccio A: usare il tempo come sequenza

Se consideri il tempo (6 time windows) come la sequenza, puoi procedere come segue:

A) Unisci canali e frequenze in un’unica dimensione di feature:

# Dati originali: (batch, canali, frequenze, tempo)
x = x.permute(0, 3, 1, 2)  # (batch, tempo, canali, frequenze)
batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # (batch, tempo, 3*38 = 114)

B) Modifica il layer di embedding:

Nel codice attuale, l'embedding è definito come:

self.embedding = nn.Linear(seq_length, d_model)
Dovrai cambiarlo in modo che mappi le dimensioni delle feature (in questo caso 114) a uno spazio latente:

self.embedding = nn.Linear(canali * frequenze, d_model)

C) Permuta per il Transformer:

Dopo l'embedding, passa l'input alla forma (seq_length, batch, d_model):

x = x.permute(1, 0, 2)  # Ora: (tempo, batch, d_model)


2) Approccio B: usare i bin di frequenza come sequenza
In alternativa, se reputi più rilevante la risoluzione spettrale, puoi considerare i 38 bin come sequenza e combinare canali e tempo:


x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
batch, frequenze, canali, tempo = x.shape
x = x.reshape(batch, frequenze, canali * tempo)  # (batch, frequenze, 3*6 = 18)

E poi procedere con un embedding layer che mappa da 18 a d_model e permutare in (frequenze, batch, d_model).

Scelta dell'approccio:
Se l'aspetto temporale è più critico, probabilmente è meglio usare l’Approccio A (sequenza di lunghezza 6).
Se invece vuoi dare maggior rilievo alla struttura spettrale, l’Approccio B potrebbe essere più indicato.

Ricorda che la scelta dipende dalla natura del tuo problema e dalla rilevanza delle informazioni temporali rispetto a quelle spettrali.
'''

import torch
import torch.nn as nn

#Scelta: In questa implementazione abbiamo deciso di usare il tempo come sequenza.
#In alternativa, potresti scegliere i bin di frequenza come sequenza, ma ciò richiederebbe una diversa riorganizzazione delle dimensioni 
#(ad esempio, un permute diverso).



class ReadMYMind(nn.Module):

    def __init__(self, d_model, num_heads, num_layers, num_classes, channels=61, freqs=26):
        
        super(ReadMYMind, self).__init__()

        # Il layer di embedding mapperà la feature dimension (channels * freqs) a d_model
        self.embedding = nn.Linear(channels * freqs, d_model)
        
        # Transformer per l'attenzione spaziale (qui si applica direttamente alla sequenza temporale)
        self.spatial_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Transformer per l'attenzione temporale (si potrebbe considerare un'iterazione successiva)
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Cross-attention per combinare le rappresentazioni
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads)
        
        # Fusione e classificazione finale
        self.fc_fusion = nn.Linear(d_model, d_model)
        self.fc_classify = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        # x: (batch, canali, frequenze, tempo)
        
        # Utilizziamo il tempo come sequenza
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        
        # Embedding: (batch, tempo, d_model)
        x = self.embedding(x)
        
        # Transformer richiede input di forma (seq_length, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # -> (tempo, batch, d_model)
        
        # Applichiamo il Transformer per l'attenzione spaziale
        x_spatial = self.spatial_transformer(x)
        # Applichiamo il Transformer per l'attenzione temporale
        x_temporal = self.temporal_transformer(x_spatial)
        
        # Cross-attention: (tempo, batch, d_model)
        x_cross, _ = self.cross_attention(x_spatial, x_temporal, x_temporal)
        
        # Fusione: per esempio, facciamo una media sul tempo (dimensione 0)
        x_fused = self.fc_fusion((x_spatial + x_temporal).mean(dim=0))  # -> (batch, d_model)
        
        # Classificazione finale
        output = self.fc_classify(x_fused)  # -> (batch, num_classes)
        
        return output
    

##### **NUOVO LOOP PER DATI NON HYPER SU CNN2D, BiLSTM e Transformer**

In [ ]:
import os
import re

import random
#perché è importante numpy.random.seed()?
#https://www.analyticsvidhya.com/blog/2021/12/what-does-numpy-random-seed-do/#:~:text=The%20numpy%20random%20seed%20is,displays%20the%20same%20random%20numbers.
from numpy.random import seed

import numpy as np
import copy as cp

from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

#importing librerie pytorch
import torch 
import torch.nn as nn #neural network module
import torch.optim as optim #ottimizzatore
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset

#from sklearn.model_selection import KFold

#importing librerie numpy, pandas, scikit-learn e matplotlib
import numpy as np


import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from tqdm import tqdm



In [ ]:
#LOOP PER CARICARE I DATI NON HYPER
data_dict = {}

# Condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

for condition in experimental_conditions:

    for data_type in ["spectrograms"]:
        
        for category in ["familiar", "unfamiliar"]:
            
            for subject_type in ["th", "pt"]:
            
                # Caricamento e suddivisione dei dati
                if data_type == "wavelet":
                    X, y = load_data(data_type, category, subject_type, wavelet_level="delta")
                else:
                    X, y = load_data(data_type, category, subject_type)

                #key = f"{condition}/{data_type}_{category}_{subject_type}"
                key = f"{condition}_{data_type}_{category}_{subject_type}"
                data_dict[key] = (X, y)

                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{key}\033[0m - Forma X: {X.shape}, Lunghezza y: {len(y)}")

In [ ]:
data_dict.keys()

In [ ]:
'''
perfetto ora, siccome ho creato data_dict nel modo di cui sopra, 
ora dentro ogni chiave, 
ci sono già tutte le chiavi associate correttamente, per estrarmi i dati e labels corrispondenti di quella combinazione di fattori lì.

infatti dentro ogni chiave c'è una tupla, con 2 elementi, il primo è l'array dei dati, il secondo è l'array delle labels
'''

data_dict['th_resp_vs_pt_resp_spectrograms_familiar_th'][0].shape

In [ ]:
'''NEW VERSION'''


'''ATTENZIONE CHE QUI HO AGGIUNTO --> "_time_frequency_" alla base_dir!'''


# Percorso base per il salvataggio
base_folder = "/home/stefano/Interrogait/spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks"
os.makedirs(base_folder, exist_ok=True)


# Condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

# Creazione della struttura delle cartelle
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)
            
            print(f"Cartella creata: \033[1m{path}\033[0m")

In [ ]:
'''VERSIONE NUOVA UFFICIALE


Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, 
lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.


'''

from wandb import Api
import torch
import numpy as np

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import io
import matplotlib.pyplot as plt
from PIL import Image


import re


    
'''
1) questa serve per plottare le metriche di loss e accuracy in ogni modello e condizione sperimentale
per salvarla dentro al dizionario 'training_plot' come buffer di memoria
'''


def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    '''
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    #Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    
    #plt.close(fig)
    
    '''
    # Salvare il plot in un buffer di memoria
    '''
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()
    
    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data


'''
2) questa serve per estrarmi le stringhe per ricostruire il nome del progetto su W&B per 
poi estrarmi le metriche ottenute sul training e validation 
da salvare sempre dentro al dizionario 'training_plot' 
'''

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "spectrograms" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        


'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
#combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
#condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

#print("Condizione:", condition_experiment)
#print("Data Type:", data_type)
#print("Soggetto:", subject_key)



'''

Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.

'''


'''
3) Dopodiché, comincia la funzione di load_best_run_results che, 
per ogni progetto e sweep del relativo modello,

si va ad estrarre le metriche del train (corregge il calcolo del train_auc)
e si calcola anche per il validation phase la confusion matrix e classification report


4) dopodichè dovrebbe richiamare la funzione di 
"plot_training_results" in modo che poi si salvi i plot di training e validation (sia loss che accuracy)
in modo che si salvi tutto in una immagine come buffer che viene spuntato fuori da quella funzione 

e poi inserito come valore dentro al dizionario training_results che sarà l'output di "load_best_run_results" 


quindi qui sotto mi manca richiamare la funzione "plot_training_results" con una variabile tipo training_plot = plot_training_results che avrà come argomenti

queste liste qua salvate come colonne del df creato dentro a 'load_best_run_results!'


loss_train_history     = df["train_loss"].tolist()
loss_val_history       = df["val_loss"].tolist()
accuracy_train_history = df["train_accuracy"].tolist()
accuracy_val_history   = df["val_accuracy"].tolist()


5) dopodiché mi serve caricare tutte queste info dentro al dizionario train_results, che sarà l'output di load_best_run_results... 
e su questo ho dei dubbi su quali chiavi del dizionario tenere separate oppure se "unirne" qualcuna, aggregando tutte le info del sweep_config assieme, 
sia che siano veri iper-parametri (learning rate etc) o parametri architetturali della rete (anche se avevano valori fissi) il più delle volte se vedi



# --- CNN1D solo quando model_name=="CNN1D" ---

sweep_config_cnn1d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN1D"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "dropout": {"values": [0.5]},
        
        "conv_out_channels":{"values":[16]},

        "conv_k1":{"values":[3]},
        "conv_k2":{"values":[3]},
        "conv_k3":{"values":[3]},

        "conv_s1":{"values":[1]},
        "conv_s2":{"values":[1]},
        "conv_s3":{"values":[1]},

        "pool_p1":{"values":[1]},
        "pool_p2":{"values":[1]},
        "pool_p3":{"values":[1]},

        "pool_type":{"values":["avg"]},
        "fc1_units":{"values":[8]},

        "cnn_act1":{"values":["relu"]},
        "cnn_act2":{"values":["relu"]},
        "cnn_act3":{"values":["relu"]}
    }
}


# --- BiLSTM solo quando model_name=="BiLSTM" ---

sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "dropout": {"values": [0.5]},

        # --- BiLSTM solo quando model_name=="BiLSTM" ---
        "hidden_size":{"values":[16]},
        "bidirectional":{"values":[0,1]}
    }
}


# --- Transformer solo quando model_name =="Transformer" ---

sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "dropout": {"values": [0.5]},
         # --- Transformer solo quando model_name=="Transformer" ---
        "d_model":{"values":[8]},
        
        #"num_heads":{"values":[2,4,6,8,10,12]}, # 6,10,12 vanno tolti, perché non divisori di tutti i d_model!
        "num_heads":{"values":[2]}, # solo divisori di tutti i d_model
        
        "num_layers":{"values":[2]},
        "ff_mult":{"values":[2]},
        "transformer_activations":{"values":["relu","gelu"]}
    }
}



'''

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def load_best_run_results(
    key, # es. "rest_vs_left_fist_spectrograms_familiar_th"
    model, # # <-- istanza PyTorch già caricata con i pesi best es. "CNN3D_LSTM_FC"
    
    sweep_config,      # <— qui richiamo lo sweep config del modello corrispondente
    
    data_loaders, # dict con DataLoader per "train" e "val"
    entity= "my_wb_entity"): # entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
    
    
    # --- 1) Parse key e ricava project name ---
    exp_cond, data_type, category_subject = parse_combination_key(key)
    
    
    '''CAMBIATA PER DATI INTERROGAIT IN RAPPRESENTAZIONE TIME DOMAIN 1D'''
    #project = f"{exp_cond}_{data_type}_channels_freqs_new_3d_grid_multiband"
    #project = f"{exp_cond}_{data_type}_time_freqs_new_imagery_3d_grid_multiband"
    
    #project = f"{exp_cond}_{data_type}_{category_subject}"
    
    
    #project=f"{exp_cond}_{data_type}_time_freqs_{category_subject}"
    
    
    project=f"{exp_cond}_{data_type}_time_frequency_{category_subject}"
    
    '''OLD APPROACH'''
    #model_name = type(model).__name__
    

    '''SE ESTRAGGO SWEEP ID A POSTERIORI DAL PROGETTO

    1) Prendo tutte le run del progetto e modello corrispondente
    2) Filtro solo quelle con config["model_name"] == model_name.
    3) Controllo che ce ne sia almeno una (altrimenti errore).
    4) Costruisce un set di tutti gli r.sweep e verifica che sia esattamente uno (altrimenti errore).
    5) Estrae quello unico (.pop()) e lo stampa insieme al numero di run.
    6) Infine, seleziona la singola best_run sulla base di val_accuracy.

    '''
    
    '''NEW APPROACH'''
    # === PATCH: accetta alias tra nome classe PyTorch e nome usato nello sweep ===
    def _get_param_list(cfg, key):
        p = cfg["parameters"][key]
        vals = p.get("values", p.get("value"))
        if isinstance(vals, str): 
            return [vals]
        return list(vals) if isinstance(vals, (list, tuple)) else [vals]

    model_class = type(model).__name__          # es. "ReadMEndYou"
    cfg_names   = _get_param_list(sweep_config, "model_name")   # es. ["BiLSTM"]
    aliases     = set([model_class, *cfg_names])                # {"ReadMEndYou","BiLSTM"}
    
    def matches_aliases(r):
        return (
            r.config.get("model_name") in aliases or
            r.config.get("model_class") in aliases or
            bool(set(r.tags or []) & aliases)
        )
    
    
    # 2) Recupero tutte le run del progetto
    api  = Api()
    runs = api.runs(f"{entity}/{project}")

    # 3) filtro solo quelle del modello giusto
    
    '''OLD APPROACH'''
    #runs_filtered = [r for r in runs if r.config.get("model_name", "") == model_name]
    #n_runs = len(runs_filtered)
    
    '''NEW APPROACH'''
    runs_filtered = [r for r in runs if matches_aliases(r)]
    n_runs = len(runs_filtered)
    

    if n_runs == 0:
        raise RuntimeError(f"Nessuna run trovata per progetto `{project}` e modello `{model_name}`")

    # 4) controllo che le run filtrate appartengano tutte allo stesso sweep
    unique_sweeps = {r.sweep for r in runs_filtered}
    if len(unique_sweeps) != 1:
        raise RuntimeError(
            f"Trovati più sweep per progetto `{project}` e modello `{model_name}`: {unique_sweeps}"
        )

    # 5) estraggo lo sweep_id
    sweep_id_unico = unique_sweeps.pop()
    #print(f"✓ Trovate \033[1m{n_runs}\033[0m runs in progetto `{project}` e modello `{model_name}`, sweep: `{sweep_id_unico}`")
    print(f"✓ Trovate \033[1m{n_runs}\033[0m runs\n")
    print(f"✓ Progetto \033[1m`{project}`\033[0m\n")
    print(f"✓ Modello \033[1m`{model_name}`\033[0m\n")
    print(f"✓ Sweep \033[1m`{sweep_id_unico}`\033[0m\n\n")

    # 6) scelgo la run con val_accuracy massima
    best_run = max(runs_filtered, key=lambda r: r.summary.get("val_accuracy", 0.0))

    # --- 7) Estraggo tutta la history (compresi i train_auc sbagliati) ---
    df = best_run.history(
        keys=[
          "train_loss","train_accuracy","train_precision",
          "train_recall","train_f1","train_auc",
          "val_loss","val_accuracy"
        ],
        pandas=True
    )
    # converto in liste
    loss_train_history     = df["train_loss"].tolist()
    loss_val_history       = df["val_loss"].tolist()
    accuracy_train_history = df["train_accuracy"].tolist()
    accuracy_val_history   = df["val_accuracy"].tolist()
    precision_train_history= df["train_precision"].tolist()
    recall_train_history   = df["train_recall"].tolist()
    f1_train_history       = df["train_f1"].tolist()
    auc_train_history      = df["train_auc"].tolist()

    # best_epoch (su val_accuracy)
    best_epoch = int(df["val_accuracy"].idxmax())

    # --- 8) Prendo il modello ottimizzato .pkl corrispondente passato in input ---
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device).eval()

    # --- 9) Ricalcolo vero train AUC–ROC sul train_loader ---
    y_t_train, y_s_train = [], []
    with torch.no_grad():
        for x,y in data_loaders["train"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            y_s_train.extend(probs)
            y_t_train.extend(y.numpy())
            
    true_auc_train = roc_auc_score(np.array(y_t_train), np.array(y_s_train))

    # Sovrascrivo il vecchio valore sbagliato
    auc_train_history[best_epoch] = true_auc_train

    # Ricostruisco best_metrics
    best_metrics = {
      "train_loss":       [round(loss_train_history[best_epoch],4)],
      "train_accuracy":   [round(accuracy_train_history[best_epoch],4)],
      "train_precision":  [round(precision_train_history[best_epoch],4)],
      "train_recall":     [round(recall_train_history[best_epoch],4)],
      "train_f1_score":   [round(f1_train_history[best_epoch],4)],
      "train_auc":        [round(true_auc_train,4)]
    }

    # --- 10) Ricreo confusion matrix e classification report su val set ---
    y_t_val, y_p_val = [], []
    with torch.no_grad():
        for x,y in data_loaders["val"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds  = (probs >= 0.5).astype(int)
            y_p_val.extend(preds)
            y_t_val.extend(y.numpy())
            
    confusion_matrix_val = confusion_matrix(y_t_val, y_p_val)
    classification_report_val = classification_report(y_t_val, y_p_val, output_dict=False)

    #Solo una nota: qui non serve per training che l'auc abbia l'average='weighted' 
    #perché è binario e stai usando score continui.
    #anche se sopra lo avevi messo in "training_sweep".
    
    #Per le altre metriche (precision, recall, f1_score invece) l'average andava bene!
    #Anche in binario: average='weighted' = fai la media pesata per supporto delle metriche per ciascuna classe (0 e 1). 
    #È sensato se hai sbilanciamento e vuoi che le metriche riflettano anche quanto è frequente ciascuna classe. 
    
    #L’unica cosa da essere consapevoli è che non stai riportando “F1 della classe positiva”, 
    #ma una F1 complessiva pesata sulle due classi. 
    #Ma va bene, basta essere coerenti e chiari nel testo della tesi/paper."
    

    # --- 10) Ricreo confusion matrix e classification report su val set ---
    
    #Per il validation set, invece, rifai il calcolo:
    
    #y_t_val = true labels (0/1).
    #y_p_val = predizioni binarie (0/1), usate per accuracy / precision / recall / f1.
    #y_s_val = score continui (probabilità o logit della classe 1), usati per il calcolo dell'AUC-ROC:
    
    #Quindi diventerà ---> val_auc = roc_auc_score(y_t_val, y_s_val)
    #Quindi qui "y_s_val" è semplicemente la lista di p(y=1) per ogni campione di validation.
    
    y_t_val, y_p_val, y_s_val = [], [], []
    with torch.no_grad():
        for x,y in data_loaders["val"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds  = (probs >= 0.5).astype(int)
            
            
            y_p_val.extend(preds) # predizioni 0/1
            y_s_val.extend(probs) # score continui per AUC
            y_t_val.extend(y.numpy()) 
            
    confusion_matrix_val = confusion_matrix(y_t_val, y_p_val)
    classification_report_val = classification_report(y_t_val, y_p_val, output_dict=False)
    
    # Metriche Validation
    #val_accuracy = accuracy_score(y_t_val, y_p_val)
    val_precision = precision_score(y_t_val, y_p_val, average='weighted')
    val_recall    = recall_score(y_t_val, y_p_val, average='weighted')
    val_f1        = f1_score(y_t_val, y_p_val, average='weighted')
    
    try:
        val_auc = roc_auc_score(y_t_val, y_s_val)   # <-- NOTHING average=... qui
    except ValueError:
        print("⚠️ AUC non calcolabile: nel val set c'è una sola classe.")
        val_auc = np.nan
    
    # Val performances alla best_epoch
    
    '''
    Qui la cosa importante è: il modello con cui stai facendo il forward su data_loaders["val"] 
    è il best model, cioè quello che hai caricato da .pkl e che dovrebbe corrispondere esattamente 
    ai pesi di best_epoch per quella run.
    
    Per cui, val_loss e val_accuracy che salvi nel dict sono proprio quelli loggati all’epoca best_epoch durante il training.
    Questi sono coerenti con “la migliore epoca secondo val_accuracy”.
    
    Mentre le altre metriche (precision, recall, f1_score, son ricalcolate in base al best model che aveva ottenuto
    a quella epoca specifica la migliore val_accuracy!
    
    '''    
    
    validation_performances = {
        # dalla history di W&B (loss/acc per quella epoch)
        "val_loss":       [round(loss_val_history[best_epoch],4)],
        "val_accuracy":   [round(accuracy_val_history[best_epoch],4)],
        
        # dalle metriche ricalcolate con il best_model
        "val_precision":  [round(val_precision,4)],
        "val_recall":     [round(val_recall,4)],
        "val_f1_score":   [round(val_f1,4)],
        "val_auc":        [round(val_auc,4)],
    }
    
        
    # --- 10) Plot delle curve loss/accuracy tra train e test ---
    training_plot = plot_training_results(
        loss_train_history,
        loss_val_history,
        accuracy_train_history,
        accuracy_val_history
    )

    # --- 11) Composizione del dict finale identico a `training()` ---
    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        
        "best_model": model,
        
        # VALIDATION
        "validation_performances": validation_performances,
        
        "confusion_matrix": confusion_matrix_val,
        "classification_report": classification_report_val,
    
        "hyperparams" : {k: best_run.config[k] for k in best_run.config.keys() if k in sweep_config["parameters"]},
            
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }
    
    '''
    Ho questo errore "Errore “cudnn RNN backward can only be called in training mode”" solo con i dati di 
    left_fist_vs_right_fist, per il modello SeparableCNN2D_LSTM_FC, 
    mentre con i dati delle altre condizioni sperimentali, ossia:
    
    rest_vs_left_fist o rest_vs_right_fist, sempre per il modello SeparableCNN2D_LSTM_FC,non succede ... come mai solo con l'ultimo succede? 
    
    cioè dove dovrei aver lasciato il modello caricato in eval.() ?
    
    probabilmente qui nella funzione load_best_train_results!?
    
    quindi qui poi alla fine dovrei rimettere il modello in un'altra modalità alla fine della funzione? 
    perché in sostanza, dovrebbe succedere che in sostanza... non succede nulla per lo stesso modello per  gli altri dati, 
    perché ogni volta che ne prendo uno lo porto in eval e vabbè.. ma poi il problema succede solo per l'ultimo caso solo, 
    perché forse l'ultimo proprio, ossia solo SeparableCNN2D_LSTM_FC usa proprio il layer LSTM e quindi da errore là,
    perché dentro a load_best_train_results è rimasto in .eval() ed ha il layer LSTM e quindi dà errore?
    
    
    
    Perché l’errore appare “solo” con l’ultima combinazione

    1. load_best_run_results() termina con:

    model.to(device).eval()   # ← il modello rimane in eval()
    
    2. In compute_gradcam_figure() tu usi il best model che hai messo in train_results["best_model"] (quello appena impostato in eval()), poi esegui:

   
    output = model(sample_input)
    ...
    target.backward()         # <-- gradiente attraverso l’LSTM
    3. Il kernel CuDNN per gli RNN (LSTM/GRU) rifiuta il backward quando il modulo è in modalità inference (eval()), e solleva:

    
    RuntimeError: cudnn RNN backward can only be called in training mode
    
    4. Per le combinazioni precedenti con lo stesso modello “SeparableCNN2D_LSTM_FC” non è esploso perché, con ogni probabilità, 
    use_lstm=False nelle relative run migliori (quindi l’LSTM non c’è e CuDNN non interviene).
    
    Nell’ultima combinazione invece la best‑run ha use_lstm=True, quindi compare l’LSTM e l’errore salta fuori.
    
    '''

    return train_results

In [ ]:
# 2.1 – Sweep config per ciascun modello

#CNN2D_LSTM_TF
sweep_config_cnn2d_lstm_tf = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
    
        "dropout": {"values": [0.5]},
    }
}


sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "dropout": {"values": [0.5]},
        "bidirectional": {"values": [False, True]},
        
        #Soluzione 1 per mettere valori agli hidden sizes
        #"hidden1": {"values": [24, 32, 48, 64]},
        #"hidden2": {"values": [48, 64, 96, 128]},
        #"hidden3": {"values": [62, 96, 128, 160]}
        # in build del modello: hidden_sizes=[hidden1, hidden2, hidden3]
        
        #Soluzione 2 per mettere valori agli hidden sizes
        
        #hidden_sizes = [24, 48, 62]
        #lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    }
}


sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "d_model": {"values": [32]},
        "num_heads": {"values": [2]},
        "num_layers": {"values": [2]},
    }
}

In [ ]:
# Imposta il seme per la riproducibilità

#Imposta il seme per i generatori casuali di PyTorch (per operazioni sui tensori e inizializzazione dei pesi dei modelli).
#Importante se vuoi garantire che l'addestramento del modello produca gli stessi risultati in diverse esecuzioni.
torch.manual_seed(32)

#Imposta il seme per NumPy, utile se NumPy viene usato per operazioni casuali (ad es. shuffling dei dati, inizializzazione di matrici, ecc.).
#Importante se usi NumPy per il preprocessing dei dati e vuoi riproducibilità.

np.random.seed(32)

#mposta il seme per il modulo random di Python (utile se si usano funzioni di randomizzazione di Python puro).
#Importante solo se usi random per operazioni come mescolamento di liste.
random.seed(32)

#Imposta il seme per i generatori casuali su GPU, se disponibile.
#Utile se stai eseguendo il codice su una GPU per garantire riproducibilità anche in quel contesto.

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(32)

       
'''

In questo caso, 

il set processed_datasets traccia i dataset già elaborati, 
e il set processed_models tiene traccia delle combinazioni già effettuate (modello + dataset). 

In questo modo, puoi escludere un dataset dal training se è già stato utilizzato in precedenza, 
anche se usato con un modello differente.
'''

# Dizionario per tracciare la standardizzazione usata per ogni combinazione d
# Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)
models_info = {}


# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Set per tenere traccia delle combinazioni già elaborate
processed_models = set()


# Path delle performance dei modelli ottimizzati con weight and biases
# Path per trovare le best performances di ogni modello per ogni combinazione dei dati

'''ATTENZIONE CHE QUI HO AGGIUNTO --> "_time_frequency" alla base_folder!'''


base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_time_frequency"

# Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder
#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


'''ATTENZIONE CHE QUI HO AGGIUNTO --> "_time_frequency" alla save_path_folder!'''

save_path_folder = "/home/stefano/Interrogait/spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks"

                                              #spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks

# --- LOOP PRINCIPALE (con minime modifiche) ---
for key, (X_data, y_data) in data_dict.items():
    
    print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")
    
    if key in processed_datasets:
        print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
        continue
        
    processed_datasets.add(key)
    
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
    print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")
    
    
    
    '''
    CREO COPIA TEST_LOADER_RAW PER I PLOT DEL POWER RAW PER BANDA E CLASSE
    '''
    # 1) salva una copia RAW dei soli dati di test PRIMA di standardizzare
    X_test_raw = X_test.copy()
    y_test_raw = y_test.copy()
    
    # 2) tensori
    X_raw_tensor = torch.tensor(X_test_raw, dtype=torch.float32)
    y_raw_tensor = torch.tensor(y_test_raw, dtype=torch.long)
    
    
    
    
    #for model_name in ["CNN2D", "BiLSTM", "Transformer"]:
    
    for model_name in ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]:
        
            
        model_key = f"{model_name}_{key}"
        if model_key in processed_models:
            print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
            continue
        processed_models.add(model_key)
        
        print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")
        
        # Prova a caricare la configurazione e i pesi ottimali dal file .pkl
        
        '''
        load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
        parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate
        
        exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
        pesi del modello e i suoi iper-parametri
        
        Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
        '''
        
        config, best_weights = load_config_if_available(key, model_name, base_folder)
        
        if config is None:
            raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")
        
        '''
        Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
        MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
        '''
        
        # Parsifica la chiave una volta sola e memorizza i valori
        exp_cond, data_type, category_subject = parse_combination_key(key)
        
        '''
        Dpodiché, 
        
        1) si carica i vari valori degli iper-parametri,
        2) si esegue la standardizzazione se servisse,
        3) prepara il modello per la divisione in train_loader etc.,
        4) si carica la configurazione dei pesi del modello, 
        5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 
        
        6) esegue il training e il test e poi
        
        7) si salva il tutto nella path corrispondente...
        
        '''
        
        '''
        PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
        DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''
        
        #---
        #model_lr = config["lr"]
        #model_weight_decay = config["weight_decay"]
        #model_n_epochs = config["n_epochs"]
        #model_patience = config["patience"]
        
        
        #model_batch_size = config["batch_size"]
        #model_standardization = config["standardization"]
        
        #model_n_epochs = config["n_epochs"]
        #model_patience = config["patience"]
        
        #model_lr = config["lr"]
        
        #'''NUOVE MODIFICHE'''
        #model_beta1 =  config["beta1"]
        #model_beta2 =  config["beta2"]
        #model_eps = config["eps"]
        #---
        
        
        model_lr = config["lr"]
        model_weight_decay = config["weight_decay"]
        model_n_epochs = config["n_epochs"]
        model_patience = config["patience"]
        
        model_batch_size = config["batch_size"]
        model_standardization = config["standardization"]
        
        '''NUOVE MODIFICHE'''
        model_beta1 = config["beta1"]
        model_beta2 = config["beta2"]
        model_eps = config["eps"]
        
        
    
        
        print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        
        # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
        models_info[model_key] = {"standardization": model_standardization}
        
        
        # 3) dataset & loader per test set (per plots power raw) –‑  IMPORTANTISSIMO: shuffle=False
        raw_dataset = TensorDataset(X_raw_tensor, y_raw_tensor)
        test_loader_raw = DataLoader(raw_dataset,
                             batch_size=model_batch_size,
                             shuffle=False)
        
        
        
        '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
        IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
        '''

        if model_standardization:
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
        else:
            print(f"\033[1mNO Standardizzazione Dati!\033[0m")
        
        # Sposta il modello sulla GPU (se disponibile)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
        # Preparazione dei dataloaders
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)
        
        #sweep_config_cnn2d_lstm_tf
        #sweep_config_bilstm
        #sweep_config_transformer
        
        '''PARAMETRI MODEL-SPECIFIC DI CNN2D_LSTM_TF, BiLSTM e Transformer, ma richiamati al momento della inizializzazione dei relativi sweep_config! '''
    
        '''
        # Appena caricato X_train, X_val, X_test, etc.
        # X_train.shape == (N, channels, freq_bins, time_steps)

        _, channels, freq_bins, time_steps = X_train.shape
        
        #Classificazione (binaria)
        
        num_classes = 2 
        
        #CNN2D_LSTM_TF
        model_conv_out_channels = config["conv_out_channels"]
        
        model_conv_k1_h = config["conv_k1_h"]
        model_conv_k1_w = config["conv_k1_w"]
         
        model_conv_k2_h = config["conv_k2_h"]
        model_conv_k2_w = config["conv_k2_w"]
        
        model_conv_k3_h = config["conv_k3_h"]
        model_conv_k3_w = config["conv_k3_w"]
        
        model_conv_s1_h = config["conv_s1_h"]
        model_conv_s1_w = config["conv_s1_w"]

        model_conv_s2_h = config["conv_s2_h"]
        model_conv_s2_w = config["conv_s2_w"]

        model_conv_s3_h = config["conv_s3_h"]
        model_conv_s3_w = config["conv_s3_w"]

        model_pool_p1_h = config["pool_p1_h"]
        model_pool_p1_w = config["pool_p1_w"]
        
        model_pool_p2_h = config["pool_p2_h"]
        model_pool_p2_w = config["pool_p2_w"]
        
        model_pool_p3_h = config["pool_p3_h"]
        model_pool_p3_w = config["pool_p3_w"]
        
        model_pool_type = config["pool_type"]
        model_fc1_units = config["fc1_units"]
                                 
        model_cnn_act1 = config["cnn_act1"]
        model_cnn_act2 = config["cnn_act2"]
        model_cnn_act3 = config["cnn_act3"]
        
        
        #BiLSTM
        model_hidden_size = config["hidden_size"]
        model_bidirectional = config["bidirectional"]
        
        
        #Transformer
        model_d_model = config["d_model"]
        model_num_heads = config["num_heads"]
        
        model_num_layers = config["num_layers"]
        model_ff_mult = config["ff_mult"]
        model_transformer_activations = config["transformer_activations"]
        
        model_dropout = config["dropout"]
        
        
        # Inizializzazione del modello
        if model_name == "CNN2D":
            #model = CNN2D(input_channels=3, num_classes=2)
            
            model = CNN2D(
                input_channels=channels,
                num_classes=num_classes,
                conv_out_channels = model_conv_out_channels,
                conv_k1_h = model_conv_k1_h, conv_k1_w = model_conv_k1_w,
                conv_k2_h = model_conv_k2_h, conv_k2_w = model_conv_k2_w,
                conv_k3_h = model_conv_k3_h, conv_k3_w = model_conv_k3_w,
                conv_s1_h = model_conv_s1_h, conv_s1_w = model_conv_s1_w,
                conv_s2_h = model_conv_s2_h, conv_s2_w = model_conv_s2_w,
                conv_s3_h = model_conv_s3_h, conv_s3_w = model_conv_s3_w,
                pool_p1_h = model_pool_p1_h, pool_p1_w = model_pool_p1_w,
                pool_p2_h = model_pool_p2_h, pool_p2_w = model_pool_p2_w,
                pool_p3_h = model_pool_p3_h, pool_p3_w = model_pool_p3_w,
                pool_type = model_pool_type,
                fc1_units = model_fc1_units,
                dropout = model_dropout,
                cnn_act1 = model_cnn_act1, 
                cnn_act2 = model_cnn_act2,
                cnn_act3 = model_cnn_act3
            )
            
        elif model_name == "BiLSTM":
            #model = ReadMEndYou(input_size= 3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
            
            model = ReadMEndYou(input_size=channels*freq_bins, 
                                hidden_size= model_hidden_size,
                                output_size=num_classes,
                                num_layers=3, 
                                dropout= model_dropout, 
                                bidirectional= model_bidirectional)
       
        #trans = ReadMYMind(num_channels=channels, num_freqs= frequency, seq_length=time,
                           #d_model=64, num_heads=8, num_layers=2, num_classes=num_classes,
                           #ff_mult=2, dropout=0.1, transformer_activations='relu')
        #out_trans = trans(x)
        #print("Transformer output:", out_trans.shape)

        #wandb.init(project = f"{condition}_{data_type}_time_frequency_{category_subject}", name = run_name, tags = tags)
            
        elif model_name == "Transformer":
            
            model = ReadMYMind(num_channels = channels, 
                               num_freqs = freq_bins, 
                               
                               seq_length = time_steps,
                               d_model = model_d_model,
                               
                               num_heads = model_num_heads, 
                               num_layers = model_num_layers,
                               
                               num_classes = num_classes,
                               
                               ff_mult = model_ff_mult,
                               dropout = model_dropout,
                               transformer_activations = model_transformer_activations)
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
        
        
        '''
        
        
        
        channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])
        
        if model_name == "CNN2D_LSTM_TF":
            
            sweep_config = sweep_config_cnn2d_lstm_tf
            
            model_dropout = float(config.get('dropout', 0.5))
            
            
            model = CNN2D_LSTM_TF(input_channels = channels, num_classes = 2, dropout = model_dropout) #input_channels = 61
            print(f"\nInizializzazione Modello \033[1mCNN2D_LSTM_TF\033[0m")
            
        elif model_name == "BiLSTM":
            
            sweep_config = sweep_config_bilstm
            
            model_dropout = float(config.get('dropout', 0.5))
            model_bidirectional = bool(config.get('bidirectional', False))
            
            #input_size= 61 * 26
            model = ReadMEndYou(input_size= channels * freqs, hidden_sizes=[24, 48, 62], output_size=2, dropout = model_dropout, bidirectional = model_bidirectional)
            
            print(f"\nInizializzazione Modello \033[1mReadMEndYou (BiLSTM)\033[0m")
            
        elif model_name == "Transformer":
            
            sweep_config = sweep_config_transformer
            
            model_d_model = int(config.get('d_model', 32))
            model_num_heads = int(config.get('num_heads', 2))
            model_num_layers  = int(config.get('num_layers', 2))
            
            model = ReadMYMind(d_model = model_d_model, num_heads = model_num_heads, num_layers = model_num_layers, num_classes=2, channels = channels, freqs= freqs) #channels = 61, freqs=26
            print(f"\nInizializzazione Modello \033[1mReadMYMind (Transformer)\033[0m")
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
            
        
        # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
        if best_weights is not None:
            try:
                model.load_state_dict(best_weights)
                print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
            except Exception as e:
                print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                continue
        
        
        # Definizione del criterio di perdita
        criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
        
        '''OLD VERSION'''
        # Definizione dell'ottimizzatore con i parametri aggiornati
        #optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)
        
        '''NEW VERSION'''
        # 1) Optimizer con betas, eps, weight_decay
    
        optimizer = optim.Adam(
            model.parameters(),
            lr = model_lr, 
            betas=(model_beta1, model_beta2),
            eps=model_eps,
            weight_decay=model_weight_decay
        )
        
        '''OLD VERSION'''
        #print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)
    
        #print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_test_results = testing(my_train_results, test_loader, criterion)
        
        
        # 1) prepara i data_loaders per train/val
        data_loaders = {
            "train": train_loader,
            "val":   val_loader
        }
        
        print(f"🏋️‍♂️Salvo le metriche del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m a seguito della ottimizzazione su W&B ...")
        
        
        #ATTENZIONE al potenziale problema di stringa, non di API: 

        #i due esempi che hai postato in realtà usano diversi caratteri “‑” (uno è il classico ASCII U+002D, l’altro è un non‑breaking hyphen U+2011 o simili), quindi quando chiami
        
        #entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
        #stai passando un nome che W&B non riconosce (e quindi api.projects(entity=…) torna vuoto), mentre con

        #entity = "stefano-bargione-universit-di-roma-tor-vergata"
        #funziona perché lì usi i semplici - ASCII.

        my_train_results = load_best_run_results(
            key=key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity = "stefano-bargione-universit-di-roma-tor-vergata"
        )
        
        
        '''
        L’entity che passi a Api().runs(f"{entity}/{project}") è semplicemente il tuo account (o l’organizzazione) su W&B,
        cioè la parte che compare subito prima del nome del progetto nell’URL.

        Per esempio, se quando apri il tuo progetto su W&B vedi un indirizzo del tipo
        
        -> https://wandb.ai/steclab/some_project_name, allora entity = "steclab".
        
        Se invece lavori sotto un’organizzazione 
        
        -> “cool‑team”, e l’URL è https://wandb.ai/cool-team/some_project_name, allora userai entity = "cool-team".

        Puoi verificarlo:

        Accedi a wandb.ai e vai sul progetto.
        Leggi la prima parte dell’URL (tra wandb.ai/ e il /project_name).
        Copiala esattamente come stringa in entity.

        Così il tuo Api().runs(f"{entity}/{project}") andrà a pescare proprio le run che hai lanciato tu.

        my_train_results = load_best_run_results(
            key= key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity= "mio-entity"
        )
        
        '''
        
        print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        # 3) usa il best_model caricato dentro `train_results` e chiama il testing
        my_test_results = testing(my_train_results, test_loader, criterion)
        
        
        
        '''
        GRADCAM COMPUTATION PER IL MODELLO CNN2D_LSTM_TF
        
        La funzione compute_gradcam_figure estrae due campioni (uno per ogni classe) e crea una figura con le due righe richieste.
        
        Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
        'save_performance_results', in modo da essere salvato nella path corretta. 
        
        La funzione 'save_performance_results' è stata modificata 
        per gestire ANCHE questo nuovo input dell'immagine 
        
        (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
        seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:
        
        - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
        - tipologia di dato EEG prelevato (i.e., spectrograms) 
        - provenienza del dato stesso (i.e., familiar_th)
        )
        
        Spiegazione:
        
        La funzione compute_gradcam_figure eseguire il calcolo di GradCAM (vedi dettagli nella sua funzione)
        e alla fine ritornerà in output una variabile 
        
        'fig_image' che sarà poi assegnata alla variabile 'gradcam_image',
        che è un oggetto buffer, che contiene i dati binari dell'immagine in formato PNG
        (poiché abbiamo usato plt.savefig con format='png'). 
        
        Quindi, quando passi gradcam_image (cioè fig_image) alla funzione 'save_performance_results',
        viene scritto direttamente su disco come file PNG.
        
        Non c'è bisogno di ri-aprire o convertire ulteriormente, a meno che tu non voglia manipolare l'immagine in seguito.
        Quindi, la soluzione è corretta così com'è:
        il buffer viene salvato come file PNG nella directory specificata, 
        e successivamente potrai aprirlo con una libreria come cv2 o PIL se necessario.        
        
        Quindi, gradcam_image (i.e., fig_image) viene quindi passato correttamente dentro al loop di training e test, 
        tramite 'save_performance_results', come input, 
        che salverà quindi poi l'immagine nella path corrispondente 

        '''
        
        # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
        gradcam_image = None
        
        #if model_name == "CNN2D":
        
        '''ATTENZIONE MODIFICA QUI'''
        
        if model_name == "CNN2D_LSTM_TF":
            
            '''
            ATTENZIONE! Qui ho aggiunto alla nuova versione di "compute_gradcam_figure" (versione del 17/09/2025)
            il test_loader_raw tra gli argomenti della funzione!
            '''
            gradcam_image = compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device)
            if gradcam_image is not None:
                print(f"Creazione di \033[1mGradCAM Image\033[0m per il modello \033[1m{model_name}\033[0m.")
                
        print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        save_performance_results(model_name,
                                 my_train_results,
                                 my_test_results,
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder = save_path_folder,
                                 gradcam_image = gradcam_image)
        
        
        '''
        N.B
        
        gradcam_image = None avverrà solo all'inizio cioè per il primo modello CNN2D, che verrà testato con una certa combinazione di dati mi sa.. 
        ma servirebbe tracciare in qualche modo 

        1) o che la gradcam_image di ogni combinazione venga ri-azzerata alla fine loop
        2) o che venga monitorato che gradcam_image di una combinazione di dati già analizzata venga esclusa poi
        (o messa in un set) in modo che rivenga per errore sovrascritta più volte.. 
        
        Forse la strada più veloce potrebbe essere la soluzione 1)
        
        La soluzione più veloce e semplice è reimpostare la variabile gradcam_image a None alla fine dell'iterazione per ogni combinazione di dati
        (cioè, all'interno del ciclo esterno che itera su key). 
         
        In questo modo, per ogni nuovo dataset la variabile viene "azzera" e viene calcolata l'immagine GradCAM solo per quella combinazione, 
        evitando di sovrascrivere accidentalmente i risultati già calcolati per combinazioni precedenti.
         
        Un'altra possibilità sarebbe tenere traccia delle chiavi (o combinazioni) per cui hai già calcolato la GradCAM,
        ad esempio usando un set, e saltare il calcolo se la combinazione è già presente. 
        
        Tuttavia, se ogni combinazione deve avere la sua immagine, 
        la soluzione più semplice è quella di reimpostare gradcam_image = None alla fine dell'iterazione.
        
        Quindi, per esempio, alla fine del ciclo per ogni dataset (key) potresti fare:
        (VEDI SOTTO)
        
        In questo modo, ti assicuri che per ogni nuova combinazione la variabile sia pulita e pronta per essere ricalcolata, 
        senza rischio di sovrascrivere o confondere i risultati
        '''
        
        # Reimposta gradcam_image a None per la prossima combinazione di dati
        gradcam_image = None

In [ ]:
print("finito")

In [ ]:
print(models_info.keys())

In [ ]:
with open('/home/stefano/Interrogait/models_info_spectrograms_time_frequency_EEG_GradCAM_Checks.pkl', 'wb') as f:
    pickle.dump(models_info, f)

##### **CREAZIONE DELLE TABLES CON INTEGRAZIONE DELLE PERFORMANCE TRAINING & TEST DEI MODELLI DENTRO DATAFRAME**

#### Integrazioni Performance Training e Test del Modello dentro DataFrame - OLD APPROACH

##### **OLD BEST APPROACH**

In [ ]:
import os
import pickle

# Definiamo le path
paths = {
    "TH_FAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/TH_FAM_UNSCALED/",
    "PT_FAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/PT_FAM_UNSCALED/",
    "TH_UNFAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/TH_UNFAM_UNSCALED/",
    "PT_UNFAM": "/home/stefano/Interrogait/PPRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/PT_UNFAM_UNSCALED/"
}


# Identificatori delle triplette
identifiers = ["1_20", "1_45", "wavelet_delta"]

# Dizionario per salvare i risultati
all_models_dict = {}

# Iteriamo su ogni path
for condition, path in paths.items():
    models_dict = {identifier: {} for identifier in identifiers}  # Dizionario per i modelli della path corrente
    
    # Controlliamo che la directory esista
    if not os.path.exists(path):
        print(f"Directory non trovata: {path}")
        continue
    
    # Otteniamo la lista di file nella directory
    files = os.listdir(path)
    
    # Filtriamo e carichiamo i file per ciascun identificatore
    for identifier in identifiers:
        for file in files:
            if file.endswith(f"{identifier}.pkl"):  # Controlliamo se il file termina con l'identificatore
                file_path = os.path.join(path, file)
                try:
                    with open(file_path, "rb") as f:
                        models_dict[identifier][file] = pickle.load(f)
                except Exception as e:
                    print(f"Errore nel caricamento di {file}: {e}")
    
    # Salviamo il dizionario della path corrente nel dizionario principale
    all_models_dict[condition] = models_dict


In [ ]:
# Ora all_models_dict contiene i dati strutturati per ogni path e identificatore
# Stampa i tipi di ogni sotto-dizionario
for path_key, identifier_dict in all_models_dict.items():
    print(f"Path: {path_key} - Tipo: {type(identifier_dict)}")
    for identifier, model_dict in identifier_dict.items():
        print(f"  Identifier: {identifier} - Tipo: {type(model_dict)}")
        for model, data in model_dict.items():
            print(f"    Model: {model} - Tipo: {type(data)}")

In [ ]:
all_models_dict.keys()

In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Definiamo le path
paths = {
    "TH_FAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/TH_FAM_UNSCALED/",
    "PT_FAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/PT_FAM_UNSCALED/",
    "TH_UNFAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/TH_UNFAM_UNSCALED/",
    "PT_UNFAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/PT_UNFAM_UNSCALED/"
}

# Identificatori delle triplette
identifiers = ["1_20", "1_45", "wavelet_delta"]

# Iteriamo su ogni path
for condition, path in paths.items():
    
    # Dizionario per i modelli della path corrente
    models_dict = {identifier: {} for identifier in identifiers}
    
    # Controlliamo che la directory esista
    if not os.path.exists(path):
        print(f"Directory non trovata: {path}")
        continue
    
    # Otteniamo la lista di file nella directory
    files = os.listdir(path)
    
    # Filtriamo e carichiamo i file per ciascun identificatore
    for identifier in identifiers:
        for file in files:
            if file.endswith(f"{identifier}.pkl"):  # Controlliamo se il file termina con l'identificatore
                file_path = os.path.join(path, file)
                try:
                    with open(file_path, "rb") as f:
                        models_dict[identifier][file] = pickle.load(f)
                except Exception as e:
                    print(f"Errore nel caricamento di {file}: {e}")

    # Ora creiamo un file separato per ogni identificatore
    for identifier in identifiers:
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}
        
        print(f"\nProcessing condition: {condition}, identifier: {identifier}\n")

        # Iteriamo sui modelli relativi a questo identificatore
        for model_name, model_data in models_dict[identifier].items():
            name_model = model_name.split("_")[0]  # Prende solo la parte prima del primo '_'
            print(f"    Processing model: {name_model}")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}

                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]

            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame per l'identificatore specifico
        df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis('off')

        # Usa pandas per creare una tabella nel grafico
        tabla = table(ax, df_performances, loc='center', colWidths=[0.2]*len(df_performances.columns))

        # Personalizza la tabella
        tabla.auto_set_font_size(True)
        tabla.set_fontsize(10)
        tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        for key, cell in tabla.get_celld().items():
            if key[0] == 0:  # Se la riga è la prima (intestazioni delle colonne)
                cell.set_text_props(weight='bold')  # Grassetto

        # Creazione della directory se non esiste
        output_dir = paths[condition]
        file_name = f"{condition}_{identifier}_models.png"
        img_file_path = os.path.join(output_dir, file_name)

        # Salva l'immagine della tabella
        fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        plt.close(fig)  # Chiudi la figura per liberare memoria

        print(f"Tabella salvata in: {img_file_path}")


#### Integrazioni Performance Training e Test del Modello dentro DataFrame - NEW APPROACH

#### Spiegazione

Ok in questo modo, model_standardization_dict dovrebbe andare a salvarsi se, i dati per quella combinazione di fattori, rispetto ad uno specifico modello, siano stati standardizzati o meno.

Di conseguenza, dentro questo loop

    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    from pandas.plotting import table

    # Base folder
    base_folder = "/home/stefano/Interrogait/time_domain_best_models_post_WB"

    # Condizioni sperimentali
    experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

    # Tipologie di dati
    data_types = ["1_20", "1_45", "wavelet_delta"]

    # Subfolders per tipologia di soggetto
    subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

    # Dizionario per salvare tutti i modelli
    all_models = {}

    # Caricamento dei modelli
    for condition in experimental_conditions:
        for data_type in data_types:
            for subfolder in subfolders:

                path = os.path.join(base_folder, condition, data_type, subfolder)

                if not os.path.exists(path):
                    print(f"Directory non trovata: {path}")
                    continue

                # Creiamo la chiave per questa combinazione
                key = f"{condition}_{data_type}_{subfolder}"
                all_models[key] = {}

                # Otteniamo la lista di file nella directory
                files = os.listdir(path)

                # Filtriamo e carichiamo i file .pkl
                for file in files:
                    if file.endswith(".pkl"):  # Controlliamo se è un file modello
                        file_path = os.path.join(path, file)
                        try:
                            with open(file_path, "rb") as f:
                                all_models[key][file] = pickle.load(f)
                        except Exception as e:
                            print(f"Errore nel caricamento di {file}: {e}")

    # Creazione delle tabelle di performance
    for key, models_dict in all_models.items():

        # Otteniamo le informazioni dalla chiave
        #condition, data_type, subfolder = key.split("_", 2)
        condition, data_type, subfolder = parse_combination_models_keys(key)

        print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")

        # Creazione della tabella
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

        # Iteriamo sui modelli caricati
        for model_name, model_data in models_dict.items():
            name_model = model_name.split("_")[0]  # Nome modello
            print(f"    Processing model: \033[1m{name_model}\033[0m")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}


                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]


            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame
        #df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        #fig, ax = plt.subplots(figsize=(10, 6))
        #ax.axis('off')
        #tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
        #tabla.auto_set_font_size(True)
        #tabla.set_fontsize(10)
        #tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        #for key, cell in tabla.get_celld().items():
        #    if key[0] == 0:
        #        cell.set_text_props(weight='bold')

        # Salva l'immagine della tabella
        path = os.path.join(base_folder, condition, data_type, subfolder)
        file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
        img_file_path = os.path.join(path, file_name)
        #fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        #plt.close(fig)

        print(f"\nTabella dei dati di \033[1m{key}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")


vorrei provare ad iterare con "zip", sia all_models che su model_standardization_dict ...? (che forse dovrebbero avere la stessa struttura, che renderebbe possibile questa cosa...?)

E, nel momento in cui si aggiungono le metriche del training e test del relativo modello, controllare rispetto a model_standardization_dict (di cui si ha la chiave per accedere all' informazione su se quel modello, per quella combinazioni di fattori che compongono quel dato) se il dato sia stato standardizzato... 

Se questo è VERO, allora nella colonna del dataframe che si riferisce al modello... vorrei che ci mettessi accanto, alla stringa che si riferisce al nome del modello (name_model) un asterisco, SOLO SE, per quel modello, allenato con quella combinazioni di fattori che compongono quel dato, i dati siano stati standardizzati...

chiaro?

#### Implementazione 

#### **CREAZIONE DELLE TABLES CON INTEGRAZIONE DELLE PERFORMANCE TRAINING & TEST DEI MODELLI DENTRO DATAFRAME**

#### **Integrazioni in Tabella delle Performance Training e Test del Modello dentro DataFrame - NEW APPROACH**

#### Spiegazione

Ok in questo modo, model_standardization_dict dovrebbe andare a salvarsi se, i dati per quella combinazione di fattori, rispetto ad uno specifico modello, siano stati standardizzati o meno.

Di conseguenza, dentro questo loop

    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    from pandas.plotting import table

    # Base folder
    base_folder = "/home/stefano/Interrogait/time_domain_best_models_post_WB"

    # Condizioni sperimentali
    experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

    # Tipologie di dati
    data_types = ["1_20", "1_45", "wavelet_delta"]

    # Subfolders per tipologia di soggetto
    subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

    # Dizionario per salvare tutti i modelli
    all_models = {}

    # Caricamento dei modelli
    for condition in experimental_conditions:
        for data_type in data_types:
            for subfolder in subfolders:

                path = os.path.join(base_folder, condition, data_type, subfolder)

                if not os.path.exists(path):
                    print(f"Directory non trovata: {path}")
                    continue

                # Creiamo la chiave per questa combinazione
                key = f"{condition}_{data_type}_{subfolder}"
                all_models[key] = {}

                # Otteniamo la lista di file nella directory
                files = os.listdir(path)

                # Filtriamo e carichiamo i file .pkl
                for file in files:
                    if file.endswith(".pkl"):  # Controlliamo se è un file modello
                        file_path = os.path.join(path, file)
                        try:
                            with open(file_path, "rb") as f:
                                all_models[key][file] = pickle.load(f)
                        except Exception as e:
                            print(f"Errore nel caricamento di {file}: {e}")

    # Creazione delle tabelle di performance
    for key, models_dict in all_models.items():

        # Otteniamo le informazioni dalla chiave
        #condition, data_type, subfolder = key.split("_", 2)
        condition, data_type, subfolder = parse_combination_models_keys(key)

        print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")

        # Creazione della tabella
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

        # Iteriamo sui modelli caricati
        for model_name, model_data in models_dict.items():
            name_model = model_name.split("_")[0]  # Nome modello
            print(f"    Processing model: \033[1m{name_model}\033[0m")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}


                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]


            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame
        #df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        #fig, ax = plt.subplots(figsize=(10, 6))
        #ax.axis('off')
        #tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
        #tabla.auto_set_font_size(True)
        #tabla.set_fontsize(10)
        #tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        #for key, cell in tabla.get_celld().items():
        #    if key[0] == 0:
        #        cell.set_text_props(weight='bold')

        # Salva l'immagine della tabella
        path = os.path.join(base_folder, condition, data_type, subfolder)
        file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
        img_file_path = os.path.join(path, file_name)
        #fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        #plt.close(fig)

        print(f"\nTabella dei dati di \033[1m{key}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")


vorrei provare ad iterare con "zip", sia all_models che su model_standardization_dict ...? (che forse dovrebbero avere la stessa struttura, che renderebbe possibile questa cosa...?)

E, nel momento in cui si aggiungono le metriche del training e test del relativo modello, controllare rispetto a model_standardization_dict (di cui si ha la chiave per accedere all' informazione su se quel modello, per quella combinazioni di fattori che compongono quel dato) se il dato sia stato standardizzato... 

Se questo è VERO, allora nella colonna del dataframe che si riferisce al modello... vorrei che ci mettessi accanto, alla stringa che si riferisce al nome del modello (name_model) un asterisco, SOLO SE, per quel modello, allenato con quella combinazioni di fattori che compongono quel dato, i dati siano stati standardizzati...

chiaro?

#### Implementazione 

In [ ]:
import pickle 
path = '/home/stefano/Interrogait/'

with open(f"{path}models_info_spectrograms_time_frequency_EEG_GradCAM_Checks.pkl", "rb") as f:
    models_info = pickle.load(f)

In [ ]:
'''
In questo codice:

model_info.get('standardization', False) cerca la chiave 'standardization' all'interno di ogni sottodizionario. 
Se non esiste, restituirà False come valore di default.
Se standardization è True, stampa la chiave associata.
'''

# Ciclo attraverso le chiavi di 'models_info'
for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiave



In [ ]:
models_info

In [ ]:
#for key, model_info in all_models.items():
#    print(key)

In [ ]:
'''
Siccome la stringa associata alla category subject è diversa tra i due.. 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt  da un lato (models_info)
th_fam, pt_fam, th_unfam, pt_unfam  dall'altro (all_models)

la corrispondenza non avverrà mai... per cui, si deve fare il mapping corrispondente tra 
le stringhe di uno e dell'altro, in modo che models_info cambi come parte della stringa della sua chiave da queste 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt
a queste
th_fam, pt_fam, th_unfam, pt_unfam 

'''

mapping_subject = {
    "familiar_th": "th_fam",
    "familiar_pt": "pt_fam",
    "unfamiliar_th": "th_unfam",
    "unfamiliar_pt": "pt_unfam"
}

def remap_key_suffix(key: str, mapping: dict) -> str:
    # prova i pattern più lunghi prima per evitare match parziali (es. "unfamiliar_th" vs "familiar_th")
    for old in sorted(mapping.keys(), key=len, reverse=True):
        if key.endswith(old):
            return key[:-len(old)] + mapping[old]   # sostituisci SOLO il suffisso
    return key

updated_models_info = { remap_key_suffix(k, mapping_subject): v
                        for k, v in models_info.items() }

models_info = updated_models_info

In [ ]:
models_info.keys()

In [ ]:
''' Ciclo attraverso le chiavi di 'models_info' AGGIORNATO!'''

for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiavi

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''

# Funzione per parsare la chiave
def parse_combination_models_keys(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    
    Il formato atteso PRIMA è:
    
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    
    Il formato atteso ORA è:
    
     "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "th_fam|th_unfam|pt_fam|pt_unfam"
     
    """
    #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$", 
    
    match = re.match(
        
        #PRIMA
        #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(1_20|1_45|wavelet)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        
        #DOPO
        
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        
        #r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        #r"^(rest_vs_both_feet|rest_vs_both_fists|both_feet_vs_both_fists)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
    return exp_cond, data_type, category_subject

In [ ]:
'''
NEW APPROACH 

Adesso replichiamo l'approccio usato prima, ma stavolta integrado tutte le combinazioni di dati. 
Andiamo a

1) iterare sulla struttura delle directory a partire da base_folder, 
2) caricare i modelli .pkl per ogni combinazione di fattori che compongono i dati
3) creare un DataFrame che raccolga le metriche di tutti i modelli relativi alla stessa combinazione di dati. 

Infine, salviamo questa tabella come immagine all'interno della cartella corrispondente
'''


import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table



def safe_load_any(path):
    # 1) Prova come salvataggio PyTorch (mappa tutto su CPU)
    try:
        return torch.load(path, map_location='cpu')
    except Exception:
        pass
    # 2) Fallback: semplice pickle
    with open(path, 'rb') as f:
        return pickle.load(f)
    
    
# Base folder
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"

base_folder = "/home/stefano/Interrogait/spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks"
                                        

# Condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

#experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]
#experimental_conditions = ["rest_vs_both_feet", "rest_vs_both_fists", "both_feet_vs_both_fists"]

# Tipologie di dati
data_types = ["spectrograms"]

#data_types = ["1_20", "1_45", "wavelet_delta"]


# Subfolders per tipologia di soggetto
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

#subfolders = ["th_fam"]

# Dizionario per salvare tutti i modelli
all_models = {}

# Caricamento dei modelli
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue
            
            # Creiamo la chiave per questa combinazione
            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            # Otteniamo la lista di file nella directory
            files = os.listdir(path)
            
            '''
            Così:
            se il file è stato creato con torch.save(...) (e magari contiene tensori su GPU), verrà rimappato su CPU;
            se è un pickle “normale”, si userà pickle.load.
            '''
            # Filtriamo e carichiamo i file .pkl
            for file in files:
                if file.endswith(".pkl"):  # Controlliamo se è un file modello
                    file_path = os.path.join(path, file)
                    
                    try:
                        all_models[key][file] = safe_load_any(file_path)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")
                        
                        #with open(file_path, "rb") as f:
                            #all_models[key][file] = pickle.load(f)
                    #except Exception as e:
                        #print(f"Errore nel caricamento di {file}: {e}")

# Creazione delle tabelle di performance
for key, models_dict in all_models.items():
    
    # Otteniamo le informazioni dalla chiave
    condition, data_type, subfolder = parse_combination_models_keys(key)
    
    print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")
    
    # Creazione della tabella
    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    # Iteriamo sui modelli caricati
    for model_name, model_data in models_dict.items():
        
        # Estrai il nome del modello dal file (ad esempio, "CNN1D" da "CNN1D_performances_...pkl")
        name_model = model_name.split("_")[0]
        
        if model_name.startswith(("CNN2D_LSTM")):
            
            # es. “CNN3D_LSTM_FC”  →  split → [“CNN3D”, “LSTM”, “FC”] → prendi i primi 2 elementi
            parts = model_name.split("_")
            name_model = "_".join(parts[:2])      # “CNN3D_LSTM” o “SeparableCNN2D_LSTM”
        else:
            name_model = model_name.split("_")[0]  # Prende solo CNN1D, CNN2D
        
        print(f"    Processing model: \033[1m{name_model}\033[0m")
        
        # Costruisci la chiave utilizzata nel dizionario models_info
        
        '''
        Nota: occorrerà che il formato della chiave sia consistente tra i due loop.
        
        Ad esempio, se nel primo loop era f"{key}_{model_name}", qui potresti dover fare:
        model_key = f"{key}_{name_model}"
        
        Oppure, se nel primo loop era f"{model_name}_{key}", qui potresti dover fare:
        model_key = f"{name_model}_{key}"
        
        '''
        model_key = f"{name_model}_{key}"
        
        # Controlla se i dati sono stati standardizzati per questo modello
        standardization_flag = models_info.get(model_key, {}).get("standardization", False)
        
        if standardization_flag:
            suffix = "" 
        else:
            suffix = "" 
        
        try:
            # Recupera i risultati di training e testing
            train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
            test_scores = model_data.get('my_test_results', {}).get('test_performances', {})
            
            # Converti i valori in float
            train_scores = {key: float(value[0]) for key, value in train_scores.items()}
            test_scores = {key: float(value[0]) for key, value in test_scores.items()}
            
            
            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_train = f"{name_model} (Training){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_train}"] = [
                train_scores["train_accuracy"],
                train_scores["train_loss"],
                train_scores["train_precision"],
                train_scores["train_recall"],
                train_scores["train_f1_score"],
                train_scores["train_auc"],
            ]

            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_test = f"{name_model} (Test){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_test}"] = [
                test_scores["test_accuracy"],
                test_scores["test_loss"],
                test_scores["test_precision"],
                test_scores["test_recall"],
                test_scores["test_f1_score"],
                test_scores["test_auc"],
            ]
        
        except Exception as e:
            print(f"    Errore nell'elaborazione di {model_name}: {e}")

    # Creazione del DataFrame
    df_performances = pd.DataFrame(df_data)

    # Crea un'immagine della tabella
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    
    # Aggiunta del titolo
    #title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}, Subject: {subfolder}"
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    # Evidenzia i nomi delle colonne
    for key, cell in tabla.get_celld().items():
        if key[0] == 0:
            cell.set_text_props(weight='bold')

    # Salva l'immagine della tabella
    path = os.path.join(base_folder, condition, data_type, subfolder)
    file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    img_file_path = os.path.join(path, file_name)
    fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
    plt.close(fig)

    print(f"\nTabella dei dati di \033[1m{condition}_{data_type}_{subfolder}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")

#### **Integrazioni in Tabelle AGGREGATE delle Performance Training e Test del Modello dentro DataFrame - NEW APPROACH**

In [ ]:
'''
perfetto ora va. ma io vorrei anche rendere le tabelle ancora più informative.. ossia

vorrei ricreare lo stesso codice ma questa volta anziché avere una tabella specifica SOLO
per un certo tipo di condizione sperimentale, tipo di dato e soggetto...

io vorrei provare quanto meno ad 'allargare' le tabelle, nel senso di mettere nella stessa tabella
la stessa condizione sperimentale e tipo di dato, per tutti e 3 i modelli, 

ma confrontando però la performance dello STESSO MODELLO per gli STESSI TIPI DI CONDIZIONE SPERIMENTALE, TIPO DI DATO e TIPI DI SOGGETTI (ossia RUOLO nel task)
... ossia ad esempio


A) Ossia.. quindi, farei prima i RUOLI di th_fam e th_unfam ...ossia

per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_45

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 




poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_20

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 




poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati delta_wavelet

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 

in modo da avere un confronto diretto visivo per la stessa condizione sperimentale, stesso tipo di feature dei dati EEG usata, 
rispetto allo stesso modello, ma confrontando però la performance tra i due soggetti che hanno fatto lo STESSO RUOLO nei 2 gruppi (controllo e sperimentale).




B) Allo stesso modo.. quindi, farei la STESSA COSA anche per i RUOLI di pt_fam e pt_unfam...ossia

 
per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_45

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 



poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_20

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 



poi per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!),, per i dati delta_wavelet

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test' 
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 
  
magari, nella prima riga metto le performance di training e test dei modelli che son con "_fam" 
e invece sotto le stesse performance dello stesso modello, condizione e tipo di dato, per chi è "_unfam", 


in modo da distinguire in base alla riga quali sono le performance di uno rispetto a quelle dell'altro soggetto, 
che avrà svolto lo stesso ruolo ma nel gruppo o di controllo o sperimentale...



In [ ]:
'''
Yes! idea chiarissima. Senza stravolgere il tuo codice, 
aggiungi un secondo pass che costruisce (e salva) le tabelle aggregate per ruolo per ogni (condizione, data_type). 

Le colonne restano i 3 modelli × (Training/Test), le righe diventano le metriche replicate per i due soggetti del ruolo (fam / unfam). 

Il simbolo * per la standardizzazione lo mettiamo dentro la cella (così può cambiare tra fam e unfam).

Incolla questo blocco dopo aver popolato all_models (puoi tenere anche le tabelle “singole” che già fai):



# ===== TABELLE AGGREGATE PER RUOLO (th_fam vs th_unfam e pt_fam vs pt_unfam) =====

MODEL_ORDER = ["CNN1D", "BiLSTM", "Transformer"]
METRICS = [
    ("Accuracy",  "train_accuracy", "test_accuracy"),
    ("Loss",      "train_loss",     "test_loss"),
    ("Precision", "train_precision","test_precision"),
    ("Recall",    "train_recall",   "test_recall"),
    ("F1-Score",  "train_f1_score", "test_f1_score"),
    ("AUC-ROC",   "train_auc",      "test_auc"),
]

def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None)."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        # match robusto: inizia con "<MODEL>_"
        if fname.startswith(model_prefix + "_"):
            return blob
    return None

def fmt(v, star=False):
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("*" if star else "")

# Gruppi di ruolo
ROLE_GROUPS = {
    "THroles": ["th_fam", "th_unfam"],
    "PTroles": ["pt_fam", "pt_unfam"],
}

for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            # Costruisci righe: una sezione per sub='..._fam' e una per '..._unfam'
            rows = []
            # Colonne: 3 modelli × (Training/Test)
            columns = ["Metriche"]
            for m in MODEL_ORDER:
                columns.append(f"{m} (Training)")
                columns.append(f"{m} (Test)")

            df_data = {c: [] for c in columns}

            for subfolder in subs:
                # intestazione “visiva” delle righe: preferisci th_fam/th_unfam ecc.
                for label, tr_key, te_key in METRICS:
                    df_data["Metriche"].append(f"{subfolder} — {label}")

                    for m in MODEL_ORDER:
                        # recupero blob salvato per quel subfolder/modello
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        # standardization flag (per cella)
                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            # niente file -> celle vuote
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            # training_performances/test_performances hanno valori come liste [val]
                            tr_val = tr.get(tr_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))
                        except Exception:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")

            # DataFrame e salvataggio
            df_performances = pd.DataFrame(df_data)

            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis('off')

            title = f"DL Models performances — {condition} — EEG: {data_type} — {role_label}"
            ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

            tabla = table(ax, df_performances, loc='center',
                          colWidths=[0.25] + [0.12]*(len(df_performances.columns)-1))
            tabla.auto_set_font_size(True)
            tabla.set_fontsize(9)
            tabla.scale(1.2, 1.2)

            for k, cell in tabla.get_celld().items():
                if k[0] == 0:
                    cell.set_text_props(weight='bold')

            out_dir = os.path.join(base_folder, condition, data_type)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
            out_path = os.path.join(out_dir, out_name)
            fig.savefig(out_path, bbox_inches='tight', dpi=300)
            plt.close(fig)

            print(f"Tabella aggregata salvata: {out_path}")

In [ ]:
'''
perfetto — qui sotto trovi il tuo script “chiavi in mano” con il secondo pass che genera anche le tabelle aggregate per ruolo 

(THroles = th_fam/th_unfam, PTroles = pt_fam/pt_unfam) per ogni coppia (condizione, data_type).

Ho aggiunto:

parse_combination_models_keys() con wavelet_delta nel regex.

Caricamento “robusto” di models_info (se non esiste, procede senza *).

Funzioni di supporto find_model_blob() e fmt().

Secondo pass che salva i PNG ..._{condition}_{data_type}_{THroles|PTroles}.png nella cartella di quella coppia.


Se vuoi nascondere completamente il vecchio primo pass e tenere solo i comparativi per ruolo, basta commentare/bloccare la sezione “Pass 1”.




Puoi commentare tutto il “Pass 1” senza problemi: il “Pass 2” non dipende da quello.
Il “Pass 2” usa solo:

all_models (riempito nel blocco di caricamento iniziale, prima del Pass 1),

models_info (per mettere l’asterisco * se standardizzato),

le funzioni helper (find_model_blob, fmt) e le costanti (MODEL_ORDER, ROLE_GROUPS, ecc.).
Quindi, finché lasci il caricamento di all_models e gli helper, funziona da solo.

Sì, le tabelle aggregate del Pass 2 vengono salvate esattamente nel percorso costruito da queste righe:

out_dir = os.path.join(base_folder, condition, data_type)
os.makedirs(out_dir, exist_ok=True)
out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"


quindi avrai file tipo:

/home/stefano/Interrogait/time_domain_1D_best_models_post_WB/th_resp_vs_pt_resp/1_45/models_performances_th_resp_vs_pt_resp_1_45_THroles.png

/home/stefano/Interrogait/time_domain_1D_best_models_post_WB/pt_resp_vs_shared_resp/wavelet_delta/models_performances_pt_resp_vs_shared_resp_wavelet_delta_PTroles.png

Se preferisci tenerle in una sottocartella tipo aggregated/, cambia così:

out_dir = os.path.join(base_folder, condition, data_type, "aggregated")


'''


import os
import re
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# ---------- Helpers ----------
def parse_combination_models_keys(combination_key: str):
    """
    Ritorna (exp_cond, data_type, category_subject) da chiavi tipo:
    th_resp_vs_pt_resp_1_45_th_fam
    """
    match = re.match(
        #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(1_20|1_45|wavelet_delta)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        combination_key
    )
    if match:
        return match.groups()
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

# Carica models_info (flag standardization per cella). Se non c'è, prosegue senza '*'
try:
    with open("/home/stefano/Interrogait/models_info_spectrograms_time_frequency_EEG_GradCAM_Checks.pkl", "rb") as f:
        models_info = pickle.load(f)
except Exception:
    print("⚠️  models_info non trovato/caricabile: le tabelle verranno create senza indicatore * di standardizzazione.")
    models_info = {}

def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None) cercando per filename prefix."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        if fname.startswith(model_prefix + "_"):
            return blob
    return None

def fmt(v, star=False):
    """Formatta un valore numerico a 3 decimali e aggiunge '*' se standardizzato."""
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("*" if star else "")

# ---------- Config ----------
base_folder = "/home/stefano/Interrogait/spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks"

experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]
#data_types = ["1_20", "1_45", "wavelet_delta"]
data_types = ["spectrograms"]
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

MODEL_ORDER = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]
METRICS = [
    ("Accuracy",  "train_accuracy", "test_accuracy"),
    ("Loss",      "train_loss",     "test_loss"),
    ("Precision", "train_precision","test_precision"),
    ("Recall",    "train_recall",   "test_recall"),
    ("F1-Score",  "train_f1_score", "test_f1_score"),
    ("AUC-ROC",   "train_auc",      "test_auc"),
]

ROLE_GROUPS = {
    "Observer_Role": ["th_fam", "th_unfam"],
    "Receiver_Role": ["pt_fam", "pt_unfam"],
}


# --- aggiungi in alto (vicino a ROLE_GROUPS / MODEL_ORDER) ---
DISPLAY_LABELS = {
    "th_fam":   "observer_fam",
    "th_unfam": "observer_unfam",
    "pt_fam":   "receiver_fam",
    "pt_unfam": "receiver_unfam",
}



# ---------- Caricamento modelli ----------
all_models = {}

for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            path = os.path.join(base_folder, condition, data_type, subfolder)
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue

            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            for file in os.listdir(path):
                if file.endswith(".pkl"):
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")

# ---------- Pass 1: tabella per singola combinazione (come avevi) ----------

'''
for key, models_dict in all_models.items():
    condition, data_type, subfolder = parse_combination_models_keys(key)
    print(f"\nProcessing: {condition} - {data_type} - {subfolder}\n")

    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    for filename, model_data in models_dict.items():
        name_model = filename.split("_")[0]  # CNN1D / BiLSTM / Transformer

        # Standardization: usa models_info[colonna] a livello di modello-subfolder
        model_key = f"{name_model}_{key}"
        standardization_flag = bool(models_info.get(model_key, {}).get("standardization", False))
        suffix = "*" if standardization_flag else ""

        try:
            train_scores = model_data.get("my_train_results", {}).get("training_performances", {})
            test_scores  = model_data.get("my_test_results", {}).get("test_performances", {})

            # convert list -> float
            train_scores = {k: float(v[0]) for k, v in train_scores.items()}
            test_scores  = {k: float(v[0]) for k, v in test_scores.items()}

            col_train = f"{name_model} (Training){suffix}"
            col_test  = f"{name_model} (Test){suffix}"

            df_data[col_train] = [
                train_scores.get("train_accuracy", float("nan")),
                train_scores.get("train_loss", float("nan")),
                train_scores.get("train_precision", float("nan")),
                train_scores.get("train_recall", float("nan")),
                train_scores.get("train_f1_score", float("nan")),
                train_scores.get("train_auc", float("nan")),
            ]
            df_data[col_test] = [
                test_scores.get("test_accuracy", float("nan")),
                test_scores.get("test_loss", float("nan")),
                test_scores.get("test_precision", float("nan")),
                test_scores.get("test_recall", float("nan")),
                test_scores.get("test_f1_score", float("nan")),
                test_scores.get("test_auc", float("nan")),
            ]
        except Exception as e:
            print(f"    Errore nell'elaborazione di {filename}: {e}")

    df_performances = pd.DataFrame(df_data)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis("off")
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc="center", colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    for kcell, cell in tabla.get_celld().items():
        if kcell[0] == 0:
            cell.set_text_props(weight="bold")

    out_dir = os.path.join(base_folder, condition, data_type, subfolder)
    os.makedirs(out_dir, exist_ok=True)
    out_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    out_path = os.path.join(out_dir, out_name)
    fig.savefig(out_path, bbox_inches="tight", dpi=300)
    plt.close(fig)

    print(f"Tabella singola salvata: {out_path}")
'''


# ---------- Pass 2: tabelle aggregate per ruolo ----------
for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            columns = ["Metriche"]
            for m in MODEL_ORDER:
                columns.append(f"{m} (Training)")
                columns.append(f"{m} (Test)")

            df_data = {c: [] for c in columns}

            for idx_sub, subfolder in enumerate(subs):
                
                '''CONVERSIONE LABELS DEL RUOLO --> da th_fam a observer_fam etc'''
                # solo per la visualizzazione converto th_fam->observer_fam, ecc.
                subfolder_disp = DISPLAY_LABELS.get(subfolder, subfolder)
                
                # per ciascun subfolder (fam / unfam) aggiungo le 6 metriche
                for label, tr_key, te_key in METRICS:
                    
                    #df_data["Metriche"].append(f"{subfolder} — {label}")
                    
                    # usa il suffisso "display" SOLO per la label della riga
                    df_data["Metriche"].append(f"{subfolder_disp} — {label}")
                    

                    for m in MODEL_ORDER:
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            tr_val = tr.get(tr_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))
                        except Exception:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")

                # riga separatrice tra fam e unfam (opzionale ma utile visivamente)
                if idx_sub == 0:
                    df_data["Metriche"].append("")  # riga vuota
                    for m in MODEL_ORDER:
                        df_data[f"{m} (Training)"].append("")
                        df_data[f"{m} (Test)"].append("")

            df_performances = pd.DataFrame(df_data)
            
            
            
            SHOW_ONLY = False  # <- True per visualizzare, False per salvare
            
            
            
            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis("off")

            title = f"DL Models performances — {condition} — EEG feature: {data_type} — {role_label}"
            ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

            tabla = table(
                ax,
                df_performances,
                loc="center",
                colWidths=[0.25] + [0.12] * (len(df_performances.columns) - 1),
            )
            tabla.auto_set_font_size(True)
            tabla.set_fontsize(9)
            tabla.scale(1.2, 1.2)

            for kcell, cell in tabla.get_celld().items():
                if kcell[0] == 0:
                    cell.set_text_props(weight="bold")
            
            
            '''Con "aggregated", io aggiungo una sotto-cartella ancora alla path di salvataggio delle tabelle'''
            
            if SHOW_ONLY:
                plt.show()
                print(f"Tabella aggregata di: models_performances_{condition}_{data_type}_{role_label}.png")
            else:
                
                out_dir = os.path.join(base_folder, condition, data_type, "aggregated")
                os.makedirs(out_dir, exist_ok=True)
                out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                out_path = os.path.join(out_dir, out_name)
                fig.savefig(out_path, bbox_inches="tight", dpi=300)
                plt.close(fig)

                print(f"Tabella aggregata salvata: {out_path}")

#### Implementazione : Versione dal 24 novembre 2025 - Versione Aggregata

In [ ]:
'''METRICHE PRIMA DI TUTTI I MODELLI SUL TRAIN ... POI SUL VALIDATION ...  E POI SUL TEST SET'''

import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ---------- Helpers ----------
def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None) cercando per filename prefix."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        if fname.startswith(model_prefix + "_"):
            return blob
    return None


def fmt(v, star=False):
    """Formatta un valore numerico a 3 decimali e aggiunge '*' se standardizzato (se vuoi)."""
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("" if star else "")


def format_col_label(label: str) -> str:
    """
    Converte 'CNN1D (Training)' -> 'CNN1D\n(Training)' per header a due righe.
    Lascia 'Metrics' invariato.
    """
    if label == "Metrics":
        return label
    if "(" in label and label.endswith(")"):
        model, phase = label.split("(", 1)
        model = model.strip()
        phase = "(" + phase.strip()
        return f"{model}\n{phase}"
    return label


def pretty_condition_name(cond: str) -> str:
    """
    th_resp_vs_pt_resp        -> 'observer resp vs receiver resp'
    th_resp_vs_shared_resp    -> 'observer resp vs shared resp'
    pt_resp_vs_shared_resp    -> 'receiver resp vs shared resp'
    """
    token_map = {
        "th_resp": "observer resp",
        "pt_resp": "receiver resp",
        "shared_resp": "shared resp",
    }
    parts = cond.split("_vs_")
    pretty_parts = [token_map.get(p, p.replace("_", " ")) for p in parts]
    return " vs ".join(pretty_parts)


# ---------- Config ----------
base_folder = "/home/stefano/Interrogait/spectrograms_time_frequency_best_models_post_WB_GradCAM_Checks"

experimental_conditions = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]

#data_types = ["1_20", "1_45", "wavelet"]
data_types = ["spectrograms"]
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

#MODEL_ORDER = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]

MODEL_ORDER = ["CNN2D_LSTM", "BiLSTM", "Transformer"]
PHASES = ["Training", "Validation", "Test"]   # <--- aggiunto per chiarezza

# (label, train_key, val_key, test_key)
METRICS = [
    ("Accuracy",  "train_accuracy",  "val_accuracy",  "test_accuracy"),
    ("Loss",      "train_loss",      "val_loss",      "test_loss"),
    ("Precision", "train_precision", "val_precision", "test_precision"),
    ("Recall",    "train_recall",    "val_recall",    "test_recall"),
    ("F1-Score",  "train_f1_score",  "val_f1_score",  "test_f1_score"),
    ("AUC-ROC",   "train_auc",       "val_auc",       "test_auc"),
]

ROLE_GROUPS = {
    "Observer_Role": ["th_fam", "th_unfam"],
    "Receiver_Role": ["pt_fam", "pt_unfam"],
}

# Etichette che appariranno nella colonna "Metrics"
DISPLAY_LABELS = {
    "th_fam":   "observers familiar group",
    "th_unfam": "observers unfamiliar group",
    "pt_fam":   "receivers familiar group",
    "pt_unfam": "receivers unfamiliar group",
}

# Etichette per il tipo di dato nel titolo
DATA_LABELS = {
    "spectrograms": "Time x Frequency"
}

SHOW_ONLY = False  # cambia in False per salvare i PNG
#SHOW_ONLY = True  # cambia in False per salvare i PNG

# ---------- Carica models_info se esiste ----------
try:
    with open("/home/stefano/Interrogait/models_info_spectrograms_time_frequency_EEG_GradCAM_Checks.pkl", "rb") as f:
        models_info = pickle.load(f)
except Exception:
    print("⚠️  models_info non trovato/caricabile: nessun indicatore di standardizzazione (*).")
    models_info = {}

# ---------- Caricamento modelli ----------
all_models = {}

for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            path = os.path.join(base_folder, condition, data_type, subfolder)
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue

            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            for file in os.listdir(path):
                if file.endswith(".pkl"):
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")


# =========================================================
#  PASS 2: tabelle aggregate per ruolo (Observer / Receiver)
# =========================================================
for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            print(f"\nProcessing aggregate table: {condition} - {data_type} - {role_label}")

            # ---------- COSTRUZIONE DF ----------
            # ORA: prima tutte le colonne Train (tutti i modelli),
            #      poi tutte le colonne Validation, poi Test.
            columns = ["Metrics"]
            for phase in PHASES:                      # <- loop su Training / Validation / Test
                for m in MODEL_ORDER:                 #    e dentro sui modelli
                    columns.append(f"{m} ({phase})")

            df_data = {c: [] for c in columns}

            for idx_sub, subfolder in enumerate(subs):

                subfolder_disp = DISPLAY_LABELS.get(subfolder, subfolder)

                for label, tr_key, val_key, te_key in METRICS:

                    df_data["Metrics"].append(f"{subfolder_disp} — {label}")

                    for m in MODEL_ORDER:
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            for phase in PHASES:
                                df_data[f"{m} ({phase})"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            va = blob.get("my_train_results", {}).get("validation_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            tr_val = tr.get(tr_key,  [None])[0]
                            va_val = va.get(val_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Validation)"].append(fmt(va_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))

                        except Exception:
                            for phase in PHASES:
                                df_data[f"{m} ({phase})"].append("-")

                # riga vuota di separazione (fam / unfam)
                if idx_sub == 0:
                    df_data["Metrics"].append("")
                    for phase in PHASES:
                        for m in MODEL_ORDER:
                            df_data[f"{m} ({phase})"].append("")

            df_performances = pd.DataFrame(df_data)

            # =========================
            #  PREPARAZIONE PER PLOT
            # =========================
            df_display = df_performances.copy()

            col_weights = []
            for col in df_display.columns:
                header_len = len(str(col))
                body_max = df_display[col].astype(str).map(len).max()
                col_weights.append(max(header_len, body_max))

            col_weights = np.array(col_weights, dtype=float)
            col_weights[0] *= 1.4  # "Metrics" più larga
            col_widths = (col_weights / col_weights.sum()) * 0.98

            # ---------- FIGURA & AX ----------
            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis("off")

            cond_pretty = pretty_condition_name(condition)
            data_pretty = DATA_LABELS.get(data_type, data_type)
            role_pretty = role_label.replace("_", " ")

            line1 = "Deep Learning Models performances for Brain Decoding of Sense of Responsibility"
            line2 = f"Experimental Conditions: {cond_pretty} — EEG Spectrogram: {data_pretty} — Subject Cohort: {role_pretty}"

            ax.set_title(
                f"{line1}\n{line2}",
                fontsize=11,
                pad=6,
            )

            col_labels = [format_col_label(c) for c in df_display.columns]

            tabla = ax.table(
                cellText=df_display.values,
                colLabels=col_labels,
                loc="upper center",
                cellLoc="center",
                colWidths=col_widths.tolist(),
            )

            tabla.auto_set_font_size(False)
            base_fontsize = 6
            header_fontsize = 6

            tabla.set_fontsize(base_fontsize)
            tabla.scale(1.1, 1.1)

            for (row, col), cell in tabla.get_celld().items():
                if row == 0:
                    cell.set_text_props(weight="bold", fontsize=header_fontsize)

            if SHOW_ONLY:
                plt.show()
                plt.close(fig)
                #print(f"Tabella aggregata mostrata: {condition} - {data_type} - {role_label}")
                #out_dir = os.path.join(base_folder, condition, data_type)
                #os.makedirs(out_dir, exist_ok=True)
                #out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                #out_path = os.path.join(out_dir, out_name)
                #print(f'out_path: {out_path}') 
            else:
                out_dir = os.path.join(base_folder, condition, data_type, "aggregated")
                os.makedirs(out_dir, exist_ok=True)
                out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                out_path = os.path.join(out_dir, out_name)
                print(f'out_path: \033[1m{out_path}\033[0m') 

                fig.savefig(out_path, bbox_inches="tight", dpi=300)
                plt.close(fig)

                print(f"Tabella aggregata salvata: {out_path}")


## Impostazione **Weight & Biases DL Training** con **Rappresentazione Electrodes x Frequencies Signal (2D)** 

## Optimization Weight and Biases - EEG Spectrograms - Electrodes x Frequencies 
### (Solo Iper-parametri Dinamici)

#### **Weight & Biases Procedure FINAL SEQUENCE OF STEPS - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

In [ ]:
#Library Importing 
    
    
#import mne 

import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 


import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

In [ ]:
print(torch.cuda.device_count())  # Numero di GPU disponibili
print(torch.cuda.current_device())  # ID della GPU in uso
print(torch.cuda.get_device_name(0))  # Nome della GPU

In [ ]:
import numpy as np
print(np.__version__)

In [ ]:
import pickle
import numpy

# Apri il file in modalità lettura binaria ('rb')

#path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/'

#path = '/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/'

path = '/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/'

with open(f"{path}new_all_th_concat_spectrograms_coupled_exp.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
# Itera sulle chiavi del dizionario principale
for condition, values in data.items():
    if isinstance(values, dict) and "data" in values and "labels" in values:
        X_shape = values["data"].shape
        y_length = len(values["labels"])
        print(f"🔹 Condizione: {condition}")
        print(f"   ➡ Shape dati: {X_shape}")
        print(f"   ➡ Lunghezza labels: {y_length}\n")


#### **Utils Functions - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
QUI DENTRO HO CONFIGURATO 
LE FUNZIONI DI CONTROLLO DELLE STRINGHE 
PER IL SALVATAGGIO DELLE PERFORMANCE DEL MODELLO
NELLE RELATIVE SUBFOLDERS

(I.E., get_subfolder_from_key, get_subfolder_from_key_hyper)

IN MODO CHE SI LEGHINO ALLA CHIAVE 'STANDARDIZATION' DELL'OGGETTO SWEEP_CONFIG

'''


import pickle
import numpy as np


def load_data_hyper(data_type, category, wavelet_level=None, condition= "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, seleziona la finestra temporale (50°-300° punto, ossia 0-1000 mms).

    Parameters:
    - data_type: str, "1_20", "1_45" o "wavelet"
    - category: str, "familiar" o "unfamiliar"
    - wavelet_level: str, "theta", "delta", ecc. (solo per dati wavelet)
    - condition: str, condizione sperimentale da selezionare

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto)
    - y: Etichette corrispondenti
    """
    # Definizione dei percorsi base
    base_paths = {
        "1_20": {
            "familiar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_20/hyper_dataset_EEG_preprocessed_1_20_familiar_{condition}.pkl",
            "unfamiliar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_20/hyper_dataset_EEG_preprocessed_1_20_unfamiliar_{condition}.pkl"
        },
        "1_45": {
            "familiar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_45/hyper_dataset_EEG_preprocessed_1_45_familiar_{condition}.pkl",
            "unfamiliar": f"/home/stefano/Interrogait/all_datas/Hyper_Datasets_EEG_1_45/hyper_dataset_EEG_preprocessed_1_45_unfamiliar_{condition}.pkl"
        },
        "wavelet": {
            "familiar": "/home/stefano/Interrogait/all_datas/Hyper_Datasets_Wavelet_Reconstructions/hyper_dataset_wavelet_familiar.pkl",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Hyper_Datasets_Wavelet_Reconstructions/hyper_dataset_wavelet_unfamiliar.pkl"
        }
    }

    # Seleziona il path corretto
    filepath = base_paths[data_type][category]
    
    # Caricamento del file
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    # Selezione della finestra temporale e delle etichette
    X = data[wavelet_level][condition]["data"][:, :, 125:200] if data_type == "wavelet" else data["data"][:, :, 50:300]
    y = data[wavelet_level][condition]["labels"] if data_type == "wavelet" else data["labels"]
        
    return X, y


'''
# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key_hyper(key, sweep_config):
    
    
    #Mi richiamo la chiave 'standardization' che ho impostato nella configurazione dell'oggetto weight and biases
    #(i.e., sweep_config['standardization']) e eseguo una procedura condizionale 
    
    #ossia che, se risulta o True o False, lui cambi le condizioni di gestione 
    #della costruzione delle path di salvataggio 
    
    
    
    
    # Controlla se i dati sono standardizzati
    if sweep_config['standardization']:
        
        #PER I DATI SCALED
            
        if '_familiar' in key:
            return 'HYPER_FAM'
        elif '_unfamiliar' in key:
            return 'HYPER_UNFAM'
        else:
            return None
    else:
        
        #PER I DATI UNSCALED
        if '_familiar' in key:
            return 'HYPER_FAM_UNSCALED'
        elif '_unfamiliar' in key:
            return 'HYPER_UNFAM_UNSCALED'
        else:
            return None
    
    
# Funzione per salvare i risultati
def save_performance_results_hyper(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition= "th_resp_vs_pt_resp"):
    
    #Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key_hyper(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "wavelet" in key:
        data_type_str = "wavelet_delta"
    elif "1_20" in key:
        data_type_str = "1_20"
    elif "1_45" in key:
        data_type_str = "1_45"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
'''






def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (50°-300° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "familiar" o "unfamiliar"
    - subject_type: str, "th" (terapisti) o "pt" (pazienti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms_channels_frequencies/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


'''ATTENZIONE MODIFICATA FUNZIONE DI STANDARDIZZAZIONE'''
# Funzione per standardizzare i dati
# Con questa modifica eviti che std==0 produca NaN e i tuoi loss torneranno numeri sensati.
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    #aggiungo eps per evitare divisione per zero
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test

# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels = 61, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=61 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=61, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    #SeparableCNN2D_LSTM_FC
    
    #CNN3D_LSTM_FC
    if model_type == "CNN3D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    ##SeparableCNN2D_LSTM_FC
    elif model_type == "SeparableCNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        
    
    elif model_type == "CNN2D":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN2D', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor



    


'''
QUESTE DUE FUNZIONI PRESE DA 

EEG Motor Movement - Imagery Dataset (EEGMMIDB) - TASK 1 - 2D GRID - ALL FREQS + 3D CONV CONV SEP.ipynb

DA SEZIONE

## Impostazione **Weight & Biases DL Training** con **Rappresentazione Tempo-Frequenza dei miei dati EEG** 
a seconda del Dataset del Task scelto
'''


# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
     # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
        
'''

QUESTE FUNZIONI TROVATE COME ERANO NEL NOTEBOOK
# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, sweep_config):
    
    
    #Mi richiamo la chiave 'standardization' che ho impostato nella configurazione dell'oggetto weight and biases
    #(i.e., sweep_config['standardization']) e eseguo una procedura condizionale 
    
    #ossia che, se risulta o True o False, lui cambi le condizioni di gestione 
    #della costruzione delle path di salvataggio 
    
    
    # Controlla se i dati sono standardizzati
    if sweep_config['standardization']:
    
        #PER I DATI SCALED
        if '_familiar_th' in key:
            return 'TH_FAM'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM'
        elif '_familiar_pt' in key:
            return 'PT_FAM'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM'
        else:
            return None
    else: 
        #PER I DATI UNSCALED

        if '_familiar_th' in key:
            return 'TH_FAM_UNSCALED'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM_UNSCALED'
        elif '_familiar_pt' in key:
            return 'PT_FAM_UNSCALED'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM_UNSCALED'
        else:
            return None
    
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "wavelet" in key:
        data_type_str = "wavelet_delta"
    elif "1_20" in key:
        data_type_str = "1_20"
    elif "1_45" in key:
        data_type_str = "1_45"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
        
'''


#### **Neural Network Models - EEG Spectrograms - Electrodes x Frequencies - previous versions**

##### **Neural Network Models - EEG Spectrograms - Electrodes x Frequencies - LSTM & Transformer (DA NON USARE)**

In [ ]:
'''
Gli LSTM si aspettano un input in forma (batch, lunghezza_sequenza, dimensione_feature). 
Dovrai quindi decidere qual è la dimensione sequenziale.

Opzione comune: usare il tempo come sequenza
Step 1: Trasponi i dati in modo da avere il tempo come dimensione sequenziale.

Dalla forma (batch, canali, frequenze, tempo) puoi fare:


x = x.permute(0, 3, 1, 2)  # Diventa (batch, tempo, canali, frequenze)

Step 2: Unisci le dimensioni dei canali e dei bin di frequenza in un’unica dimensione di feature:


batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # Ora: (batch, tempo, canali*frequenze)

Nel tuo caso, per 3 canali e 38 bin di frequenza: input_size = 3 * 38 = 114 e lunghezza sequenza = 6.

Nota: Se invece preferisci usare i bin di frequenza come sequenza, potresti fare:

x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
x = x.reshape(batch, frequenze, canali * tempo)  # Sequence length = 38, feature size = 3*6 = 18
La scelta dipende dal tipo di informazione temporale o spettrale che vuoi evidenziare.

'''

class ReadMEndYou(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.5, bidirectional=False):
        """
        input_size: dimensione delle feature per time-step (dovrà essere canali * frequenze)
        hidden_sizes: lista con le dimensioni degli hidden state, es. [24, 48, 62]
        output_size: numero di classi
        
        """
    
        super(ReadMEndYou, self).__init__()
        
        self.bidirectional = bidirectional # Impostazione della bidirezionalità    
        
        # Adattiamo hidden_size in base alla bidirezionalità
        self.hidden_sizes = [
            hidden_sizes[0] * 2 if bidirectional else hidden_sizes[0],
            hidden_sizes[1] * 2 if bidirectional else hidden_sizes[1],
            hidden_sizes[2] * 2 if bidirectional else hidden_sizes[2]
        ]
        
        self.lstm1 = nn.LSTM(input_size=input_size, 
                             hidden_size=self.hidden_sizes[0], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0, 
                             bidirectional=bidirectional)
        self.lstm2 = nn.LSTM(input_size=self.hidden_sizes[0] * 2 if bidirectional else self.hidden_sizes[0],
                             hidden_size=self.hidden_sizes[1], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        self.lstm3 = nn.LSTM(input_size=self.hidden_sizes[1] * 2 if bidirectional else self.hidden_sizes[1],
                             hidden_size=self.hidden_sizes[2],
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_sizes[2] * 2 if bidirectional else self.hidden_sizes[2], output_size)
    
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Trasponi per avere il tempo come dimensione sequenziale:
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        # Ora input_size deve essere channels * freqs (es. 3 * 26 = 78)
        
        # LSTM 1
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        
        # LSTM 2
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        
        # LSTM 3
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        
        # Estraiamo l'output dell'ultimo time-step
        out = out[:, -1, :]
        
        # Dropout prima del layer fully connected    
        out = self.dropout(out)
        
        # Passaggio attraverso il layer finale per la previsione
        out = self.fc(out)
        return out
        


'''
Il modulo Transformer in PyTorch lavora tipicamente su input di forma (seq_length, batch, embedding_dim).

Nel codice attuale, si parte da una forma simile a (batch, canali, seq_length), ma dovrai adattarla alla nuova struttura.

Possibili approcci:

1) Approccio A: usare il tempo come sequenza

Se consideri il tempo (6 time windows) come la sequenza, puoi procedere come segue:

A) Unisci canali e frequenze in un’unica dimensione di feature:

# Dati originali: (batch, canali, frequenze, tempo)
x = x.permute(0, 3, 1, 2)  # (batch, tempo, canali, frequenze)
batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # (batch, tempo, 3*38 = 114)

B) Modifica il layer di embedding:

Nel codice attuale, l'embedding è definito come:

self.embedding = nn.Linear(seq_length, d_model)
Dovrai cambiarlo in modo che mappi le dimensioni delle feature (in questo caso 114) a uno spazio latente:

self.embedding = nn.Linear(canali * frequenze, d_model)

C) Permuta per il Transformer:

Dopo l'embedding, passa l'input alla forma (seq_length, batch, d_model):

x = x.permute(1, 0, 2)  # Ora: (tempo, batch, d_model)


2) Approccio B: usare i bin di frequenza come sequenza
In alternativa, se reputi più rilevante la risoluzione spettrale, puoi considerare i 38 bin come sequenza e combinare canali e tempo:


x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
batch, frequenze, canali, tempo = x.shape
x = x.reshape(batch, frequenze, canali * tempo)  # (batch, frequenze, 3*6 = 18)

E poi procedere con un embedding layer che mappa da 18 a d_model e permutare in (frequenze, batch, d_model).

Scelta dell'approccio:
Se l'aspetto temporale è più critico, probabilmente è meglio usare l’Approccio A (sequenza di lunghezza 6).
Se invece vuoi dare maggior rilievo alla struttura spettrale, l’Approccio B potrebbe essere più indicato.

Ricorda che la scelta dipende dalla natura del tuo problema e dalla rilevanza delle informazioni temporali rispetto a quelle spettrali.
'''

import torch
import torch.nn as nn

#Scelta: In questa implementazione abbiamo deciso di usare il tempo come sequenza.
#In alternativa, potresti scegliere i bin di frequenza come sequenza, ma ciò richiederebbe una diversa riorganizzazione delle dimensioni 
#(ad esempio, un permute diverso).

class ReadMYMind(nn.Module):

    def __init__(self, d_model, num_heads, num_layers, num_classes, channels=61, freqs=26):
        
        super(ReadMYMind, self).__init__()

        # Il layer di embedding mapperà la feature dimension (channels * freqs) a d_model
        self.embedding = nn.Linear(channels * freqs, d_model)
        
        # Transformer per l'attenzione spaziale (qui si applica direttamente alla sequenza temporale)
        self.spatial_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Transformer per l'attenzione temporale (si potrebbe considerare un'iterazione successiva)
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Cross-attention per combinare le rappresentazioni
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads)
        
        # Fusione e classificazione finale
        self.fc_fusion = nn.Linear(d_model, d_model)
        self.fc_classify = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        # x: (batch, canali, frequenze, tempo)
        
        # Utilizziamo il tempo come sequenza
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        
        # Embedding: (batch, tempo, d_model)
        x = self.embedding(x)
        
        # Transformer richiede input di forma (seq_length, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # -> (tempo, batch, d_model)
        
        # Applichiamo il Transformer per l'attenzione spaziale
        x_spatial = self.spatial_transformer(x)
        
        # Applichiamo il Transformer per l'attenzione temporale
        x_temporal = self.temporal_transformer(x_spatial)
        
        # Cross-attention: (tempo, batch, d_model)
        x_cross, _ = self.cross_attention(x_spatial, x_temporal, x_temporal)
        
        # Fusione: per esempio, facciamo una media sul tempo (dimensione 0)
        x_fused = self.fc_fusion((x_spatial + x_temporal).mean(dim=0))  # -> (batch, d_model)
        
        # Classificazione finale
        output = self.fc_classify(x_fused)  # -> (batch, num_classes)
        
        return output
    

In [ ]:
'''
Ecco un codice che fornisce dati di input fittizi a ciascuna rete neurale, 
stampa le dimensioni a ogni passaggio e verifica che gli output abbiano le forme attese.

Ho mantenuto le forme coerenti con i tuoi parametri:


Batch size: 8
Numero di canali EEG: 3
Numero di frequenze: 38
Numero di timepoints (campioni temporali): 100
Numero di classi: 2

'''


import torch
import torch.nn as nn
import torch.nn.functional as F

# Parametri
batch_size = 44
input_channels = 61  # Canali EEG
num_freqs = 45       # Numero di frequenze
num_classes = 2       # Numero di classi

# Creazione di dati fittizi per il test
x = torch.randn(batch_size, num_freqs, input_channels)  # (batch, frequenze, channels)
print(f"Input iniziale: {x.shape}\n")

# ---- CNN2D ----
cnn_model = CNN2D(input_channels=input_channels, num_classes=num_classes)
cnn_output = cnn_model(x)
print(f"Output CNN2D: {cnn_output.shape}\n")  # Atteso: (batch_size, num_classes)

'''
# ---- ReadMEndYou (LSTM) ----
hidden_sizes = [24, 48, 62]
lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
lstm_output = lstm_model(x)
print(f"Output ReadMEndYou (LSTM): {lstm_output.shape}\n")  # Atteso: (batch_size, num_classes)

# ---- ReadMYMind (Transformer) ----
d_model = 64   # Dimensione embedding
num_heads = 8   # Numero di teste di attenzione
num_layers = 3  # Numero di strati Transformer

transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)
transformer_output = transformer_model(x)
print(f"Output ReadMYMind (Transformer): {transformer_output.shape}\n")  # Atteso: (batch_size, num_classes)
'''


##### **Neural Network Models - EEG Spectrograms - Electrodes x Frequencies - CNN2D (DA USARE!)**

In [ ]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI 2D FREQUENCY-CHANNELS (LUGLIO 2025!)



Ora però, ragionandoci, potrei inserire dei valori da cui pescare, 

durante l'ottimizzazione degli iper-parametri della mia rete, che si riferiscono 

1) a valori di alcuni parametri generale dell'apprendimento delle reti
2) a valori dei parametri architetturali di ciascuna delle mie singole reti neurali testate



                                                                ***CNN2D NEW*** 

1) All'interno di ogni layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

a) il numero di output channels (ossia 16 impostato di default qui sotto, ma che potrebbe variare da 16 a 32 con step di 4 
come grandezza della feature map sostanzialmente

b) la grandezza del kernel size (tra 2 e 8 con step di 2)
c) la grandezza dello stride (metti solo valori tra 1 e 2) 


2) Per il layer di batch normalisation del relativo layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#batchnorm1d

deve avere il valore del numero di features di quel layer di batch normalisation
(che deve corrispondere come valore a quello dell'output channels del layer convolutivo che lo precede sostanzialmente) 


3) Al layer di pooling del relativo strato della della CNN1D, far variare la scelta tra

a) max pooling ed average pooling 

b) Il valore del kernel_size del layer di max od average pooling (a seconda di quello che viene scelto tra i due), 
che può variare tra 1 e 2 

4) Al solo primo layer fully connected della CNN1D, far variare la scelta del suo valore 
(che nella mia rete sarebbe "self.fc1 = nn.LazyLinear(8)") in questo set di valori, ossia tra i valori 8,10,12,14,16

5) Il valore del dropout layer (con valori tra  0.0 e 0.5) 


6) Il valore della possibile funzione di attivazione tra 3 (relu, selu ed elu)

 a) per gli strati convolutivi (3) +
 b) per il primo fully connected layer (FC1) (prendendone una a caso tra quelle 3 possibili



TABELLA FINALE RIASSUNTIVA - CNN1D 


| Iper-parametro                     | Descrizione                                             | Valori possibili                 |
| ---------------------------------- | ------------------------------------------------------- | -------------------------------- |
| `conv_out_channels`                | Numero di feature-map di base                           | `[16, 20, 24, 28, 32]`           |
| `conv_k1`, `conv_k2`, `conv_k3`    | Kernel size rispettivamente per i 3 blocchi convolutivi | `[2, 4, 6, 8]`                   |
| `conv_s1`, `conv_s2`, `conv_s3`    | Stride rispettivamente per i 3 blocchi convolutivi      | `[1, 2]`                         |
| `pool_type`                        | Tipo di pooling                                         | `["max","avg"]`                  |
| `pool_p1`, `pool_p2`, `pool_p3`    | Kernel size rispettivamente per i 3 blocchi di pooling  | `[1, 2]`                         |
| `fc1_units`                        | Numero di unità nel primo fully-connected               | `[8, 10, 12, 14, 16]`            |
| `cnn_act1`, `cnn_act2`, `cnn_act3` | Funzione di attivazione per ciascun blocco (layer1,2,3) | `["relu","selu","elu"]`          |
| **+ comune**                       | `dropout`                                               | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |



'''

'''




                                                                ***OLD CNN2D***


Una CNN2D si aspetta input in forma (batch, frequenze, canali). 
Nel tuo caso, puoi interpretare l’"altezza" come i bin di frequenza (45)
e la "larghezza" come i canali (61)

Quindi, la tua CNN2D lavorerebbe direttamente con:
Shape: (batch, frequenze, canali)



class CNN2D(nn.Module):
    
    def __init__(self, input_channels, num_classes):
        
        super(CNN2D, self).__init__()
        
        # Ipotizziamo kernel 3x3 con padding per mantenere le dimensioni (puoi adattare a tuo piacimento)
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(32, 48, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))
        
        # Utilizzo LazyLinear per evitare di calcolare manualmente la dimensione piatta finale
        self.fc1 = nn.LazyLinear(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)

        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = F.elu(x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = F.elu(x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = F.elu(x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.elu(x)
       

        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x
        
'''




'''CNN2D CON LE FUNZIONI DI ATTIVAZIONI DINAMICHE (LAYER CONVOLUTIVI + FC1) E DINAMISMO DEI KERNEL SIZE DI CONV e POOL LAYER e STRIDE



Scusami non ho capito la differenza tra la mia e la tua, che la mia tratta le convoluzione per ogni canale separatamente
e quindi vede le features per ogni canale mentre la tua vede le relazioni in frequenza tra i canali?

mi fai capire meglio?


ChatGPT ha detto:

Ottima domanda, e sì: sei sulla strada giusta! Vediamo con precisione la differenza concettuale tra le due versioni, perché hanno implicazioni molto diverse sul tipo di informazione che la rete convoluzionale impara.

🧠 Contesto: input (batch, freq, channels) → (B, 45, 61)
✅ Versione A: .unsqueeze(1) → input = (B, 1, 45, 61)

x = x.unsqueeze(1)
# Output: (batch_size, 1, 45, 61)

✅ Come la interpreta la CNN:
Tratta tutta la matrice frequenze × canali come un'immagine 2D.

Applica filtri convoluzionali 2D che coprono sia le frequenze che i canali contemporaneamente.

La convoluzione può imparare pattern locali e globali che coinvolgono più canali EEG e bande di frequenza assieme.

✅ Esempio di pattern che può catturare:
"La banda theta (4–8 Hz) è più attiva nei canali posteriori rispetto a quelli frontali"

"Un pattern a L tra canali temporali e frequenze gamma"

➕ Vantaggi:
Cattura interazioni tra frequenze e canali.

Molto potente per pattern spatial-temporal-frequenziali complessi.

➖ Svantaggi:
Più complesso da interpretare.

Può "mescolare troppo" se le relazioni tra canali non sono forti.





✅ Versione B: .permute(0, 2, 1).unsqueeze(3) → input = (B, 61, 45, 1)


x = x.permute(0, 2, 1).unsqueeze(3)
# Output: (batch_size, 61, 45, 1)

✅ Come la interpreta la CNN:
Ogni canale EEG (dei 61) è trattato come un’immagine verticale di 45 pixel (cioè le frequenze).

**Ogni filtro convoluzionale lavora indipendentemente su ciascun canale EEG.

È come applicare 61 CNN monodimensionali parallele sulle frequenze di ogni canale.

✅ Esempio di pattern che può catturare:
“Nel canale Fz, la banda alpha ha un picco”

“Nel canale Pz, la potenza decresce linearmente con la frequenza”

➕ Vantaggi:
Cattura pattern locali per canale (molto utile se ogni canale è considerato indipendente).

Meno soggetto a overfitting se i dati sono limitati.

➖ Svantaggi:
Non cattura le relazioni tra canali.

Meno potente se la distribuzione spaziale tra canali è importante (es. P300, N170, ecc.).

🔍 Riassunto Visuale

Variante	        Input Shape	      Tratta i canali come	       Cattura relazioni tra canali?	    Cattura pattern cross-frequenza?
.unsqueeze(1)	    (B, 1, 45, 61)	    Griglia 2D	                               ✅ Sì	                             ✅ Sì
.permute...	        (B, 61, 45, 1)	   Canali separati	                           ❌ No	               ✅ Solo all'interno del canale

🧠 Quale scegliere?
Usa .unsqueeze(1) (griglia 2D) se:
Vuoi sfruttare relazioni tra canali EEG (spazialità).

Stai analizzando pattern che coinvolgono distribuzioni topografiche (Pz vs Cz, emisferi, ecc.).

Usa .permute(...).unsqueeze(3) se:
Vuoi analizzare ogni canale indipendentemente.

Hai pochi dati e vuoi una CNN più semplice.

Tratti i canali EEG come “sorgenti indipendenti”.

✅ Conclusione
Hai centrato la differenza:

La tua versione **focalizza sui pattern di frequenza all'interno di ciascun canale.

L’altra versione esplora pattern condivisi tra canali e frequenze, come se fosse un'immagine EEG frequenze × elettrodi.

Se vuoi, posso anche mostrarti un esempio visivo (matplotlib) per rendere ancora più chiara la differenza.



Quindi, nella mia versione originale è come se dessi 61 immagini singolo canale 'verticali', 
nel senso che per analogia, sarebbe come dare la rappresentazione in formato vettore verticale di ogni canale rispetto ai pixel in frequenza 
(ossia i bins di frequenza)..

Nella tua, invece, forniamo un input solo, ossia una immagine sola,
con la rappresentazione dei bin di frequenza di ogni canale e quindi ci consente di vedere le relazioni tra i canali stessi?


****************** ****************** ****************** ****************** ****************** ****************** ******************

✅ Esattamente! Hai centrato il punto con una metafora visiva perfetta. Ti spiego meglio facendo proprio leva su questa analogia dell’immagine.

🎯 Tua versione originale: (B, 61, 45, 1) → 61 “immagini verticali”
✅ Metafora:
È come dare 61 immagini strette e alte (una per ogni canale EEG), in cui:

Altezza = 45: le frequenze (tipo pixel verticali)

Larghezza = 1: inesistente (una sola colonna)

Ogni immagine è indipendente

👉 In pratica:
La rete analizza separatamente ogni canale EEG.

Impara “per ogni canale, com’è distribuita la potenza nelle varie frequenze?”

Non sa nulla di cosa succede in altri canali.

🎯 La mia versione (classica CNN2D): (B, 1, 45, 61) → una sola immagine F×C
✅ Metafora:
È come dare una singola immagine 2D:

Asse verticale = 45 bin di frequenza

Asse orizzontale = 61 canali EEG

👉 In pratica:
La rete guarda l’intera griglia frequenza × canale come un’immagine.

Può imparare pattern tipo:

“La potenza in alpha aumenta da O1 verso Cz”

“Un cluster di attivazione nei canali frontali nella gamma”

Le convoluzioni possono spostarsi sia tra frequenze che tra canali, quindi può apprendere interazioni spaziali.

👁️‍🗨️ Visualizzazione (semplificata)
Tua:
scss
Copia
Modifica
Canale Fz →  [ delta ]  
              [ theta ]  
              [ alpha ]  
              [ beta  ]  
              [ gamma ]

Canale Cz →  [ delta ]  
              [ theta ]  
              ...
(61 ripetizioni)
Mia:
matlab
Copia
Modifica
        C1   C2   C3   ...  C61
delta   ■    □    ■    ...  □
theta   □    ■    □    ...  ■
alpha   ■    ■    ■    ...  □
beta    □    □    ■    ...  ■
gamma   ■    □    □    ...  □
Una griglia piena, dove ogni pixel rappresenta la potenza in una frequenza per un canale.

✅ Conclusione
Hai descritto benissimo:

Versione	Interpretazione	Vantaggio principale
Tua (permute + unsqueeze)	61 immagini verticali (1 per canale)	Focus dettagliato su pattern per canale
Classica (unsqueeze(1))	Una sola immagine 2D F×C	Cattura pattern e relazioni tra canali

Se vuoi fare un passo in più, puoi anche combinare le due visioni con multi-branch CNN (una per canale, una per immagine globale). 
Fammi sapere se vuoi un esempio anche per questo!

****************** ****************** ****************** ****************** ****************** ****************** ******************

'''


#https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class CNN2D(nn.Module):
    def __init__(
        self,
        input_channels: int,
        num_classes: int,

        # da sweep: numero di feature map di base
        conv_out_channels: int,

        # da sweep: kernel size H×W per i 3 blocchi
        conv_k1_h: int, conv_k1_w: int,
        conv_k2_h: int, conv_k2_w: int,
        conv_k3_h: int, conv_k3_w: int,

        # da sweep: stride H×W per i 3 blocchi
        conv_s1_h: int, conv_s1_w: int,
        conv_s2_h: int, conv_s2_w: int,
        conv_s3_h: int, conv_s3_w: int,

        # da sweep: pool kernel H×W per i 3 blocchi
        pool_p1_h: int, pool_p1_w: int,
        pool_p2_h: int, pool_p2_w: int,
        pool_p3_h: int, pool_p3_w: int,

        # da sweep: tipo di pooling
        pool_type: str,  # "max" o "avg"

        # fully‑connected
        fc1_units: int,
        dropout: float,

        # attivazioni per i 3 blocchi
        cnn_act1: str,
        cnn_act2: str,
        cnn_act3: str,
    ):
        super().__init__()
        mapping = {'relu': F.relu, 'selu': F.selu, 'elu': F.elu}
        self.act_fns = [
            mapping[cnn_act1],
            mapping[cnn_act2],
            mapping[cnn_act3],
        ]
        
        # calcolo padding “quasi‐same” per ciascun blocco
        p1_h = (conv_k1_h - 1) // 2
        p1_w = (conv_k1_w - 1) // 2
        p2_h = (conv_k2_h - 1) // 2
        p2_w = (conv_k2_w - 1) // 2
        p3_h = (conv_k3_h - 1) // 2
        p3_w = (conv_k3_w - 1) // 2
        
        # Primo blocco
        self.conv1 = nn.Conv2d(
            input_channels, conv_out_channels,
            kernel_size = (conv_k1_h, conv_k1_w),
            stride = (conv_s1_h, conv_s1_w),
            #padding='same'
            padding = (p1_h, p1_w)
        )
        self.bn1   = nn.BatchNorm2d(conv_out_channels)
        self.pool1 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p1_h, pool_p1_w))

        # Secondo blocco (×2 feature map)
        self.conv2 = nn.Conv2d(
            conv_out_channels, conv_out_channels*2,
            kernel_size=(conv_k2_h, conv_k2_w),
            stride=(conv_s2_h, conv_s2_w),
            #padding='same'
            padding = (p2_h, p2_w) 
        )
        self.bn2   = nn.BatchNorm2d(conv_out_channels*2)
        self.pool2 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p2_h, pool_p2_w))

        # Terzo blocco (×3 feature map)
        self.conv3 = nn.Conv2d(
            conv_out_channels*2, conv_out_channels*3,
            kernel_size=(conv_k3_h, conv_k3_w),
            stride=(conv_s3_h, conv_s3_w),
            #padding='same'
            padding = (p3_h, p3_w)
        )
        self.bn3   = nn.BatchNorm2d(conv_out_channels*3)
        self.pool3 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p3_h, pool_p3_w))

        # FC finale
        self.fc1     = nn.LazyLinear(fc1_units)
        self.dropout = nn.Dropout(dropout)
        self.fc2     = nn.LazyLinear(num_classes)
    
    
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        #🔁 Prima:
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        #x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        #x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)
        
        #✅ Ora:
        #Siccome i dati arrivano come (B, 45, 61) — cioè frequenze × canali, non serve permutare. Ti basta:
        
        # Aggiungiamo una dimensione per il canale "immagine"
        x = x.unsqueeze(1)  # → (B, 1, 45, 61)
            
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = self.act_fns[0](x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = self.act_fns[1](x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = self.act_fns[2](x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.relu(x)
       
        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x


In [ ]:
'''
Ecco un codice che fornisce dati di input fittizi a ciascuna rete neurale, 
stampa le dimensioni a ogni passaggio e verifica che gli output abbiano le forme attese.
'''

import torch.nn.functional as F

# Testing shapes
batch, frequency, channels, num_classes = 44, 45, 61, 2

x = torch.randn(batch, channels, frequency)

print("Input:", x.shape)

cnn = CNN2D(input_channels = 1, num_classes = num_classes,
            conv_out_channels=16,
            conv_k1_h=3,conv_k1_w=5,
            conv_k2_h=3,conv_k2_w=5,
            conv_k3_h=3,conv_k3_w=5,
            conv_s1_h=1,conv_s1_w=2,
            conv_s2_h=1,conv_s2_w=2,
            conv_s3_h=1,conv_s3_w=2,
            pool_p1_h=1,pool_p1_w=2,
            pool_p2_h=1,pool_p2_w=2,
            pool_p3_h=1,pool_p3_w=1,
            pool_type='max',fc1_units=10,dropout=0.1,
            cnn_act1='relu',cnn_act2='relu',cnn_act3='elu')

out_cnn = cnn(x)
print("CNN2D output:", out_cnn.shape)


##### **Neural Network Models - EEG Spectrograms - Electrodes x Frequencies - CNN3D e CNN2D_Sep (RIADATTATA PER SHAPE batch, frequenze, canali)**

In [ ]:
'''ATTENZIONE, BISOGNA

1) RICREARE UNA GRIGLIA EXCEL, PER I DATI EEG DI INTERROGAIT


a) Carica questo file e ricrea i dati 

path = '/home/stefano/Interrogait/all_datas/'

with open(f"{path}EEG_channels_names.pkl", "rb") as f:
    EEG_channels_names = pickle.load(f)
    

b) Tieni presente che tu hai sia


Familiar_Spectrograms_channels_frequencies
Unfamiliar_Spectrograms_channels_frequencies 

e per ciascuna hai coppie di condizioni sperimentali....non so se valga la pena investirci...pensaci bene


Forse conviene ri-adattare le reti neurali per fare convoluzioni separabili,
ma sull'input frequenza x canali base ottenuto da Interrogait...




2) VERIFICARE IL NUMERO DI ELETTRODI CORRISPONDA A QUELLO USATO PER TASK MOTORIO!
3) CREARE LA GRIGLIA CON LA DISPOSIZIONE ELETTRODICA DEI DATI DI INTERROGAIT
4) USARE LE STESSE ARCHITETTURE DI CONVOLUZIONI SEPARABILI PER IL TASK DI INTERROGAIT!

'''

In [ ]:
'''

Perfetto — ti ho preparato l’excel con le coordinate x,y per solo i canali che hai elencato (ordine identico alla tua lista).
Ho usato il tuo file “EEG_grid (1).xlsx” come formato di riferimento e ho estratto/ricostruito le posizioni in modo coerente 
con la topologia 10–20 che avevi già mappato sulla griglia. 

Gli unici due canali che mancavano nella griglia erano AF3 e AF4: li ho inseriti come punti a metà strada fra (Fp1, F3) e (Fp2, F4)
rispettivamente, cioè sul “ring” AF tra FP e F (scelta standard nella 10–10).




In [ ]:
'''

VERSIONE CONVOLUZIONE 3D PURA e CONVOLUZIONI SEPARABILI 19 LUGLIO 2025


Due versioni dell’architettura:

CNN3D_LSTM_FC: usa nn.Conv3d per eseguire una vera convoluzione 3D sui cinque depth (bande di frequenza), 
mantenendo il resto del flusso identico.

SeparableCNN2D_LSTM_FC: applica in sequenza una convoluzione depthwise (gruppi = canali) e una pointwise (1×1) 
per fondere i cinque canali in modo efficiente.

Entrambe le classi si integrano con il tuo blocco LSTM e il classificatore come nella versione originale.



******** PROBLEMA SUL GRADCAM NELLE ANALISI PRE 19 LUGLIO ********


Per ottenere un Grad‑CAM “3D” su ciascuna delle 5 bande (cioè un volume 9×9×5) 
invece di schiacciare tutto in una mappa 9×9, bisogna:

Non appiattire la dimensione di profondità (“depth” = bande) con cam.mean(dim=1).

Calcolare i pesi medi dei gradienti solo su altezza e larghezza, non su depth, in modo da preservare D=5.

Upsample (solo) le due dimensioni spaziali H×W, lasciando inalterata la profondità D.

(Opzionale) 

Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!





******** PROBLEMA SUL CONV3D NELLE ANALISI PRE 19 LUGLIO ********


Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!


Se vuoi davvero avere D=5 in uscita, devi cambiare conv1 in:


# ❌ kernel_size=(5,3,3), padding=(0,1,1) → D_out = 1

self.conv1 = nn.Conv3d(1, 32, kernel_size=(3,3,3), padding=(1,1,1))
così la profondità si conserva da 5→5.



1) Perché in conv1 useremo padding=(1,1,1) e negli altri layer padding=(0,1,1)
Obiettivo: mantenere la profondità (numero di bande, D = 5) costante lungo tutta la rete.

In conv1, abbiamo scelto kernel_size=(3,3,3) perché vogliamo che il filtro “scorra” su tutti e tre gli assi (D,H,W).

Con kernel_depth=3, per avere

𝐷out = (𝐷in + 2⋅𝑃 depth − 𝐾 depth)/ 𝑆 + 1 = 5

Da qui (1,1,1) per (depth, height, width).

Negli altri layer 3D (conv2a, conv2b, conv3) il kernel depth = 1 (kernel_size=(1,3,3)), 
quindi la profondità non cambia se mettiamo padding_depth=0 con padding (0,1,1) nel layer conv2 e conv3

In altre parole, su quell’asse non serve alcun padding:

se P dept = 0 allora diventa infatti

𝐷out = (𝐷in + 2⋅0 − 𝐾 depth)/ 𝑆 + 1 = 5

2⋅0


Non è che la tua rete “CNN3D_LSTM_FC” sia sbagliata in senso assoluto, 
ma — proprio a causa di quel primo Conv3d con kernel_size=(5,3,3) e padding=(0,1,1) — 

stai automaticamente comprimendo tutte e 5 le bande nella singola fetta di profondità:


self.conv1 = nn.Conv3d(
    in_channels=1, out_channels=32,
    kernel_size=(5, 3, 3),  # → D_out = (5 − 5 + 2·0)/1 + 1 = 1
    padding=(0, 1, 1)
)
Quindi il tuo tensore (B, 1, 5, 9, 9) diventa (B, 32, 1, 9, 9): la dimensione depth (5) si riduce a 1 subito.

Se invece vuoi davvero preservare le 5 “fette” come vera terza dimensione spaziale, hai due possibili correzioni:

Usare un kernel 3×3×3 (o 1×3×3) in conv1, in modo da non “abbracciare” tutta la profondità d’ingresso:


- self.conv1 = nn.Conv3d(1, 32, kernel_size=(5, 3, 3), padding=(0, 1, 1))
+ # preserva D: depth out = depth in = 5
+ self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))

Oppure, se vuoi mantenere le bande completamente indipendenti in questo layer,


self.conv1 = nn.Conv3d(1, 32, kernel_size=(1, 3, 3), padding=(0, 1, 1))
che trasforma (B,1,5,9,9) → (B,32,5,9,9).

Lasciare com’è, sapendo però che la rete “fonderà” le 5 bande in un’unica mappa di profondità: 
non è un bug, è una scelta architetturale.



La SeparableCNN2D_LSTM_FC
Quella architettura non comprime mai le bande all’interno di un’unica fetta, perché:

Le bande diventano canali di un Conv2d depth‑wise:


x = x.permute(0, 3, 1, 2)  # (B,5,9,9)
self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
Ogni “fetta” (banda) resta separata fino al pointwise e agli strati successivi.

Quindi se il tuo obiettivo è avere un’uscita per banda (e poi poter plottare un Grad‑CAM 2D per ciascuna),
la SeparableCNN2D è già configurata correttamente.

Se invece vuoi un Grad‑CAM “volumetrico” 3D (5×9×9) direttamente dal modello 3D puro,
la vera modifica necessaria è solo sul primo Conv3d, come mostrato sopra.

Fammi sapere quale dei due setup stai usando e ti aiuto a integrare il Grad‑CAM 3D di conseguenza!

'''



'''
Ecco l’implementazione completa di CNN3D_LSTM_FC (“approccio sequenza di profondità”) in cui:

mantieni il tuo primo blocco 3D con kernel (3,3,3) e padding (1,1,1), quindi D rimane 5 fino alla fine;

riduci le spatial singleton dims (H=1,W=1) e trasformi la depth D=5 in una sequenza di lunghezza 5;

imposti l’input_size=128 nell’LSTM (feature per time‑step = 128);


Con questa versione:

la sequenza per l’LSTM ha lunghezza D=5;

ogni passo ha 128 feature, esattamente input_size=128;

non servono trucchi di reshape su scala globale.


'''


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3D_LSTM_FC(nn.Module):
    """
    Version with pure 3D convolutions treating the 5 frequency bands
    as a sequence (depth) for the LSTM.
    Input: Tensor of shape (B, 9, 9, 5) --> reshaped to (B, 1, 5, 9, 9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 (3D) ---
        self.conv1   = nn.Conv3d(1,  32, kernel_size=(3,3,3), padding=(1,1,1))
        self.bn1     = nn.BatchNorm3d(32)
        self.pool3d  = nn.MaxPool3d((1,2,2))  # non tocca D

        # --- Block 2 (3D Residual) ---
        self.res_conv3d = nn.Conv3d(32, 64, kernel_size=1, bias=False)
        self.res_bn3d   = nn.BatchNorm3d(64)
        self.conv2a     = nn.Conv3d(32, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2a       = nn.BatchNorm3d(64)
        self.conv2b     = nn.Conv3d(64, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2b       = nn.BatchNorm3d(64)

        # --- Block 3 (3D) ---
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn3   = nn.BatchNorm3d(128)

        # LSTM o FC finale
        if self.use_lstm:
            # input_size = feature_dim per time‑step = 128
            self.lstm       = nn.LSTM(input_size=128,
                                      hidden_size=self.hidden_size,
                                      num_layers=1,
                                      batch_first=True)
            self.classifier = nn.LazyLinear(num_classes)
        else:
            self.classifier = nn.LazyLinear(num_classes)

    def forward(self, x):
        # x: (B, 9, 9, 5)
        if x.ndim == 4:
            # -> (B,1,D=5,H=9,W=9)
            x = x.permute(0, 3, 1, 2).unsqueeze(1)

        # --- Block 1 ---
        x = F.relu(self.bn1(self.conv1(x)))  # (B,32,5,9,9)
        x = self.pool3d(x)                   # (B,32,5,4,4)

        # --- Block 2 (Residual) ---
        res = self.res_bn3d(self.res_conv3d(x))  # (B,64,5,4,4)
        x   = F.relu(self.conv2a(x))             # (B,64,5,4,4)
        x   = self.bn2b(self.conv2b(x))          # (B,64,5,4,4)
        x   = F.relu(x + res)                    # (B,64,5,4,4)
        x   = self.pool3d(x)                     # (B,64,5,2,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))      # (B,128,5,2,2)
        x = self.pool3d(x)                       # (B,128,5,1,1)

        # Stampa delle dimensioni prima di passare al classifier
        #print(f"Dimensioni prima del classifier: {x.shape}")

        if self.use_lstm:
            # x: (B,128,5,1,1)
            # -> squeeze spatial dims → (B,128,5)
            x = x.squeeze(-1).squeeze(-1)
            # -> permute per batch_first → (B, seq_len=5, feat=128)
            x = x.permute(0, 2, 1)
            x = self.dropout(x)
            out, _ = self.lstm(x)               # out: (B,5,hidden_size)
            last    = out[:, -1, :]             # prendo l’ultimo time-step
            logits  = self.classifier(last)     # (B, num_classes)
        else:
            # x: (B,128,5,1,1) → flatten → (B,128)
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

    

class SeparableCNN2D_LSTM_FC(nn.Module):
    """
    Version with depthwise + pointwise separable convolutions
    across the 5 channels.
    Input: Tensor of shape (B, 9, 9, 5) -> (B,5,9,9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 separabile ---
        self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
        self.pw_conv1 = nn.Conv2d(5, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        # --- Block 2 (residuo) ---
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn = nn.BatchNorm2d(64)
        self.bn2a = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.lstm = nn.LSTM(input_size=128 * 5, hidden_size=self.hidden_size, num_layers=1, batch_first=True)

        if self.use_lstm:
            self.lstm = nn.LSTM(
                input_size=128 * 1,
                hidden_size=self.hidden_size,
                num_layers=1,
                batch_first=True
            )
            self.classifier = nn.Linear(self.hidden_size, num_classes)
        else:
            self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)  # -> (B,5,9,9)

        x = F.relu(self.dw_conv1(x))
        x = F.relu(self.bn1(self.pw_conv1(x)))
        x = self.pool(x)

        res = self.res_bn(self.res_conv(x))
        x = F.relu(self.conv2a(self.bn2a(x)))
        x = self.bn2b(self.conv2b(x))
        x = F.relu(x + res)
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)  # (B,128,1,1)

        if self.use_lstm:
            x = x.permute(0, 2, 1, 3).reshape(x.size(0), 1, -1)  # (B,1,128)
            out, _ = self.lstm(self.dropout(x))
            last = out[:, -1, :]
            logits = self.classifier(last)
        else:
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

In [ ]:
# Quick test
if __name__ == "__main__":
    cnn = CNN3D_LSTM_FC()
    sep_conv = SeparableCNN2D_LSTM_FC()
    
    # Test both input orders
    x2 = torch.randn(8, 9, 9, 5)
    print(cnn(x2).shape)   # -> (8,2)
    print(sep_conv(x2).shape)  # -> (8,2)

#### **Early Stopping - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
DEFINIZIONE EARLY STOPPING
'''

import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle
import numpy as np


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
        
        """
        :param patience: Numero di epoche da attendere prima di interrompere il training se non c'è miglioramento
        
        Esempio: il training si interromperà se non si osserva un miglioramento per (N = 5) epoche consecutive.
        
        :param min_delta: Variazione minima richiesta per considerare un miglioramento
        
        definisce il miglioramento minimo richiesto per essere considerato significativo. 
        Se il miglioramento è inferiore a min_delta, non viene considerato un vero miglioramento.
        
        Il parametro min_delta in una configurazione di early stopping indica 
        la minima variazione del valore di una metrica 
        (ad esempio, la perdita o l'accuratezza) 
        che deve verificarsi tra un'epoca e la successiva 
        per continuare l'allenamento. 
        
        In genere, il valore di min_delta dipende dal tipo di modello e dai dati specifici, 
        ma di solito si trova in un intervallo tra 0.001 e 0.01.
    
            - Se stai cercando di evitare che l'allenamento si fermi troppo presto,
            puoi impostare un valore più basso per min_delta (come 0.001), 
            - Se vuoi essere più conservativo e permettere fluttuazioni nei valori della metrica,
            un valore più alto (come 0.01) potrebbe essere appropriato.

        Un buon punto di partenza potrebbe essere 0.001, e poi fare dei test per capire quale valore funziona meglio
        nel tuo caso specifico!
        
        :param mode: 'min' per monitorare la loss (minimizzazione), 'max' per l'accuracy (massimizzazione)
        
        'max' → ottimizza metriche da massimizzare (es. accuracy, F1-score, AUC).
        'min' → ottimizza metriche da minimizzare (es. loss).
        
        """
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True


#### **Weight & Biases Login & REMOTE CHECKS - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

WEIGHT AND BIASES LOGIN

Il messaggio che stai ricevendo indica 
che sei già connesso al tuo account Weights & Biases (wandb).

Se vuoi forzare il login, puoi usare il comando suggerito:

wandb login --relogin

Questo comando ti permetterà di reinserire le credenziali e riconnetterti al tuo account.
Se non hai bisogno di disconnetterti o di cambiare l'account,
puoi semplicemente continuare a usare wandb senza ulteriori passaggi. 
Hai bisogno di ulteriore assistenza con wandb o con il tuo progetto?
'''


import wandb
wandb.login()
print("✅ Weights & Biases login effettuato con successo!")

In [ ]:
import wandb
print(wandb.__version__)

##### **Weight & Biases Login ERRORE CANCELLAZIONE SWEEPS - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
COME CANCELLARE GLI SWEEP DA REMOTO!

                                                                                IO
Quindi un modo potrebbe essere :

1) tracciarmi il codice univoco delle sweep che avevo associate ad un progetto e salvare in una lista tipo sfruttando la base di questo codice

import wandb

# Sostituisci con il nome del tuo progetto e dell'utente
project_name = 'th_resp_vs_pt_resp_spectrograms_channels_freqs'
user_name = 'stefano-bargione-universit-di-roma-tor-vergata'  # Può essere vuoto se è il tuo account

api = wandb.Api()
runs = api.runs(f"{user_name}/{project_name}")

# Stampa tutte le run e il loro numero
for run in runs:
    print(run.id, run.name)

print(f"Totale runs: {len(list(runs))}")



2) Poi creare un loop iterando sugli sweep di quel progetto e poi fare

wandb delete <sweep_id>

3) In questo modo sarei sicuro da remoto che gli sweep (e quindi anche le runs associate) 
verrano eliminate e di conseguenza potrei esser sicuro che, se ri-avvio il kernel, avrò sweep id nuovi, 
ma solo quelli (e non anche quelli vecchi)



                                                                                CHATGPT

Passaggi:
Salvare gli sweep ID associati al progetto:
Utilizzando il codice che hai condiviso, puoi raccogliere gli ID delle run e sweep 
(anche se, in W&B, gli ID delle run sono associati agli sweep, quindi si tratta della stessa cosa). 

Li puoi salvare in una lista, così da avere il riferimento completo per il progetto specifico.

Eliminare gli sweep:
Dopo aver raccolto gli ID, puoi iterare su di essi ed eliminarli usando il comando wandb delete <sweep_id>.
Questo rimuoverà tutti gli sweep associati al progetto.

Assicurarti che al riavvio del kernel vengano creati solo nuovi sweep:
Una volta eliminati tutti gli sweep, quando ri-eseguirai il codice per la creazione di nuovi sweep, verranno generati con nuovi ID,
senza sovrapporsi a quelli precedenti.


In [ ]:
import wandb


#Sweeps dei Progetti da ELIMINARE
#1)th_resp_vs_pt_resp_spectrograms_channels_freqs
#2 th_resp_vs_shared_resp_spectrograms_channels_freqs
#3) pt_resp_vs_shared_resp_spectrograms_channels_freqs


# Nome del Progetto 
project_name = 'th_resp_vs_pt_resp_spectrograms_channels_freqs' 

# Nome Utente
user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Recupera tutte le run del progetto
runs = api.runs(f"{user_name}/{project_name}")

# Salva gli sweep_id delle run in una lista (per tracciare gli sweep ID)
to_delete_sweep_ids = []

for run in runs:
    print(f"Sweep ID: {run.sweep.id} - Run ID: {run.id} - Run Name: {run.name}")
    to_delete_sweep_ids.append(run.sweep.id)  # Aggiungi l'ID dello sweep alla lista

# Numero totale di sweep trovati
print(f"Totale sweep: {len(to_delete_sweep_ids)}")


# Elimina ogni sweep trovato
for sweep_id in to_delete_sweep_ids:
    print(f"Eliminando sweep ID: {sweep_id}")
    wandb.delete(sweep_id)  # Elimina lo sweep

print("Tutti gli sweep sono stati eliminati.")

In [ ]:
'''

----

                                                                                IO

import wandb


Sweeps dei Progetti da ELIMINARE
#1)th_resp_vs_pt_resp_spectrograms_channels_freqs
#2 th_resp_vs_shared_resp_spectrograms_channels_freqs
#3) pt_resp_vs_shared_resp_spectrograms_channels_freqs


# Nome del Progetto 
project_name = 'th_resp_vs_pt_resp_spectrograms_channels_freqs' 

# Nome Utente
user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Recupera tutte le run del progetto
runs = api.runs(f"{user_name}/{project_name}")

# Salva gli sweep_id delle run in una lista (per tracciare gli sweep ID)
to_delete_sweep_ids = []

for run in runs:
    print(f"Sweep ID: {run.sweep.id} - Run ID: {run.id} - Run Name: {run.name}")
    to_delete_sweep_ids.append(run.sweep.id)  # Aggiungi l'ID dello sweep alla lista

# Numero totale di sweep trovati
print(f"Totale sweep: {len(to_delete_sweep_ids)}")


# Elimina ogni sweep trovato
for sweep_id in to_delete_sweep_ids:
    print(f"Eliminando sweep ID: {sweep_id}")
    wandb.delete(sweep_id)  # Elimina lo sweep

print("Tutti gli sweep sono stati eliminati.")

                                                                                IO


mi dice 

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[8], line 4
      2 for sweep_id in to_delete_sweep_ids:
      3     print(f"Eliminando sweep ID: {sweep_id}")
----> 4     wandb.delete(sweep_id)  # Elimina lo sweep
      6 print("Tutti gli sweep sono stati eliminati.")

AttributeError: module 'wandb' has no attribute 'delete'
ChatGPT ha detto:


                                                                                CHATGPT




Il messaggio di errore che hai ricevuto è dovuto al fatto che la libreria WandB non ha un metodo chiamato delete() per eliminare direttamente gli sweep. 
In realtà, non esiste un metodo diretto per eliminare un sweep tramite l'API Python di WandB. L'API di WandB supporta l'eliminazione delle run, 
ma non degli sweep direttamente.

Quindi, per risolvere il problema, puoi eliminare le run individuali (che sono associate agli sweep), e questo rimuoverà indirettamente anche gli sweep,
dato che ogni run è parte di uno sweep. Ecco come puoi procedere:

Codice aggiornato per eliminare le run:

        import wandb

        # Nome del progetto e utente
        project_name = 'th_resp_vs_pt_resp_spectrograms_channels_freqs'
        user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

        # Connessione all'API di W&B
        api = wandb.Api()

        # Recupera tutte le run del progetto
        runs = api.runs(f"{user_name}/{project_name}")

        # Salva gli ID delle run da eliminare
        run_ids = []

        for run in runs:
            print(f"Eliminando run ID: {run.id} - Run Name: {run.name}")
            run_ids.append(run.id)  # Aggiungi l'ID della run alla lista

        # Elimina le run
        for run_id in run_ids:
            run = api.run(f"{user_name}/{project_name}/{run_id}")
            run.delete()  # Elimina la run

        print("Tutte le run sono state eliminate.")
        
        
Dettagli del codice:
run.delete(): Questo comando elimina la run specificata. Poiché una run è associata a uno sweep, eliminando la run si rimuove anche lo sweep.

run.id: L'ID della run viene raccolto per poter essere eliminato successivamente.

api.run(f"{user_name}/{project_name}/{run_id}"): Carica la run specificata dall'ID e la elimina.

Come funziona:
Eliminazione delle run: Quando esegui run.delete(), WandB elimina la run e rimuove tutte le informazioni associate, inclusi gli sweep. 
Questo è il comportamento indiretto che ti consente di eliminare gli sweep, dato che le run sono legate a questi.

Considerazioni:
Verifica le eliminazioni: Dopo aver eseguito lo script, puoi fare un altro controllo per verificare che tutte le run siano state effettivamente rimosse.

Nuove run/sweep: Dopo aver eliminato le run esistenti, quando ri-eseguirai il codice, verranno creati nuovi sweep con nuovi ID, senza conflitti con quelli vecchi.

Questo dovrebbe risolvere il problema e consentire l'eliminazione delle run/sweep in modo corretto!



                                                                                IO

quindi questo mi assicura che ANCHE GLI SWEEPS (ossia i codici univoci ad essi collegati durante  la loro creazione) verranno eliminati? 


                                                                                CHATGPT
Sì, eliminando le run tramite il metodo run.delete(), gli sweep associati verranno eliminati indirettamente. 

Questo succede perché:

Ogni sweep è composto da una serie di run (dove ogni run rappresenta un'esecuzione del modello in un determinato set di condizioni).
Quando elimini una run, questa non è più associata a uno sweep specifico, e se non ci sono altre run associate a quello stesso sweep,
lo sweep verrà rimosso automaticamente.

Perché succede:
Quando crei uno sweep, viene generato un ID unico per quello sweep. Ogni volta che una run viene eseguita come parte di uno sweep, 
viene collegata a quel sweep tramite l'ID di quest'ultimo. 
Se elimini tutte le run di uno sweep, quell'ID di sweep non avrà più run associate e verrà rimosso dal sistema di WandB.

Cosa succede quindi:
Eliminando le run, gli sweep ad esse collegati saranno eliminati, dato che uno sweep non ha senso senza run associate.

Se esegui di nuovo il codice per creare nuovi sweep e run, verranno creati nuovi ID di sweep senza alcun conflitto con i vecchi.

In breve:
Eliminando le run come nel codice che ti ho fornito, eliminerai anche gli sweep associati, e i nuovi sweep che creerai avranno nuovi ID univoci, 
garantendo che non ci siano sovrapposizioni con quelli eliminati.


'''

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies**

###### **BOZZE**

###### **IMPLEMENTAZIONE**

In [ ]:
###TUTTE ASSIEME


import wandb

# Nomi dei progetti
project_names = [
    'th_resp_vs_pt_resp_spectrograms_channels_freqs',
    'th_resp_vs_shared_resp_spectrograms_channels_freqs',
    'pt_resp_vs_shared_resp_spectrograms_channels_freqs'
]

user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Itera su tutti i progetti
for project_name in project_names:
    print(f"\nSelezione delle run per il progetto: \033[1m{project_name}\033[0m")
    
    # Recupera tutte le run del progetto
    runs = api.runs(f"{user_name}/{project_name}")
    
    # Salva gli ID delle run da eliminare
    run_ids_to_delete = []
    
    for run in runs:
        #print(f"Estrazione run ID: {run.id} - Run Name: {run.name}")
        run_ids_to_delete.append(run.id)  # Aggiungi l'ID della run alla lista
    
    print(f"\nTotale runs da eliminare: \033[1m{len(run_ids_to_delete)}\033[0m")
    
    # Elimina le run
    for run_id in run_ids_to_delete:
        run = api.run(f"{user_name}/{project_name}/{run_id}")
        run.delete()  # Elimina la run
        #print(f"Eliminata la run con ID: {run_id}")
    print(f"Eliminazione runs completata")
        

#### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

Sì, è perfettamente normale: in W&B un progetto è semplicemente un contenitore di run e di sweep, 
e rimane visibile (anche se vuoto) fintanto che non lo archivi o lo cancelli esplicitamente. 

Cancellare tutte le run e tutti i sweep in un progetto non elimina il progetto stesso — lascia solo un progetto “vuoto” con zero run/sweep.


Come rimuovere (o archiviare) anche i progetti


1) Dal web UI

Vai nella pagina del progetto che vuoi rimuovere.

Clicca sui tre puntini (⋯) accanto al nome del progetto.
Scegli “Archive project” (per nasconderlo) o, se disponibile, “Delete project” per eliminarlo definitivamente.


2) Da CLI
Se hai una versione recente del wandb CLI, puoi usare (da terminale):


wandb project delete <ENTITY>/<PROJECT_NAME>

Esempio:

wandb project delete stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp_1_20_familiar_th




Programmaticamente
Attualmente W&B non espone un’API Python ufficiale per cancellare progetti; l’opzione più rapida resta il CLI o la UI.

In sintesi
Cancellare run/sweep → lascia il progetto “vuoto” (0 run, 0 sweep).

Progetto visibile → finché non lo archivi o elimini esplicitamente, lo vedrai nella lista.

Per toglierlo di mezzo → archivia o cancella il progetto tramite i tre puntini in UI, oppure con wandb project delete da CLI.



IO:
ok ma quindi se ricreo il progetto con lo stesso nome semplicemente ripopola quel progetto ma con sweeps e runs nuovi giusto?


ChatGPT:

Esatto. In W&B il progetto è identificato dal solo nome (più l’entity) e non ha uno “stato” permanente di esistenza: se tu:

Elimini o archivi totalmente il progetto (tramite UI o wandb project delete),

E poi lanci di nuovo uno wandb.init(..., project="quel_nome"),

W&B creerà un nuovo progetto con lo stesso nome, vuoto all’inizio, in cui andranno a confluire i tuoi nuovi sweep e run.

Quindi:

Se archivi il vecchio progetto, è solo nascosto ma rimane là;

Se lo cancelli definitivamente, sparisce e puoi ricrearlo identico per nome, e in quel momento sarà “pulito” (0 sweep, 0 run)
pronto a riempirsi con i tuoi esperimenti successivi.


'''


import sys
import subprocess
from itertools import product
import wandb

# 1) Parti delle stringhe da combinare
prefixes = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]
middles = [
    "spectrograms_channels_freqs",
]

suffixes = [
    "familiar_th",
    "familiar_pt",
    "unfamiliar_th",
    "unfamiliar_pt",
]

# 2) Genera tutti i nomi di progetto
projects = [
    f"{p}_{m}_{s}"
    for p, m, s in product(prefixes, middles, suffixes)
]

# 3) Configura l’API e l’entity
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# 4) Itera su ogni progetto: svuota le run e poi cancella gli sweep
for proj in projects:
    project_path = f"{entity}/{proj}"
    print(f"\n→ Progetto: {project_path}")

    # 4.1 Cancella tutte le run via Python API
    try:
        runs = api.runs(project_path)
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                except Exception as e:
                    print(f"     – Errore cancellando run {run.id}: {e}")
                else:
                    print(f"     – run {run.id} eliminata")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Impossibile caricare le run: {e}")

    # 4.2 Cancella tutti gli sweep via CLI Python module
    #    Evitiamo di chiamare un eseguibile esterno, usiamo `python -m wandb`
    #    Lo stesso interprete che esegue questo script è in sys.executable
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", project_path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    # Ogni riga di res.stdout ha uno sweep_id come primo token
    for line in res.stdout.splitlines():
        sweep_id = line.split()[0]
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{project_path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    print(f"  ✅ Run e sweep eliminati per {project_path}")


In [ ]:
import sys
import subprocess
from itertools import product
import wandb

entity   = "stefano-bargione-universit-di-roma-tor-vergata"
api      = wandb.Api()

# il solo progetto di cui voglio ripulire sweep+run
target = "th_resp_vs_pt_resp_spectrograms_channel_freqs_familiar_th"
project_path = f"{entity}/{target}"

print(f"\n→ Progetto: {project_path}")

# 4.1 Cancella tutte le run via Python API
try:
    runs = api.runs(project_path)
    if runs:
        print(f"   • Eliminando {len(runs)} run…")
        for run in runs:
            try:
                run.delete()
            except Exception as e:
                print(f"     – Errore cancellando run {run.id}: {e}")
            else:
                print(f"     – run {run.id} eliminata")
    else:
        print("   (nessuna run trovata)")
except Exception as e:
    print(f"   ⚠️ Impossibile caricare le run: {e}")

# 4.2 Cancella tutti gli sweep via CLI (usando python -m wandb)
cmd_list = [
    sys.executable, "-m", "wandb", "sweep",
    "--project", project_path, "--list"
]
res = subprocess.run(cmd_list, capture_output=True, text=True)

if res.returncode != 0 or not res.stdout.strip():
    print("   • Nessuno sweep trovato o progetto inesistente")
else:
    for line in res.stdout.splitlines():
        sweep_id = line.split()[0]
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{project_path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

print(f"  ✅ Run e sweep eliminati per {project_path}")


#### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e ANCHE SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies V2**

In [ ]:
print('finito')

In [ ]:
'''
Perfetto, hai due script:

uno funzionale e robusto che elimina run e sweep, ma usa pattern statici,

uno che usa prefisso, medio e suffisso per generare i nomi dei progetti, ma è meno dettagliato.

Ti creo una versione unificata che:

Usa prefissi, medii e suffissi per generare i nomi dei progetti (come nel secondo script).

Per ogni progetto, cancella tutte le run (come nel primo script).

Cancella tutti gli sweep usando wandb sweep --delete.

Verifica che gli sweep siano stati eliminati correttamente.


✅ Script finale combinato:
    
✅ Cosa fa questo script:

Genera i progetti combinando prefix, middle, suffix.

Cancella le run usando l’API Python (run.delete()).

Cancella gli sweep usando wandb sweep --delete, richiamato tramite subprocess.

Verifica se gli sweep sono stati realmente eliminati, usando proj.sweeps().

📝 Dipendenze e prerequisiti:

wandb dev'essere installato e autenticato (wandb login)

Lo script va eseguito in un ambiente con accesso alla CLI di wandb (es. terminale Python)


'''


import sys
import subprocess
from itertools import product
import wandb

# --- Configurazione ---
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# --- Parti del nome progetto ---
prefixes = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]
middles = [
    "spectrograms_channels_freqs",
]
suffixes = [
    "familiar_th",
    "familiar_pt",
    "unfamiliar_th",
    "unfamiliar_pt",
]

# --- Genera i nomi dei progetti ---
project_names = [
    f"{p}_{m}_{s}"
    for p, m, s in product(prefixes, middles, suffixes)
]

# --- Itera su ogni progetto ---
for proj_name in project_names:
    path = f"{entity}/{proj_name}"
    print(f"\n→ Progetto: {path}")

    # --- 1. Elimina tutte le run ---
    try:
        runs = api.runs(path, per_page=None)
        runs = list(runs)
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                    print(f"     – Run {run.id} eliminata")
                except Exception as e:
                    print(f"     – Errore eliminando run {run.id}: {e}")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Errore caricando le run: {e}")
        continue  # salta alla prossima

    # --- 2. Ottieni e cancella gli sweep tramite CLI ---
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    sweep_ids = []
    for line in res.stdout.strip().splitlines():
        sweep_id = line.split()[0]
        sweep_ids.append(sweep_id)
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    # --- 3. Verifica cancellazione sweep ---
    print("   • Verifica sweep attivi dopo la cancellazione...")
    try:
        project_obj = next(p for p in api.projects(entity=entity) if p.name == proj_name)
        remaining_sweeps = project_obj.sweeps()
        if not remaining_sweeps:
            print("   ✅ Nessuno sweep attivo trovato.")
        else:
            print(f"   ⚠️ Sweep ancora attivi: {remaining_sweeps}")
    except Exception as e:
        print(f"   ⚠️ Errore nella verifica sweep: {e}")

    print(f"  ✅ Run e sweep eliminati per {path}")

#### **Weight & Biases Login - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

WEIGHT AND BIASES LOGIN

Il messaggio che stai ricevendo indica 
che sei già connesso al tuo account Weights & Biases (wandb).

Se vuoi forzare il login, puoi usare il comando suggerito:

wandb login --relogin

Questo comando ti permetterà di reinserire le credenziali e riconnetterti al tuo account.
Se non hai bisogno di disconnetterti o di cambiare l'account,
puoi semplicemente continuare a usare wandb senza ulteriori passaggi. 
Hai bisogno di ulteriore assistenza con wandb o con il tuo progetto?
'''


import wandb
wandb.login()
print("✅ Weights & Biases login effettuato con successo!")

In [ ]:
'''
Per modificare il percorso in cui W&B salva i dati localmente,
puoi configurare la variabile di ambiente WANDB_DIR.

Questo ti permette di specificare una directory personalizzata in cui W&B salva tutti i file associati al tuo run, inclusi i dati e i log.
'''

import os

# Imposta la directory per i dati W&B:
# Questo cambierà la cartella in cui W&B salva i dati per quella sessione di esecuzione

# Definisci la cartella principale
WB_dir = "/home/stefano/Interrogait/WB_spectrograms_analyses_channels_frequencies"
os.makedirs(WB_dir, exist_ok=True)


os.environ["WANDB_DIR"] = WB_dir


In [ ]:
import pickle
import numpy

# Apri il file in modalità lettura binaria ('rb')

#path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/'

#path = '/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/'

path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms_channels_frequencies/'

with open(f"{path}new_all_th_concat_spectrograms_coupled_exp.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
# Itera sulle chiavi del dizionario principale
for condition, values in data.items():
    if isinstance(values, dict) and "data" in values and "labels" in values:
        X_shape = values["data"].shape
        y_length = len(values["labels"])
        print(f"🔹 Condizione: {condition}")
        print(f"   ➡ Shape dati: {X_shape}")
        print(f"   ➡ Lunghezza labels: {y_length}\n")


#### **Datasets Loading - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
##################### CODICE UFFICIALE DEL 04/03/2025 ORE 9:30 #####################
                                 ##################### SENZA DETTAGLI SCRITTI V°3 #####################
        
'''ATTENZIONE: 

HO SOSTITUITO LE VARIABILI DI 

    1) DATASET_TRAIN_LOADER -->  TRAIN_LOADER
    2) DATASET_VAL_LOADER -->  VAL_LOADER

    VEDI FUNZIONE 'PREPARE_DATA_FOR_MODEL --> NOMI DELLE VARIABILI DEI TORCH TENSOR DATASET LOADER SON  'TRAIN_LOADER' E VAL_LOADER!!!  

'''
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy as cp
import numpy as np

import wandb
import random
import copy as cp


# Definisci le lista delle coppie di condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

# Inizializza il dizionario per caricare i dati
data_dict = {}

# Definisci la cartella principale


#base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results"

base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies"

os.makedirs(base_dir, exist_ok=True)

'''LOOP DI CARICAMENTO DATI'''

for condition in experimental_conditions:
    # Crea la cartella per la condizione sperimentale
    condition_dir = os.path.join(base_dir, condition)
    os.makedirs(condition_dir, exist_ok=True)
    
    # Aggiungi un livello di annidamento per ogni condizione
    data_dict[condition] = {}
    
    for data_type in ["spectrograms"]:
        
        # Crea la cartella per il tipo di dato
        data_dir = os.path.join(condition_dir, data_type)
        os.makedirs(data_dir, exist_ok=True)
        
        for category in ["familiar", "unfamiliar"]:
            # Crea la cartella per la categoria
            #category_dir = os.path.join(data_dir, category)
            #os.makedirs(category_dir, exist_ok=True)
            
            for subject_type in ["th", "pt"]:
                # Caricamento e suddivisione dei dati
                
                #if data_type == "spectrograms":
                    
                print(f"Caricamento dati per: {condition} - {data_type} - {category}_{subject_type}")
                X, y = load_data(data_type, category, subject_type, condition=condition)
                
                
                # Creazione della chiave per il dizionario annidato
                data_dict[condition][data_type] = data_dict[condition].get(data_type, {})
                data_dict[condition][data_type][f"{category}_{subject_type}"] = (X, y)
                
                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{condition}\033[0m_\033[1m{data_type}\033[0m_\033[1m{category}_{subject_type}\033[0m - Shape X: \033[1m{X.shape}\033[0m, Shape y: \033[1m{len(y)}\033[0m\n")

#### **Creazione Griglia 2D per Interrogait - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
import pandas as pd

path = '/home/stefano/Interrogait/all_datas/'

with open(f"{path}EEG_channels_names.pkl", "rb") as f:
    EEG_channels_names = pickle.load(f)
    
# Caricare file xlsx con pickle
path_xlsx = f'{path}EEG_grid_interrogait.xlsx'

# Caricamento del file in un DataFrame
EEG_file_interrogait = pd.read_excel(path_xlsx)

In [ ]:
EEG_file_interrogait

In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List

def _build_grid_maps(
    eeg_grid_df: pd.DataFrame,
    eeg_channels_names: List[str],
    grid_shape: Tuple[int, int] = (9, 9),
):
    """
    Crea:
      - label_grid: matrice (9x9) di etichette (elettrodi o 'EMPTY')
      - electrode_grid_map: dict {elettrodo -> (y, x)} (solo elettrodi reali)
      - placement_idx: matrice (9x9) di indici canale (>=0) o -1 per EMPTY/non presenti
    """
    df = eeg_grid_df.copy()
    df["Electrode"] = df["Electrode"].astype(str).str.strip()

    H, W = grid_shape
    label_grid = np.full((H, W), "", dtype=object)
    electrode_grid_map = {}

    # Mappa canale -> indice colonna in X_data
    ch_to_idx = {ch: i for i, ch in enumerate(eeg_channels_names)}

    # Matrice con indici canale (per riempimento veloce delle griglie)
    placement_idx = np.full((H, W), -1, dtype=int)

    for _, row in df.iterrows():
        elec = row["Electrode"]
        x = int(round(row["grid_x"] * (W - 1)))
        y = int(round(row["grid_y"] * (H - 1)))

        label_grid[y, x] = "" if elec == "EMPTY" else elec

        if elec != "EMPTY":
            electrode_grid_map[elec] = (y, x)
            if elec in ch_to_idx:
                placement_idx[y, x] = ch_to_idx[elec]
            # se l'elettrodo non è nella lista canali, placement resta -1 (verrà messo 0 in griglia)

    # Controllo elettrodi presenti nell'Excel ma non nei dati
    excel_elec = set(df["Electrode"].unique()) - {"EMPTY"}
    missing = sorted(elec for elec in excel_elec if elec not in ch_to_idx)
    if missing:
        print("⚠️ Elettrodi nel file Excel ma non presenti in EEG_channels_names:", missing)

    return label_grid, electrode_grid_map, placement_idx


def convert_fft_images_to_2d_grids_all_freqs_interrogait(
    data_dict: Dict[str, Dict[str, Dict[str, Tuple[np.ndarray, np.ndarray]]]],
    eeg_grid_df: pd.DataFrame,
    eeg_channels_names: List[str],
    grid_shape: Tuple[int, int] = (9, 9),
    fs: int = 250,
    n_fft_points: int = 250,
    bands: Dict[str, Tuple[float, float]] = None,
    verbose: bool = True,
) -> Tuple[Dict, np.ndarray, Dict]:
    """
    Converte OGNI X_data nella struttura annidata di `data_dict`:
      (B, n_freqs, n_channels)  →  (B, 9, 9, 5)
    sommando la potenza sulle frequenze per ciascuna banda EEG e mappando
    i canali nelle posizioni (y,x) definite dal file Excel della griglia.

    Struttura in input (immutata nelle chiavi):
      data_dict[condition][data_type][category_subject] = (X_data, y_data)

    In output mantiene la stessa struttura ma con X_data trasformato:
      X_grid: (B, 9, 9, 5),  y invariato.

    Ritorna:
      - new_data_dict: stessa struttura annidata con X trasformati
      - label_grid: matrice (9x9) con le etichette
      - electrode_grid_map: dict {elettrodo -> (y, x)}
    """
    if bands is None:
        # Ordine fisso (profondità = 5)
        bands = {
            "delta": (1, 4),
            "theta": (4, 8),
            "alpha": (8, 13),
            "beta":  (13, 30),
            "gamma": (30, 45),
        }
    band_order = ["delta", "theta", "alpha", "beta", "gamma"]

    # Precostruisco mappe della griglia
    label_grid, electrode_grid_map, placement_idx = _build_grid_maps(
        eeg_grid_df=eeg_grid_df,
        eeg_channels_names=eeg_channels_names,
        grid_shape=grid_shape,
    )

    H, W = grid_shape

    # Trovo un esempio per determinare n_freqs effettivi (bins) e costruire le maschere
    example_found = False
    n_freqs_example = None
    n_channels_example = None

    for condition, data_types in data_dict.items():
        for data_type, categories in data_types.items():
            for category_subject, (X_data, y_data) in categories.items():
                if X_data is not None and len(X_data) > 0:
                    n_freqs_example = X_data.shape[1]
                    n_channels_example = X_data.shape[2]
                    example_found = True
                    break
            if example_found:
                break
        if example_found:
            break

    if not example_found:
        raise ValueError("Impossibile determinare n_freqs/n_channels: data_dict è vuoto?")

    # Frequenze in Hz per i bins RFFT (tronco ai primi n_freqs effettivi)
    all_freqs_full = np.fft.rfftfreq(n_fft_points, d=1.0 / fs)
    all_freqs = all_freqs_full[:n_freqs_example]

    # Maschere per ciascuna banda sull'asse delle frequenze
    band_masks = {
        b: (all_freqs >= fmin) & (all_freqs <= fmax) for b, (fmin, fmax) in bands.items()
    }

    # Avvisi utili
    if verbose:
        print(f"fs={fs} Hz, n_fft_points={n_fft_points}")
        print(f"n_freqs in X_data = {n_freqs_example} (verranno usati i primi {n_freqs_example} bins di rfftfreq)")
        print("Bande usate:", {b: bands[b] for b in band_order})

    # Trasformazione
    new_data_dict = {}
    for condition, data_types in data_dict.items():
        new_data_dict.setdefault(condition, {})
        for data_type, categories in data_types.items():
            new_data_dict[condition].setdefault(data_type, {})

            for category_subject, (X_data, y_data) in categories.items():
                
                # X_data: (B, n_freqs, n_channels)
                if X_data is None or X_data.size == 0:
                    new_data_dict[condition][data_type][category_subject] = (X_data, y_data)
                    continue

                B, n_freqs, n_channels = X_data.shape
                if n_freqs != n_freqs_example:
                    # Le maschere sono state costruite su n_freqs_example; se differisce, le rigenero on-the-fly
                    all_freqs_local = np.fft.rfftfreq(n_fft_points, d=1.0 / fs)[:n_freqs]
                    band_masks_local = {
                        b: (all_freqs_local >= bands[b][0]) & (all_freqs_local <= bands[b][1])
                        for b in band_order
                    }
                else:
                    band_masks_local = band_masks

                if n_channels != len(eeg_channels_names):
                    print(
                        f"⚠️ Attenzione: n_channels={n_channels} "
                        f"diverso da len(EEG_channels_names)={len(eeg_channels_names)} "
                        f"per {condition} / {data_type} / {category_subject}. "
                        f"Userò SOLO i canali presenti in placement_idx (gli altri verranno ignorati)."
                    )

                # Output per questo blocco: (B, H, W, 5)
                X_out = np.zeros((B, H, W, len(band_order)), dtype=X_data.dtype)

                # Precompute posizione valide nella griglia (non EMPTY)
                valid_pos = placement_idx >= 0
                idx_lin = placement_idx[valid_pos]  # indici canale per le posizioni valide
                yy, xx = np.where(valid_pos)       # coordinate y,x da riempire

                for b in range(B):
                    # Per ciascuna banda: somma lungo le frequenze → vettore (n_channels,)
                    per_band_grids = []
                    sample = X_data[b]  # (n_freqs, n_channels)

                    for bi, band_name in enumerate(band_order):
                        mask = band_masks_local[band_name]
                        if not np.any(mask):
                            # nessun bin in banda → griglia a zero
                            continue

                        # potenza totale per canale nella banda
                        band_power_per_ch = sample[mask, :].sum(axis=0)  # (n_channels,)

                        # riempi griglia rapidamente con indicizzazione
                        grid = np.zeros((H, W), dtype=sample.dtype)
                        # assegna solo posizioni con elettrodi mappati (placement_idx >= 0)
                        grid[yy, xx] = band_power_per_ch[idx_lin]

                        X_out[b, :, :, bi] = grid

                new_data_dict[condition][data_type][category_subject] = (X_out, y_data)

                if verbose:
                    print(
                        f"[OK] {condition} / {data_type} / {category_subject} : "
                        f"{X_data.shape}  →  {X_out.shape}"
                    )

    return new_data_dict, label_grid, electrode_grid_map


In [ ]:
# 3) converti TUTTI i blocchi del tuo data_dict
data_dict, label_grid, electrode_grid_map = convert_fft_images_to_2d_grids_all_freqs_interrogait(
    data_dict,
    eeg_grid_df=EEG_file_interrogait,
    eeg_channels_names=EEG_channels_names,
    grid_shape=(9, 9),
    fs=250,
    n_fft_points=250,
    verbose=True
)

In [ ]:
import pickle

path = '/home/stefano/Interrogait/all_datas/'

# Salvare l'intero dizionario annidato con pickle
with open(f'{path}final_EEG_electrodes_grid_interrogait.pkl', 'wb') as f:
    pickle.dump(label_grid, f)
    
# Salvare l'intero dizionario annidato con pickle
with open(f'{path}electrode_grid_map_interrogait.pkl', 'wb') as f:
    pickle.dump(electrode_grid_map, f)

In [ ]:
'''

Se vuoi costruire una matrice 2D indicizzata (es. grid[y, x]) per usarla come immagine 2D o input per modelli CNN:

→ Devi usare indici interi

Quindi sì, devi fare:

x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
y = int(round(row['grid_y'] * (grid_shape[0] - 1)))

Questo perché una matrice NumPy grid_2d[y, x] non può accettare float come indici
'''

import matplotlib.pyplot as plt
import numpy as np

def plot_eeg_grid_labels(label_grid: np.ndarray, title: str = "Posizione elettrodi sulla griglia EEG"):
    """
    Visualizza una griglia 2D con le etichette degli elettrodi (senza potenza).
    
    Args:
        label_grid (np.ndarray): Griglia 2D (grid_y x grid_x) con stringhe degli elettrodi.
        title (str): Titolo del grafico.
    """
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    # Disegna la griglia e annota le etichette
    for y in range(label_grid.shape[0]):
        for x in range(label_grid.shape[1]):
            label = label_grid[y, x]
            rect = plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='gray', lw=1)
            ax.add_patch(rect)
            if label != '':
                ax.text(x + 0.5, y + 0.5, label, ha='center', va='center', fontsize=8)

    ax.set_xlim(0, label_grid.shape[1])
    ax.set_ylim(label_grid.shape[0], 0)  # Inverti y per avere origine in alto a sinistra
    plt.tight_layout()
    plt.show()

In [ ]:
plot_eeg_grid_labels(label_grid)

In [ ]:
#🔄 Se vuoi aggiungere le etichette degli elettrodi sulla griglia:

def plot_eeg_grid_any(trial_img, label_grid, band=None, mode="single", title=None):
    """
    trial_img: (9,9) oppure (9,9,5)
    band: 'delta'|'theta'|'alpha'|'beta'|'gamma' (usata se mode='single')
    mode: 'single' (una banda), 'sum', 'mean', 'all' (griglia 2x3 con tutte le bande)
    """
    import matplotlib.pyplot as plt
    band_map = {"delta":0, "theta":1, "alpha":2, "beta":3, "gamma":4}
    if trial_img.ndim == 2:
        img2d = trial_img
        plt.figure(figsize=(6,6))
        im = plt.imshow(img2d, cmap='viridis', origin='upper')
        for y in range(label_grid.shape[0]):
            for x in range(label_grid.shape[1]):
                lab = label_grid[y, x]
                if lab != "":
                    plt.text(x, y, lab, ha='center', va='center', color='white', fontsize=8)
        plt.colorbar(im, label="Potenza Totale")
        plt.title(title or "EEG grid")
        plt.axis('off'); plt.tight_layout(); plt.show()
        return

    # trial_img è (9,9,5)
    if mode in ("sum", "mean"):
        img2d = trial_img.sum(axis=-1) if mode=="sum" else trial_img.mean(axis=-1)
        return plot_eeg_grid_any(img2d, label_grid, title=title or f"Aggregato ({mode})")

    if mode == "all":
        import matplotlib.pyplot as plt
        bands = ["delta","theta","alpha","beta","gamma"]
        fig = plt.figure(figsize=(10,7))
        for i,b in enumerate(bands):
            ax = fig.add_subplot(2,3,i+1)
            im = ax.imshow(trial_img[:,:,band_map[b]], cmap='viridis', origin='upper')
            for y in range(label_grid.shape[0]):
                for x in range(label_grid.shape[1]):
                    lab = label_grid[y, x]
                    if lab != "":
                        ax.text(x, y, lab, ha='center', va='center', color='white', fontsize=7)
            ax.set_title(b); ax.axis('off')
        fig.suptitle(title or "Tutte le bande")
        plt.tight_layout(); plt.show()
        return

    # mode == 'single'
    assert band in band_map, f"band deve essere in {list(band_map.keys())}"
    img2d = trial_img[:, :, band_map[band]]
    return plot_eeg_grid_any(img2d, label_grid, title=title or f"Banda: {band}")


In [ ]:
#th_resp_vs_pt_resp / spectrograms / familiar_th : (1586, 45, 61)  →  (1586, 9, 9, 5)

In [ ]:
#plot_eeg_grid_any(data_dict["th_resp_vs_pt_resp"]["spectrograms"]["familiar_th"][0], label_grid, "delta")

X_grid, y = data_dict["th_resp_vs_shared_resp"]["spectrograms"]["familiar_th"]
trial_3d = X_grid[5]  # (9,9,5)


plot_eeg_grid_any(trial_3d, label_grid, band="alpha", mode="single",
                  title="th_resp_vs_shared_resp - Trial 0 - alpha")

In [ ]:
data_dict['th_resp_vs_pt_resp'].keys()

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            if category_subject not in sweep_ids[condition][data_type]:
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['th_resp_vs_pt_resp'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp'].keys())
print(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys())
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['th_resp_vs_pt_resp']['spectrograms']['familiar_th']))

#### **Sweep Configuration - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS (PER CNN2D)**

In [ ]:
'''OGNI IPER-PARAMETRO DI OGNI RETE


ALLO STESSO LIVELLO DI PARAMETERS!


                                                                POST 22 SETTEMBRE 2025
                                                                
                                                                
                                                                
                                                                ***CNN2D NEW*** 

1) All'interno di ogni layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

a) il numero di output channels (ossia 16 impostato di default qui sotto, ma che potrebbe variare da 16 a 32 con step di 4 
come grandezza della feature map sostanzialmente

b) la grandezza del kernel size (tra 2 e 8 con step di 2)
c) la grandezza dello stride (metti solo valori tra 1 e 2) 


2) Per il layer di batch normalisation del relativo layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#batchnorm1d

deve avere il valore del numero di features di quel layer di batch normalisation
(che deve corrispondere come valore a quello dell'output channels del layer convolutivo che lo precede sostanzialmente) 


3) Al layer di pooling del relativo strato della della CNN1D, far variare la scelta tra

a) max pooling ed average pooling 

b) Il valore del kernel_size del layer di max od average pooling (a seconda di quello che viene scelto tra i due), 
che può variare tra 1 e 2 

4) Al solo primo layer fully connected della CNN1D, far variare la scelta del suo valore 
(che nella mia rete sarebbe "self.fc1 = nn.LazyLinear(8)") in questo set di valori, ossia tra i valori 8,10,12,14,16

5) Il valore del dropout layer (con valori tra  0.0 e 0.5) 


6) Il valore della possibile funzione di attivazione tra 3 (relu, selu ed elu)

 a) per gli strati convolutivi (3) +
 b) per il primo fully connected layer (FC1) (prendendone una a caso tra quelle 3 possibili



TABELLA FINALE RIASSUNTIVA - CNN1D 


| Iper-parametro                     | Descrizione                                             | Valori possibili                 |
| ---------------------------------- | ------------------------------------------------------- | -------------------------------- |
| `conv_out_channels`                | Numero di feature-map di base                           | `[16, 20, 24, 28, 32]`           |
| `conv_k1`, `conv_k2`, `conv_k3`    | Kernel size rispettivamente per i 3 blocchi convolutivi | `[2, 4, 6, 8]`                   |
| `conv_s1`, `conv_s2`, `conv_s3`    | Stride rispettivamente per i 3 blocchi convolutivi      | `[1, 2]`                         |
| `pool_type`                        | Tipo di pooling                                         | `["max","avg"]`                  |
| `pool_p1`, `pool_p2`, `pool_p3`    | Kernel size rispettivamente per i 3 blocchi di pooling  | `[1, 2]`                         |
| `fc1_units`                        | Numero di unità nel primo fully-connected               | `[8, 10, 12, 14, 16]`            |
| `cnn_act1`, `cnn_act2`, `cnn_act3` | Funzione di attivazione per ciascun blocco (layer1,2,3) | `["relu","selu","elu"]`          |
| **+ comune**                       | `dropout`                                               | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |







Ok, adesso invece dovrei cercare di modificare lo "sweep config" associato al modello CNN2D che processerà invece i dati EEG, nella sua rappresentazione frequenza x canali che ho creato. 
In questo formulazione di dato di input, i segnali EEG hanno shape (batch, frequenze, canali). 


Lo sweep config dovrebbe avere questi valori nella sua configurazione "generica"/"generale"


sweep_config_cnn2d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        #ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
    
        
    }
}


a cui, però,  vorrei che venissero aggiunti anche nello sweep config stesso i parametri architetturali specifici della rete CNN2D che ho creato quando ho generato il costruttore della rete stessa,z che sarebbe questa qui



class CNN2D(nn.Module):
    def __init__(
        self,
        input_channels: int,
        num_classes: int,

        # da sweep: numero di feature map di base
        conv_out_channels: int,

        # da sweep: kernel size H×W per i 3 blocchi
        conv_k1_h: int, conv_k1_w: int,
        conv_k2_h: int, conv_k2_w: int,
        conv_k3_h: int, conv_k3_w: int,

        # da sweep: stride H×W per i 3 blocchi
        conv_s1_h: int, conv_s1_w: int,
        conv_s2_h: int, conv_s2_w: int,
        conv_s3_h: int, conv_s3_w: int,

        # da sweep: pool kernel H×W per i 3 blocchi
        pool_p1_h: int, pool_p1_w: int,
        pool_p2_h: int, pool_p2_w: int,
        pool_p3_h: int, pool_p3_w: int,

        # da sweep: tipo di pooling
        pool_type: str,  # "max" o "avg"

        # fully‑connected
        fc1_units: int,
        dropout: float,

        # attivazioni per i 3 blocchi
        cnn_act1: str,
        cnn_act2: str,
        cnn_act3: str,
    ):
        super().__init__()
        mapping = {'relu': F.relu, 'selu': F.selu, 'elu': F.elu}
        self.act_fns = [
            mapping[cnn_act1],
            mapping[cnn_act2],
            mapping[cnn_act3],
        ]
        
        # calcolo padding “quasi‐same” per ciascun blocco
        p1_h = (conv_k1_h - 1) // 2
        p1_w = (conv_k1_w - 1) // 2
        p2_h = (conv_k2_h - 1) // 2
        p2_w = (conv_k2_w - 1) // 2
        p3_h = (conv_k3_h - 1) // 2
        p3_w = (conv_k3_w - 1) // 2
        
        # Primo blocco
        self.conv1 = nn.Conv2d(
            input_channels, conv_out_channels,
            kernel_size = (conv_k1_h, conv_k1_w),
            stride = (conv_s1_h, conv_s1_w),
            #padding='same'
            padding = (p1_h, p1_w)
        )
        self.bn1   = nn.BatchNorm2d(conv_out_channels)
        self.pool1 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p1_h, pool_p1_w))

        # Secondo blocco (×2 feature map)
        self.conv2 = nn.Conv2d(
            conv_out_channels, conv_out_channels*2,
            kernel_size=(conv_k2_h, conv_k2_w),
            stride=(conv_s2_h, conv_s2_w),
            #padding='same'
            padding = (p2_h, p2_w) 
        )
        self.bn2   = nn.BatchNorm2d(conv_out_channels*2)
        self.pool2 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p2_h, pool_p2_w))

        # Terzo blocco (×3 feature map)
        self.conv3 = nn.Conv2d(
            conv_out_channels*2, conv_out_channels*3,
            kernel_size=(conv_k3_h, conv_k3_w),
            stride=(conv_s3_h, conv_s3_w),
            #padding='same'
            padding = (p3_h, p3_w)
        )
        self.bn3   = nn.BatchNorm2d(conv_out_channels*3)
        self.pool3 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p3_h, pool_p3_w))

        # FC finale
        self.fc1     = nn.LazyLinear(fc1_units)
        self.dropout = nn.Dropout(dropout)
        self.fc2     = nn.LazyLinear(num_classes)
    
    
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        #🔁 Prima:
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        #x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        #x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)
        
        #✅ Ora:
        #Siccome i dati arrivano come (B, 45, 61) — cioè frequenze × canali, non serve permutare. Ti basta:
        
        # Aggiungiamo una dimensione per il canale "immagine"
        x = x.unsqueeze(1)  # → (B, 1, 45, 61)
            
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = self.act_fns[0](x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = self.act_fns[1](x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = self.act_fns[2](x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.relu(x)
       
        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x


Vorrei quindi che nello sweep config io dia un SOLO valore ad ogni parametro architetturale (quindi CNN2D specifico) della rete stessa, in modo che in realtà nello sweep config sia 'fisso',  in modo che quando lo richiamo dovrebbe avere questi valori qui 

cnn = CNN2D(input_channels = 1, num_classes = num_classes,
            conv_out_channels=16,
            conv_k1_h=3,conv_k1_w=5,
            conv_k2_h=3,conv_k2_w=5,
            conv_k3_h=3,conv_k3_w=5,
            conv_s1_h=1,conv_s1_w=2,
            conv_s2_h=1,conv_s2_w=2,
            conv_s3_h=1,conv_s3_w=2,
            pool_p1_h=1,pool_p1_w=2,
            pool_p2_h=1,pool_p2_w=2,
            pool_p3_h=1,pool_p3_w=1,
            pool_type='max',fc1_units=10,dropout=0.5,
            cnn_act1='relu',cnn_act2='relu',cnn_act3='relu')

 quindi, se ragiono correttamente....


il mio sweep config dovrebbe diventare così


sweep_config_cnn2d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        

a cui si dovrebbero aggiungere però appunto i valori dei parametri architetturali della CNN2D, e quindi dovrebbero essere


        # --- CNN1D solo quando model_name=="CNN2D" ---
        "conv_out_channels":{"values":[16]},

        "conv_k1_h":{"values":[3]},
        "conv_k1_w":{"values":[5]},
        
        "conv_k2_h":{"values":[3]},
        "conv_k2_w":{"values":[5]},
        
        "conv_k3_h":{"values":[3]},
        "conv_k3_w":{"values":[5]},

        "conv_s1_h":{"values":[1]},
        "conv_s1_w": {"values":[2]},
        
        "conv_s2_h":{"values":[1]},
        "conv_s2_w": {"values":[2]},
        
        "conv_s3_h":{"values":[1]},
        "conv_s3_w": {"values":[2]},
        
        "pool_p1_h":{"values":[1]},
        "pool_p1_w":{"values":[2]},
        
        "pool_p2_h":{"values":[1]},
        "pool_p2_w":{"values":[2]},
    
        
        "pool_p3_h":{"values":[1]},
        "pool_p3_w":{"values":[1]},

        "pool_type":{"values":["max","avg"]},
        "fc1_units":{"values":[10]},

        "cnn_act1":{"values":["relu"]},
        "cnn_act2":{"values":["relu"]},
        "cnn_act3":{"values":["relu"]},
        
     
        # comune
        "dropout":{"values":[0.5]}
    }
}

giusto?



'''


sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        # --- setup generale ---
        "model_name":   {"values": ["CNN2D"]},
        "n_epochs":     {"value": 100},
        "patience":     {"value": 12},
        "batch_size":   {"values": [32, 48, 64, 96]},
        "standardization": {"value": True},   # fisso a True

        # --- ottimizzatore ---
        "lr":           {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "beta1":        {"values": [0.9, 0.95]},
        "beta2":        {"values": [0.99, 0.995]},
        "eps":          {"values": [1e-8, 1e-7]},

        # --- iperparametri architettura CNN2D (fissi) ---
        "conv_out_channels": {"value": 16},

        "conv_k1_h": {"value": 3}, "conv_k1_w": {"value": 5},
        "conv_k2_h": {"value": 3}, "conv_k2_w": {"value": 5},
        "conv_k3_h": {"value": 3}, "conv_k3_w": {"value": 5},

        "conv_s1_h": {"value": 1}, "conv_s1_w": {"value": 2},
        "conv_s2_h": {"value": 1}, "conv_s2_w": {"value": 2},
        "conv_s3_h": {"value": 1}, "conv_s3_w": {"value": 2},

        "pool_p1_h": {"value": 1}, "pool_p1_w": {"value": 2},
        "pool_p2_h": {"value": 1}, "pool_p2_w": {"value": 2},
        "pool_p3_h": {"value": 1}, "pool_p3_w": {"value": 1},

        "pool_type":  {"values": ["max", "avg"]},     # se vuoi fissarlo; se vuoi provarlo, usa {"values":["max","avg"]}
        "fc1_units":  {"value": 12},
        "cnn_act1":   {"value": "relu"},
        "cnn_act2":   {"value": "relu"},
        "cnn_act3":   {"value": "relu"},
        "dropout":    {"value": 0.5}
    }
}


    
'''SWEEP_IDS_FOR_MODELS'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

for condition in sweep_ids_for_models:
    for data_type in sweep_ids_for_models[condition]:
        for category_subject in sweep_ids_for_models[condition][data_type]:
            for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                sweep_ids[condition][data_type][category_subject] = []
                    
print(f"\nAggiornato \033[1msweep_ids\033[0m")

In [ ]:
best_models

In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

In [ ]:
import pprint
pprint.pprint(sweep_config)

import pprint
pprint.pprint(sweep_config)**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE: A DIFFERENZA DI PRIMA, DOVE GLI SWEEPS ERANO CREATI SOLO PER OGNI CONDIZIONE SPERIMENTALE,
ADESSO INVECE VENGONO CREATI PER OGNI COMBINAZIONI DI FATTORI, CHE INCLUDONO:

1) DATI DI COPPIE DI CONDIZIONI SPERIMENTALI
2) PROVEVIENZA DEI DATI (IN QUESTO SPETTOGRAMMI TIME-FREQUENCY
3) PROVENIENZA DEI DATI STESSI (FAMILIAR VS UNFAMILIAR; THERAPIST VS PATIENT)

'''

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_{data_type}_channels_freqs_{category_subject}")

                    '''QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                     CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA '''
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['th_resp_vs_pt_resp'].keys()

In [ ]:
best_models

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA) OLD VERSION**

##### **Training Function Edits - EEG Spectrograms - Electrodes x Frequencies BOTH HYPER-PARAMS & MODEL PARAMAS**

Allora adesso, credo ci sia una delle parti più complesse, ossia: 

aggiungere una stringa formata da 'v_' e 'un valore numerico progressivo' al nome del file, quindi a 'best_model_name' quindi in riferimento a questa parte del codice qui...


    # Salva un dizionario contenente sia i pesi che le configurazioni
        torch.save({
            "state_dict": best_model.state_dict(),
            "config": training_config,
            "model_config": model_config
        }, model_file)

in base alla configurazione dei parametri della architettura.. l'idea è questa:

è possibile che venga trovato lo stesso modello (ossia gli stessi valori di model_config ossia dei PARAMETRI del modello), per la stessa combinazioni di fattori che costituiscono in dat (LEGGI SOTTO BENE PER CAPIRE QUESTO PASSAGGIO) MA con diverse configurazioni di IPER-PARAMETRI.... giusto?

quindi, potrei salvarmi i modelli, in base alla configurazione dei loro PARAMETRI... e tenere traccia della loro combinazione di valori dentro ad un set, che contiene la combinazione dei valori associati ai parametri di UN CERTO MODELLO...

dovendo provare a scegliere quale tra I DUE (immaginiamo che lo sweep crea lo STESSO MODELLO più volte) lui dovrà SALVARE (o sovrascrivere, quindi, secondo la logica che ho scritto) quello che, tra i due, abbia ad esempio ottenuto una migliore VALIDATION ACCURACY...

ora abbiamo detto che  


1) best_model_name dovrà avere alla fine un 'v_' e 'un valore numerico progressivo' al nome del file che indentifica quale tipologia di modello CNN2D è stato configurato...

quindi sarà tipo alla fine "v_1"

a questo punto si dovrebbe, credo, creare una ulteriore funzione a parte che 

1) accetta come argomento 

A) il 'model_config' corrente (quindi quello che è associato alla creazione di 'best_model_name' con l'aggiunta del "v_1" FINALE, giusto?) e 
B) la stringa appunto del nome del modello ( che sarà diventata appunto "best_model_name" con il suffisso "v_1")

A questo punto, questa funzione potrebbe creare un set, che tiene traccia appunto di questa configurazione creata, appena creata per il modello CNN2D, ma che tiene conto e che si riferisce allo specifico modello di una certa combinazione di fattori che costituiscono i DATI! 

perché RICORDATI CHE 

best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
!

quindi, l'idea è che
 
2) se non l'aveva mai trovata quella combinazioni di PARAMETRI per il modello CNN2D, per quella combinazioni di dati, si salva la stringa del nome del modello dentro al set (con anche il suffisso) e la configurazione di parametri!

3) SUCCESSIVAMENTE, mettiamo siamo nel caso in cui siamo nell'ipotesi di PRIMA, ossia che 

è possibile che venga trovato lo stesso modello (ossia gli stessi valori di model_config ossia dei PARAMETRI del modello), di una certa combinazione di fattori che costituiscono i DATI, MA con diverse configurazioni di IPER-PARAMETRI.... giusto?
 
ALLORA, se capita questa cosa, lui dovrebbe fare il confronto tra 

1) la prima instanza del modello CNN2D (dentro al set) con la stessa configurazione e QUELLA DEL MODELLO CORRENTE, che ha la stessa configurazione dei PARAMETRI DEL MODELLO.... MA magari ha una diversa configurazione di IPER-PARAMETRI

2) si dovrebbe confrontare appunto QUALE validation accuracy sia la migliore tra i DUE MODELLI e SE LA VALIDATION ACCURACY DEL MODELLO CORRENTE, che ha la stessa configurazione dei PARAMETRI DEL MODELLO.... MA magari ha una diversa configurazione di IPER-PARAMETRI è MIGLIORE di quella DEL MODELLO confrontato dentro al SET.. ALLORA SI ANDRA' A SOVRASCRIVERE IL FILE CON 

LO STESSO MODELLO (STESSA CONFIGURAZIONE DI PARAMETRI) MA ORA AVRA' UNA NUOVA (MIGLIORE) CONFIGURAZIONE DI IPER-PARAMETRI....

l'idea è replicare la stessa logica che si usava prima, ma in questo caso è più complesso perché la verifica non è più solo SUGLI IPER-PARAMETRI ma anche SUI PARAMETRI STESSI DEL MODELLO, che sì possono cambiare, MA POSSONO ANCHE RIPETERSI MAGARI NEI VARI SWEEPS ...

SPERO DI ESSER STATO CHIARO




<br>
<br>

Aspetta ti do il codice che ho ad ora:

    "import re

    def parse_combination_key(combination_key):
        """
        Estrae condition_experiment e subject_key da combination_key
        dove il data_type è fisso a "spectrograms".

        Esempio di chiave: 
        "pt_resp_vs_shared_resp_spectrograms_familiar_th"
        """
        match = re.match(
            r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
            combination_key
        )
        if match:
            condition_experiment = match.group(1)
            subject_key = match.group(2)
            return condition_experiment, subject_key
        else:
            raise ValueError(f"Formato non valido: {combination_key}")


    def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

        sweep_id, combination_key = sweep_tuple

        exp_cond, category_subject = parse_combination_key(combination_key)

        data_type = "spectrograms"

        if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
            raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")

        run_name = f"{exp_cond}_{data_type}_{category_subject}_channels_freqs"
        tags = [exp_cond, data_type, category_subject]

        wandb.init(project=f"{exp_cond}_spectrograms_channels_freqs", name=run_name, tags=tags)

        print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
        print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
        print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

        config = wandb.config

        try:
            X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
            print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
            print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
            print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
            print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
        except KeyError:
            raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


        if config.standardization:
            # Standardizzazione
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
        else:
            print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

        # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
        )

        '''ULTIMA VERSIONE: QUI CARICO LA FUNZIONE PER CREARE
         LA CONFIGURAZIONE DI PARAMETRI (RANDOMICA) DEL MODELLO
         A PARTIRE DA SWEEP CONFIG
        '''

        #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo

        if config.model_name == "CNN2D":
            model = build_cnn2d(config)
            print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
            print(f"\Configurazione Modello CNN2D: \n{dict(config)}")

        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        criterion = nn.CrossEntropyLoss()

        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Parametri di training
        n_epochs = config.n_epochs
        patience = config.patience
        early_stopping = EarlyStopping(patience=patience, mode='max')

        best_model = None
        max_val_acc = 0
        best_epoch = 0

        pbar = tqdm(range(n_epochs))

        for epoch in pbar:
            train_loss_tmp = []
            correct_train = 0
            y_true_train_list, y_pred_train_list = [], []

            for x, y in train_loader:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()

                y_pred = model(x)
                loss = criterion(y_pred, y.view(-1))
                loss.backward()
                optimizer.step()

                train_loss_tmp.append(loss.item())
                _, predicted_train = torch.max(y_pred, 1)
                correct_train += (predicted_train == y).sum().item()
                y_true_train_list.extend(y.cpu().numpy())
                y_pred_train_list.extend(predicted_train.cpu().numpy())

            accuracy_train = correct_train / len(train_loader.dataset)
            loss_train = np.mean(train_loss_tmp)

            precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
            recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
            f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
            auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')

            loss_val_tmp = []
            correct_val = 0
            y_true_val_list, y_pred_val_list = [], []

            with torch.no_grad():
                for x, y in val_loader:
                    x, y = x.to(device), y.to(device)
                    y_pred = model(x)

                    loss = criterion(y_pred, y.view(-1))
                    loss_val_tmp.append(loss.item())
                    _, predicted_val = torch.max(y_pred, 1)

                    correct_val += (predicted_val == y).sum().item()
                    y_true_val_list.extend(y.cpu().numpy())
                    y_pred_val_list.extend(predicted_val.cpu().numpy())

            accuracy_val = correct_val / len(val_loader.dataset)
            loss_val = np.mean(loss_val_tmp)

            wandb.log({
                "epoch": epoch,
                "train_loss": loss_train,
                "train_accuracy": accuracy_train,
                "train_precision": precision_train,
                "train_recall": recall_train,
                "train_f1": f1_train,
                "train_auc": auc_train,
                "val_loss": loss_val,
                "val_accuracy": accuracy_val
            })

            if accuracy_val > max_val_acc:
                max_val_acc = accuracy_val
                best_epoch = epoch
                best_model = cp.deepcopy(model)

            early_stopping(accuracy_val)
            if early_stopping.early_stop:
                print("🛑 Early stopping attivato!")
                break


            # Crea un dizionario separato per prelevarsi i correnti valori della configurazione interna della rete
            model_config = {
                "conv_channels": config.conv_channels,
                "kernel_sizes": config.kernel_sizes,
                "strides": config.strides,
                "paddings": config.paddings,
                "pooling_type": config.pooling_type
                "dropout_rate": config.dropout_rate,
                "activations": config.activations,

            }


            training_config = {key: config[key] for key in config if key not in model_config}

            if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

                # Salvo il primo best_model per quella combinazione
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": cp.deepcopy(model),
                    "max_val_acc": accuracy_val,
                    "best_epoch": epoch,
                    "config": training_config,        # Iperparametri di training
                    "model_config": model_config      # Parametri del modello

                }

                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

                os.makedirs(model_path, exist_ok=True)

                model_file = f"{model_path}/{best_model_name}.pkl"

                # Salva un dizionario contenente sia i pesi che la configurazione
                torch.save({
                    "state_dict": best_model.state_dict(),
                    "config": training_config,
                    "model_config": model_config
                }, model_file)

                print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")


            elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                    best_models[exp_cond][data_type][category_subject][config.model_name] = {
                        "model": best_model,
                        "max_val_acc": accuracy_val,
                        "best_epoch": best_epoch,
                        "config": training_config,        # Iperparametri di training
                        "model_config": model_config      # Parametri del modello
                    }

                    best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                    model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                    os.makedirs(model_path, exist_ok=True)

                    print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                    print(f"\nHa un MIGLIORAMENTO!")

                    model_file = f"{model_path}/{best_model_name}.pkl"

                    if os.path.exists(model_file):

                        print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                        # Salva un dizionario contenente sia i pesi che la configurazione
                        torch.save({
                            "state_dict": best_model.state_dict(),
                            #"config": dict(config)
                            "config": training_config,
                            "model_config": model_config
                        }, model_file)

                        print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                    else:
                        continue

            else:
                ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                model_file = f"{model_path}/{best_model_name}.pkl"
                print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

        wandb.finish()

        return best_models

    " 







    ORA, voglio capire come queste funzioni che mi hai proposto, si vadano ad integrare nel codice che già ho


    "import json

    # Funzione per ottenere una stringa chiave univoca dalla configurazione del modello.
    def get_model_config_key(model_config):
        # Convertiamo il dizionario in una stringa in modo canonico
        return json.dumps(model_config, sort_keys=True)

    # Funzione per aggiornare (o assegnare) la versione del modello
    def update_model_version(model_config, base_model_name, current_val_acc, data_key, model_versions):
        """
        - model_config: dizionario con i parametri interni del modello
        - base_model_name: stringa base (es. "CNN2D_pt_resp_vs_shared_resp_spectrograms_familiar_th")
        - current_val_acc: validation accuracy della run corrente
        - data_key: chiave identificativa dei dati, ad esempio una stringa composta da exp_cond, data_type, category_subject
        - model_versions: dizionario che tiene traccia delle versioni per ciascuna configurazione
        """
        # Creiamo una chiave unica per il model_config
        model_config_key = get_model_config_key(model_config)
        # La chiave completa include anche la combinazione di fattori dati e il nome base del modello.
        full_key = (data_key, model_config_key)

        if full_key not in model_versions:
            # Nuova configurazione per questi dati: assegnamo la versione 1
            version = 1
            new_model_name = f"{base_model_name}_v_{version}"
            model_versions[full_key] = {
                "version": version,
                "best_val_acc": current_val_acc,
                "model_name": new_model_name
            }
            return new_model_name
        else:
            # Configurazione già esistente: confrontiamo la validation accuracy
            record = model_versions[full_key]
            if current_val_acc > record["best_val_acc"]:
                # Aggiorniamo la best accuracy, manteniamo la stessa versione (lo stesso suffisso)
                record["best_val_acc"] = current_val_acc
                return record["model_name"]
            else:
                # Il modello corrente è peggiore, ritorna il nome già salvato
                return record["model_name"]

    # Esempio di uso nel training loop:
    # Supponiamo di avere:
    # - config: il dizionario di configurazione completo (contenente sia iperparametri che parametri del modello)
    # - exp_cond, data_type, category_subject: identificatori dei dati
    # - best_models: il dizionario in cui salvi i modelli migliori
    # - model_versions: un dizionario globale (o locale) che tiene traccia delle versioni per ciascuna configurazione
    # - accuracy_val: validation accuracy della run corrente
    # - model: il modello corrente, best_model: il modello best salvato

    # Preleviamo la configurazione del modello
    model_config = {
        "conv_channels": config.conv_channels,
        "kernel_sizes": config.kernel_sizes,
        "strides": config.strides,
        "paddings": config.paddings,
        "dropout_rate": config.dropout_rate,
        "activations": config.activations,
        "pooling_type": config.pooling_type  # se lo hai aggiunto
    }

    # Filtriamo gli iperparametri di training
    training_config = {key: config[key] for key in config if key not in model_config}

    # Costruiamo la chiave dei dati, per esempio:
    data_key = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

    # Ora usiamo la funzione di versioning per ottenere il nome del modello (con suffisso v_X)
    # model_versions deve essere un dizionario definito prima, per esempio all'esterno del loop.
    # Esempio: model_versions = {} (all'inizio dell'esperimento)
    base_model_name = data_key
    new_model_name = update_model_version(model_config, base_model_name, accuracy_val, data_key, model_versions)

    # Quindi, il nome finale del file sarà new_model_name:
    model_file = os.path.join(base_dir, exp_cond, data_type, category_subject, f"{new_model_name}.pkl")

    # Ora, applica la logica per il salvataggio, ad esempio:
    if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": epoch,
            "config": training_config,        # Iperparametri di training
            "model_config": model_config      # Parametri del modello
        }

        os.makedirs(os.path.join(base_dir, exp_cond, data_type, category_subject), exist_ok=True)
        torch.save({
            "state_dict": best_model.state_dict(),
            "config": training_config,
            "model_config": model_config
        }, model_file)
        print(f"Il modello \033[1m{new_model_name}\033[0m verrà salvato in \033[1m{model_file}\033[0m")

    elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": best_model,
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": training_config,
            "model_config": model_config
        }

        os.makedirs(os.path.join(base_dir, exp_cond, data_type, category_subject), exist_ok=True)
        print(f"Il modello in {os.path.join(base_dir, exp_cond, data_type, category_subject)} ha un MIGLIORAMENTO!")

        if os.path.exists(model_file):
            print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{new_model_name}\033[0m verrà AGGIORNATO in \033[1m{model_file}\033[0m")
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": training_config,
                "model_config": model_config
            }, model_file)
            print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{new_model_name}\033[0m")
        else:
            continue
    "

Nello specifico:

1) get_model_config_key cosa fa esattamente e perché c'è 'sort_keys=True'?

2) se capisco bene 'update_model_version' si crea la chiave unica full_key basandosi su 

a) model_config_key che però non capisco che cosa sarebbe alla fine di base...?
b) la combinazione di stringhe che si riferiscono ai fattori da cui provengono i dati ...? 

cioè, se capisco, tu semplifichi la creazione delle "versioni" date dalla combinazione di parametri del modello e set di iper-parametri, perché sai che qui tanto il modello sarà sempre e solo uno, ossia CNN2D?

dopodiché, vedo che metti quella versione del modello dentro 'model_versions' se non è mai esistita.. e questo è ok, ma 

1)  il valore di 'version' non deve essere sempre necessariamente 1, ma sarebbe un numero progressivo.. nel senso che, la versione del modello è dipesa principalmente, nella mia idea originale, dal set di valori che compongo i parametri del modello che viene creato...

(NOTA BENE: 

il suffisso "v_" e "digit" progressivo è solo per tenermi diciamo traccia di quante combinazioni di modelli (ognuna con la sua specific combinazioni di parametri) viene 'ri-creata' magari più volte all'interno degli sweep...

è un modo diciamo per sapere alla fine di quanti modelli CNN2D siano stati effettivamente creati.. )

Quindi, mettiamo caso che la versione 1 è stata creata ed ok.. ora nello sweep magari, ne viene creata un'altra nuova... questa versione sarà quella con valore "2", (i.e., v_2),  in modo da dare un identificativo univoco a questo NUOVO modello, che avrà una sua NUOVA specifica combinazione di parametri (rispetto a quella 'v_1')....

Ora, immagina che, magari, negli sweep lo stesso modello (supponiamo che sia la versione 2 ossia "v_2" ) viene re-instanziata, più volte.... mettiamo il caso che capiti che venga re-instanziata una seconda volta..


ma, in questa seconda volta, magari questa versione "v_2" avrà una NUOVA combinazione di iper-parametri rispetto alla prima volta in cui era stata istanziata (sempre la "v_2" ma il cui file già esiste, perché è stata già incontrata almeno una volta...) 

ora, mettiamo il caso che, questa "v_2", con questa NUOVA combinazione di iper-parametri 

ha una accuratezza MIGLIORE 

di quella stessa versione, ma rispetto alla prima volta in cui era stata istanziata ... e che, come ti dicevo è già stata salvata dentro 'model_versions' (che sarebbe in questo esempio sempre la versione 2) , 

allora.. vorrei che, quello che il codice facesse è che, per questa versione del modello (v2) verrà salvata 

a) la NUOVA configurazione di iper-parametri
b) i pesi e bias che hanno contribuito al MIGLIORAMENTO della validation accuracy... 
c) vedi se necessita di essere ANCHE ri-salvata nuovamente anche la stessa configurazioni di parametri del modello (che sarebbe in questo esempio sempre la versione 2, e per cui è già in memoria nel file che era già stato salvato precedentemente..) 

l'importante è che, 

b) il salvataggio dei pesi e bias e
a) della NUOVA configurazione di iper-parametri (che ha portato al miglioramenteo della validation accuracy per lo stesso modello "v_2")  avvenga rispetto a quello stesso associato alla modello della stessa versione 

ora, ti chiedo appunto.. come potrebbe essere integrata questa cosa, nel codice che già ho ?

<br>
<br>

**CHATGPT**


Ti spiego nel dettaglio cosa fanno le due funzioni e come integrarle nel tuo codice.

1. Funzione get_model_config_key

        def get_model_config_key(model_config):
            # Convertiamo il dizionario in una stringa in modo canonico,
            # usando sort_keys=True per garantire che l’ordinamento delle chiavi
            # sia sempre lo stesso indipendentemente dall’ordine in cui sono state definite.
            return json.dumps(model_config, sort_keys=True)
        
Cosa fa e perché sort_keys=True
Obiettivo: Convertire il dizionario model_config in una stringa che rappresenti in modo univoco la configurazione interna del modello.
sort_keys=True:
Questo parametro assicura che le chiavi siano ordinate alfabeticamente. In questo modo, due dizionari che contengono gli stessi elementi (anche se inseriti in ordine diverso) daranno sempre la stessa stringa. Questo è fondamentale per creare una chiave univoca e consistente.

2. Funzione update_model_version

        def update_model_version(model_config, base_model_name, current_val_acc, data_key, model_versions):
            """
            - model_config: dizionario con i parametri interni del modello.
            - base_model_name: stringa base, ad esempio "CNN2D_pt_resp_vs_shared_resp_spectrograms_familiar_th".
            - current_val_acc: validation accuracy della run corrente.
            - data_key: stringa identificativa dei dati (ad es. combinazione di exp_cond, data_type, category_subject e model_name).
            - model_versions: dizionario che tiene traccia delle versioni per ciascuna configurazione.

            Ritorna il nome del modello con suffisso versionale (es. "_v_1", "_v_2", ecc.).
            """
            # Creiamo una chiave univoca per la configurazione del modello
            model_config_key = get_model_config_key(model_config)
            # La chiave completa include anche la combinazione dei fattori dati (data_key)
            full_key = (data_key, model_config_key)

            if full_key not in model_versions:
                # Se non abbiamo mai visto questa combinazione di dati e configurazione del modello,
                # la versione inizia a 1.
                version = 1
                new_model_name = f"{base_model_name}_v_{version}"
                model_versions[full_key] = {
                    "version": version,
                    "best_val_acc": current_val_acc,
                    "model_name": new_model_name
                }
                return new_model_name
            else:
                # Se esiste già questa configurazione
                record = model_versions[full_key]
                if current_val_acc > record["best_val_acc"]:
                    # Se la nuova validation accuracy è migliore, aggiorniamo il record.
                    record["best_val_acc"] = current_val_acc
                    # Manteniamo lo stesso suffisso (la stessa versione) e ritorniamo il nome.
                    return record["model_name"]
                else:
                    # Altrimenti, non aggiorniamo e ritorniamo il nome già salvato.
                    return record["model_name"]
                    
Cosa fa update_model_version
model_config_key: È la stringa canonica (ottenuta con get_model_config_key) che rappresenta i parametri interni del modello.
full_key: È una tupla composta da data_key (che identifica la combinazione dei dati, ad esempio "CNN2D_pt_resp_vs_shared_resp_spectrograms_familiar_th") e dalla stringa della configurazione del modello.

In questo modo, la chiave è univoca per:
La combinazione dei dati (ad esempio: exp_cond, data_type, category_subject, model_name)
La configurazione interna del modello (i parametri architetturali)

Versioning:
Se questa chiave non è presente in model_versions, viene assegnata la versione 1 (ossia suffisso "_v_1"). Se già esiste, si confronta la validation accuracy:
Se la nuova accuracy è migliore, si aggiorna il record (ma si mantiene lo stesso suffisso).
Altrimenti, si ritorna il nome già salvato.


3. Integrazione nel Training Loop
All'interno del tuo loop di training, subito prima del salvataggio, puoi fare quanto segue:

Creare model_config (che contiene i parametri del modello)
Filtrare training_config (gli iperparametri di training)
Costruire la chiave data_key (ad esempio usando:

    data_key = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"


Utilizzare update_model_version per ottenere il nome del modello con suffisso versionale.
Ecco come integrare queste parti nel tuo codice:


        import json  # Assicurati di importare json all'inizio del file

        # Supponiamo che model_versions sia definito all'esterno del loop (o all'inizio dell'esperimento)
        # Ad esempio, all'inizio del file:
        model_versions = {}

        # ... all'interno del loop (dopo early stopping, ad esempio) ...

        # Crea il dizionario dei parametri del modello:
        model_config = {
            "conv_channels": config.conv_channels,
            "kernel_sizes": config.kernel_sizes,
            "strides": config.strides,
            "paddings": config.paddings,
            "pooling_type": config.pooling_type,
            "dropout_rate": config.dropout_rate,
            "activations": config.activations
        }

        # Filtra gli iperparametri di training:
        training_config = {key: config[key] for key in config if key not in model_config}

        # Costruisci la chiave dei dati:
        data_key = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
        base_model_name = data_key

        # Ottieni il nome del modello con suffisso versione (es. v_1, v_2, etc.)
        new_model_name = update_model_version(model_config, base_model_name, accuracy_val, data_key, model_versions)

        # Costruisci il percorso del file:
        model_file = os.path.join(base_dir, exp_cond, data_type, category_subject, f"{new_model_name}.pkl")

        # Ora, integra la logica di salvataggio:
        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                "config": training_config,        # Iperparametri di training
                "model_config": model_config      # Parametri del modello
            }

            os.makedirs(os.path.join(base_dir, exp_cond, data_type, category_subject), exist_ok=True)
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": training_config,
                "model_config": model_config
            }, model_file)
            print(f"Il modello \033[1m{new_model_name}\033[0m verrà salvato in \033[1m{model_file}\033[0m")

        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": best_model,
                "max_val_acc": accuracy_val,
                "best_epoch": best_epoch,
                "config": training_config,
                "model_config": model_config
            }

            os.makedirs(os.path.join(base_dir, exp_cond, data_type, category_subject), exist_ok=True)
            print(f"Il modello in {os.path.join(base_dir, exp_cond, data_type, category_subject)} ha un MIGLIORAMENTO!")

            if os.path.exists(model_file):
                print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{new_model_name}\033[0m verrà AGGIORNATO in \033[1m{model_file}\033[0m")
                torch.save({
                    "state_dict": best_model.state_dict(),
                    "config": training_config,
                    "model_config": model_config
                }, model_file)
                print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{new_model_name}\033[0m")
            else:
                continue
        else:
            # Se non c'è miglioramento, stampi un messaggio.
            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \033[1m{model_path}\033[0m, ossia \033[1m{model_file}\033[0m")
            
            
Riepilogo delle risposte alle tue domande specifiche:
get_model_config_key

Converte il dizionario model_config in una stringa con le chiavi ordinate (grazie a sort_keys=True).
Questo assicura che la stessa configurazione (anche se definita in ordine diverso) dia sempre la stessa stringa, usata come chiave univoca.
update_model_version

Calcola model_config_key usando get_model_config_key.
Costruisce una chiave completa (full_key) composta da data_key (identificativo dei dati) e dalla stringa della configurazione del modello.
Se full_key non esiste in model_versions, assegna la versione 1 (quindi il nome sarà "base_model_name_v_1").
Se esiste già, confronta la validation accuracy: se la nuova è migliore, aggiorna best_val_acc (senza cambiare il suffisso) e ritorna il nome già salvato; altrimenti, ritorna il nome già esistente.
In questo modo tieni traccia di quante volte (e quale versione) una specifica configurazione del modello è stata ottenuta e aggiornata se migliora.
Integrazione nel codice

Dopo ogni epoch (o al termine dell’addestramento), costruisci i dizionari model_config e training_config.
Costruisci una chiave identificativa data_key basata su exp_cond, data_type, category_subject e model_name.
Usa update_model_version per ottenere il nome del file (con suffisso versione, es. "_v_1", "_v_2", ecc.).
Salva il modello usando questo nome e aggiorna il dizionario best_models.
Con questo schema, se lo sweep istanzia lo stesso modello (stessa configurazione architetturale) più volte con iperparametri differenti, verrà usato lo stesso suffisso (versione) e, se la nuova run ha una migliore validation accuracy, verrà aggiornata la configurazione di iperparametri e i pesi del modello associato a quella versione.

Spero che ora il meccanismo sia chiaro! Se hai ulteriori dubbi o desideri altre modifiche, fammi sapere!


<br>
<br>


**IO**
Quindi, se capisco bene 

1) "get_model_config_key" crea sostanzialmente in forma stringa json, tutto il set di parametro del modello CNN2D corrente, giusto?

e questo serve per avere un formato UNIVOCO e CANONICO del di parametri interni della rete
e si impone con 'sort_keys=True:' per dare una STESSA SEQUENZA all'ORDINAMENTO delle STRINGHE associati ai PARAMETRI INTERNI DELLA RETE... giusto?

2) "update_model_version" richiama "get_model_config_key" (che sarebbe la stringa json, che sarebbe il set di STRINGHE (UNIVOCO e CANONICO ed ORDINATO con la STESSA SEQUENZA) dei parametri interni della rete.

Costruisce poi, con "full_key" una tupla che contiene tutta la STRINGA che si riferisce a

1) nome del modello e combinazione dei fattori dei dati  + la rappresentazione canonica della configurazione del modello (in formato stringa)... quindi immagino sarà una stringa bella lunga...?

se questa non si trova dentro 'model_versions' allora la salva dentro a 'model_versions'..

però forse non son stato chiaro o forse non ti ho posto la domanda... 

quando io creo la 'version' IO intendevo che la rappresentazione canonica della configurazione del modello è costituita sì dalla stessa combinazioni di parametri, e quello va bene...

ma ci dovrebbe essere anche il check di rispetto al fatto che,  due modelli per essere della stessa VERSION, devono per forza (ma quella è comune a tutti)

1) la rappresentazione canonica della configurazione del modello, ma oltretutto quella rappresentazione canonica della configurazione del modello deve avere, per ogni parametro che la costituisce, LO STESSO SET DI VALORI.. perché è quello che rende 

della STESSA VERSIONE due modelli.. cioè la categorizzazione di due modelli come della stessa VERSION deriva non tanto dalla rappresentazione canonica della configurazione del modello (che è COMUNE per tutti diciamo), ma dal fatto che quei due modelli abbiano 

GLI STESSI VALORI nella rappresentazione canonica della configurazione DEI PARAMETRI del modello....

forse mancava specificare questo passaggio....

per questo dicevo che, il suffisso può esser diverso... perché io posso avere, per uno stesso set di dati (es. combinazione di exp_cond, data_type, category_subject e il nome base del modello) diverse VERSIONI di instanziazione del modello, ma con DIVERSI VALORI (appunto) all'interno della rappresentazione canonica della configurazione DEI PARAMETRI del modello stesso...

<br>
<br>


**CHATGPT**


Esattamente, hai colto il punto. Riepilogo e chiarisco:

get_model_config_key:

Questa funzione converte il dizionario dei parametri interni del modello (model_config) in una stringa JSON.
L'uso di sort_keys=True garantisce che le chiavi vengano ordinate sempre nello stesso ordine, così che due dizionari contenenti gli stessi valori (anche se definiti in ordine diverso) producano esattamente la stessa stringa.
In questo modo, ottieni una rappresentazione univoca e canonica della configurazione del modello.
update_model_version:

Richiama get_model_config_key per ottenere quella rappresentazione canonica (model_config_key).
Quindi costruisce una full_key che è una tupla formata da:
a) un identificatore dei dati (ad esempio una stringa costruita con il nome base del modello e i fattori dati, come "CNN2D_pt_resp_vs_shared_resp_spectrograms_familiar_th")
b) la stringa canonica dei parametri del modello ottenuta (che conterrà tutti i valori dei parametri interni, nell'ordine standardizzato).
Questa full_key identifica in modo univoco una configurazione del modello per quella specifica combinazione di dati.
Se full_key non esiste in model_versions, viene assegnata la versione 1 (il suffisso "_v_1") e si memorizza quella combinazione.
Se la full_key è già presente, si confronta la validation accuracy:
Se la nuova run migliora la best accuracy per quella configurazione, si aggiorna il record (ma si mantiene lo stesso suffisso, perché la configurazione interna è identica).
Se invece la run corrente non migliora, viene semplicemente restituito il nome già salvato.
Quindi, per due modelli affinché siano considerati della stessa versione (ad es. "v_1"), devono avere esattamente lo stesso set di valori nella rappresentazione canonica dei parametri interni. Se invece la rappresentazione cambia (cioè, i valori effettivi differiscono), la full_key sarà diversa e verrà assegnata una nuova versione (ad esempio "v_1" per quella nuova chiave, indipendentemente dal fatto che un'altra configurazione con la stessa chiave sia già presente per altri dati).

In sintesi:

La versione (suffisso "v_X") è determinata dalla combinazione di:

I fattori dei dati (exp_cond, data_type, category_subject, model_name)
La rappresentazione canonica dei parametri del modello (ottenuta con get_model_config_key)
Due modelli avranno lo stesso suffisso (stessa versione) se e solo se la loro rappresentazione canonica dei parametri interni è identica.
Se i valori interni sono diversi, anche se il modello è dello stesso tipo (CNN2D), avranno full_key differenti e quindi verrà considerata una nuova versione.

Questo meccanismo ti permette di tenere traccia in modo univoco delle diverse istanze (o "versioni") di modelli creati in base alla configurazione dei parametri della rete, e di aggiornare il salvataggio (con la migliore validation accuracy) per ogni versione specifica.

Spero che questo chiarisca come funzionano e come si integrano le due funzioni nel tuo codice. Se hai altri dubbi o necessiti ulteriori dettagli, chiedi pure!


<br>
<br>

**IO**

ok ma allora domanda, tu hai detto

"
La versione (suffisso "v_X") è determinata dalla combinazione di:

I fattori dei dati (exp_cond, data_type, category_subject, model_name)
La rappresentazione canonica dei parametri del modello (ottenuta con get_model_config_key)

Due modelli avranno lo stesso suffisso (stessa versione) SE E SOLO SE la loro rappresentazione canonica dei parametri interni è identica.
Se i valori interni sono diversi, anche se il modello è dello stesso tipo (CNN2D), avranno full_key differenti e quindi verrà considerata una nuova versione.
"

ma allora, se i il modello è lo stesso ed i fattori dei dati (exp_cond, data_type, category_subject, model_name) sono gli stessi MA SOLO i valori interni dei parametri sono diversi...

DATO CHE ho già una versione "v_1".. per non confondere.. non conviene aumentare il valore progressivo del numero di di "v_*" dove * sarebbe il digit progressivo?

te lo dico perché il salvataggio è stato fatto in modo che ogni modello, relativo ad una certa combinazioni di fattori dei dati, venga salvata in una directory specifica...

automatizzando questa "aumento progressivo" del valore del digit nella striga di salvataggio del modello aiuta così a capire quante diverse versioni di modelli son state provate per ogni configurazione di fattori dei dati.. giusto?

se è corretto il mio ragionamento.. allora, si può fare questa modifica?

in questo modo, la mia idea è che, alla fine della procedura su Weight & Biases, io abbia 

1) per ogni combinazione di fattori dei dati, 
2) provenienti da un certo soggetto, si avranno 

3) specifiche versioni dei modelli CNN2D (e non solo una diciamo) e si salverà, così, 

per ognuna delle versioni, non solo 

i migliori modelli con la loro relativa configurazione di parametri (e dei suoi valori), ma anche degli iper-parametri associati... giusto? 

<br>
<br>



##### **Training Function IMPLEMENTATION - EEG Spectrograms - Electrodes x Frequencies PESI E HYPER-PARAMS**

##### **PARTI DA QUI**

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

#Test
combination_key = "pt_resp_vs_shared_resp_spectrograms_familiar_th"
condition_experiment, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Soggetto:", subject_key)


In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''OLD VERSION'''
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''NEW VERSION
    
    Questo assicura la coerenza tra la creazione degli sweep e le run che li eseguono,
    e permette di tracciare meglio ogni combinazione anche su W&B.
    '''
    wandb.init(project = f"{condition}_{data_type}_channels_freqs_{category_subject}", name = run_name, tags = tags)
    
    
    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo
    
    '''OLD VERSION'''
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
        #model = CNN2D(input_channels = 61, num_classes = 2)
        #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")

    #class CNN2D(nn.Module):
        #def __init__(
            #self,
            #input_channels: int,              # numero di canali (es. 61)
            #num_classes: int,                 # numero di classi di output
            #conv_out_channels: int,           # parametro dallo sweep
            #conv2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #conv2d_stride: tuple,             # es. (h, w) dallo sweep
            #pool_type: str,                   # "max" o "avg" dallo sweep
            #pool2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #fc1_units: int,                   # unità del primo fully connected
            #dropout: float,                   # dropout dallo sweep
            #activations: tuple                # tupla di 3 stringhe, es. ('relu','selu','elu')
        #):
    
    '''PRENDO LA SHAPE DEI DATI PER FORNIRE VALORI GIUSTI PER OGNI INPUt DI CIASCUNA RETE'''
    
    # Appena caricato X_train, X_val, X_test, etc.
    # X_train.shape == (N, freq_bins, channels)
    
    _, freq_bins, channels = X_train.shape
    
    '''NEW VERSION'''
    if config.model_name == "CNN2D":
        
        #model = CNN2D(
            #input_channels   = 1,
            #num_classes      = 2,
            #conv_out_channels= config.conv_out_channels,
            #conv2d_kernel_size = tuple(config.conv2d_kernel_size),
            #conv2d_stride      = tuple(config.conv2d_stride),
            #pool_type        = config.pool_type,
            #pool2d_kernel_size = tuple(config.pool2d_kernel_size),
            #fc1_units        = config.fc1_units,
            #dropout          = config.dropout,
            #activations      = tuple(config.activations)
        #)
        #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
        
    
        model = CNN2D(
                input_channels   = 1,
                num_classes      = num_classes,
                conv_out_channels= config.conv_out_channels,

                conv_k1_h = config.conv_k1_h, 
                conv_k1_w = config.conv_k1_w,

                conv_k2_h = config.conv_k2_h, 
                conv_k2_w = config.conv_k2_w,

                conv_k3_h = config.conv_k3_h,
                conv_k3_w = config.conv_k3_w,

                conv_s1_h = config.conv_s1_h, 
                conv_s1_w = config.conv_s1_w,

                conv_s2_h = config.conv_s2_h,
                conv_s2_w = config.conv_s2_w,

                conv_s3_h = config.conv_s3_h,
                conv_s3_w = config.conv_s3_w,

                pool_p1_h = config.pool_p1_h,
                pool_p1_w = config.pool_p1_w,

                pool_p2_h = config.pool_p2_h,
                pool_p2_w = config.pool_p2_w,

                pool_p3_h = config.pool_p3_h,
                pool_p3_w = config.pool_p3_w,

                pool_type = config.pool_type,

                fc1_units = config.fc1_units,
                dropout   = config.dropout,

                cnn_act1  = config.cnn_act1,
                cnn_act2  = config.cnn_act2,
                cnn_act3  = config.cnn_act3,
            )
    
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    # 1) Optimizer con betas, eps, weight_decay
    optimizer = optim.Adam(
        model.parameters(),
        lr=config.lr,
        betas=(config.beta1, config.beta2),
        eps=config.eps,
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss()
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    '''OLD VERSION'''
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')

        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)

        wandb.log({
            "epoch": epoch,
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            "val_loss": loss_val,
            "val_accuracy": accuracy_val
        })
        
        
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break
            
        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

    
        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
        
    return best_models

#### **Weight & Biases Procedure Edits - EEG Spectrograms - Electrodes x Frequencies BOTH HYPER-PARAMS & MODEL PARAMAS (NON GUARDARE!)**

##### **PRE AGGIORNAMENTO: Weight & Biases Procedure Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS (VEDI SOTTO!!!)**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config,
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        
                
# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                sweep_id, combination_key = sweep_tuple
                
                # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione
                
                
                # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                if num_gpus > 1:
                    
                    # Assegna la GPU in modo rotazionale
                    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{condition}_spectrograms_channels_freqs", count=100)
                    
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=200)
                    
                    # Passa alla prossima GPU per il prossimo sweep
                    gpu_counter = (gpu_counter + 1) % num_gpus

                else:
                    # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                    
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=200)

                    
                # Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
                #def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
                    #def train_wrapper():
                        
                        # Qui chiamiamo la funzione di training con i parametri appropriati
                        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
                        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
                        #training_sweep(
                            #data_dict_preprocessed, 
                            #sweep_config,
                            #sweep_ids,
                            #sweep_id,
                            #sweep_tuple,
                            #best_models  # Best models viene aggiornato all'interno della funzione
                        #)
                    #return train_wrapper
                
                # Crea la funzione wrapper per l'agent
                '''COMMENTATO'''
                #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                
                
                # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                   ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''
                
                print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")
                
                '''COMMENTATO'''
                #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)
                
                print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")
                
                torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

In [ ]:
print("finito")

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA) NEW VERSION (PER CNN2D)**

##### **Training Function Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''OLD VERSION'''
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''NEW VERSION
    
    Questo assicura la coerenza tra la creazione degli sweep e le run che li eseguono,
    e permette di tracciare meglio ogni combinazione anche su W&B.
    '''
    wandb.init(project = f"{condition}_{data_type}_channels_freqs_{category_subject}", name = run_name, tags = tags)
    
    
    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo
    
    '''OLD VERSION'''
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
        #model = CNN2D(input_channels = 61, num_classes = 2)
        #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")

    #class CNN2D(nn.Module):
        #def __init__(
            #self,
            #input_channels: int,              # numero di canali (es. 61)
            #num_classes: int,                 # numero di classi di output
            #conv_out_channels: int,           # parametro dallo sweep
            #conv2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #conv2d_stride: tuple,             # es. (h, w) dallo sweep
            #pool_type: str,                   # "max" o "avg" dallo sweep
            #pool2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #fc1_units: int,                   # unità del primo fully connected
            #dropout: float,                   # dropout dallo sweep
            #activations: tuple                # tupla di 3 stringhe, es. ('relu','selu','elu')
        #):
    
    '''PRENDO LA SHAPE DEI DATI PER FORNIRE VALORI GIUSTI PER OGNI INPUt DI CIASCUNA RETE'''
    
    # Appena caricato X_train, X_val, X_test, etc.
    # X_train.shape == (N, freq_bins, channels)
    
    _, freq_bins, channels = X_train.shape
    
    '''NEW VERSION'''
    if config.model_name == "CNN2D":
        
        #model = CNN2D(
            #input_channels   = 1,
            #num_classes      = 2,
            #conv_out_channels= config.conv_out_channels,
            #conv2d_kernel_size = tuple(config.conv2d_kernel_size),
            #conv2d_stride      = tuple(config.conv2d_stride),
            #pool_type        = config.pool_type,
            #pool2d_kernel_size = tuple(config.pool2d_kernel_size),
            #fc1_units        = config.fc1_units,
            #dropout          = config.dropout,
            #activations      = tuple(config.activations)
        #)
        #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
        
    
        model = CNN2D(
                input_channels   = 1,
                num_classes      = num_classes,
                conv_out_channels= config.conv_out_channels,

                conv_k1_h = config.conv_k1_h, 
                conv_k1_w = config.conv_k1_w,

                conv_k2_h = config.conv_k2_h, 
                conv_k2_w = config.conv_k2_w,

                conv_k3_h = config.conv_k3_h,
                conv_k3_w = config.conv_k3_w,

                conv_s1_h = config.conv_s1_h, 
                conv_s1_w = config.conv_s1_w,

                conv_s2_h = config.conv_s2_h,
                conv_s2_w = config.conv_s2_w,

                conv_s3_h = config.conv_s3_h,
                conv_s3_w = config.conv_s3_w,

                pool_p1_h = config.pool_p1_h,
                pool_p1_w = config.pool_p1_w,

                pool_p2_h = config.pool_p2_h,
                pool_p2_w = config.pool_p2_w,

                pool_p3_h = config.pool_p3_h,
                pool_p3_w = config.pool_p3_w,

                pool_type = config.pool_type,

                fc1_units = config.fc1_units,
                dropout   = config.dropout,

                cnn_act1  = config.cnn_act1,
                cnn_act2  = config.cnn_act2,
                cnn_act3  = config.cnn_act3,
            )
    
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    # 1) Optimizer con betas, eps, weight_decay
    optimizer = optim.Adam(
        model.parameters(),
        lr=config.lr,
        betas=(config.beta1, config.beta2),
        eps=config.eps,
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss()
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    '''OLD VERSION'''
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')

        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)

        wandb.log({
            "epoch": epoch,
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            "val_loss": loss_val,
            "val_accuracy": accuracy_val
        })
        
        
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break
            
        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

    
        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
        
    return best_models

#### **Weight & Biases Procedure Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS - NEW VERSION (PER CNN2D)**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config,
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        
                
# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                sweep_id, combination_key = sweep_tuple
                
                # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione
                
                
                # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                if num_gpus > 1:
                    
                    # Assegna la GPU in modo rotazionale
                    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                
                    #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project=f"{condition}_spectrograms_channels_freqs", count=100)
                    
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=200)
                    
                    # Passa alla prossima GPU per il prossimo sweep
                    gpu_counter = (gpu_counter + 1) % num_gpus

                else:
                    # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                    
                    agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                    
                    wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=200)

                    
                # Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
                #def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject):
                    #def train_wrapper():
                        
                        # Qui chiamiamo la funzione di training con i parametri appropriati
                        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
                        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
                        #training_sweep(
                            #data_dict_preprocessed, 
                            #sweep_config,
                            #sweep_ids,
                            #sweep_id,
                            #sweep_tuple,
                            #best_models  # Best models viene aggiornato all'interno della funzione
                        #)
                    #return train_wrapper
                
                # Crea la funzione wrapper per l'agent
                '''COMMENTATO'''
                #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)
                
                
                # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                   ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''
                
                print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")
                
                '''COMMENTATO'''
                #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)
                
                print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")
                
                torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

#### **Sweep Configuration - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**


#### **Sweep separati per ciascuno dei modelli CNN3D e CNN Sep**

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''



from sklearn.model_selection import train_test_split

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Modelli che useremo nei sweep
MODEL_LIST = ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]


# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            # 1. Prepara spazio nei dizionari: sotto category_subject, un dict per ogni modello
            
            data_dict_preprocessed[condition][data_type][category_subject] = None
            
            if category_subject not in sweep_ids[condition][data_type]:
                
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }

                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids_for_models[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test Set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['th_resp_vs_shared_resp'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['th_resp_vs_shared_resp'].keys())
print(data_dict_preprocessed['th_resp_vs_shared_resp']['spectrograms'].keys())
print(type(data_dict_preprocessed['th_resp_vs_shared_resp']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['th_resp_vs_shared_resp']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['th_resp_vs_shared_resp']['spectrograms']['familiar_th']))

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
''' 

                                                                    AGGIORNATA AL 19 LUGLIO
                                                                    
                                                                    
#"learning rate : {"value"[1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]"}
#"n_epochs": {"value": 100},
# "patience": {"value": 12},
#"batch_size": {"values": [16, 24, 32, 48, 64, 72, 84, 96]}
#"standardization": {"values": [True, False]}, 
# "beta1": {"values": [0.8, 0.85, 0.9, 0.95]},
#  "beta2": {"values": [0.98, 0.99, 0.995, 0.999]},
#  "eps": {"value": [1e-8, 1e-7, 1e-6, 1e-5]}                                                                                                                            



sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]}, # fissato al valore di default del paper

        "weight_decay":  {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        
        
        "model_name":{"values": ['CNN3D_LSTM_FC']},

        "batch_size": {"values": [32, 48, 64, 96]},

        "standardization":{"values": [True, False]},

        "beta1": {"values": [0.9, 0.95]},

        "beta2": {"values": [0.99, 0.995]},
        
        "eps": {"values": [1e-8, 1e-7]},
        
        #In questo modo:
        
        "use_lstm":      {"values":[True, False]},
        "lstm_hidden":   {"values":[32]},
        "dropout":       {"values":[0.5]},
        
    }
}


'''


#Tutti gli sweep saranno organizzati sotto lo stesso progetto,
#che corrisponde alla coppia di condizioni sperimentali corrente (i.e., exp_cond).

#Questo significa che tutte le runs che verranno lanciate con quello sweep, 
#saranno associate a quella specifica coppia di condizioni sperimentali corrente.

#Dato che sto iterando su ogni coppia di condizioni sperimentali, 
#ogni sweep verrà automaticamente salvato all'interno del progetto corrispondente 
#della specifica condizione sperimentale (exp_cond).

#In pratica, se hai più condizioni sperimentali 
#(ad esempio, "Condizione_A", "Condizione_B", ecc.),
#WandB creerà automaticamente sweep separati all'interno dei rispettivi progetti


#Creo la configurazione dello sweep e la eseguo:

#uno per il modello CNN3D_LSTM_FC, uno oer 


# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}





    
'''SWEEP_IDS_FOR_MODELS

# 2) Popolo sweep_ids_for_models in base a MODEL_LIST (già inizializzato nella prima cella)
'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

#for condition in sweep_ids_for_models:
    #for data_type in sweep_ids_for_models[condition]:
        #for category_subject in sweep_ids_for_models[condition][data_type]:
            #for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                #if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    #sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


'''BEST_MODELS

# 3) Creo best_models da sweep_ids_for_models
'''

#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


#'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
#for condition in sweep_ids:
    #for data_type in sweep_ids[condition]:
        #for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            #if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                #sweep_ids[condition][data_type][category_subject] = []
                    
#print(f"\nAggiornato \033[1msweep_ids\033[0m")


In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
print(best_models)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE CHE A QUESTO PUNTO


1) sweep_ids[cond][dtype][cat][model_name] contiene le tuple (sweep_id, combo_key) per ciascun modello, che ancora non esistono perché devo esser create durante la creazione degli sweeps, ma ho solo una lista

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': []}}, 
'rest_vs_right_fist': {'spectrograms': {'familiar_th': []}}, 
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': []}}}


2) sweep_ids_for_models e best_models sono paralleli a sweep_ids con lo stesso livello model_name

ossia 

sweep_ids_for_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}},
'rest_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}},
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}}}

best_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'rest_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}}, 

'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}}}


'''


In [ ]:
'''
Popolamento di sweep_ids e lancio degli agenti:

Obiettivo: 

Per ogni combinazione (condition, data_type, category_subject, model_name), 
Se la lista è vuota, crei uno sweep usando wandb.sweep(sweep_config, project=condition) e lo inserisci nella lista. 
In seguito, iteri su quella lista (che ora contiene IL TUO SPECIFICO sweep_id) e lanci wandb.agent() per eseguire il training.



Nota importante:
L'ID restituito da wandb.sweep() è una STRINGA UNIVOCA generata automaticamente da WandB.
Non puoi assegnargli direttamente una stringa personalizzata, ma puoi comunque usarlo per mappare nel tuo dizionario la combinazione di fattori! 

In questo ciclo, il fatto che la lista parta vuota è normale: il codice la popola se necessario e poi lancia l'agente per ogni sweep_id presente.


****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******
INOLTRE, BISOGNA CONTROLLARE CHE SI STIA ITERANDO CORRETTAMENTE SOLO SULLA COMBINAZIONE CORRENTE DI 

                CONDITION, DATA_TYPE, CATEGORY_SUBJECT E MODEL_NAME
                
QUESTO PERCHÉ SE UN CICLO SI RIPETE PER UNA CONDIZIONE IN PIÙ UNA COMBINAZIONE, POTREBBE GENERARE PIÙ  SWEEP IDS DI QUELLI CHE TI ASPETTI!
****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******



SOLUZIONE:

Un buon approccio per evitare la creazione ripetuta di Sweep ID 
per la stessa combinazione di condition, data_type, category_subject e model_name 
è quello di utilizzare un SET per tenere traccia delle combinazioni già processate.
Se una combinazione è già presente nel set, non dovresti creare un nuovo Sweep ID, ma semplicemente saltare quella parte del codice


Inoltre, ho avuto una idea ad un certo punto! 


****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************


Quando creo ogni sweep singolarmente, si genera una stringa univoca di quello sweep, che si riferisce ad un dataset che è il prodotto di diversi fattori:

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Di conseguenza, iterando su ogni sweep_ids (che ho fatto in modo avesse la STESSA struttura dei miei dati già splittati i.e, data_dict_preprocessed
io posso, 

1) da un lato eseguire la creazione della stringa univoca associata a quello sweep,
2) crearmi una 'combination_key', che sarebbe l'insieme delle stringhe che descrivono quel dataset specifico di data_dict_preprocessed

che sarà costituito da

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Poiché quindi so già la corrispondenza tra ogni Sweep ID e la sua combinazione di fattori (condition, data_type, category_subject), 
posso creare un MAPPING, che associ, ad certo Sweep ID e la stringa che descrive i suoi fattori associati!


In questo modo, forse, si riesce a risolvere il PROBLEMA 2 NELLA CELLA DI CREAZIONE DELLA FUNZIONE DI TRAINING (VEDI SOTTO!)



                                                        ******IMPORTANTE MODIFICA*****
                                                        
Ora lo sweep_ids non si deve sdoppiare ora, perché sostanzialmente, 
per ogni modello si creano gli sweeps ids corrispondenti e salvati come valore
dentro la chiave del modello corrispondente, sotto forma di tupla...

cioè non più così

"sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))"

ma una cosa del genere

"sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))




COME ERA PRIMA

#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
'''


'''
ADESSO


Cosa fa questo snippet

Cicla su ogni (condition, data_type, category_subject) una volta sola grazie a created_combinations.

All’interno, fa un sottoloop su MODEL_LIST (i tuoi due modelli).

In base a model_name, sceglie sweep_config_cnn3d o sweep_config_cnn_sep.

Chiama wandb.sweep(...) con il config giusto e salva il risultato in


sweep_ids[condition][data_type][category_subject][model_name]
anziché nella lista “piatta” che avevi prima.


In questo modo:

sweep_ids[cond][dtype][cat] resta un dict con due chiavi ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")

Ognuna di quelle chiavi punta a una propria lista di tuple (sweep_id, combo_key)

Non serve sdoppiare l’intero sweep_ids, perché tiene già separati gli sweep di ciascun modello

Più tardi, quando lancerai gli agent, ti basterà:


for model_name, sweeps in sweep_ids[cond][dtype][cat].items():
    for sweep_id, combo_key in sweeps:
        # qui scegli il train_fn in base a model_name
        wandb.agent(sweep_id, function=train_fn_map[model_name], count=200)
e ogni modello girerà solo i suoi sweep.



Alla fine, sweep_ids avrà la forma:

{
  'rest_vs_left_fist': {
    'spectrograms': {
      'familiar_th': {
         'CNN3D_LSTM_FC':       [(sweep_id_1, 'rest_vs_left_fist_spectrograms_familiar_th')],
         'SeparableCNN2D_LSTM_FC': [(sweep_id_2, 'rest_vs_left_fist_spectrograms_familiar_th')]
      }
    }
  },
  …
}
'''


#Ecco come puoi riscrivere solo la TERZA CELLA (quella in cui crei effettivamente gli sweep) 
#mantenendo la tua struttura “a celle” e usando per ognuno il sweep_config giusto in base al model_name.

#Creazione degli sweep (Terza cella)
#Ecco il solo snippet che devi usare per creare gli sweep ripartiti per modello, usando i due sweep_config_*:


'''
Per mantenere la stessa logica di prima ma tenendo conto che ora stai lavorando con modelli separati, 
dovresti modificare il controllo in modo che verifichi se una combinazione di condition, data_type, category_subject
è già stata processata per ciascun modello.

Quindi, il controllo dovrebbe essere fatto separatamente per ogni modello dentro il loop che itera sui modelli (MODEL_LIST).
Di seguito ti mostro la versione modificata che tiene conto di questo:



#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
                
                
'''

                    
'''

Cosa è stato cambiato rispetto alla versione precedente?
Controllo della combinazione di modello:
La logica del controllo della combinazione (combination_key, model_name) nel set created_combinations è corretta, 
perché vogliamo evitare di creare più volte lo stesso sweep per una combinazione di condition, data_type, category_subject, e model_name.

Controllo e creazione dello sweep:
Il codice controlla prima se la combinazione con il modello non è stata già processata 
con il controllo if (combination_key, model_name) not in created_combinations. 

Se non è stata processata, procede a creare lo sweep corrispondente. 
Se la combinazione esiste già, salta la creazione dello sweep per quel modello.

Aggiunta del nuovo sweep ID:
Una volta creato il nuovo sweep per il modello, viene aggiunto correttamente 
alla lista del modello specifico sotto sweep_ids[condition][data_type][category_subject][model_name].

Aggiunta al set delle combinazioni:
Dopo aver creato lo sweep, aggiungiamo (combination_key, model_name) al set created_combinations
per tenere traccia delle combinazioni già elaborate.

Verifica della logica:
La combinazione (combination_key, model_name) deve essere unica per ciascun modello, 
e quindi il controllo che evita duplicazioni nel set è corretto.

La creazione dello sweep per ciascun modello separato è mantenuta, 
e viene applicata solo quando la combinazione specifica non è già stata elaborata per quel modello.

In questo modo, la logica funziona come nel codice precedente, ma ora si tiene conto anche dei modelli separati, 
creando un sweep per ciascuno di essi e mantenendo la traccia delle combinazioni in modo appropriato.

'''


'''
COME ERA PER CNN2D


created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_{data_type}_channels_freqs_{category_subject}")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                     #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")



'''



created_combinations = set()

# Per semplicità, tieni MODEL_LIST a portata di mano
MODEL_LIST = ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"
            
            # per ciascun modello, creo uno sweep separato
            for model_name in MODEL_LIST:

                # Controlla se la combinazione di condition, data_type, category_subject + modello è già stata elaborata
                if (combination_key, model_name) not in created_combinations:

                    # Scegli il config in base al model_name
                    if model_name == "CNN3D_LSTM_FC":
                        sweep_conf = sweep_config_cnn3d
                        
                    else:  # SeparableCNN2D_LSTM_FC
                        sweep_conf = sweep_config_cnn_sep
                    
                    # Controllo se la lista per il modello specifico è vuota
                    if not sweep_ids[condition][data_type][category_subject][model_name]:

                        # Crea lo sweep e lo appendo nella lista dedicata a quel modello
                        #new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_channels_freqs_new_imagery_3d_grid_multiband")
                        
                        new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_{data_type}_channels_freqs_{category_subject}")
                        
                        #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                        #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                        
                        sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))

                    print(f"▶ Sweep \033[1m{new_sweep_id}\033[0m creato per \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
                    
                    # Aggiungi la combinazione al set per evitare duplicazioni
                    created_combinations.add((combination_key, model_name))  # Aggiungi la combinazione con il modello
                else:
                    # Se la combinazione è già stata creata, salta
                    print(f"⚠️ {combination_key} già processato per il modello {model_name}, skip.")
                    continue


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['th_resp_vs_pt_resp'].keys()

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA) NEW VERSION (PER CNN3D e CNN2D Sep)**

##### **Training Function Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(pt_resp_vs_shared_resp|th_resp_vs_pt_resp|th_resp_vs_shared_resp)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''OLD VERSION'''
    #wandb.init(project=f"{exp_cond}_spectrograms", name=run_name, tags=tags)
    
    '''NEW VERSION
    
    Questo assicura la coerenza tra la creazione degli sweep e le run che li eseguono,
    e permette di tracciare meglio ogni combinazione anche su W&B.
    '''
    wandb.init(project = f"{condition}_{data_type}_channels_freqs_{category_subject}", name = run_name, tags = tags)
    
    
    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo
    
    '''OLD VERSION'''
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
        #model = CNN2D(input_channels = 61, num_classes = 2)
        #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")

    #class CNN2D(nn.Module):
        #def __init__(
            #self,
            #input_channels: int,              # numero di canali (es. 61)
            #num_classes: int,                 # numero di classi di output
            #conv_out_channels: int,           # parametro dallo sweep
            #conv2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #conv2d_stride: tuple,             # es. (h, w) dallo sweep
            #pool_type: str,                   # "max" o "avg" dallo sweep
            #pool2d_kernel_size: tuple,        # es. (h, w) dallo sweep
            #fc1_units: int,                   # unità del primo fully connected
            #dropout: float,                   # dropout dallo sweep
            #activations: tuple                # tupla di 3 stringhe, es. ('relu','selu','elu')
        #):
    
    if config.model_name == "CNN3D_LSTM_FC":
        #model = CNN2D_LSTM_FC(n_freq =45, input_channels=64, num_classes=2, dropout=0.2)
        
        '''OCCHIO QUI CAMBIATO PER GRIGLIA 3D'''
    
        model = CNN3D_LSTM_FC(
            num_classes=2,
            dropout=config.dropout,
            hidden_size=config.lstm_hidden,
            use_lstm=config.use_lstm
        )

        print(f"\nInizializzazione Modello \033[1mCNN3D_LSTM_FC\033[0m")
    
    
    elif config.model_name == "SeparableCNN2D_LSTM_FC":
        model = SeparableCNN2D_LSTM_FC(
            num_classes=2,
            dropout=config.dropout,
            hidden_size=config.lstm_hidden,
            use_lstm=config.use_lstm
        )
        print(f"\nInizializzazione Modello \033[1mSeparableCNN2D_LSTM_FC\033[0m")

    else:
        raise ValueError(f"Modello sconosciuto: {config.model_name}")
    
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    # 1) Optimizer con betas, eps, weight_decay
    optimizer = optim.Adam(
        model.parameters(),
        lr=config.lr,
        betas=(config.beta1, config.beta2),
        eps=config.eps,
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss()
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    '''OLD VERSION'''
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    #'''AGGIORNAMENTI FINALI'''
    #from sklearn.metrics import roc_auc_score

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        # ---------------------- TRAIN ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.train()  
        
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
            
            #'''AGGIORNAMENTI FINALI'''
            
            # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
            # OPZIONE A: puoi usare la Softmax per avere le probabilità,
            # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
            
            # Opzione A: usare le probabilità (softmax) 
            
            #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
            
            #probs_train = torch.softmax(y_pred, dim=1)
            #y_score_train_list.extend(probs_train[:, 1].detach().cpu().numpy())
            
            # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
            
            #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
            
            # y_score_train_list.extend(y_pred[:, 1].detach().cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        '''come dovrebbe essere calcolato se non si dovesse passare al load_best_run_results'''
        #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list)
        
        '''come è stato calcolato se si dovesse passare al load_best_run_results'''
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        
        
        #'''AGGIORNAMENTI FINALI'''
        #try:
            #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel train set c'è una sola classe.")
            #auc_val = np.nan
        
        # ---------------------- VALIDATION ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.eval()
        
        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []
        
                
        #'''AGGIORNAMENTI FINALI'''
        #y_score_val_list = []  # per AUC valida

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())
                
                #'''AGGIORNAMENTI FINALI'''
                
                # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
                
                # OPZIONE A: puoi usare la Softmax per avere le probabilità,
                # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
                
                # Opzione A: usare le probabilità (softmax) 
                
                #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
                
                #probs_val = torch.softmax(y_pred, dim=1)
                #y_score_val_list.extend(probs_val[:, 1].detach().cpu().numpy())
                
                # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
                
                #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
                # y_score_val_list.extend(y_pred[:, 1].detach().cpu().numpy())
                

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)
        
        #'''AGGIORNAMENTI FINALI'''
        #precision_val = precision_score(y_true_val_list, y_pred_val_list, average='weighted')
        #recall_val    = recall_score(y_true_val_list, y_pred_val_list, average='weighted')
        #f1_val        = f1_score(y_true_val_list, y_pred_val_list, average='weighted')
        
        #try:
            # ATTENZIONE: qui usiamo gli score continui, NON le etichette
            #auc_val = roc_auc_score(y_true_val_list, y_score_val_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel validation set c'è una sola classe.")
            #auc_val = np.nan

        wandb.log({
            "epoch": epoch,
            
            # TRAIN
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            
            # VALIDATION
            
            "val_loss": loss_val,
            "val_accuracy": accuracy_val,
            
            # se vuoi loggare anche queste (consigliato):
            
            #"val_precision": precision_val,
            #"val_recall": recall_val,
            #"val_f1": f1_val,
            #"val_auc": auc_val,
        })
        
        #Nota: questa patch qua sopra (correzione su train e validation) rende corretto anche train_auc per le run future, 
        #quindi non avrai più bisogno della “correzione a posteriori” in load_best_run_results 
        #per i nuovi esperimenti (ma la puoi lasciare per compatibilità coi vecchi run).
        
        
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break
            
        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

    
        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
        
    return best_models

#### **Weight & Biases Procedure Final Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**
#### **Sweep separati per ciascuno dei modelli CNN3D e CNN Sep**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''



'''

Per modificare il loop in modo che accetti i sweeps per ogni modello e gestisca correttamente
l'esecuzione del training per ciascun modello con il relativo sweep, dobbiamo fare alcune modifiche.


Modifiche principali:

1) Funzione make_train_wrapper:
La funzione dovrà essere adattata per passare correttamente la configurazione di sweep per ogni modello, 
invece di passare un'unica configurazione generica (sweep_config).

2) Identificazione corretta del modello: 
Nel loop, per ogni combinazione (condition, data_type, category_subject)
e per ogni modello (ad esempio, CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC), 

dobbiamo passare al wandb.agent il relativo sweep ID per il modello e la sua configurazione.

3) Modifica della funzione make_train_wrapper per gestire ogni modello separatamente: 
Ogni modello avrà il proprio sweep e la propria configurazione.


Spiegazione delle modifiche:

1) Funzione make_train_wrapper:

Adesso prende anche model_name per passare il relativo sweep_config dal dizionario sweep_config_dict.
Passa il sweep_config corretto per ogni modello, a seconda del model_name passato nel ciclo.

2) Dizionario sweep_config_dict:

Ho creato un dizionario sweep_config_dict che associa ciascun modello ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")
alla sua configurazione di sweep (sweep_config_cnn3d e sweep_config_cnn_sep).
Questo permette di usare la corretta configurazione per ogni modello.

3) Modifica nel ciclo:

Il ciclo ora scorre su model_name (i.e., i modelli CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC) 
per ogni combinazione di condition, data_type, category_subject.

Per ogni modello, il relativo sweep viene creato ed eseguito.


Risultato:
Ora, per ogni combinazione di condition, data_type, e category_subject, 
il codice creerà e gestirà separatamente gli sweeps per ciascun modello,
e li eseguirà utilizzando la funzione training_sweep con la relativa configurazione specifica per ogni modello.

Questa modifica ti consente di avere il corretto flusso di lavoro per eseguire
il training separato per ogni modello con la sua configurazione.


'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili

'''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper'''

def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config_dict[model_name], # Prendi la configurazione per il modello specifico
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        

# Dizionari di configurazione per ogni modello
sweep_config_dict = {
    "CNN3D_LSTM_FC": sweep_config_cnn3d,
    "SeparableCNN2D_LSTM_FC": sweep_config_cnn_sep
}

# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for model_name in sweep_ids[condition][data_type][category_subject]:  # Aggiunto loop per il modello
                
                #for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                 for sweep_tuple in sweep_ids[condition][data_type][category_subject][model_name]:  # Itera sugli sweep per ciascun modello

                    # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                    sweep_id, combination_key = sweep_tuple
                    
                    
                    combination_key = f"{condition}_{data_type}_{category_subject}"
                    
                    # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                    # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                    # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione


                    # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                    if num_gpus > 1:

                        '''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper''' 
                        
                        # Assegna la GPU in modo rotazionale
                        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                    
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_imagery_3d_grid_multiband", count=200)
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=202)
                        
                        
                        # Passa alla prossima GPU per il prossimo sweep
                        gpu_counter = (gpu_counter + 1) % num_gpus

                    else:
                        
                        # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                        
                        
                        
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_imagery_3d_grid_multiband", count=200)
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project = f"{condition}_{data_type}_channels_freqs_{category_subject}", count=202)


                    # Crea la funzione wrapper per l'agent
                    '''COMMENTATO'''
                    #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)


                    # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                    '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                       ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''

                    print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")

                    '''COMMENTATO'''
                    #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)

                    print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")

                    torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

## Impostazione **Recupero DL Optimized Models** - EEG Spectrograms - **Electrodes x Frequencies (2D)**

### IMPLEMENTAZIONE DEI BEST MODELS DOPO W&B - EEG SPECTROGRAMS **+ GRADCAM FREQUENCY x CHANNELS (ALL SUBJECTS)**! 

In [1]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

In [2]:
path = '/home/stefano/Interrogait/all_datas/'

with open(f"{path}EEG_channels_names.pkl", "rb") as f:
    EEG_channels_names = pickle.load(f)

In [3]:
import os
file_size = os.stat(f"{path}EEG_channels_names.pkl").st_size
print(f"File size: {file_size} bytes")

File size: 352 bytes


In [4]:
len(EEG_channels_names)

61

##### **NUOVE MODIFICHE SPECIFICHE PER I DATI NON HYPER POST W&B CON GRADCAM**

Allora le modifche che ho ultimato quindi sono:

- **1)Creazione della classe GradCAM**


    **GRADCAM CLASS**

        import torch
        import torch.nn.functional as F
        import cv2
        import numpy as np
        import matplotlib.pyplot as plt

        class GradCAM:
            def __init__(self, model, target_layer):
                self.model = model
                self.target_layer = target_layer
                self.activations = None
                self.gradients = None
                # Registra hook per catturare attivazioni e gradienti
                self.target_layer.register_forward_hook(self.save_activation)
                self.target_layer.register_backward_hook(self.save_gradient)

            def save_activation(self, module, input, output):
                self.activations = output.detach()

            def save_gradient(self, module, grad_input, grad_output):
                self.gradients = grad_output[0].detach()


- **2)** Creazione della funzione per generare delle immagini associate alla GradCAM compution**

    
    **FUNCTION FOR CREATING GRAD-CAM MAPS & FIGURES ASSOCIATED TO GRADCAM COMPUTATION**

        import cv2
        import numpy as np
        import matplotlib.pyplot as plt
        import io

        def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):

            """
            Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
            calcola la GradCAM e costruisce una figura con:
              - Riga 1: Heatmap per classe 0 e classe 1.
              - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
            I titoli della figura vengono personalizzati con exp_cond, data_type, category_subject.
            """

            # Assumiamo che il modello sia CNN2D e che il layer target sia model.conv3
            target_layer = model.conv3
            gradcam = GradCAM(model, target_layer)

            # Dizionari per salvare il campione per ogni classe
            samples = {}      # Salveremo il sample input per ogni classe
            labels_found = {} # Per tenere traccia delle etichette già trovate

            # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                for i, label in enumerate(labels):
                    label_int = int(label.item())
                    if label_int not in labels_found:
                        samples[label_int] = inputs[i].unsqueeze(0)  # salva come tensore 4D
                        labels_found[label_int] = True
                    if 0 in labels_found and 1 in labels_found:
                        break
                if 0 in labels_found and 1 in labels_found:
                    break

            # Se non troviamo entrambi gli esempi, esci con un messaggio
            if 0 not in samples or 1 not in samples:
                print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
                return None

            # Per ciascun campione, calcola GradCAM
            cams = {}
            overlays = {}
            for cls in [0, 1]:
                sample_input = samples[cls]
                sample_input.requires_grad = True  # Abilita gradiente per il campione
                cam = gradcam.generate_cam(sample_input)
                cams[cls] = cam

                # Converti il sample in immagine numpy per la visualizzazione
                img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
                # Normalizza l'immagine in scala 0-255
                img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))
                # Applica la heatmap
                heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
                heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
                # Sovrapponi la heatmap all'immagine originale
                overlay = cv2.addWeighted(img_norm, 0.6, heatmap, 0.4, 0)
                overlays[cls] = overlay

            # Crea la figura con due righe e due colonne
            fig, axs = plt.subplots(2, 2, figsize=(12, 10))

            # Titolo per la prima riga
            title_row1 = f"Grad-CAM mapping of experimental condition {exp_cond}, EEG {data_type}, Subject {category_subject}"
            # Titolo per la seconda riga
            title_row2 = f"Grad-CAM mapping superimposition over EEG Spectrogram of experimental condition {exp_cond}, Subject {category_subject}"

            # Prima riga: solo le heatmap
            for j, cls in enumerate([0, 1]):
                axs[0, j].imshow(cv2.cvtColor(cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB))
                axs[0, j].set_title(f"Class {cls} Heatmap")
                axs[0, j].axis('off')
            axs[0, 0].set_ylabel(title_row1, fontsize=10)

            # Seconda riga: overlay della heatmap sullo spettrogramma originale
            for j, cls in enumerate([0, 1]):
                axs[1, j].imshow(overlays[cls])
                axs[1, j].set_title(f"Class {cls} Overlay")
                axs[1, j].axis('off')
            axs[1, 0].set_ylabel(title_row2, fontsize=10)

            # Ottimizza la disposizione della figura
            plt.tight_layout()

            # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            fig_image = buf.getvalue()
            buf.close()
            plt.close(fig)

            return fig_image


- **3) Modifica delle funzioni per il salvataggio delle immagini create tramite la GradCAM compution**

    **FUNCTIONS FOR GRADCAM COMPUTATION & SAVING**
    
    Questa modifica consente di creare ed adattare le path di salvataggio ANCHE delle immagini calcolate dalla classe customizzata di GradCAM, 
    delle mappe di attivazione prodotte dalle feature maps e della sovrapposizione delle stesse aree decisionali
    rilevanti per la migliore classificazione dei dati di esempio di una certa classe,
    a partire da un certo dataset composto da una certa combinazione di fattori
    (i.e., exp_cond, data_type, category_subject)


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

    **Funzione per determinare a quale subfolder appartiene la chiave**
    def get_subfolder_from_key(key, model_standardization):

        #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
        if '_familiar_th' in key:
            return 'th_fam'
        elif '_unfamiliar_th' in key:
            return 'th_unfam'
        elif '_familiar_pt' in key:
            return 'pt_fam'
        elif '_unfamiliar_pt' in key:
            return 'pt_unfam'
        else:
            return None


    from PIL import Image
    import io
    import pickle
    import os

    **Funzione per salvare i risultati**
    def save_performance_results(model_name, 
                                 my_train_results,
                                 my_test_results, 
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder,
                                 gradcam_image = None):
        """
        Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
        Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
        """

        # Identificazione del subfolder in base alla chiave
        subfolder = get_subfolder_from_key(key, model_standardization)

        # Debug: controllo sulla subfolder
        print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")

        if subfolder is None:
            print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
            return

        # Determinazione del tipo di dato direttamente dalla chiave
        if "spectrograms" in key:
            data_type_str = "spectrograms"
        else:
            print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
            return

        # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
        if model_standardization:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        else:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)

        # Verifica se la cartella di destinazione esiste, altrimenti creala
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        file_path = os.path.join(folder_path, file_name)

        # Creazione del dizionario con i risultati
        results_dict = {
            'my_train_results': my_train_results,
            'my_test_results': my_test_results
        }

        # Salvataggio del dizionario con i risultati
        try:
            with open(file_path, 'wb') as f:
                pickle.dump(results_dict, f)
            print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
        except Exception as e:
            print(f"❌Errore durante il salvataggio dei risultati: {e}")

        # Se è stata fornita l'immagine GradCAM, salvala come file PNG
        if gradcam_image is not None:
            if model_standardization:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
            else:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"

            gradcam_file_path = os.path.join(folder_path, gradcam_file_name)

            #try:
            #    with open(gradcam_file_path, "wb") as f_img:
            #        f_img.write(gradcam_image)
            #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")

            try:

                '''
                Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
                Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
                Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
                👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()

                Per maggior info leggi cella successiva!
                '''

                # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
                if isinstance(gradcam_image, io.BytesIO):
                    gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                    gradcam_image = Image.open(gradcam_image)

                print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
                # 🔄 Salvare l'immagine nel percorso specificato
                gradcam_image.save(gradcam_file_path, format = "PNG")

            except Exception as e:
                print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")


- **4) Integrazione nel loop di training e test dei punti 1), 2) e 3)**

    **INTEGRATION OF GRADCAM COMPUTATION IN THE TRAINING E FOR LOOP**

    Nel loop che esegue il training ed il testing, integrazione della parte di inizializzazione della classe custom di GradCAM, con cui si esegue 

    il calcolo delle mappe di attivazione e della sovrapposizione delle mappe di attivazione stesse sullo spettogramma originale, 
    riportate poi in due immagini distinte create nella stessa figura che vengono salvate correttamente nella stessa directory path. 

    Le due immagini dovrebbero rappresentare l'heatmap activation e la sovrapposizione della mappa di attivazione sullo spettogramma originale,
    relativo ad un esempio rappresentativo per ciascuna delle due classi possibili presenti nello stesso dataset correntemente iterato.

    Il loro contributo è di descrivere se la CNN2D abbia identificato delle (possibili) differenti aree decisionali delle feature maps 
    (e dunque dello spettrogramma) maggiormente utili ai fini della discriminazione delle due condizioni sperimentali inserite all'interno del dataset correntemente iterato.


        ** Dizionario per tracciare la standardizzazione usata per ogni combinazione di dati**
        ** Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)**

        models_info = {}

        ** Set per tenere traccia dei dataset già elaborati**
        processed_datasets = set()

        ** Set per tenere traccia delle combinazioni già elaborate**
        processed_models = set()

        ** Path delle performance dei modelli ottimizzati con weight and biases**
        ** Path per trovare le best performances di ogni modello per ogni combinazione dei dati**
        base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results"

        ** Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder**
        save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


        ** --- LOOP PRINCIPALE (con minime modifiche) ---**
        for key, (X_data, y_data) in data_dict.items():

            print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")

            if key in processed_datasets:
                print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
                continue

            processed_datasets.add(key)

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")

            for model_name in ["CNN2D", "BiLSTM", "Transformer"]:

                model_key = f"{model_name}_{key}"
                if model_key in processed_models:
                    print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
                    continue
                processed_models.add(model_key)

                print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")

                # Prova a caricare la configurazione e i pesi ottimali dal file .pkl

                '''
                load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
                parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate

                exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
                pesi del modello e i suoi iper-parametri

                Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
                '''

                config, best_weights = load_config_if_available(key, model_name, base_folder)

                if config is None:
                    raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")

                '''
                Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
                MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
                '''

                # Parsifica la chiave una volta sola e memorizza i valori
                exp_cond, data_type, category_subject = parse_combination_key(key)

                '''
                Dpodiché, 

                1) si carica i vari valori degli iper-parametri,
                2) si esegue la standardizzazione se servisse,
                3) prepara il modello per la divisione in train_loader etc.,
                4) si carica la configurazione dei pesi del modello, 
                5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 

                6) esegue il training e il test e poi

                7) si salva il tutto nella path corrispondente...

                '''

                '''
                PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
                DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''

                model_batch_size = config["batch_size"]
                model_n_epochs = config["n_epochs"]
                model_patience = config["patience"]
                model_lr = config["lr"]
                model_weight_decay = config["weight_decay"]
                model_standardization = config["standardization"]

                print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")

                # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
                models_info[model_key] = {"standardization": model_standardization}


                '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
                IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
                '''

                if model_standardization:
                    X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
                    print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
                else:
                    print(f"\033[1mNO Standardizzazione Dati!\033[0m")

                # Sposta il modello sulla GPU (se disponibile)
                device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


                # Preparazione dei dataloaders
                train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
                    X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)

                # Inizializzazione del modello
                if model_name == "CNN2D":
                    model = CNN2D(input_channels=3, num_classes=2)
                elif model_name == "BiLSTM":
                    model = ReadMEndYou(input_size= 3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
                elif model_name == "Transformer":
                    model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
                else:
                    raise ValueError(f"Modello {model_name} non riconosciuto.")

                # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
                if best_weights is not None:
                    try:
                        model.load_state_dict(best_weights)
                        print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
                    except Exception as e:
                        print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                        continue


                # Definizione del criterio di perdita
                criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)

                # Definizione dell'ottimizzatore con i parametri aggiornati
                optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)

                print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)

                print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_test_results = testing(my_train_results, test_loader, criterion)

                '''
                GRADCAM COMPUTATION PER IL MODELLO CNN2D

                La funzione compute_gradcam_figure estrae due campioni (uno per ogni classe) e crea una figura con le due righe richieste.

                Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
                'save_performance_results', in modo da essere salvato nella path corretta. 

                La funzione 'save_performance_results' è stata modificata 
                per gestire ANCHE questo nuovo input dell'immagine 

                (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
                seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:

                - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
                - tipologia di dato EEG prelevato (i.e., spectrograms) 
                - provenienza del dato stesso (i.e., familiar_th)
                )

                '''

                # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
                gradcam_image = None

                if model_name == "CNN2D":
                    gradcam_image = compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device)
                    if gradcam_image is not None:
                        print(f"GradCAM image computed successfully for {model_name}.")

                print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                save_performance_results(model_name,
                                         my_train_results,
                                         my_test_results,
                                         key,
                                         exp_cond,
                                         model_standardization,
                                         base_folder = save_path_folder,
                                         gradcam_image = gradcam_image)

##### **UTILS DATI NON HYPER**

In [5]:
import pickle
import numpy as np


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (50°-300° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "familiar" o "unfamiliar"
    - subject_type: str, "th" (terapisti) o "pt" (pazienti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/",
            "unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms_channels_frequencies/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

'''ATTENZIONE MODIFICATA FUNZIONE DI STANDARDIZZAZIONE'''
# Funzione per standardizzare i dati
# Con questa modifica eviti che std==0 produca NaN e i tuoi loss torneranno numeri sensati.
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    #aggiungo eps per evitare divisione per zero
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test


# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels=61, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    '''ATTENZIONE CAMBIATO QUI!'''
    #if model_type == "CNN2D":
    
    if model_type == "CNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "TopomapNet":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "CNN3D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "SeparableCNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor



'''
OLD VERSIONS BEFORE GRADCAM COMPUTATION ON CNN2D

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
    
     
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, exp_cond, model_standardization, base_folder):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
'''


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None


from PIL import Image
import io
import pickle
import os
     
# Funzione per salvare i risultati
def save_performance_results(model_name, 
                             my_train_results,
                             my_test_results, 
                             key,
                             exp_cond,
                             model_standardization,
                             base_folder,
                             gradcam_image = None):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
    
    # Se è stata fornita l'immagine GradCAM, salvala come file PNG
    if gradcam_image is not None:
        if model_standardization:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
        else:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"
        
        gradcam_file_path = os.path.join(folder_path, gradcam_file_name)
        
        #try:
        #    with open(gradcam_file_path, "wb") as f_img:
        #        f_img.write(gradcam_image)
        #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
        
        try:
            
            '''
            Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
            Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
            Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
            👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()
            
            Per maggior info leggi cella successiva!
            '''
            
            # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
            if isinstance(gradcam_image, io.BytesIO):
                gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                gradcam_image = Image.open(gradcam_image)
            
            '''
            Il messaggio di errore indica che il tuo oggetto gradcam_image è di tipo bytes e non ha il metodo save(), 
            che è tipico di un oggetto PIL. 
            
            Per risolvere questo, devi convertire i byte in un'immagine PIL. 
            Per farlo, controlla se gradcam_image sia un oggetto di tipo bytes e,
            in tal caso, usa io.BytesIO per creare un buffer da passare a Image.open(). 
            
            Inserisci questa conversione all'interno del blocco che salva l'immagine, così da assicurarti che,
            indipendentemente dal tipo, gradcam_image diventi un oggetto PIL e possa chiamare il metodo save().
            '''
            
            if isinstance(gradcam_image, bytes):
                gradcam_image = io.BytesIO(gradcam_image)
                gradcam_image.seek(0)
                gradcam_image = Image.open(gradcam_image)
            
            
            print(f"\n📸Immagine \033[1mGradCAM salvata\033[0m con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
            # 🔄 Salvare l'immagine nel percorso specificato
            gradcam_image.save(gradcam_file_path, format = "PNG")
            
        except Exception as e:
            print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")

##### **NUOVE UTILS DATI NON HYPER POST W&B**

###### **SUGGERIMENTI DI MODIFICA CHATGPT DELLE UTILS DATI NON HYPER POST W&B**

###### **IMPLEMENTAZIONE ADOTTATA (ONLY HYPERPARAMS)**

In [6]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''
import re 

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$", 
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
combination_key = "pt_resp_vs_shared_resp_spectrograms_familiar_th"
condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Data Type:", data_type)
print("Soggetto:", subject_key)

Condizione: pt_resp_vs_shared_resp
Data Type: spectrograms
Soggetto: familiar_th


In [7]:
'''
Verifica del file .pkl:
La funzione load_config_if_available cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. 

Se il file esiste, allora viene passata poi a load_model_config_and_weights, 
che carica il dizionario di partenza 
e da questo estrae i 2 sotto-dizionari 'config' e 'state_dict'.
'''

def load_config_if_available(dataset_key, model_name, base_path):
    """
    Data una chiave (es. "th_resp_vs_pt_resp_wavelet_familiar_th") e il nome del modello,
    cerca il file .pkl corrispondente e ritorna (config, state_dict).
    Se non esiste, restituisce (None, None).
    """
    try:
        exp_cond, data_type, category_subject = parse_combination_key(dataset_key)
        config, state_dict = load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path)
        print(f"✅ File .pkl trovato per \033[1m{model_name}\033[0m su \033[1m{dataset_key}\033[0m")
        
        return config, state_dict
    except Exception as e:
        print(f"⚠️ Nessun file .pkl per {model_name} su {dataset_key} - uso parametri di default. ({e})")
        return None, None

In [8]:
'''
Caricamento del file .pkl:
La funzione load_model_config_and_weights cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. Se il file esiste, vengono restituiti config e state_dict.
'''

# Funzione per caricare il file .pkl con la configurazione e i pesi ottimali
def load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path):
    """
    Costruisce il path usando:
        base_path / exp_cond / data_type / category_subject
    e il nome del file:
        {model_name}_{exp_cond}_{data_type}_{category_subject}.pkl
    Se il file esiste, lo carica e restituisce (config, state_dict).
    """
    
    file_name = f"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
    file_path = os.path.join(base_path, exp_cond, data_type, category_subject, file_name)
    
    if os.path.exists(file_path):
        print(f"🕵️‍♂️🔍Caricamento file .pkl: \033[1m{file_path}\033[0m")
        
        # Il file .pkl è stato salvato con torch.save() e contiene un dizionario con chiavi al suo interno che sono: "config" e "state_dict"
        with open(file_path, "rb") as f:
            data = torch.load(f)
        return data["config"], data["state_dict"]
    else:
        raise FileNotFoundError(f"File {file_path} non trovato.")

###### **IMPLEMENTAZIONE ADOTTATA (PARAMS ED HYPERPARAMS)**

In [ ]:
#1) Prima si cerca il modello migliore con queste "scan_folder_for_best_model" (e "clean_config")


import os
import torch
import json

def clean_config(config):
    """Rimuove la chiave '_wandb' dal dizionario di configurazione, se presente."""
    if "_wandb" in config:
        del config["_wandb"]
    return config

def scan_folder_for_best_model(folder_path):
    
    """
    Scansiona la cartella folder_path per file .pkl, estrae il valore di 'max_val_acc'
    da ciascun file e restituisce (best_file, best_val) dove best_file è il file con 
    il valore più alto di max_val_acc.
    """
    best_val = -float('inf')
    best_file = None
    
    # Imposta la device per la GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for file in os.listdir(folder_path):
        if file.endswith(".pkl"):
            file_path = os.path.join(folder_path, file)
            
            try:
            #    data = torch.load(file_path, map_location = torch.device('cpu'))
            
                with torch.serialization.safe_globals({"CNN2D": CNN2D}):
                    data = torch.load(file_path, map_location = device, weights_only=False)

                # Pulizia opzionale della configurazione
                data['config'] = clean_config(data['config'])
                
                current_val = data.get("max_val_acc", -float('inf'))
                
                print(f"File {file}: max_val_acc = {current_val}")
                
                if current_val > best_val:
                    best_val = current_val
                    best_file = file_path
                    
            except Exception as e:
                print(f"Errore nel caricamento di {file}: {e}")
    
    if best_file:
        print(f"\nIl file con la migliore accuratezza (max_val_acc) è \n\033[1m{best_file}\033[0m con un valore di \033[1m{best_val}\033[0m.")
    return best_file #questa è una stringa!

# Esempio di utilizzo:
#folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies/th_resp_vs_pt_resp/spectrograms/familiar_th"
#best_file, best_val = scan_folder_for_best_model(folder)
#print(f"\nIl file con il miglior modello è: {best_file} con max_val_acc = {best_val}")

In [ ]:
#2) Si deve passare a 'load_config_if_available' "best_file" trovato da 'scan_folder_for_best_model' 
#e si deve estrapolare dalla sua stringa il suffisso che è '_v_*' dove * è un stringa di un numero variabile (da 1 a 15)

import re

'''
# Funzione per estrarre il suffisso
def extract_suffix(string):
    match = re.search(r'_v_\d+', string)  # Cerca "_v_" seguito da uno o più numeri
    if match:
        return match.group(0)  # Restituisce il suffisso trovato
    return None  # Se non trovato, restituisce None

# Stringa di esempio
#s1 = 'CNN2D_th_resp_vs_pt_resp_spectrograms_familiar_th_v_1'
#s2 = 'CNN2D_th_resp_vs_pt_resp_spectrograms_familiar_th_v_10'

# Test
#print(extract_suffix(s1))  # Output: _v_1
#print(extract_suffix(s2))  # Output: _v_10
'''


def extract_suffix(filename):
    match = re.search(r'_v_\d+', filename)
    if match:
        suffix = match.group(0)
        print(f"\n\033[1mSuffix estratto\033[0m da \n{filename}: \033[1m{suffix}\033[0m\n")
        return suffix
    print(f"Nessun suffisso trovato in {filename}")
    return ""


In [ ]:
#3) Dentro a  'load_config_if_available", un volta estratto anche il suffisso da "best_file", 

#4) A quel punto, 'load_config_if_available' si fa PRIMA il parsing delle key corrente 
# (richiamando 'parse_combination_key') 


'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''
import re 

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$", 
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

#'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
#combination_key = "pt_resp_vs_shared_resp_spectrograms_familiar_th"
#condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

#print("Condizione:", condition_experiment)
#print("Data Type:", data_type)
#print("Soggetto:", subject_key)


#5) A quel punto, si chiama 'load_model_config_and_weights', 
# la quale si porta appresso ANCHE il 'suffix' tra i suoi argomenti di input per accedere al file .pkl corrispondente 


#6)Vengono ritornati in output da 'load_model_config_and_weights':

#a) optimized_model
#b)state_dict
#c) model_config
#d) training_config

#che saranno che sono le variabili, il cui valore stringa, viene usato per accedere alle chiavi del dizionario dentro il file .pkl per prelevarsi, rispettivamente

#- la versione del modello migliore corrente  
#- pesi e bias della versione migliore del modello corrente della specifica combinazione di dati (i..e, state_dict ) 
#- valori dei parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, model_config) 
#- valore degli iper-parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, training_config )



def load_config_if_available(dataset_key, model_name, base_path, best_file):
    
    """
    Si estrapola il suffisso del file della versione del modello migliore per la specifica combinazione di dati
    che è variabile come '_v*' dove * è un unità od un indice numerico (tra 1 e 15)
    
    
    Data una chiave (es. "th_resp_vs_pt_resp_wavelet_familiar_th") e il nome del modello,
    1) cerca il file .pkl corrispondente (con suffisso opzionale)
    2) e ritorna (config, state_dict).
    Se non esiste, restituisce (None, None).
    """
    
    suffix = extract_suffix(best_file)
    
    try:
        exp_cond, data_type, category_subject = parse_combination_key(dataset_key)
        
        # Carica state_dict, model_config e training_config usando il suffisso
        #optimized_model, state_dict, model_config, training_config = load_model_config_and_weights(
        #    exp_cond, data_type, category_subject, model_name, base_path, suffix = suffix
        #)
        
        optimized_model, model_config, training_config = load_model_config_and_weights(
            exp_cond, data_type, category_subject, model_name, base_path, suffix = suffix
        )
        
        print(f"✅ File .pkl trovato per {model_name} su {dataset_key} con suffisso: {suffix}")
    
        # Ritorna un dizionario con, al suo interno, i dati caricati
        #return {"optimized_model": optimized_model, "state_dict": state_dict, "model_config": model_config, "config": training_config}
        return {"optimized_model": optimized_model, "model_config": model_config, "config": training_config}
        
    except Exception as e:
        print(f"⚠️ Nessun file .pkl per {model_name} su {dataset_key} con suffisso '{suffix}' - uso parametri di default. ({e})")
        return None, None, None    

    
def load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path, suffix = ""):
    
    """
    Costruisce il path usando:
    base_path / exp_cond / data_type / category_subject
    e il nome del file:
    {model_name}_{exp_cond}_{data_type}_{category_subject}{suffix}.pkl
    Se il file esiste, lo carica e restituisce (config, state_dict).
    """

    file_name = f"{model_name}_{exp_cond}_{data_type}_{category_subject}{suffix}.pkl"
    file_path = os.path.join(base_path, exp_cond, data_type, category_subject, file_name)

    if os.path.exists(file_path):
        print(f"\n🕵️‍♂️🔍 Caricamento file .pkl: \033[1m{file_path}\033[0m")

        #with open(file_path, "rb") as f:
        #    data = torch.load(f)
        
        '''
        Questa parte di codice serve per caricare un modello salvato precedentemente in un file,
        utilizzando la funzionalità di serializzazione sicura di PyTorch. 
        
        Vediamo cosa succede in dettaglio:

        Contesto sicuro con torch.serialization.safe_globals:

            La funzione safe_globals consente di caricare il file di modello in un contesto controllato. 
            Questo è particolarmente utile se il modello salvato fa riferimento a classi o funzioni che non sono definite nel contesto corrente, 
            evitando problemi di sicurezza o errori di caricamento.

            Nel tuo caso, il contesto sicuro assicura che la classe CNN2D, che è il modello, sia disponibile durante il caricamento del file. 
            Senza questa precauzione, se la classe CNN2D non fosse definita, PyTorch non saprebbe come ricostruirla dal file salvato.

        Caricamento del modello con torch.load:

            torch.load(file_path, map_location=torch.device('cpu')) carica il modello salvato dal percorso specificato (file_path), 
            assicurandosi che venga mappato sulla CPU 
            (INDIPENDENTEMENTE da dove sia stato salvato originariamente, se su GPU, ad esempio,
            il parametro map_location si assicura che il modello venga caricato sulla CPU).

            Quindi, questo codice che hai fornito permette di 
            
            1) caricare un modello SALVATO, che è la versione "modello migliore" ,che vuoi confrontare successivamente con 
            
            2) il modello che sarà RICOSTRUITO dinamicamente dalla funzione load_best_cnn2d  
            
            (presumibilmente una funzione che carica o costruisce il modello da zero o da alcuni parametri).
        
        '''
        # Usa il contesto safe_globals per permettere il caricamento della classe CNN2D
        with torch.serialization.safe_globals({"CNN2D": CNN2D}):
            data = torch.load(file_path, map_location=torch.device('cpu'), weights_only = False)    
    
        optimized_model = data['model']
        #state_dict = data["state_dict"]
        model_config = data["model_config"]
        training_config = data["config"]
            
        #return optimized_model, state_dict, model_config, training_config
        return optimized_model, model_config, training_config
    else:
        #raise FileNotFoundError(f"File {file_path} non trovato.")
        #raise ValueError(f"⚠️ FileNotFoundError(f"File {file_path} non trovato.")
        raise ValueError(f"⚠️ FileNotFoundError: \nFile {file_path} non trovato.")

#Alla fine, questi 3 valori vengono ritornati da 'load_model_config_and_weights' come nuove variabili
#(best_state_dict, best_model_config, best_training_config) e....

In [ ]:
######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### 

#NEL FRATTEMPO, si chiamerà anche il costruttore dinamico del modello CNN2D ---> VEDI CELLA SOTTO 

                                        #MODELLI CNN2D, BILSTM e TRANSFORMER


######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### ######### 


In [ ]:
#Quindi a quel punto, come dicevo si forniscono in ingresso questi valori

#- best_state_dict : pesi e bias della versione migliore del modello corrente della specifica combinazione di dati (i..e, state_dict ) 
#- best_model_config: valori dei parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, model_config) 
#- best_training_config; valore degli iper-parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, training_config )


#def load_best_cnn2d(best_model, best_state_dict, best_model_config, best_training_config):

def load_best_cnn2d(best_model, best_model_config, best_training_config):

    """
    Si carica i 4 dizionari che contengono
    
    - la versione migliore del modello ottimizzato corrente della specifica combinazione di dati (i..e, optimized_model ) 
    - pesi e bias della versione migliore del modello corrente della specifica combinazione di dati (i..e, state_dict ) 
    - valori dei parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, model_config) 
    - valore degli iper-parametri della versione migliore del modello corrente della specifica combinazione di dati (i..e, training_config )
    
    Args:
        model_path (dict): I 3 dizionari estratti dal file .pkl.
        
    Returns:
        model (CNN2D): Modello con i suoi pesi caricati.
        training_config (dict): Iperparametri usati nel training (per quella versione di modello migliore di quella combinazione di dati)
    """
       
    #Qui, si inizializza la versione del modello migliore corrente, richiamando il dizionario che contiene i valori dei parametri interni del modello stesso

    #model = build_cnn2d(best_model_config)
    
    model = best_model
    
    #Qui, facciamo un ulteriore check, per verificare che effettivamente, il modello ricostruito dinamicamente corrisponda proprio allo stesso modello
    #che era stato salvato in precedenza
    
    '''
    Sì, hai ragione nel voler fare un controllo per verificare che la struttura del modello appena ricostruito sia la stessa di quella salvata nel best_model. Esistono diversi modi per fare questo controllo, ma dobbiamo tenere presente che la struttura del modello, se ricostruita correttamente tramite build_cnn2d(best_model_config), dovrebbe essere identica a quella del modello salvato, e i parametri di configurazione dovrebbero essere quelli corretti.

    Per verificare che la struttura sia la stessa, puoi eseguire due controlli:

    1) Controllo della struttura (architettura del modello): 
        
        Questo si può fare confrontando l'architettura del modello appena ricostruito (model) con quella del modello salvato (best_model)
        In pratica, possiamo confrontare la lista dei layer o gli attributi del modello.
        
        Poiché entrambi dovrebbero essere della stessa classe e configurazione, un confronto diretto potrebbe rivelarsi utile.
        
        # Confronta l'architettura del modello ricostruito con quella del modello salvato
        if model.__class__ != best_model.__class__:
            raise ValueError("Le architetture dei modelli non sono uguali!")


    2) Controllo dei parametri: 
        Confrontare i parametri del modello, come i pesi e le configurazioni (iper-parametri). 
        Questo controllo assicura che il modello ricostruito abbia esattamente gli stessi valori nei parametri di configurazione e nei pesi.
        
    '''
    
    #Questa soluzione ti permette di fare un check completo sulla corrispondenza tra 
    #la struttura del modello,
    #i parametri di configurazione e i pesi,
    #assicurandoti che il modello ricostruito sia identico a quello salvato
    
    # Verifica che l'architettura sia la stessa, tra modello ricostruito al momento correte (model) e modellom salvato (best_model)
    #if model.__class__ != best_model.__class__:
    #    raise ValueError("Le architetture dei modelli non sono uguali!")
    
    
    # Verifica che i pesi siano gli stessi, tra modello ricostruito al momento correte (model) e modellom salvato (best_model)
    #if not all(torch.equal(model.state_dict()[key], best_state_dict[key]) for key in model.state_dict()):
    #    raise ValueError("I pesi del modello ricostruito non corrispondono a quelli salvati!")
        
        
    '''
    Dopodiché, una volta che si è caricato i pesi e bias di quella versione migliore del modello, ‘load_best_cnn2d’ si tira fuori in output
    
    1) La versione MIGLIORE del modello ESTRATTO CORRENTEMENTE da 'load_best_cnn2d con già configurato con 
        a) i valori corretti dei suoi parametri interni
        b) il caricamento dei suoi pesi e bias
    
    2) Gli iper-parametri associati al modello (che dovranno esser caricati successivamente nella fase di training del modello stesso)
    '''
    
    #data.keys()
    #dict_keys(['state_dict', 'config', 'model_config', 'max_val_acc', 'best_epoch'])
    
    #data['model_config'].keys()
    #dict_keys(['conv_channels', 'kernel_sizes', 'strides', 'paddings', 'pooling_type', 'dropout_rate', 'activations'])
    
    #data['config'].keys()
    #dict_keys(['_wandb', 'batch_size', 'lr', 'model_name', 'n_epochs', 'patience', 'standardization', 'weight_decay'])
    
    #if best_state_dict and best_model_config and best_training_config is not None:
    #    try:
    #        model.load_state_dict(best_state_dict)
    #        print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
    
    if best_model and best_model_config and best_training_config is not None:
        #DESCRIZIONE DEI PARAMETRI DELLA VERSIONE DEL MIGLIORE MODELLO SELEZIONATO
        print(f"\nParametri Modello \033[1m{model_name}\033[0m:")
        print(f"conv_channels per \033[1m{best_model_config['conv_channels']}\033[0m")
        print(f"kernel_sizes = \033[1m{best_model_config['kernel_sizes']}\033[0m")
        print(f"strides = \033[1m{best_model_config['strides']}\033[0m")
        print(f"paddings = \033[1m{best_model_config['paddings']}\033[0m")
        print(f"pooling_type = \033[1m{best_model_config['pooling_type']}\033[0m")
        print(f"dropout_rate = \033[1m{best_model_config['dropout_rate']}\033[0m")
        print(f"activations = \033[1m{best_model_config['activations']}\033[0m")
        
        #DESCRIZIONE DEGLI IPER-PARAMETRI DELLA VERSIONE DEL MIGLIORE MODELLO SELEZIONATO
        print(f"\nIperparametri Modello \033[1m{model_name}\033[0m:") 
        print(f"batch size = \033[1m{best_training_config['batch_size']}\033[0m")
        print(f"patience = \033[1m{best_training_config['patience']}\033[0m")
        print(f"learning rate = \033[1m{best_training_config['lr']}\033[0m")
        print(f"weight decay = \033[1m{best_training_config['weight_decay']}\033[0m")
        print(f"standardization = \033[1m{best_training_config['standardization']}\033[0m")
           
    else: 
        raise ValueError(f"⚠️ Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
            
    return model

##### **MODELLI CNN2D, BiLSTM e Transformer (PARAMS E HYPEPARAMS)**

In [ ]:
'''

QUELLA ORIGINALE DI PARTENZA
class CNN2D(nn.Module):
    
    def __init__(self, input_channels, num_classes):
        
        super(CNN2D, self).__init__()
        
        # Ipotizziamo kernel 3x3 con padding per mantenere le dimensioni (puoi adattare a tuo piacimento)
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(32, 48, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))
        
        # Utilizzo LazyLinear per evitare di calcolare manualmente la dimensione piatta finale
        self.fc1 = nn.LazyLinear(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = F.elu(x)
        x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh

        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x
    
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- -----  ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
'''

import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN2D(nn.Module):
    def __init__(self, input_channels = 61, num_classes=2,
                 # Parametri fissi:
                 conv_channels = [16, 32, 48],
                 kernel_sizes = [(2,2), (2,2), (2,2)],
                 strides = [[(1,1), (1,1), (1,1)], [(2,2), (2,2), (2,2)]],
                 paddings = [[1, 1, 1], [2, 2, 2]],
                 max_layers = 3,
                 
                 # Parametri dinamici (da sweep config):
                 activations = ["elu", "elu", "tanh"],
                 dropout_rate = 0.5,
                 pooling_type = "max"):
        """
        Costruttore della rete CNN2D con configurazione parzialmente dinamica.
        
        Parametri fissi (definiti internamente):
          - conv_channels: [16, 32, 48]
          - kernel_sizes: [(2,2), (2,2), (2,2)]
          - max_layers: 3
        
        Parametri dinamici (da ottimizzare tramite sweep config):
          - strides: [[(1,1), (1,1), (1,1)], [(2,2), (2,2), (2,2)]],
          - paddings: [[1, 1, 1], [2, 2, 2]],
          - activations: lista di funzioni di attivazione da usare per ogni layer
          - dropout_rate: valore del dropout
          - pooling_type: "max" oppure "avg"
        """
        super(CNN2D, self).__init__()
        
        # Salva i parametri fissi
    
        '''
        Anche se conv_channels e kernel_sizes hanno valori fissi nello sweep config,
        assegnarli esplicitamente all'istanza del modello permette di evitare ambiguità
        quando la rete viene inizializzata con configurazioni diverse.
        '''
        
        self.conv_channels = conv_channels
        self.kernel_sizes = kernel_sizes

        # Salva i parametri dinamici
        
        self.activations = activations
        self.dropout_rate = dropout_rate
        self.pooling_type = pooling_type
        
        self.strides = strides  # Aggiungi il salvataggio del parametro 'strides'
        self.paddings = paddings  # Aggiungi il salvataggio del parametro 'paddings'
        
        self.layers = nn.ModuleList()
        in_channels = input_channels
        
        # Costruisci max_layers blocchi convoluzionali usando parametri fissi
        for i in range(max_layers):
            out_channels = conv_channels[i]
            ks = kernel_sizes[i]
            stride = strides[i]
            padding = paddings[i]
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=ks, stride=stride, padding=padding)
            bn = nn.BatchNorm2d(out_channels)
            
            '''
            In questo modo, il kernel_size per ogni layer di pooling verrà scelto dinamicamente, 
            in base al valore presente nella lista kernel_sizes,
            proprio come per i kernel_size delle convoluzioni.
            
            Perché questa modifica?
            Flessibilità: Ora puoi configurare il kernel_size per ogni livello di pooling in modo indipendente, 
            come nel caso delle convoluzioni, permettendo un controllo maggiore sull'architettura del modello.
            
            Configurabilità: Puoi passare liste diverse di kernel_sizes per le convoluzioni e il pooling,
            e ognuna sarà applicata correttamente al suo livello.
            
            Con questa modifica, il comportamento del pooling sarà molto più simile al comportamento delle convoluzioni, 
            ed entrambe le operazioni potranno essere configurate dinamicamente tramite i parametri di input.
            '''
            
            # Selezione dinamica del pooling
            if pooling_type.lower() == "avg":
                pool = nn.AvgPool2d(kernel_size=ks)  # Usa lo stesso kernel_size per pooling
            else:
                pool = nn.MaxPool2d(kernel_size=ks)
            
            block = nn.Sequential(conv, bn, pool)
            self.layers.append(block)
            in_channels = out_channels
        
        self.fc1 = nn.LazyLinear(8)  # LazyLinear deduce automaticamente la dimensione d'ingresso
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.LazyLinear(num_classes)
    
    def forward(self, x):
        
        # Adattamento dell'input: 
        
        # Original Shape: (batch, frequenze, canali)
        
        # Novel Shape: (batch, canali, frequenze, 1)
        x = x.permute(0, 2, 1)  # (batch, canali, frequenze)
        x = x.unsqueeze(3)      # (batch, canali, frequenze, 1)
        
        for i, block in enumerate(self.layers):
            # Il blocco contiene conv, bn e pooling.
            x = block[0](x)      # Convoluzione
            x = block[1](x)      # Batch Normalization
            
            # Applica la funzione di attivazione mappata dalla stringa
            act_fn = self.get_activation(self.activations[i])
            x = act_fn(x)
            x = block[2](x)      # Pooling (Max o Average in base a pooling_type)
        
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.elu(x)  # Qui potresti parametrizzare anche questa attivazione se lo desideri
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    #def get_activation(self, act_str):
        """
        Mappa la stringa dell'attivazione alla funzione corrispondente.
        Le chiavi sono in minuscolo per garantire la corrispondenza (es. "relu", "elu", "selu", "tanh").
        """
   #     mapping = {
   #         "relu": F.relu,
   #         "elu": F.elu,
   #         "selu": F.selu,
   #         "tanh": torch.tanh
   #     }
   #     return mapping.get(act_str.lower(), F.elu)

    
    '''
    La nuova versione della funzione get_activation è più robusta perché gestisce in modo chiaro e diretto le stringhe 
    senza ambiguità legate alla capitalizzazione. 
    
    Inoltre, sollevare un'eccezione (ValueError) in caso di attivazione sconosciuta è una scelta efficace
    per intercettare errori e mantenere il codice più sicuro.
    '''
    
    def get_activation(self, activation_name):
        if activation_name == 'relu':
            return F.relu
        elif activation_name == 'elu':
            return F.elu
        elif activation_name == 'selu':
            return F.selu
        elif activation_name == 'tanh':
            return torch.tanh
        else:
            raise ValueError("Unknown activation function: " + activation_name)


In [ ]:
'''
È buona norma definire una funzione factory separata che, 
a partire dalla configurazione (ottenuta ad es. da wandb.config),
istanzia la rete.

Questa funzione può essere definita in un file di utilità o all’inizio del file di training e poi richiamata nel loop di training.

Dove metterla?
La funzione può essere definita in un modulo (ad es. model_factory.py) oppure all’inizio del file in cui gestisci il training. 

Nel tuo loop di training, dopo aver ottenuto config = wandb.config, puoi usarla così:


if config.model_name == "CNN2D":
    model = build_cnn2d(config)
    print(f"\nInizializzazione Modello CNN2D con configurazione: {dict(config)}")



#ATTENZIONE CHE QUI NELLA FUNZIONE DI RICHIAMO, DEVO IMPORRE INPUT_CHANNELS A 61 E NUM_CHANNELS = 2 

def build_cnn2d(config):
    """
    Costruisce una CNN2D usando i parametri da config.
    Assumiamo che config contenga:
       - activations, dropout_rate, pooling_type (dinamici)
    Gli altri parametri sono fissi.
    """
    
    return CNN2D(
        input_channels = 61,
        num_classes = 2,
        conv_channels = [16, 32, 48],
        kernel_sizes = [(2,2), (2,2), (2,2)],
        strides = config.strides, 
        paddings = config.strides,
        max_layers = 3,
        activations=config.activations,
        dropout_rate=config.dropout_rate,
        pooling_type=config.pooling_type
    )
'''


'''
Sì, conviene rendere anche conv_channels e kernel_sizes dinamici in build_cnn2d se vuoi garantire che 
tutti i parametri chiave della rete siano tracciati in modo chiaro dallo sweep_config. Questo aiuta per diversi motivi:

Maggiore trasparenza – Tutti i parametri vengono centralizzati in config, semplificando il debugging e il logging.

Flessibilità futura – Anche se ora sono fissi, in futuro potresti voler sperimentare diverse architetture CNN senza dover modificare la funzione.

Coerenza con il resto del codice – Stai già passando strides, activations, dropout_rate e pooling_type da config, 
quindi è logico che anche conv_channels e kernel_sizes seguano lo stesso principio.

Quindi, la nuova versione della funzione è più modulare e allineata con la gestione dinamica dei parametri.

In questo modo, tutto è centralizzato in config, rendendo il codice più pulito e scalabile! 🚀



def build_cnn2d(config):
    """
    Costruisce una CNN2D usando i parametri da config.
    Assumiamo che config contenga:
       - activations, dropout_rate, pooling_type (dinamici)
    Gli altri parametri sono fissi.
    """
    
    return CNN2D(
        input_channels = 61,
        num_classes = 2,
        conv_channels = config.conv_channels,
        kernel_sizes = config.kernel_sizes,
        strides = config.strides, 
        paddings = config.strides,
        max_layers = 3,
        activations=config.activations,
        dropout_rate=config.dropout_rate,
        pooling_type=config.pooling_type
    )
    
'''

#7) A quel punto, best_state_dict, best_model_config e best_training_config vengono forniti in ingresso a ‘load_best_cnn2d’, 
    #che al suo interno richiamerà la funzione per instanziare il modello che è ‘build_cnn2d’

    
# Funzione factory per creare il modello CNN2D con parametri dinamici migliori tra i 15 modelli della relativa path!

'''
def build_cnn2d(best_model_config):
    
    """
    Costruisce una CNN2D usando i parametri da config.
    Assumiamo che config contenga:
       - activations, dropout_rate, pooling_type (dinamici)
    Gli altri parametri sono fissi.
    """
    
    return CNN2D(
        input_channels = 61,
        num_classes = 2,
        conv_channels = [16, 32, 48],
        kernel_sizes =[(2,2), (2,2), (2,2)],
        strides = best_model_config['strides'],
        paddings = best_model_config['paddings'],
        max_layers = 3,
        activations = best_model_config['activations'],
        dropout_rate = best_model_config['dropout_rate'],
        pooling_type = best_model_config['pooling_type']
    )
'''


'''

Passando conv_channels e kernel_sizes come parametri dinamici,
puoi adattare meglio la funzione build_cnn2d per supportare una maggiore flessibilità con i parametri definiti nel tuo best_model_config.
La nuova versione della funzione ti permetterà di personalizzare più facilmente il modello, 
modificando anche la configurazione delle convoluzioni (numero di canali e dimensioni dei kernel), oltre agli altri parametri che restano fissi.


'''

def build_cnn2d(best_model_config):
    
    """
    Costruisce una CNN2D usando i parametri da config.
    Assumiamo che config contenga:
       - activations, dropout_rate, pooling_type (dinamici)
       - conv_channels, kernel_sizes, strides, paddings (ora configurabili dinamicamente)
    """
    
    return CNN2D(
        input_channels = 61,
        num_classes = 2,
        conv_channels = best_model_config['conv_channels'],  # Fissato dinamicamente
        kernel_sizes = best_model_config['kernel_sizes'],  # Fissato dinamicamente
        strides = best_model_config['strides'],
        paddings = best_model_config['paddings'],
        max_layers = 3,
        activations = best_model_config['activations'],
        dropout_rate = best_model_config['dropout_rate'],
        pooling_type = best_model_config['pooling_type']
    )


#Dove ‘strides’, ‘paddings’, ‘activations’, ‘dropout_rate’, ‘pooling_type’ sono quelli estratti 
#dalla versione migliore del modello corrente trovato nella path della specifica combinazione di dati


##### **MODELLO CNN2D o CNN3D_FC_LST o CNN_2D Sep (ONLY HYPERPARAMS)**

In [9]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI 2D FREQUENCY-CHANNELS (LUGLIO 2025!)



Ora però, ragionandoci, potrei inserire dei valori da cui pescare, 

durante l'ottimizzazione degli iper-parametri della mia rete, che si riferiscono 

1) a valori di alcuni parametri generale dell'apprendimento delle reti
2) a valori dei parametri architetturali di ciascuna delle mie singole reti neurali testate



                                                                ***CNN2D NEW*** 

1) All'interno di ogni layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

a) il numero di output channels (ossia 16 impostato di default qui sotto, ma che potrebbe variare da 16 a 32 con step di 4 
come grandezza della feature map sostanzialmente

b) la grandezza del kernel size (tra 2 e 8 con step di 2)
c) la grandezza dello stride (metti solo valori tra 1 e 2) 


2) Per il layer di batch normalisation del relativo layer convolutivo (https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#batchnorm1d

deve avere il valore del numero di features di quel layer di batch normalisation
(che deve corrispondere come valore a quello dell'output channels del layer convolutivo che lo precede sostanzialmente) 


3) Al layer di pooling del relativo strato della della CNN1D, far variare la scelta tra

a) max pooling ed average pooling 

b) Il valore del kernel_size del layer di max od average pooling (a seconda di quello che viene scelto tra i due), 
che può variare tra 1 e 2 

4) Al solo primo layer fully connected della CNN1D, far variare la scelta del suo valore 
(che nella mia rete sarebbe "self.fc1 = nn.LazyLinear(8)") in questo set di valori, ossia tra i valori 8,10,12,14,16

5) Il valore del dropout layer (con valori tra  0.0 e 0.5) 


6) Il valore della possibile funzione di attivazione tra 3 (relu, selu ed elu)

 a) per gli strati convolutivi (3) +
 b) per il primo fully connected layer (FC1) (prendendone una a caso tra quelle 3 possibili



TABELLA FINALE RIASSUNTIVA - CNN1D 


| Iper-parametro                     | Descrizione                                             | Valori possibili                 |
| ---------------------------------- | ------------------------------------------------------- | -------------------------------- |
| `conv_out_channels`                | Numero di feature-map di base                           | `[16, 20, 24, 28, 32]`           |
| `conv_k1`, `conv_k2`, `conv_k3`    | Kernel size rispettivamente per i 3 blocchi convolutivi | `[2, 4, 6, 8]`                   |
| `conv_s1`, `conv_s2`, `conv_s3`    | Stride rispettivamente per i 3 blocchi convolutivi      | `[1, 2]`                         |
| `pool_type`                        | Tipo di pooling                                         | `["max","avg"]`                  |
| `pool_p1`, `pool_p2`, `pool_p3`    | Kernel size rispettivamente per i 3 blocchi di pooling  | `[1, 2]`                         |
| `fc1_units`                        | Numero di unità nel primo fully-connected               | `[8, 10, 12, 14, 16]`            |
| `cnn_act1`, `cnn_act2`, `cnn_act3` | Funzione di attivazione per ciascun blocco (layer1,2,3) | `["relu","selu","elu"]`          |
| **+ comune**                       | `dropout`                                               | `[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]` |



'''

'''




                                                                ***OLD CNN2D***


Una CNN2D si aspetta input in forma (batch, frequenze, canali). 
Nel tuo caso, puoi interpretare l’"altezza" come i bin di frequenza (45)
e la "larghezza" come i canali (61)

Quindi, la tua CNN2D lavorerebbe direttamente con:
Shape: (batch, frequenze, canali)



class CNN2D(nn.Module):
    
    def __init__(self, input_channels, num_classes):
        
        super(CNN2D, self).__init__()
        
        # Ipotizziamo kernel 3x3 con padding per mantenere le dimensioni (puoi adattare a tuo piacimento)
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(32, 48, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))
        
        # Utilizzo LazyLinear per evitare di calcolare manualmente la dimensione piatta finale
        self.fc1 = nn.LazyLinear(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)

        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = F.elu(x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = F.elu(x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = F.elu(x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.elu(x)
       

        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x
        
'''




'''CNN2D CON LE FUNZIONI DI ATTIVAZIONI DINAMICHE (LAYER CONVOLUTIVI + FC1) E DINAMISMO DEI KERNEL SIZE DI CONV e POOL LAYER e STRIDE



Scusami non ho capito la differenza tra la mia e la tua, che la mia tratta le convoluzione per ogni canale separatamente
e quindi vede le features per ogni canale mentre la tua vede le relazioni in frequenza tra i canali?

mi fai capire meglio?


ChatGPT ha detto:

Ottima domanda, e sì: sei sulla strada giusta! Vediamo con precisione la differenza concettuale tra le due versioni, perché hanno implicazioni molto diverse sul tipo di informazione che la rete convoluzionale impara.

🧠 Contesto: input (batch, freq, channels) → (B, 45, 61)
✅ Versione A: .unsqueeze(1) → input = (B, 1, 45, 61)

x = x.unsqueeze(1)
# Output: (batch_size, 1, 45, 61)

✅ Come la interpreta la CNN:
Tratta tutta la matrice frequenze × canali come un'immagine 2D.

Applica filtri convoluzionali 2D che coprono sia le frequenze che i canali contemporaneamente.

La convoluzione può imparare pattern locali e globali che coinvolgono più canali EEG e bande di frequenza assieme.

✅ Esempio di pattern che può catturare:
"La banda theta (4–8 Hz) è più attiva nei canali posteriori rispetto a quelli frontali"

"Un pattern a L tra canali temporali e frequenze gamma"

➕ Vantaggi:
Cattura interazioni tra frequenze e canali.

Molto potente per pattern spatial-temporal-frequenziali complessi.

➖ Svantaggi:
Più complesso da interpretare.

Può "mescolare troppo" se le relazioni tra canali non sono forti.





✅ Versione B: .permute(0, 2, 1).unsqueeze(3) → input = (B, 61, 45, 1)


x = x.permute(0, 2, 1).unsqueeze(3)
# Output: (batch_size, 61, 45, 1)

✅ Come la interpreta la CNN:
Ogni canale EEG (dei 61) è trattato come un’immagine verticale di 45 pixel (cioè le frequenze).

**Ogni filtro convoluzionale lavora indipendentemente su ciascun canale EEG.

È come applicare 61 CNN monodimensionali parallele sulle frequenze di ogni canale.

✅ Esempio di pattern che può catturare:
“Nel canale Fz, la banda alpha ha un picco”

“Nel canale Pz, la potenza decresce linearmente con la frequenza”

➕ Vantaggi:
Cattura pattern locali per canale (molto utile se ogni canale è considerato indipendente).

Meno soggetto a overfitting se i dati sono limitati.

➖ Svantaggi:
Non cattura le relazioni tra canali.

Meno potente se la distribuzione spaziale tra canali è importante (es. P300, N170, ecc.).

🔍 Riassunto Visuale

Variante	        Input Shape	      Tratta i canali come	       Cattura relazioni tra canali?	    Cattura pattern cross-frequenza?
.unsqueeze(1)	    (B, 1, 45, 61)	    Griglia 2D	                               ✅ Sì	                             ✅ Sì
.permute...	        (B, 61, 45, 1)	   Canali separati	                           ❌ No	               ✅ Solo all'interno del canale

🧠 Quale scegliere?
Usa .unsqueeze(1) (griglia 2D) se:
Vuoi sfruttare relazioni tra canali EEG (spazialità).

Stai analizzando pattern che coinvolgono distribuzioni topografiche (Pz vs Cz, emisferi, ecc.).

Usa .permute(...).unsqueeze(3) se:
Vuoi analizzare ogni canale indipendentemente.

Hai pochi dati e vuoi una CNN più semplice.

Tratti i canali EEG come “sorgenti indipendenti”.

✅ Conclusione
Hai centrato la differenza:

La tua versione **focalizza sui pattern di frequenza all'interno di ciascun canale.

L’altra versione esplora pattern condivisi tra canali e frequenze, come se fosse un'immagine EEG frequenze × elettrodi.

Se vuoi, posso anche mostrarti un esempio visivo (matplotlib) per rendere ancora più chiara la differenza.



Quindi, nella mia versione originale è come se dessi 61 immagini singolo canale 'verticali', 
nel senso che per analogia, sarebbe come dare la rappresentazione in formato vettore verticale di ogni canale rispetto ai pixel in frequenza 
(ossia i bins di frequenza)..

Nella tua, invece, forniamo un input solo, ossia una immagine sola,
con la rappresentazione dei bin di frequenza di ogni canale e quindi ci consente di vedere le relazioni tra i canali stessi?


****************** ****************** ****************** ****************** ****************** ****************** ******************

✅ Esattamente! Hai centrato il punto con una metafora visiva perfetta. Ti spiego meglio facendo proprio leva su questa analogia dell’immagine.

🎯 Tua versione originale: (B, 61, 45, 1) → 61 “immagini verticali”
✅ Metafora:
È come dare 61 immagini strette e alte (una per ogni canale EEG), in cui:

Altezza = 45: le frequenze (tipo pixel verticali)

Larghezza = 1: inesistente (una sola colonna)

Ogni immagine è indipendente

👉 In pratica:
La rete analizza separatamente ogni canale EEG.

Impara “per ogni canale, com’è distribuita la potenza nelle varie frequenze?”

Non sa nulla di cosa succede in altri canali.

🎯 La mia versione (classica CNN2D): (B, 1, 45, 61) → una sola immagine F×C
✅ Metafora:
È come dare una singola immagine 2D:

Asse verticale = 45 bin di frequenza

Asse orizzontale = 61 canali EEG

👉 In pratica:
La rete guarda l’intera griglia frequenza × canale come un’immagine.

Può imparare pattern tipo:

“La potenza in alpha aumenta da O1 verso Cz”

“Un cluster di attivazione nei canali frontali nella gamma”

Le convoluzioni possono spostarsi sia tra frequenze che tra canali, quindi può apprendere interazioni spaziali.

👁️‍🗨️ Visualizzazione (semplificata)
Tua:
scss
Copia
Modifica
Canale Fz →  [ delta ]  
              [ theta ]  
              [ alpha ]  
              [ beta  ]  
              [ gamma ]

Canale Cz →  [ delta ]  
              [ theta ]  
              ...
(61 ripetizioni)
Mia:
matlab
Copia
Modifica
        C1   C2   C3   ...  C61
delta   ■    □    ■    ...  □
theta   □    ■    □    ...  ■
alpha   ■    ■    ■    ...  □
beta    □    □    ■    ...  ■
gamma   ■    □    □    ...  □
Una griglia piena, dove ogni pixel rappresenta la potenza in una frequenza per un canale.

✅ Conclusione
Hai descritto benissimo:

Versione	Interpretazione	Vantaggio principale
Tua (permute + unsqueeze)	61 immagini verticali (1 per canale)	Focus dettagliato su pattern per canale
Classica (unsqueeze(1))	Una sola immagine 2D F×C	Cattura pattern e relazioni tra canali

Se vuoi fare un passo in più, puoi anche combinare le due visioni con multi-branch CNN (una per canale, una per immagine globale). 
Fammi sapere se vuoi un esempio anche per questo!

****************** ****************** ****************** ****************** ****************** ****************** ******************

'''


#https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class CNN2D(nn.Module):
    def __init__(
        self,
        input_channels: int,
        num_classes: int,

        # da sweep: numero di feature map di base
        conv_out_channels: int,

        # da sweep: kernel size H×W per i 3 blocchi
        conv_k1_h: int, conv_k1_w: int,
        conv_k2_h: int, conv_k2_w: int,
        conv_k3_h: int, conv_k3_w: int,

        # da sweep: stride H×W per i 3 blocchi
        conv_s1_h: int, conv_s1_w: int,
        conv_s2_h: int, conv_s2_w: int,
        conv_s3_h: int, conv_s3_w: int,

        # da sweep: pool kernel H×W per i 3 blocchi
        pool_p1_h: int, pool_p1_w: int,
        pool_p2_h: int, pool_p2_w: int,
        pool_p3_h: int, pool_p3_w: int,

        # da sweep: tipo di pooling
        pool_type: str,  # "max" o "avg"

        # fully‑connected
        fc1_units: int,
        dropout: float,

        # attivazioni per i 3 blocchi
        cnn_act1: str,
        cnn_act2: str,
        cnn_act3: str,
    ):
        super().__init__()
        mapping = {'relu': F.relu, 'selu': F.selu, 'elu': F.elu}
        self.act_fns = [
            mapping[cnn_act1],
            mapping[cnn_act2],
            mapping[cnn_act3],
        ]
        
        # calcolo padding “quasi‐same” per ciascun blocco
        p1_h = (conv_k1_h - 1) // 2
        p1_w = (conv_k1_w - 1) // 2
        p2_h = (conv_k2_h - 1) // 2
        p2_w = (conv_k2_w - 1) // 2
        p3_h = (conv_k3_h - 1) // 2
        p3_w = (conv_k3_w - 1) // 2
        
        # Primo blocco
        self.conv1 = nn.Conv2d(
            input_channels, conv_out_channels,
            kernel_size = (conv_k1_h, conv_k1_w),
            stride = (conv_s1_h, conv_s1_w),
            #padding='same'
            padding = (p1_h, p1_w)
        )
        self.bn1   = nn.BatchNorm2d(conv_out_channels)
        self.pool1 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p1_h, pool_p1_w))

        # Secondo blocco (×2 feature map)
        self.conv2 = nn.Conv2d(
            conv_out_channels, conv_out_channels*2,
            kernel_size=(conv_k2_h, conv_k2_w),
            stride=(conv_s2_h, conv_s2_w),
            #padding='same'
            padding = (p2_h, p2_w) 
        )
        self.bn2   = nn.BatchNorm2d(conv_out_channels*2)
        self.pool2 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p2_h, pool_p2_w))

        # Terzo blocco (×3 feature map)
        self.conv3 = nn.Conv2d(
            conv_out_channels*2, conv_out_channels*3,
            kernel_size=(conv_k3_h, conv_k3_w),
            stride=(conv_s3_h, conv_s3_w),
            #padding='same'
            padding = (p3_h, p3_w)
        )
        self.bn3   = nn.BatchNorm2d(conv_out_channels*3)
        self.pool3 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p3_h, pool_p3_w))

        # FC finale
        self.fc1     = nn.LazyLinear(fc1_units)
        self.dropout = nn.Dropout(dropout)
        self.fc2     = nn.LazyLinear(num_classes)
    
    
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        #🔁 Prima:
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        #x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        #x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)
        
        #✅ Ora:
        #Siccome i dati arrivano come (B, 45, 61) — cioè frequenze × canali, non serve permutare. Ti basta:
        
        # Aggiungiamo una dimensione per il canale "immagine"
        x = x.unsqueeze(1)  # → (B, 1, 45, 61)
            
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = self.act_fns[0](x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = self.act_fns[1](x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = self.act_fns[2](x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.relu(x)
       
        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x
    
    
    
'''

VERSIONE CONVOLUZIONE 3D PURA e CONVOLUZIONI SEPARABILI 19 LUGLIO 2025


Due versioni dell’architettura:

CNN3D_LSTM_FC: usa nn.Conv3d per eseguire una vera convoluzione 3D sui cinque depth (bande di frequenza), 
mantenendo il resto del flusso identico.

SeparableCNN2D_LSTM_FC: applica in sequenza una convoluzione depthwise (gruppi = canali) e una pointwise (1×1) 
per fondere i cinque canali in modo efficiente.

Entrambe le classi si integrano con il tuo blocco LSTM e il classificatore come nella versione originale.



Per ottenere un Grad‑CAM “3D” su ciascuna delle 5 bande (cioè un volume 9×9×5) 
invece di schiacciare tutto in una mappa 9×9, bisogna:

Non appiattire la dimensione di profondità (“depth” = bande) con cam.mean(dim=1).

Calcolare i pesi medi dei gradienti solo su altezza e larghezza, non su depth, in modo da preservare D=5.

Upsample (solo) le due dimensioni spaziali H×W, lasciando inalterata la profondità D.

(Opzionale) 

Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!

Se vuoi davvero avere D=5 in uscita, devi cambiare conv1 in:


# ❌ kernel_size=(5,3,3), padding=(0,1,1) → D_out = 1
self.conv1 = nn.Conv3d(1, 32, kernel_size=(3,3,3), padding=(1,1,1))
così la profondità si conserva da 5→5.



1) Perché in conv1 useremo padding=(1,1,1) e negli altri layer padding=(0,1,1)
Obiettivo: mantenere la profondità (numero di bande, D = 5) costante lungo tutta la rete.

In conv1, abbiamo scelto kernel_size=(3,3,3) perché vogliamo che il filtro “scorra” su tutti e tre gli assi (D,H,W).

Con kernel_depth=3, per avere

𝐷out = (𝐷in + 2⋅𝑃 depth − 𝐾 depth)/ 𝑆 + 1 = 5

Da qui (1,1,1) per (depth, height, width).

Negli altri layer 3D (conv2a, conv2b, conv3) il kernel depth = 1 (kernel_size=(1,3,3)), 
quindi la profondità non cambia se mettiamo padding_depth=0 con padding (0,1,1) nel layer conv2 e conv3

In altre parole, su quell’asse non serve alcun padding:

se P dept = 0 allora diventa infatti

𝐷out = (𝐷in + 2⋅0 − 𝐾 depth)/ 𝑆 + 1 = 5

2⋅0


Non è che la tua rete “CNN3D_LSTM_FC” sia sbagliata in senso assoluto, 
ma — proprio a causa di quel primo Conv3d con kernel_size=(5,3,3) e padding=(0,1,1) — 

stai automaticamente comprimendo tutte e 5 le bande nella singola fetta di profondità:


self.conv1 = nn.Conv3d(
    in_channels=1, out_channels=32,
    kernel_size=(5, 3, 3),  # → D_out = (5 − 5 + 2·0)/1 + 1 = 1
    padding=(0, 1, 1)
)
Quindi il tuo tensore (B, 1, 5, 9, 9) diventa (B, 32, 1, 9, 9): la dimensione depth (5) si riduce a 1 subito.

Se invece vuoi davvero preservare le 5 “fette” come vera terza dimensione spaziale, hai due possibili correzioni:

Usare un kernel 3×3×3 (o 1×3×3) in conv1, in modo da non “abbracciare” tutta la profondità d’ingresso:


- self.conv1 = nn.Conv3d(1, 32, kernel_size=(5, 3, 3), padding=(0, 1, 1))
+ # preserva D: depth out = depth in = 5
+ self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))

Oppure, se vuoi mantenere le bande completamente indipendenti in questo layer,


self.conv1 = nn.Conv3d(1, 32, kernel_size=(1, 3, 3), padding=(0, 1, 1))
che trasforma (B,1,5,9,9) → (B,32,5,9,9).

Lasciare com’è, sapendo però che la rete “fonderà” le 5 bande in un’unica mappa di profondità: 
non è un bug, è una scelta architetturale.

La SeparableCNN2D_LSTM_FC
Quella architettura non comprime mai le bande all’interno di un’unica fetta, perché:

Le bande diventano canali di un Conv2d depth‑wise:


x = x.permute(0, 3, 1, 2)  # (B,5,9,9)
self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
Ogni “fetta” (banda) resta separata fino al pointwise e agli strati successivi.

Quindi se il tuo obiettivo è avere un’uscita per banda (e poi poter plottare un Grad‑CAM 2D per ciascuna),
la SeparableCNN2D è già configurata correttamente.

Se invece vuoi un Grad‑CAM “volumetrico” 3D (5×9×9) direttamente dal modello 3D puro,
la vera modifica necessaria è solo sul primo Conv3d, come mostrato sopra.

Fammi sapere quale dei due setup stai usando e ti aiuto a integrare il Grad‑CAM 3D di conseguenza!

'''



'''
Ecco l’implementazione completa di CNN3D_LSTM_FC (“approccio sequenza di profondità”) in cui:

mantieni il tuo primo blocco 3D con kernel (3,3,3) e padding (1,1,1), quindi D rimane 5 fino alla fine;

riduci le spatial singleton dims (H=1,W=1) e trasformi la depth D=5 in una sequenza di lunghezza 5;

imposti l’input_size=128 nell’LSTM (feature per time‑step = 128);


Con questa versione:

la sequenza per l’LSTM ha lunghezza D=5;

ogni passo ha 128 feature, esattamente input_size=128;

non servono trucchi di reshape su scala globale.


'''


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3D_LSTM_FC(nn.Module):
    """
    Version with pure 3D convolutions treating the 5 frequency bands
    as a sequence (depth) for the LSTM.
    Input: Tensor of shape (B, 9, 9, 5) --> reshaped to (B, 1, 5, 9, 9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 (3D) ---
        self.conv1   = nn.Conv3d(1,  32, kernel_size=(3,3,3), padding=(1,1,1))
        self.bn1     = nn.BatchNorm3d(32)
        self.pool3d  = nn.MaxPool3d((1,2,2))  # non tocca D

        # --- Block 2 (3D Residual) ---
        self.res_conv3d = nn.Conv3d(32, 64, kernel_size=1, bias=False)
        self.res_bn3d   = nn.BatchNorm3d(64)
        self.conv2a     = nn.Conv3d(32, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2a       = nn.BatchNorm3d(64)
        self.conv2b     = nn.Conv3d(64, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2b       = nn.BatchNorm3d(64)

        # --- Block 3 (3D) ---
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn3   = nn.BatchNorm3d(128)

        # LSTM o FC finale
        if self.use_lstm:
            # input_size = feature_dim per time‑step = 128
            self.lstm       = nn.LSTM(input_size=128,
                                      hidden_size=self.hidden_size,
                                      num_layers=1,
                                      batch_first=True)
            self.classifier = nn.LazyLinear(num_classes)
        else:
            self.classifier = nn.LazyLinear(num_classes)

    def forward(self, x):
        # x: (B, 9, 9, 5)
        if x.ndim == 4:
            # -> (B,1,D=5,H=9,W=9)
            x = x.permute(0, 3, 1, 2).unsqueeze(1)

        # --- Block 1 ---
        x = F.relu(self.bn1(self.conv1(x)))  # (B,32,5,9,9)
        x = self.pool3d(x)                   # (B,32,5,4,4)

        # --- Block 2 (Residual) ---
        res = self.res_bn3d(self.res_conv3d(x))  # (B,64,5,4,4)
        x   = F.relu(self.conv2a(x))             # (B,64,5,4,4)
        x   = self.bn2b(self.conv2b(x))          # (B,64,5,4,4)
        x   = F.relu(x + res)                    # (B,64,5,4,4)
        x   = self.pool3d(x)                     # (B,64,5,2,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))      # (B,128,5,2,2)
        x = self.pool3d(x)                       # (B,128,5,1,1)

        # Stampa delle dimensioni prima di passare al classifier
        #print(f"Dimensioni prima del classifier: {x.shape}")

        if self.use_lstm:
            # x: (B,128,5,1,1)
            # -> squeeze spatial dims → (B,128,5)
            x = x.squeeze(-1).squeeze(-1)
            # -> permute per batch_first → (B, seq_len=5, feat=128)
            x = x.permute(0, 2, 1)
            x = self.dropout(x)
            out, _ = self.lstm(x)               # out: (B,5,hidden_size)
            last    = out[:, -1, :]             # prendo l’ultimo time-step
            logits  = self.classifier(last)     # (B, num_classes)
        else:
            # x: (B,128,5,1,1) → flatten → (B,128)
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

    

class SeparableCNN2D_LSTM_FC(nn.Module):
    """
    Version with depthwise + pointwise separable convolutions
    across the 5 channels.
    Input: Tensor of shape (B, 9, 9, 5) -> (B,5,9,9)
    
    
    groups=5 → impone che ogni canale venga convoluto indipendentemente dagli altri → depthwise ✅

    kernel_size=1 → combina i 5 canali in un nuovo spazio di 32 feature maps → pointwise ✅


    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 separabile ---
        self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
        self.pw_conv1 = nn.Conv2d(5, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        # --- Block 2 (residuo) ---
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn = nn.BatchNorm2d(64)
        self.bn2a = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.lstm = nn.LSTM(input_size=128 * 5, hidden_size=self.hidden_size, num_layers=1, batch_first=True)

        if self.use_lstm:
            self.lstm = nn.LSTM(
                input_size=128 * 1,
                hidden_size=self.hidden_size,
                num_layers=1,
                batch_first=True
            )
            self.classifier = nn.Linear(self.hidden_size, num_classes)
        else:
            self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)  # -> (B,5,9,9)

        x = F.relu(self.dw_conv1(x))
        x = F.relu(self.bn1(self.pw_conv1(x)))
        x = self.pool(x)

        res = self.res_bn(self.res_conv(x))
        x = F.relu(self.conv2a(self.bn2a(x)))
        x = self.bn2b(self.conv2b(x))
        x = F.relu(x + res)
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)  # (B,128,1,1)

        if self.use_lstm:
            x = x.permute(0, 2, 1, 3).reshape(x.size(0), 1, -1)  # (B,1,128)
            out, _ = self.lstm(self.dropout(x))
            last = out[:, -1, :]
            logits = self.classifier(last)
        else:
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

##### **TRAINING (NON DEVI ESEGUIRLA VAI DIRETTAMENTE AL TESTING!)**

###### **VERSIONE PRE- WEIGHT AND BIASES (W&B)**

###### **VERSIONE POST- WEIGHT AND BIASES (W&B)**

In [ ]:
'''UFFICIALE - VERSIONE POST- WEIGHT AND BIASES SENZA COMMENTI'''


import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True
                

def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    # Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    #plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    '''
    
    # Salvare il plot in un buffer di memoria
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()

    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data



def training(model, dataset_train_loader, dataset_val_loader, optimizer, criterion, n_epochs = 100, patience = 10):
    
    # Sposta il modello sulla GPU (se disponibile)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    model.to(device)
    
    #Setta il modello in fase di training
    model.train()
    
    # Storico delle metriche per ogni epoca
    loss_train_history = []  # History of Training loss
    loss_val_history = []    # History of Validation loss
    accuracy_train_history = []  # History of Training Accuracy
    accuracy_val_history = []    # History of Validation Accuracy
    
    early_stopping = EarlyStopping(patience=patience, mode='max')
    
    # Liste per le metriche di valutazione (precision, recall, F1, AUC)
    precision_train_history = []
    recall_train_history = []
    f1_train_history = []
    auc_train_history = []
    
    #Questa sarebbe la migliore accuratezza ottenuta sul validation set
    #in base alla quale viene preso il modello migliore!
    
    max_val_acc = 0
    best_model = None
    
    best_epoch = 0  # Epoca con la migliore validazione
    
    best_metrics = {} # Dizionario con le metriche del migliore modello nel set di validazione
    
    # Variabili per memorizzare le etichette vere e predette per l'intero training
    y_true_train_list = []
    y_pred_train_list = []
    
    
    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        #Create a list for temporary monitoring of train loss and accuracy at each epoch
        train_loss_tmp = [] 
        correct_train = 0 
        
        
        #'''STARTING OF THE TRAINING PHASE'''
        
        #Iterating for every batch inside dataset_train_loader
        for x, y in dataset_train_loader:
            
            x, y = x.to(device), y.to(device)
            
            #Run forward pass through my network and get a prediction
            y_pred = model(x)
            
            train_loss = criterion(y_pred, y.view(-1))
            optimizer.zero_grad() #so essentially finding where gradients is 0
                                  #we're looking for minimum's there

            train_loss.backward() #performing the backprop step
            optimizer.step() #update the model's hyperparameters based off of the step
        
            train_loss_tmp.append(train_loss.item()) #append the loss at each epoch in the temporary train loss list inside each epoch
            
            # Calculate the Accuracy Score during the Training Phase
                
            #qui il "_,"
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            
            # Aggiungere le etichette vere e quelle predette alla lista
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
        
        # Save the results of training set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        loss_train_history.append(np.mean(train_loss_tmp))
        accuracy_train = correct_train / len(dataset_train_loader.dataset)
        accuracy_train_history.append(accuracy_train)
        
        # Calcolare precision, recall, F1-score e AUC durante il training
        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        precision_train_history.append(precision_train)
        recall_train_history.append(recall_train)
        f1_train_history.append(f1_train)
        auc_train_history.append(auc_train)
        
        # '''STARTING OF THE VALIDATION PHASE'''
        
        #Setta il modello in fase di validation
        #model.eval() 
        
        loss_tmp_val = []  #create a list for temporary val list at each epoch
        correct_val = 0
        
        y_true_list = []
        y_pred_list = []
        
        #Here we disable gradient computation for the validation phase!
        with torch.no_grad():
            
            for x, y in dataset_val_loader:
                
                x, y = x.to(device), y.to(device)
                
                #Run forward pass through my network and get a prediction
                y_pred = model(x)

                #Calculate Validation Loss

                #remember: since we use CrossEntropyLoss we DO NOT need
                #to do any ONE HOT ENCODING between y_pred and y_train 
                
                #loss = criterion(y_pred.to(device), y.view(-1).to(device))
                
                val_loss = criterion(y_pred, y.view(-1))

                #Perform Backpropagation

                #HOW TO ADJUST THE VALUES (weights and biases)?
                #well, at every step the gradients will accumulate with every backprop,
                #so to prevent 'compounding', we need to reset the stored gradient for each new epoch!

                loss_tmp_val.append(val_loss.item()) #append the loss at each epoch in the temporary val loss list inside each epoch 
                
                # Calculate the Accuracy Score during the Validation Phase
                _, predicted_val = torch.max(y_pred, 1)
                correct_val += (predicted_val == y).sum().item()
                
                # Aggiungi le etichette e le predizioni per la confusion matrix
                y_true_list.extend(y.cpu().numpy())
                y_pred_list.extend(predicted_val.cpu().numpy())

                
        # Save the results of validation set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        
        loss_val_history.append(np.mean(loss_tmp_val)) 
        accuracy_val = correct_val / len(dataset_val_loader.dataset)
        accuracy_val_history.append(accuracy_val)
        
        #L'early stopping deve essere basato sulla val accuracy,
        #ma quando il training si interrompe, 
        #dobbiamo salvare le migliori performance ottenute sul training in corrispondenza dell'epoca in cui
        #la val accuracy era massima
        
        # Controllo della miglior validazione
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            
            best_metrics = {
                "train_loss": [round(loss_train_history[best_epoch], 4)],
                "train_accuracy": [round(accuracy_train_history[best_epoch], 4)],
                "train_precision": [round(precision_train, 4)],
                "train_recall": [round(recall_train, 4)],
                "train_f1_score": [round(f1_train, 4)],
                "train_auc": [round(auc_train, 4)]
            }
            best_model = cp.deepcopy(model)  # Salvo il miglior modello

        # Controllo Early Stopping
        early_stopping(accuracy_val)
        if early_stopping.early_stop:
            print(f"⚠️ Early stopping attivato all'epoca \033[1m{epoch}\033[0m, recupero il modello dell'epoca \033[1m{best_epoch}\033[0m")
            break

        # Update of the progress bar
        pbar.set_description(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {loss_train_history[-1]:.4f}, Val Loss: {loss_val_history[-1]:.4f}, Train Acc: {accuracy_train:.4f}, Val Acc: {accuracy_val:.4f}")

        # Calculate the confusion matrix and the classification report after all epochs in the Validation Phase
        conf_matrix = confusion_matrix(y_true_list, y_pred_list)
        class_report = classification_report(y_true_list, y_pred_list)

    # Salvataggio della configurazione del modello e iper-parametri
    model_config = {
        "model_architecture": str(model),
        "batch_size_train": train_loader.batch_size,
        "batch_size_val": val_loader.batch_size,
        "batch_size_test": test_loader.batch_size,
        "n_epochs": n_epochs
    }

    # Dizionario degli iper-parametri
    hyperparams = {
    "optimizer": str(optimizer),
    "loss_function": str(criterion),
    "learning_rate": optimizer.param_groups[0]['lr'],
   }

    
    # Plot dei risultati
    #plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history, exp_cond_1, exp_cond_2)
    training_plot = plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history)

    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        "best_model": best_model,
        "confusion_matrix_val": conf_matrix,
        "classification_report": class_report,
        "model_configuration": model_config,
        "hyperparameters": hyperparams,
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }

    return train_results


##### **TESTING**

In [10]:
'''
TESTING FUNCTION: CORRETTA ANCHE PER IL GRAD-CAM

SUCCESSIVAMENTE, DENTRO AL FOR LOOP DEL TRAINING E TESTING, 
SI RICHIAMA LA FUNZIONE DIRETTAMENTE DI 

1) compute_gradcam_figure, LA QUALE AL SUO INTERNO PRESENTA GIÀ 
TUTTO QUELLO CHE SERVE PER CALCOLARE IL GRADCAM, DI MODO CHE VADA A 

Selezionare esempi rappresentativi per ciascuna classe.
Calcolare le mappe GradCAM e gli overlay.
Creare una figura con le heatmap e le sovrapposizioni, completa di titoli esplicativi.
Restituire un'immagine (buffer) pronta per essere salvata

SUCCESSIVAMENTE, QUINDI, IL PROCEDIMENTO DIVENTA COME SEGUE:

1) Si esegue il TESTING, per ottenere le metriche e salvare i risultati (senza GradCAM)

2) Nel loop principale di TRAINING & TESTING, se il modello è CNN2D, allora 

 - richiama la funzione 'compute_gradcam_figure', la quale va a
    - calcolare le mappe di attivazione e successivamente creo le immagini che gli ho chiesto
    - passa l'immagine ottenuta da GradCAM alla funzione 'save_performance_results', la quale va a 
        - salvare i risultati di test ottenuti dalla funzione di 'testing'
        - salvare l'immagine risultatante del GradCAM e la sovrapposizione del GradCAM sullo spettrogramma originale della classe risultante
        
        
Questo approccio garantisce chiarezza e separa la parte di performance (testing) dalla parte di explainability (GradCAM).


'''

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import io
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


def testing(results, test_loader, criterion):
    
    # Recupera il miglior modello ottenuto durante la validazione
    model = results['best_model']
    model.to(device)
    
    model.eval()  # Imposta il modello in modalità valutazione

    y_true_list = []  # Lista per salvare le etichette reali
    y_pred_list = []  # Lista per salvare le previsioni del modello
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
    y_score_list = []   # <— Lista per salvare gli score per le probabilità della classe positiva (per auc-roc!)
    
    total_loss = 0
    correct = 0
    
    test_performances = {
        "test_loss": [],
        "test_accuracy": [],
        "test_precision": [],
        "test_recall": [],
        "test_f1_score": [],
        "test_auc": []
    }
    

    with torch.no_grad():
        
        pbar = tqdm(test_loader, desc="Testing")
        
        for inputs, labels in pbar:
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Ottenere le predizioni del modello
            outputs = model(inputs)
            
            '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
            # aggiungi queste due righe
            probs = torch.softmax(outputs, dim=1)
            y_score_list.extend(probs[:,1].cpu().numpy())

            # Calcolare la loss
            test_loss = criterion(outputs, labels)
            total_loss += test_loss.item()

            # Memorizzare predizioni ed etichette vere
            _, predicted = torch.max(outputs, 1)
            y_pred_list.extend(predicted.cpu().numpy())
            y_true_list.extend(labels.cpu().numpy())

            # Aggiornare il numero di predizioni corrette
            correct += (predicted == labels).sum().item()

            pbar.set_description(f"Loss: {test_loss.item():.4f}")

    # Calcolare l'accuratezza complessiva
    accuracy = correct / len(test_loader.dataset)
    
    
    # Calcolare precision, recall, F1-score, AUC durante il testing
    precision_test = precision_score(y_true_list, y_pred_list, average='weighted')
    recall_test = recall_score(y_true_list, y_pred_list, average='weighted')
    f1_test = f1_score(y_true_list, y_pred_list, average='weighted')
    
    '''OLD VERSION'''
    #auc_test = roc_auc_score(y_true_list, y_pred_list, average='weighted')  # Assicurati che il problema sia binario o multi-class
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC
    
    In questo modo l’roc_auc_score calcola l’area sotto tutta la curva ROC (tutte le soglie), 
    invece di valutare un solo punto corrispondente alla soglia 0.5
    '''
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    auc_test = roc_auc_score(y_true_list, y_score_list)


    # Aggiungere questi valori nel dizionario delle performance (arrotondando a 4 decimali)
    test_performances["test_loss"].append(round(total_loss / len(test_loader), 4))  # Media della loss
    test_performances["test_accuracy"].append(round(accuracy, 4))
    test_performances["test_precision"].append(round(precision_test, 4))
    test_performances["test_recall"].append(round(recall_test, 4))
    test_performances["test_f1_score"].append(round(f1_test, 4))
    test_performances["test_auc"].append(round(auc_test, 4))
    
    # Creare la confusion matrix
    conf_matrix = confusion_matrix(y_true_list, y_pred_list)
    
    # Stampare classification report
    class_report = classification_report(y_true_list, y_pred_list)

    print(f"\nTest Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", class_report)

    # Visualizzare la confusion matrix
    #plt.figure(figsize=(8, 6))
    #sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    #plt.title("Confusion Matrix")
    #plt.xlabel("Predicted")
    #plt.ylabel("True")
    #plt.show()
    
    # Salviamo l'immagine della confusion matrix in un buffer
    #buf = io.BytesIO()
    #plt.savefig(buf, format='png')
    #buf.seek(0)
    #conf_matrix_image_data = buf.getvalue()
    #buf.close()
    
    
    # Salviamo l'immagine della confusion matrix in un buffer
    buf = io.BytesIO()
    plt.figure(figsize=(8, 6))  # Nuova figura per evitare sovrapposizioni
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(buf, format='png')  # Salva l'immagine nel buffer
    buf.seek(0)  # Torna all'inizio del buffer
    conf_matrix_image_data = buf.getvalue()  # Ottieni l'immagine in formato binario
    buf.close()  # Chiudi il buffer

    # Mostra la confusion matrix (opzionale)
    #plt.show()
    
    # Salvataggio della configurazione del modello e iper-parametri
    '''COMMENTATO'''
    #model_config = {
        #"model_architecture": str(model),
        #"batch_size_test": test_loader.batch_size,
    #}
    
    '''COMMENTATO'''
    # Dizionario degli iper-parametri
    #hyperparams = {
        #"optimizer": str(optimizer),
        #"loss_function": str(criterion),
        #"learning_rate": optimizer.param_groups[0]['lr'],
    #}

    
    '''COMMENTATO'''
    # Restituisci i risultati come dizionario
    #test_results = {
        #"test_performances": test_performances,  # Aggiungi il dizionario delle performance
        #"confusion_matrix": conf_matrix,
        #"classification_report": class_report,
        #"model_configuration": model_config,
        #"hyperparameters": hyperparams,  # Aggiunti i due nuovi dizionari
        #"confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    #}
    
    
    # Restituisci i risultati come dizionario
    test_results = {
        "test_performances": test_performances,  # Aggiungi il dizionario delle performance
        "confusion_matrix": conf_matrix,
        "classification_report": class_report,
        "confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    }   
        
    return test_results


##### **CREAZIONE CLASSE GRADCAM**

In [11]:
##### **CREAZIONE CLASSE GRADCAM**

'''
Creazione della classe GradCAM

-----1. Costruttore (init)-----

Cosa fa:

Salva il modello e il layer target (ad esempio, l'ultimo strato convoluzionale) su cui calcolare le mappe di attivazione.

A) Inizializza due variabili, 

1) self.activations e 2) self.gradients, che verranno usate per memorizzare rispettivamente 
1) le attivazioni (feature maps) e 2) i gradienti di quel layer

B) Registra due hook sul target_layer:

1) Forward Hook: Quando il modello effettua la forward pass, viene eseguito save_activation per salvare le attivazioni
2) Backward Hook: Durante la backward pass, save_gradient viene chiamato per salvare i gradienti


-----2. Hook per Salvare Attivazioni e Gradienti-----

B) Save Activation

def save_activation(self, module, input, output):
    self.activations = output.detach()

Cosa fa:

Quando viene eseguita la forward pass sul target_layer, questo hook cattura l'output (le attivazioni) del layer.
Usa detach() per ottenere una copia dei dati senza il tracking dei gradienti, in modo da non interferire con la retropropagazione.

C) Save Gradient

def save_gradient(self, module, grad_input, grad_output):
    self.gradients = grad_output[0].detach()


Cosa fa:

Durante la backward pass, questo hook cattura i gradienti che fluiscono attraverso il target_layer.
grad_output è una tupla; solitamente il primo elemento contiene i gradienti utili. 

Anche qui si usa detach() per isolare i dati dai grafi di calcolo.


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

##### **DRAFT IMPLEMENTATIONS OF GRADCAM COMPUTATION**

In [ ]:
### INITIAL IMPLEMENTATIONS OF GRADCAM

'''
import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
        
        
1) Funzione generate_cam (interna alla classe GradCAM)

    La differenza chiave tra i due approcci è proprio la selezione dell'input su cui viene calcolata la Grad-CAM. Ti riassumo le due opzioni:

    1️⃣ Approccio attuale (generate_cam)
    Viene passato un singolo input_tensor, e il Grad-CAM viene calcolato su di esso.
    Se target_class non è specificata, viene selezionata la classe predetta dal modello per quell'input.
    Il calcolo del Grad-CAM si basa su una backward pass del gradiente rispetto alla classe target.
    
    def generate_cam(self, input_tensor, target_class=None):
        # Effettua la forward pass
        output = self.model(input_tensor)
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        # Azzeramento dei gradienti
        self.model.zero_grad()
        # Calcola il gradiente per la classe target
        target = output[0, target_class]
        target.backward()

        # Calcola i pesi come media dei gradienti su width e height
        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        # Somma pesata delle attivazioni
        cam = torch.sum(weights * self.activations, dim=1)
        cam = F.relu(cam)

        # Normalizza la mappa
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)

        # Upsample alla dimensione dell'immagine di input
        cam = F.interpolate(cam.unsqueeze(1), size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = cam.squeeze().cpu().numpy()
        return cam
    
    
2) Funzione compute_gradcam_figure (esterna alla classe GradCAM)   
    
2️⃣ Alternativa proposta (compute_gradcam_figure)
Seleziona esplicitamente un esempio per ciascuna classe (0 e 1) iterando sul test_loader.
Questo garantisce che il Grad-CAM sia calcolato su esempi rappresentativi di entrambe le classi.
La visualizzazione finale confronta le heatmap delle due classi, sovrapponendole agli spettrogrammi.


import cv2
import numpy as np
import matplotlib.pyplot as plt
import io

def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):
    
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
      - Riga 1: Heatmap per classe 0 e classe 1.
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
    I titoli della figura vengono personalizzati con exp_cond, data_type, category_subject.
    """
    
    # Assumiamo che il modello sia CNN2D e che il layer target sia model.conv3
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)

    # Dizionari per salvare il campione per ogni classe
    samples = {}      # Salveremo il sample input per ogni classe
    labels_found = {} # Per tenere traccia delle etichette già trovate

    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int not in labels_found:
                samples[label_int] = inputs[i].unsqueeze(0)  # salva come tensore 4D
                labels_found[label_int] = True
            if 0 in labels_found and 1 in labels_found:
                break
        if 0 in labels_found and 1 in labels_found:
            break

    # Se non troviamo entrambi gli esempi, esci con un messaggio
    if 0 not in samples or 1 not in samples:
        print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
        return None

    # Per ciascun campione, calcola GradCAM
    cams = {}
    overlays = {}
    for cls in [0, 1]:
        sample_input = samples[cls]
        sample_input.requires_grad = True  # Abilita gradiente per il campione
        cam = gradcam.generate_cam(sample_input)
        cams[cls] = cam

        # Converti il sample in immagine numpy per la visualizzazione
        img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
        # Normalizza l'immagine in scala 0-255
        img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))
        # Applica la heatmap
        heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        # Sovrapponi la heatmap all'immagine originale
        overlay = cv2.addWeighted(img_norm, 0.6, heatmap, 0.4, 0)
        overlays[cls] = overlay

    # Crea la figura con due righe e due colonne
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Titolo per la prima riga
    title_row1 = f"Grad-CAM mapping of experimental condition {exp_cond}, EEG {data_type}, Subject {category_subject}"
    # Titolo per la seconda riga
    title_row2 = f"Grad-CAM mapping superimposition over EEG Spectrogram of experimental condition {exp_cond}, Subject {category_subject}"
    
    # Prima riga: solo le heatmap
    for j, cls in enumerate([0, 1]):
        axs[0, j].imshow(cv2.cvtColor(cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB))
        axs[0, j].set_title(f"Class {cls} Heatmap")
        axs[0, j].axis('off')
    axs[0, 0].set_ylabel(title_row1, fontsize=10)
    
    # Seconda riga: overlay della heatmap sullo spettrogramma originale
    for j, cls in enumerate([0, 1]):
        axs[1, j].imshow(overlays[cls])
        axs[1, j].set_title(f"Class {cls} Overlay")
        axs[1, j].axis('off')
    axs[1, 0].set_ylabel(title_row2, fontsize=10)
    
    # Ottimizza la disposizione della figura
    plt.tight_layout()
    
    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

''' 

In [ ]:
'''SOLUZIONE ? INTEGRARE LA 2) IN 1)


🛠️ Cosa conviene fare?
Se il tuo obiettivo è sempre confrontare le attivazioni per entrambe le classi, 
allora conviene integrare compute_gradcam_figure dentro la classe GradCAM e rimuovere generate_cam come metodo separato.

📌 Quindi suggerirei di fare così:

Rendere compute_gradcam_figure un metodo della classe GradCAM.
Rimuovere generate_cam, perché il calcolo della CAM viene già eseguito all'interno del loop che seleziona i campioni.
Mantenere la logica che seleziona i campioni da entrambe le classi, perché è più robusta rispetto a calcolare la CAM su un singolo input arbitrario.
⚠️ Attenzione a una cosa però!
Il metodo generate_cam fa un passaggio importante che non è presente in compute_gradcam_figure:

python
Copia
Modifica
weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
cam = torch.sum(weights * self.activations, dim=1)
cam = F.relu(cam)
🔹 Bisogna assicurarsi che questa logica venga mantenuta nel nuovo metodo!
Attualmente compute_gradcam_figure chiama gradcam.generate_cam(sample_input), quindi se generate_cam viene eliminato, questa parte va spostata nel nuovo metodo.

📌 In sintesi, cosa farei
✅ Modificare la classe GradCAM e aggiungere direttamente compute_gradcam_figure.
✅ Eliminare generate_cam, ma mantenere la sua logica di calcolo della CAM.
✅ Garantire che il calcolo dei pesi e della CAM sia integrato nel nuovo metodo.
✅ Mantenere la selezione di un campione per ciascuna classe, per una migliore interpretabilità.



Ha senso integrare compute_gradcam_figure direttamente come metodo della classe GradCAM ed eliminare generate_cam, perché:

Selezione più rappresentativa dei campioni

Il metodo compute_gradcam_figure assicura che vengano selezionati esempi di entrambe le classi (0 e 1), cosa che generate_cam non fa.
Questo approccio fornisce una migliore interpretabilità della Grad-CAM confrontando diverse classi.
Chiarezza e modularità

generate_cam è attualmente chiamato da compute_gradcam_figure, ma possiamo integrare direttamente la logica dentro GradCAM.
Questo evita la duplicazione del codice e rende più chiaro il flusso.
Ottimizzazione del calcolo

La pipeline di compute_gradcam_figure gestisce direttamente la forward pass e il calcolo del gradiente per entrambi i campioni in un'unica operazione, evitando di dover chiamare generate_cam separatamente.
Prossimi passi:
Spostiamo compute_gradcam_figure dentro GradCAM come metodo della classe.
Eliminiamo generate_cam e integriamo direttamente la logica di forward pass e backward pass dentro compute_gradcam_figure.

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION**

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION: FREQUENCIES X ELECTRODES**

In [ ]:
N.B.

Certo, ti spiego nel dettaglio il motivo per cui la modifica che hai fatto con il codice sample_input.permute(0, 2, 1).unsqueeze(3) è importante.

Contesto: Formato dei dati di input nel modello
Nel caso delle reti neurali convoluzionali 2D (CNN2D), l'input che viene passato al modello deve avere una forma specifica affinché il modello possa elaborarlo correttamente. La forma tipica dell'input per un modello CNN2D è:

arduino
Copia
Modifica
(batch_size, num_channels, height, width)
batch_size: Numero di campioni nel batch (ad esempio, 32 immagini per batch).

num_channels: Numero di canali dell'immagine (ad esempio, 1 per immagini in bianco e nero, 3 per immagini RGB).

height: Altezza dell'immagine.

width: Larghezza dell'immagine.

Nel tuo caso specifico, stai lavorando con dati EEG che sono organizzati in spettrogrammi, dove ogni esempio ha la forma (batch_size, num_channels, height, width).

Problema
Nel codice che hai mostrato prima, il formato originale dell'input potrebbe non essere quello previsto dal modello. Supponiamo che i dati siano in un formato come questo:

css
Copia
Modifica
(batch_size, height, width)  # (batch, 45, 61) in questo caso, senza i canali espliciti
Qui:

batch_size è il numero di campioni nel batch.

height e width rappresentano la dimensione spaziale dei dati EEG (ad esempio, frequenze e tempo nello spettrogramma).

Tuttavia, il modello CNN2D si aspetta che l'input abbia 4 dimensioni, ovvero (batch_size, num_channels, height, width).

Se i tuoi dati sono di forma (batch_size, height, width), ciò significa che manca la dimensione per i canali (in pratica, il modello non sa come trattare i tuoi dati se non ha l'informazione sui canali). In effetti, il modello si aspetta un tensore di 4 dimensioni: un canale per ciascun dato.

Soluzione: Perché devi fare sample_input.permute(0, 2, 1).unsqueeze(3)?
La funzione permute(0, 2, 1) serve a cambiare l'ordine delle dimensioni. Se inizialmente hai:

arduino
Copia
Modifica
(batch_size, height, width)  # (batch, 45, 61)
La chiamata permute(0, 2, 1) cambia l'ordine delle dimensioni, facendo diventare l'input:

arduino
Copia
Modifica
(batch_size, width, height)  # (batch, 61, 45)
Questo è utile perché il modello può aspettarsi che la dimensione dell'input per la larghezza venga prima della dimensione dell'altezza (ad esempio, come nel caso delle immagini 2D tradizionali).

Poi, la chiamata a unsqueeze(3) aggiunge una dimensione extra per il canale (che è la quarta dimensione). In pratica, stai dicendo che ogni campione ha un solo canale (perché il dato originale è un’immagine in scala di grigi, e non RGB). La forma finale diventa:

arduino
Copia
Modifica
(batch_size, width, height, 1)  # (batch, 61, 45, 1)
In questo modo, il tuo input è pronto per essere passato a un modello CNN2D, che ora può trattarlo come se fosse un'immagine in scala di grigi (un solo canale).

Sintesi
La ragione per cui devi fare questa modifica è che il modello CNN2D richiede che l'input abbia una forma di 4 dimensioni, ovvero (batch_size, num_channels, height, width). Poiché i tuoi dati erano inizialmente di forma (batch_size, height, width), è necessario trasformarli in modo che includano un canale (se il dato è in scala di grigi) e per adattarli all'input del modello.

Con sample_input.permute(0, 2, 1).unsqueeze(3), ottieni il formato giusto, che consente al modello di elaborare correttamente i tuoi dati EEG.
                                                                                                              
                                                                                                              

In [ ]:
'''
Creazione della funzione per generare le immagini associate alla GradCAM compution

FINAL VERSION WITH ULTIMATED EDITING PHASES


Spiegazione:

1) Selezione dei Campioni:
La funzione itera sul test_loader e salva il primo campione trovato per ciascuna delle due classi (0 e 1).

2) Calcolo GradCAM per ciascun campione:

Per ogni campione, si abilita il gradiente e si esegue la forward pass.
Viene scelto il target (se non specificato, quello predetto) e si esegue la backward pass per calcolare i gradienti.

- I pesi vengono calcolati come la media dei gradienti lungo le dimensioni spaziali (dim=(2,3)) e usati per eseguire una somma pesata sulle attivazioni.
- La mappa risultante viene passata attraverso una ReLU, normalizzata e upsampled per avere la stessa dimensione dell’input.

Creazione degli Overlay:
Viene normalizzata l’immagine originale e viene applicata una heatmap (usando OpenCV), quindi l’overlay viene ottenuto con cv2.addWeighted.

Costruzione della Figura:
Viene creata una figura con due righe e due colonne:

- La prima riga mostra le heatmap per ciascuna classe.
- La seconda riga mostra le sovrapposizioni (overlay) tra heatmap e spettrogramma originale.

I titoli sono personalizzati in base a exp_cond, data_type e category_subject.

Questa struttura mantiene tutta la logica necessaria (incluso il calcolo dei pesi) e la rende simile alla versione precedente,
con la differenza che il calcolo della CAM viene eseguito per campioni rappresentativi di entrambe le classi. 




Ecco una versione modificata della funzione che tiene conto che:

L'input originale ha forma (batch, frequenze, canali) con frequenze = 45 e canali = 61.

Dopo il preprocessing nella rete (permute e unsqueeze) il modello lavora con tensori di forma (batch, 61, 45, 1), cioè:

asse dei canali = 61 (che ora costituirà l’asse x della visualizzazione),

asse delle frequenze = 45 (che sarà l’asse y).

Nella visualizzazione degli overlay imposteremo l’extent su 0,61,0,45 (oltre a ruotare l’immagine per far sì che l’asse y rappresenti le frequenze).

In aggiunta, ti fornisco uno spunto su come estrarre i nomi dei canali da una tripletta di file in formato BrainVision usando MNE, 
in modo da poterli usare per etichettare l’asse x.


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io


#La funzione compute_gradcam_figure serve a calcolare e visualizzare 
#le mappe di attivazione Grad-CAM per un modello CNN2D, applicandole a spettrogrammi EEG. 

#In particolare, seleziona un campione per ciascuna classe (0 e 1), calcola la Grad-CAM e costruisce una figura con:

#Prima riga → Heatmap della Grad-CAM per entrambe le classi.
#Seconda riga → Heatmap sovrapposta allo spettrogramma originale.
#Questa visualizzazione aiuta a interpretare su quali parti dell'immagine il modello si sta concentrando per prendere decisioni.



#Questa funzione aiuta a visualizzare le regioni attivate dalla rete CNN su immagini di spettrogrammi EEG,
#evidenziando le aree più importanti per la classificazione.

#🔹 Esempio finale:
#La figura risultante avrà due righe:

#Heatmap puro della Grad-CAM.
#Heatmap sovrapposta allo spettrogramma EEG originale.

def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device, channel_names = None):
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
    
      - Riga 1: Heatmap per classe 0 e classe 1.
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
      
    I titoli e le etichette degli assi sono personalizzati:
    
    - L'asse x rappresenta il tempo (ms) e l'asse y le frequenze (Hz) (solo per la riga overlay)    
    - I titoli dei subplot usano i nomi delle condizioni estratte automaticamente da 'exp_cond'
        (assumendo che exp_cond sia del tipo "th_resp_vs_pt_resp"), data_type e category_subject
    
    Il calcolo della CAM include il passaggio:
       weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
       cam = torch.sum(weights * activations, dim=1)
       cam = F.relu(cam)
    """
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    # Imposta il layer target (ad esempio conv3) e crea un'istanza di GradCAM
    #target_layer = model.conv3
    
    target_layer = model.layers[-1][0]
    gradcam = GradCAM(model, target_layer)
    
    # Estrai i nomi delle condizioni separando exp_cond (es: "th_resp_vs_pt_resp")
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    
    '''SOLO UN ESEMPIO'''
    # Dizionari per salvare un campione per ciascuna classe
    #samples = {}      # Qui salveremo il sample input per ogni classe 
    #labels_found = {} # Per tracciare se abbiamo già trovato un esempio per ciascuna classe di etichette
    
    '''CON MEDIA'''
    
    #Ora che ogni classe ha una sua chiave nel dizionario samples, non c'è più bisogno di usare labels_found 
    #per verificare la presenza di entrambe le classi.
    #In precedenza, stavi iterando nel test_loader e verificando la presenza di almeno un esempio per entrambe le classi (0 e 1),
    #ma ora i dati vengono direttamente organizzati nel dizionario in base alla loro classe. Quindi, se la classe non esiste nel dataset,
    #semplicemente non avrà una chiave nel dizionario samples.
    #Il controllo finale if 0 not in samples or 1 not in samples: è ancora necessario per assicurarsi che entrambe le classi siano presenti.
    #Se manca una classe, possiamo ancora uscire con un messaggio di errore.
    

    # Dizionari per salvare tutti i campioni per ciascuna classe
    samples = {0: [], 1: []}

    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                samples[label_int].append(inputs[i].unsqueeze(0))
            
            '''SOLO UN ESEMPIO'''
            #if label_int not in labels_found:
                #samples[label_int] = inputs[i].unsqueeze(0)  # Salva come tensore 4D
                #labels_found[label_int] = True
            #if 0 in labels_found and 1 in labels_found:
            #    break
        #if 0 in labels_found and 1 in labels_found:
        #    break

    # Se non troviamo entrambi gli esempi, esci con un messaggio
    #if 0 not in samples or 1 not in samples:
    #    print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
    #    return None

    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Per ciascun campione, calcola la GradCAM
    #cams = {} # Qui salveremo la mappa CAM per ogni classe
    #overlays = {} # Qui salveremo l'overlay (CAM + spettrogramma)
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''

    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}

    
    for cls in [0, 1]:
        
        for sample_input in samples[cls]:

            '''SOLO UN ESEMPIO'''    
            #sample_input = samples[cls]

            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            #print(f"\033[1mSHAPE OF SAMPLE_INPUT: {sample_input.shape}\033[0m")

            # Assicurati che sample_input sia nel formato atteso dal modello (come nel forward)
            #if sample_input.dim() == 3:  # Supponiamo che abbia shape (batch, 45, 61)
            #    sample_input = sample_input.permute(0, 2, 1).unsqueeze(3)  # Ora (batch, 61, 45, 1)


            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)

            #Passaggio 5: Normalizzazione e upsampling

            #La mappa CAM viene normalizzata tra 0 e 1.
            #Viene ridimensionata (upsampling) per adattarsi alla dimensione originale dell'immagine

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            # -------------------------------
            # Passaggio 5: Normalizzazione e upsampling della CAM
            # ---------------------------

            # Normalizza la mappa
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            '''
            # Upsample alla dimensione dell'immagine di input: usa la shape originale degli input convoluzionali (frequenze, larghezza)


            Nel tuo caso, il dato originale che esce dal test_loader è di forma (1,45,61), dove 
            - 45 rappresenta le frequenze 
            - 61 i canali

            Nel forward del modello:

            Il modello parte da un input 3D  (batch,45,61)
            lo trasforma con 'permute' in (batch, 61, 45)
            e poi aggiunge una dimensione con unsqueeze(3) per ottenere (batch, 61, 45, 1)

            Quindi il modello lavora internamente su un "immagine" con dimensioni spaziali (45,1)
            (dove 61 è il numero di canali, non le dimensioni spaziali).


            Per il calcolo della GradCAM:

            L'obiettivo è quello di upsamplare la mappa CAM per poterla sovrapporre al dato originale (lo spettrogramma), che ha forma 
            (45,61)

            Se usi sample_input.shape[2:] su un tensore di forma (1,45, 61) otterrai (61,),
            cioè solo la seconda dimensione! 
            Mentre ciò che ti serve è una tupla di due valori: le frequenze e i canali, cioè (45,61).

            Quindi, per F.interpolate devi usare come target size la tupla

            (sample_input.shape[1], sample_input.shape[2]), che, per il tuo dato, è (45, 61)


            Quindi, modifica la chiamata a F.interpolate in questo modo:

                target_size = (sample_input.shape[1], sample_input.shape[2])  # (45, 61)
                cam = F.interpolate(cam.unsqueeze(1), size=target_size, mode='bilinear', align_corners=False)

            In questo modo, l'upsampling della mappa CAM avverrà alla dimensione corretta per poterla sovrapporre al dato originale, che è (45,61).
            '''


            #cam = F.interpolate(cam.unsqueeze(1), size=sample_input.shape[2:], mode='bilinear', align_corners=False)

            target_size = (sample_input.shape[1], sample_input.shape[2])
            cam = F.interpolate(cam.unsqueeze(1), size = target_size, mode='bilinear', align_corners=False)

            cam = cam.squeeze().cpu().numpy()

            '''SOLO UN ESEMPIO'''
            #cams[cls] = cam

            '''CON MEDIA'''
            # Aggiungi la mappa alla lista per la classe
            cams_list[cls].append(cam)


            #Passaggio 6: Creazione dell’overlay Grad-CAM

            #L'immagine originale viene convertita in un array numpy.
            #La mappa CAM viene colorata con COLORMAP_JET.
            #Si sovrappone l'heatmap all'immagine originale.

            #🔹 Esempio pratico:
            #Se il CAM ha valori alti in alcune regioni, il colormap evidenzierà in rosso le aree più attivate.

            # -------------------------------
            # Passaggio 6: Creazione dell'Overlay
            # -------------------------------

            # Converte l'immagine originale in numpy; considerando che l'input è (batch, canali, frequenze, 1)
            # Dopo squeeze, otteniamo (61, 45). Poiché vogliamo:
            # - asse x: canali (61)
            # - asse y: frequenze (45)
            # invertiamo le dimensioni per ottenere (45, 61)

            # Prepara l'immagine originale per la visualizzazione
            #img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
            img = sample_input.squeeze().cpu().detach().numpy().transpose()  # ora (45, 61)

            # Normalizza l'immagine in scala 0-255
            '''
                                                        SOLUZIONE
            img_norm: Questa è l'immagine originale in scala di grigi, che ha 2 dimensioni (ad esempio, shape (45, 61)).

            1) ANDIAMO A CONVERTIRE img_norm da scala di grigi (2D) a un'immagine a 3 canali (RGB):

            img_norm_color = cv2.cvtColor(img_norm, cv2.COLOR_GRAY2RGB)
            '''


            img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))

            img_norm_color = cv2.cvtColor(img_norm, cv2.COLOR_GRAY2RGB)


            # Applica la heatmap usando OpenCV
            #Per l'Overlay possiamo scegliere un colormap alternativo,
            # ad esempio COLORMAP_HOT o COLORMAP_INFERNO, per contrastare lo spettrogramma originale

            '''
            Il processo è lo stesso di quello descritto per le cam:

            I valori del CAM (normalizzati) vengono scalati a 255 e convertiti in un'immagine in scala di grigi.
            Il colormap INFERNO viene applicato per ottenere una rappresentazione colorata (dove i valori elevati diventano in genere rossi/gialli).
            La conversione BGR→RGB assicura una visualizzazione corretta
            '''

            '''
                                                SOLUZIONE
            heatmap: Questa è l'immagine a colori, che ha 3 dimensioni (ad esempio, shape (45, 61, 3)
            '''

            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_VIRIDIS)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

            '''NOTA BENE: 
            print(f"img_norm_color shape: {img_norm_color.shape}")
            print(f"heatmap shape: {heatmap.shape}")

            img_norm_color shape: (61, 45, 3)
            heatmap shape: (45, 61, 3)

            QUINDI ANDIAMO AD INVERTIRE LE SHAPES DI img_norm_color_shape PER FARLE CORRISPONDERE A QUELLE DI heatmap 
            '''

            # Inverti le due prime dimensioni di img_norm_color
            # Ridimensiona una delle immagini per farla corrispondere
            # if img_norm_color.shape != heatmap.shape:
            # img_norm_color = cv2.resize(img_norm_color, (heatmap.shape[1], heatmap.shape[0]))

            img_norm_color = img_norm_color.transpose(1, 0, 2)


            # Sovrapponi la heatmap all'immagine originale
            # Crea l'overlay: scegliendo pesi diversi per ottenere un contrasto chiaro


            '''
            Overlay troppo sfocato e colori discordanti
            Il problema che descrivi (overlay con toni azzurri/turchesi anziché il rosso della heatmap) può derivare da:

            Differenza di colormap e blending:
            L'overlay viene creato con una combinazione di due immagini: 
                1)lo spettrogramma originale (che potrebbe avere un proprio mapping di colori) e
                2) la heatmap

            Se il bilanciamento (i pesi) è 0.5-0.5, l'influenza dello spettrogramma può "modificare" i colori della heatmap.

            Suggerimenti:

            a) Modifica i pesi in cv2.addWeighted:
            Ad esempio, prova con 0.3 per l'immagine originale e 0.7 per la heatmap, in modo che il colore della heatmap (ad es. il rosso) prevalga.

            b) Uniforma il formato dell'immagine originale:
            Se lo spettrogramma originale è in scala di grigi o usa un colormap diverso,
            considera di convertirlo in un'immagine in scala di grigi a 8 bit prima di creare l'overlay.

            c) Usa lo stesso colormap: 
            Se vuoi che l'overlay abbia colori simili a quelli della heatmap, 
            usa lo stesso colormap (qui COLORMAP_INFERNO) per entrambe e regola il blending.
            '''

            '''
                                                        SOLUZIONE 
            overlay: Questo è il risultato finale che sovrappone la heatmap sull'immagine originale. 
            La soluzione proposta suggerisce di usare cv2.addWeighted
            per combinare img_norm_color (l'immagine in scala di grigi convertita a 3 canali) e heatmap.
            '''

            '''
            2) ANDIAMO A combinare img_norm_color (l'immagine ora a 3 canali) con la heatmap usando cv2.addWeighted:

            overlay = cv2.addWeighted(img_norm_color, 0.4, heatmap, 0.6, 0)
            '''

            overlay = cv2.addWeighted(img_norm_color, 0.4, heatmap, 0.6, 0)

            #overlay = cv2.addWeighted(img_norm, 0.4, heatmap, 0.6, 0)
            #overlay = cv2.addWeighted(img_norm, 0.5, heatmap, 0.5, 0)


            '''
            3) ANDIAMO ad assegnare l'overlay al dizionario overlays (che sta memorizzando gli overlay per ciascuna classe):
            overlays[cls] = overlay
            '''

            '''SOLO UN ESEMPIO'''
            #overlays[cls] = overlay

            '''CON MEDIA'''
            # Aggiungi l'overlay alla lista per la classe
            overlays_list[cls].append(overlay)
    
    
    '''CON MEDIA'''
    mean_cams = {}
    mean_overlays = {}    
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
        mean_overlays[cls] = np.mean(np.array(overlays_list[cls]), axis=0).astype(np.uint8)
        
    #Passaggio 7: Creazione della figura finale
    
    #La prima riga mostra solo le heatmap Grad-CAM.
    #La seconda riga mostra le heatmap sovrapposte agli spettrogrammi.

    # Crea la figura con due righe e due colonne

    # -------------------------------
    # Passaggio 7: Creazione della figura finale
    # -------------------------------
    # Creiamo una figura con 2 righe e 2 colonne:
    # - Prima riga: le heatmap CAM (da 0 a 1) per ciascuna condizione.
    # - Seconda riga: l'overlay (CAM + spettrogramma) per ciascuna condizione, con etichette per gli assi.
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Imposta un titolo generale per la figura
    
    #plt.suptitle(f"Grad-CAM Mapping - Experimental Condition: {exp_cond} - Subject: {category_subject}", fontsize=12)
    
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG trial Spectrogram\nExperimental Condition: {exp_cond} - Subject: {category_subject}",
    #fontsize=10,
    #y=0.95  # Puoi regolare la posizione verticale se necessario
    #)
    
    plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    # Impostiamo l'estensione: x da 0 a 61 (canali) e y da 0 a 45 (frequenze)
    extent = [0, 61, 0, 45]
    
    
    '''
    Questo codice visualizza solo le heatmap di Grad-CAM (senza l'overlay), utilizzando il colormap "INFERNO" 
    (o un altro che preferisci) e applica l'inversione verticale (np.flipud()) per ottenere la giusta visualizzazione (se necessario).
    '''
    
    # Prima riga: Visualizza solo le heatmap (CAM)
    for j, cls in enumerate([0, 1]):
        
        # Qui usiamo il colormap INFERNO per la CAM, ma puoi modificare se preferisci
        
        '''
        np.uint8(255 * cams[cls]):
        La mappa CAM (calcolata e normalizzata) ha valori compresi tra 0 e 1.
        Moltiplicando per 255 e convertendo in uint8, ottieni un'immagine in scala di grigi a 8 bit (0-255).
        
        cv2.applyColorMap(..., cv2.COLORMAP_INFERNO):
        Applica il colormap INFERNO che trasforma la scala di grigi in un'immagine a colori, 
        dove i valori bassi saranno scuri e quelli alti appariranno in toni caldi (ad es. giallo/rosso).
        
        cv2.cvtColor(..., cv2.COLOR_BGR2RGB):
        OpenCV usa il formato BGR per impostazione predefinita. 
        Convertire in RGB assicura che l'immagine venga visualizzata correttamente (matplotlib si aspetta RGB).
        
        '''
        
        '''SOLO UN ESEMPIO'''
        #cam_img = cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_INFERNO)
        
        '''CON MEDIA'''
        cam_img = cv2.applyColorMap(np.uint8(255 * mean_cams[cls]), cv2.COLORMAP_INFERNO)
        cam_img = cv2.cvtColor(cam_img, cv2.COLOR_BGR2RGB)
        
        cam_img = np.flipud(cam_img)  # Inverte verticalmente
        
        
        #axs[0, j].imshow(cam_img, extent=extent, aspect='auto')
        #axs[0, j].set_title(f"Grad-CAM for {condition_names[cls]}", fontsize=12)
        #axs[0, j].axis('off')
        
        '''QUI AGGIUNGIAMO L'INVERSIONE DEGLI ASSI'''
        # Se necessario, inverti gli assi per ottenere la visualizzazione desiderata
        # Invertiamo verticalmente per avere le frequenze in ordine crescente (se necessario)
        
        #cam_img = np.flipud(cam_img)  # Inverte verticalmente
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[0, j].imshow(cam_img)
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        #axs[0, j].imshow(cam_img, extent=[0, 1000, 0, 26], aspect='auto')
        
        axs[0, j].imshow(cam_img, extent = extent, aspect='auto')
        
        axs[0, j].set_title(f"Grad-CAM Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[0, j].axis('off')
    
    
    '''
    Questo codice visualizza ANCHE l'overlay e le heatmap di Grad-CAM,  utilizzando il colormap "INFERNO" 
    (o un altro che preferisci) e applica l'inversione verticale (np.flipud()) per ottenere la giusta visualizzazione (se necessario).
    
    
    Per ridurre la grandezza della stringa dei canali all'interno dell'asse x (EEG Channels), puoi intervenire su vari aspetti della visualizzazione, come la dimensione del font, l'orientamento e, se necessario, l'abbreviazione o il ridimensionamento dei nomi dei canali.

    1. Ridurre la dimensione del font
    Puoi facilmente ridurre la grandezza del testo delle etichette sugli assi xticks utilizzando l'argomento fontsize dentro set_xticklabels.

    Ecco un esempio di come applicarlo:
    
        axs[1, j].set_xticklabels(channel_names, rotation=90, fontsize=6)
    
    Puoi provare a modificare la dimensione di fontsize fino a trovare quella più adatta per visualizzare i nomi dei canali 
    senza che risultino troppo grandi o sovrapposti.

    2. Abbreviare i nomi dei canali
    Se i nomi dei canali sono troppo lunghi e non si adattano all'asse x, puoi abbreviarli. 
    Ad esempio, puoi creare una lista di nomi abbreviati (ad esempio, "Fz" al posto di "Frontal Z") e usarla per l'asse x.

    Esempio:

    # Creare abbreviazioni per i canali
    abbreviated_channel_names = [name[:3] for name in channel_names]

    # Impostare le etichette abbreviate
    axs[1, j].set_xticklabels(abbreviated_channel_names, rotation=90, fontsize=6)
    
    In questo caso, i nomi dei canali verranno abbreviati ai primi 3 caratteri di ogni nome.

    3. Aumentare lo spazio tra le etichette (se necessario)
    Se le etichette sono ancora troppo vicine, puoi anche aumentare la distanza tra le etichette usando set_xticks:

    axs[1, j].set_xticks(np.arange(0.5, extent[1], 2))  # Spaziatura maggiore
    In questo modo, le etichette saranno meno affollate sull'asse x.
        '''
    
    # Seconda riga: Visualizza gli overlay con etichette degli assi
    for j, cls in enumerate([0, 1]):
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[1, j].imshow(overlays[cls])
        
        # Qui, se vuoi che l'asse y (frequenze) venga ordinato in modo crescente,
        # puoi anche invertire l'immagine verticalmente, se non è già corretto.
        
        '''SOLO UN ESEMPIO'''
        #overlay_img = np.flipud(overlays[cls])
        
        '''CON MEDIA'''
        overlay_img = np.flipud(mean_overlays[cls])
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        #axs[1, j].imshow(overlay_img, extent=[0, 1000, 0, 26], aspect='auto')
        axs[1, j].imshow(overlay_img, extent=extent, aspect='auto')
        
        axs[1, j].set_title(f"Overlay of Grad-CAM Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[1, j].set_xlabel("EEG Channels", fontsize=10)
        
        axs[1, j].set_ylabel("Frequency (Hz)", fontsize=10)
        #axs[1, j].axis('on')
        
        '''
        # Se sono disponibili i nomi dei canali, impostiamo le xticks:
        if channel_names is not None and len(channel_names) == extent[1]:
            axs[1, j].set_xticks(np.arange(0.3, extent[1], 2))
            axs[1, j].set_xticklabels(channel_names, rotation=90, fontsize = 6)
        '''
        
        # Calcola le posizioni in modo che il numero di tick corrisponda al numero di canali
        # Se sono disponibili i nomi dei canali, impostiamo le xticks:
        if channel_names is not None and len(channel_names) == extent[1]:
            num_channels = len(channel_names)
            ticks = np.linspace(0.5, extent[1] - 1, num_channels)  # crea num_channels posizioni equidistanti

            # Imposta i tick e le etichette
            axs[1, j].set_xticks(ticks)
            axs[1, j].set_xticklabels(channel_names, rotation=90, fontsize=6)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION: FREQUENCIES X ELECTRODES PER EEG STATS**

In [ ]:
'''

PER CNN2D


                                                                        NEW VERSION 27/06/2025
                                                                        
                                                                  
                                                                    VERSION FREQUENCY x CHANNELS
                                                                    
                                                                                VERSIONE C
                                                                            
                                                                    
                                                                    NORMALIZZAZIONE SOLO SU SCALA DI COLORI 
                                                                    PER HEATMAP MEDIA DEL GRADCAM 
                                                                
                                                                    SCALA LOGARITMICA PER SPETTOGRAMMA
                                                                        CONGIUNTA PER DUE CLASSI
                                                                    
                                                                    ATTENZIONE CHE VIENE FATTA IMPORTATA 
                                                                        MA PICCOLA MODIFICA,
                                                                        
                                                                    DATO CHE ORA LA RETE SAREBBE FIXED 
                                                                    E NON PIU' DINAMICA COME PRIMA!
                                                                    
                                                                    
                                                                            PER LA CNN 2D 
                                                                    
-    # -------------------------------
-    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
-    # -------------------------------
-    # Imposta il layer target (ad esempio conv3) e crea un'istanza di GradCAM
-    #target_layer = model.conv3
-    
-    target_layer = model.layers[-1][0]
-    gradcam = GradCAM(model, target_layer)

+    # -------------------------------
+    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
+    # -------------------------------
+    # Qui prendiamo direttamente il conv3 della tua CNN:
+    target_layer = model.conv3
+    gradcam = GradCAM(model, target_layer)
                                                                        
                                                                        
                                                                        
Creazione della funzione per generare le immagini associate alla GradCAM compution

FINAL VERSION WITH ULTIMATED EDITING PHASES


Spiegazione:

1) Selezione dei Campioni:
La funzione itera sul test_loader e salva il primo campione trovato per ciascuna delle due classi (0 e 1).

2) Calcolo GradCAM per ciascun campione:

Per ogni campione, si abilita il gradiente e si esegue la forward pass.
Viene scelto il target (se non specificato, quello predetto) e si esegue la backward pass per calcolare i gradienti.

- I pesi vengono calcolati come la media dei gradienti lungo le dimensioni spaziali (dim=(2,3)) e usati per eseguire una somma pesata sulle attivazioni.
- La mappa risultante viene passata attraverso una ReLU, normalizzata e upsampled per avere la stessa dimensione dell’input.

Creazione degli Overlay:
Viene normalizzata l’immagine originale e viene applicata una heatmap (usando OpenCV), quindi l’overlay viene ottenuto con cv2.addWeighted.

Costruzione della Figura:
Viene creata una figura con due righe e due colonne:

- La prima riga mostra le heatmap per ciascuna classe.
- La seconda riga mostra le sovrapposizioni (overlay) tra heatmap e spettrogramma originale.

I titoli sono personalizzati in base a exp_cond, data_type e category_subject.

Questa struttura mantiene tutta la logica necessaria (incluso il calcolo dei pesi) e la rende simile alla versione precedente,
con la differenza che il calcolo della CAM viene eseguito per campioni rappresentativi di entrambe le classi. 




Ecco una versione modificata della funzione che tiene conto che:

L'input originale ha forma (batch, frequenze, canali) con frequenze = 45 e canali = 61.

Dopo il preprocessing nella rete (permute e unsqueeze) il modello lavora con tensori di forma (batch, 61, 45, 1), cioè:

asse dei canali = 61 (che ora costituirà l’asse x della visualizzazione),

asse delle frequenze = 45 (che sarà l’asse y).

Nella visualizzazione degli overlay imposteremo l’extent su 0,61,0,45 (oltre a ruotare l’immagine per far sì che l’asse y rappresenti le frequenze).

In aggiunta, ti fornisco uno spunto su come estrarre i nomi dei canali da una tripletta di file in formato BrainVision usando MNE, 
in modo da poterli usare per etichettare l’asse x.


******

    #  - Riga 1: Istogramma della distribuzione dei valori della heatmap media RAW per ciascuna classe 
    #            rispetto alla distribuzione congiunta!
    
    #  - Riga 2: GradCAM medio della distribuzione dei valori della heatmap media per ogni classe, 
    #            a seguito della NORMALIZZAZIONE rispetto alla distribuzione congiunta!
    
    #  - Riga 3: Spettrogramma medio (RAW) rispetto ai Trial della Stessa Classe, su range logaritmico 

******

'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io


#La funzione compute_gradcam_figure serve a calcolare e visualizzare 
#le mappe di attivazione Grad-CAM per un modello CNN2D, applicandole a spettrogrammi EEG. 

#In particolare, seleziona un campione per ciascuna classe (0 e 1), calcola la Grad-CAM e costruisce una figura con:

#Prima riga → Heatmap della Grad-CAM per entrambe le classi
#Seconda riga → Heatmap sovrapposta allo spettrogramma originale
#Questa visualizzazione aiuta a interpretare su quali parti dell'immagine il modello si sta concentrando per prendere decisioni.


#Questa funzione aiuta a visualizzare le regioni attivate dalla rete CNN su immagini di spettrogrammi EEG,
#evidenziando le aree più importanti per la classificazione.

#🔹 Esempio finale:
#La figura risultante avrà due righe:

#Heatmap puro della Grad-CAM.
#Heatmap sovrapposta allo spettrogramma EEG originale.


'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''

def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, channel_names = None):
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
    
      - Riga 1: Heatmap per classe 0 e classe 1
      
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
      
      - Riga 3: Istogramma della distribuzione dei valori della heatmap media per ogni classe, 
                prima della normalizzazione centrata sulla mediana (o sul picco) della distribuzione? 
      
      - Riga 4: Δ-GradCAM della distribuzione dei valori della heatmap media per ogni classe, 
                a seguito della normalizzazione centrata sulla mediana (o sul picco) della distribuzione? 
                
      - Riga 5: Spettrogramma medio (raw) per i trial di ciascuna classe.
      
      
      
    I titoli e le etichette degli assi sono personalizzati:
    
    - L'asse x rappresenta il tempo (ms) e l'asse y le frequenze (Hz) (solo per la riga overlay)    
    - I titoli dei subplot usano i nomi delle condizioni estratte automaticamente da 'exp_cond'
        (assumendo che exp_cond sia del tipo "th_resp_vs_pt_resp"), data_type e category_subject
    
    Il calcolo della CAM include il passaggio:
       weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
       cam = torch.sum(weights * activations, dim=1)
       cam = F.relu(cam)
    """
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    # Imposta il layer target (ad esempio conv3) e crea un'istanza di GradCAM
    
    '''SCOMMENTATO QUESTA RIGA'''
    target_layer = model.conv3
    
    '''COMMENTATO QUESTA RIGA'''
    #target_layer = model.layers[-1][0]
    
    gradcam = GradCAM(model, target_layer)
    
    # Estrai i nomi delle condizioni separando exp_cond (es: "th_resp_vs_pt_resp")
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    
    '''SOLO UN ESEMPIO'''
    # Dizionari per salvare un campione per ciascuna classe
    #samples = {}      # Qui salveremo il sample input per ogni classe 
    #labels_found = {} # Per tracciare se abbiamo già trovato un esempio per ciascuna classe di etichette
    
    '''CON MEDIA'''
    
    #Ora che ogni classe ha una sua chiave nel dizionario samples, non c'è più bisogno di usare labels_found 
    #per verificare la presenza di entrambe le classi.
    #In precedenza, stavi iterando nel test_loader e verificando la presenza di almeno un esempio per entrambe le classi (0 e 1),
    #ma ora i dati vengono direttamente organizzati nel dizionario in base alla loro classe. Quindi, se la classe non esiste nel dataset,
    #semplicemente non avrà una chiave nel dizionario samples.
    #Il controllo finale if 0 not in samples or 1 not in samples: è ancora necessario per assicurarsi che entrambe le classi siano presenti.
    #Se manca una classe, possiamo ancora uscire con un messaggio di errore.
    

    # Dizionari per salvare tutti i campioni per ciascuna classe
    
    
    '''DATI ORIGINALI DEL TEST LOADER'''
    samples = {0: [], 1: []}

    # Itera sul test_loader fino a trovare gli esempi per ciascuna classe (0 e 1)
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                samples[label_int].append(inputs[i].unsqueeze(0))
            
            '''SOLO UN ESEMPIO'''
            #if label_int not in labels_found:
                #samples[label_int] = inputs[i].unsqueeze(0)  # Salva come tensore 4D
                #labels_found[label_int] = True
            #if 0 in labels_found and 1 in labels_found:
            #    break
        #if 0 in labels_found and 1 in labels_found:
        #    break
        
    
    '''TEST_LOADER RAW (DATI NON STANDARDIZZATI)'''
    samples_raw = {0: [], 1: []}
    
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
    
    
    # Se non troviamo entrambi gli esempi, esci con un messaggio
    #if 0 not in samples or 1 not in samples:
    #    print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
    #    return None

    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Per ciascun campione, calcola la GradCAM
    #cams = {} # Qui salveremo la mappa CAM per ogni classe
    #overlays = {} # Qui salveremo l'overlay (CAM + spettrogramma)
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''

    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}

    
    for cls in [0, 1]:
        
        for sample_input in samples[cls]:

            '''SOLO UN ESEMPIO'''    
            #sample_input = samples[cls]

            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            #print(f"\033[1mSHAPE OF SAMPLE_INPUT: {sample_input.shape}\033[0m")

            # Assicurati che sample_input sia nel formato atteso dal modello (come nel forward)
            #if sample_input.dim() == 3:  # Supponiamo che abbia shape (batch, 45, 61)
            #    sample_input = sample_input.permute(0, 2, 1).unsqueeze(3)  # Ora (batch, 61, 45, 1)


            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)
            
            
            '''
            
            TUTTO IL PASSAGGIO DELLO STEP 5 
            
            OSSIA NORMALIZZAZIONE i.e.,  NEL SENSO DI RISCALATURA NEL RANGE 0-1 + UPSAMPLING 
            
            (CHE SERVIVA PER UNIFORMARE I VALORI E ADATTARSI ALLA DIMENSIONE DELLA IMMAGINE ORIGINALE 
            PER VEDERE UN SOLO ESEMPIO DELLA CLASSE RISPETTO ALLA MAPPA DI ATTIVAZIONE E ALL'OVERLAY
            DEL GRADCAM RISPETTO ALLA IMMAGINE ORIGINALE)

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            
            NON SERVE PIU', AD ECCEZIONE DI QUESTE RIGHE CHE ORA TI RIMETTO QUI SOTTO!'''
            
            '''
            
            
            ✅ Cosa fa correttamente questo codice:
            
            Estrae i campioni da test_loader separandoli in samples[0] e samples[1].
            
            Per ogni campione di ogni classe:
            
            Calcola la Grad-CAM raw (senza riscaling),
            La interpola per adattarla alla dimensione originale (n_freq, n_time)
            Applica ReLU per tenere solo le attivazioni positive (come da standard Grad-CAM)
            La converte in NumPy e la salva in cams_list[cls].
            
            Alla fine, fa la media delle CAM raw per ciascuna classe:
            
            mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
            
            🔍 Stato attuale del dato
            
            cams_list[cls] → lista di array cam 2D non normalizzati, uno per ogni trial.
            mean_cams[cls] → array 2D (frequenza × tempo), media dei trial per ciascuna classe.
            
            La normalizzazione Z-score congiunta la farai dopo, sulla base di mean_cams.
            
            '''

            target_size = (sample_input.shape[1], sample_input.shape[2])
            cam = F.interpolate(cam.unsqueeze(1), size = target_size, mode='bilinear', align_corners=False)

            
            # squeeze 
            cam = cam.squeeze()                 # tensor 2D
            
            
            # Infine sposti su CPU e passi a numpy
            cam = cam.cpu().numpy()

            '''SOLO UN ESEMPIO'''
            #cams[cls] = cam

            '''CON MEDIA'''
            # Aggiungi la mappa del singolo esempio alla lista per la classe (per poi dopo farci la media dentro mean_cams!)
            cams_list[cls].append(cam)
    
    # ============================================================
    # Calcolo dello heatmap media dei valori (raw) per ciascuna classe
    # ============================================================
    
    mean_cams = {}
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)


    # =======================================================
    # Calcolo dello spettrogramma medio (raw) per ciascuna classe
    # =======================================================
    
    '''
    
    Cosa fa questo codice?
    ✅ Calcola lo spettrogramma medio per ogni classe (0 e 1) prendendo i trials da samples e facendo la media sulla prima dimensione (batch).
    ✅ Plotta lo spettrogramma medio nella quarta riga del grafico finale, con una colonna per ogni classe.
    ✅ Usa una colormap jet per una migliore visualizzazione.
    ✅ Evita errori: Se una classe non ha dati, non plotta nulla per quella colonna.
    
    '''
    
    #Sì, l'errore indica che stai cercando di convertire un tensore PyTorch che richiede il calcolo del gradiente
    #in un array NumPy direttamente con .numpy(), cosa che non è permessa.
    
    
    mean_raw_spectrograms = {}
    for cls in [0, 1]:
        
        '''ATTENZIONE CHE QUESTO DIVENTA samples_raw'''
        if len(samples_raw[cls]) > 0:
            # Stacka tutti i trials per la classe e calcola la media sul batch (dimensione 0)
            
            mean_raw_spectrograms[cls] = torch.cat(samples_raw[cls], dim=0).mean(dim = 0).detach().cpu().numpy()
        else:
            mean_raw_spectrograms[cls] = None
            
            '''
            Nel tuo caso l'input ha la forma:

                (batch, 45, 61), dove:

                45 rappresenta le frequenze (asse y)
                61 rappresenta i canali (asse x)

            Se vuoi visualizzare lo "spettrogramma medio" per ogni classe come un'immagine 2D (45 × 61), 
            devi mediare solo sul batch (cioè, sui trial) e lasciare intatte le dimensioni di frequenza e canali. 
            
            Aver mediato anche sui canali (usando mean(dim=(0,1))) porterebbe a un vettore 1D (di forma (61,)), mentre quello che ti serve è una matrice 2D.

            Quindi, la riga corretta per calcolare lo spettrogramma medio è:

                mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=0).detach().cpu().numpy()
            
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************
            SPIEGAZIONE
            
            In questo modo:

                torch.cat(samples[cls], dim=0) concatena tutti i trial per quella classe, ottenendo un tensore di forma (num_trials, 45, 61).
                .mean(dim=0) calcola la media lungo la dimensione del batch, restituendo un tensore di forma (45, 61).
                Infine, .detach().cpu().numpy() converte il risultato in un array NumPy, adatto per imshow.
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************

            '''
            
    '''
    # =======================================================
    # Passaggio Finale: Creazione della figura finale
    # Ora la figura ha 3 righe:
    
    #  - Riga 1: Istogramma della distribuzione dei valori della heatmap media per ciascuna classe 
    #            normalizzata rispetto alla distribuzione congiunta!
    
    #  - Riga 2: GradCAM medio della distribuzione dei valori della heatmap media per ogni classe, 
    #            a seguito della normalizzazione rispetto alla distribuzione congiunta!
    
    #  - Riga 3: Spettrogramma medio (raw) rispetto ai Trial della Stessa Classe, su range logaritmico 
    # =======================================================
    
    
    Quando devo plottare l'istogramma dei valori di ogni heatmap media solamente (riga 3), 
    devo plottarli in base alla normalizzazione rispetto alla distribuzione congiunta.
    
    Quindi, devo plottarli in base al range minimo e massimo della intera distribuzione congiunta, quando è stata normalizzata!
    Di conseguenza devo fare
    
    1) Prendere la Media delle CAM per ogni classe (già fatto)
    2) Costruzione distribuzione congiunta raw
    3) Calcolo Media e Deviazione Standard della Distribuzione Congiunta
    4) Normalizzazione Z-score della Distribuzione Congiunta
    
    5) Prendo il range minimo e massimo della Distribuzione Congiunta Normalizzata
    
    Ossia, il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta,
    dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione.
    
    Quindi, dovrei ricreare un'altra variabile che contiene i valori normalizzati di entrambe le distribuzioni assieme,
    ossia una cosa del tipo
    
    normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    e da questa prendere il minimo ed il massimo!
    
    
    '''
    
    
    # Creiamo una figura con 4 righe e 2 colonne
    #fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    #plt.suptitle(f"Grad-CAM Mapping and Overlay over EEG Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    #fig, axs = plt.subplots(4, 2, figsize=(12, 20))
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    # Creiamo una figura con 3 righe e 2 colonne
    fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    plt.suptitle(f"Grad-CAM Mapping over EEG Trials\nExperimental Conditions: {exp_cond}", fontsize=15)
    
    plt.tight_layout()  # Regola automaticamente la spaziatura globale
    plt.subplots_adjust(hspace = 0.5, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    
    
    
    # Impostiamo l'estensione: x da 0 a 61 (canali) e y da 0 a 45 (frequenze)
    extent = [0, 61, 0, 45]
    
    #extent = [0, 64, 0, 45]
    
    
   
    # PLOT RIGA 1: Visualizzazione degli istogrammi della distribuzione dei valori delle heatmap medie RAW
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA
        
    '''
    Questi valori rappresentano la distribuzione delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta!

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Istogramma dei valori raw medi di ogni classe (su distribuzione congiunta!): Stai visualizzando un istogramma di questi valori medi, 
                                                           sulla distribuzione congiunta, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                          
                                                           - calcolo minimo e massimo a seguito della normalizzazione (?) e non prima
                                                           - faccio i plot di entrambe delle heatmap medie raw,
                                                             ma rispetto a distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello RAW!
                                                        
    
    N.B. PER IL NOME DEL TITOLO DEL PLOT
    
    Perché "Grad-CAM value" può creare confusione:
    Il termine "Grad-CAM value" potrebbe sembrare che faccia riferimento direttamente ai valori generati dalla mappa Grad-CAM finale. 
    Ma in realtà, i valori che stai trattando sono le attivazioni mediate e clippate, che formano la heatmap. 
    L'istogramma che stai tracciando rappresenta la distribuzione delle attivazioni prima della normalizzazione.

    Riepilogo
    Quindi, questi valori sono attivazioni pesate per ciascun pixel della mappa Grad-CAM, e mediati per classe. 
    Il processo di normalizzazione che segue (basato sui percentili) serve a enfatizzare il contrasto in modo da focalizzarsi sulle aree più significative 
    per la previsione.

    Per rispondere alla tua domanda: sì, è corretto dire che stai visualizzando la distribuzione delle attivazioni pesate prima della normalizzazione
    per migliorare il contrasto, ma è meglio riferirsi a questi valori come valori di attivazione della mappa Grad-CAM o valori della heatmap Grad-CAM, 
    piuttosto che "Grad-CAM value" che potrebbe risultare ambiguo.

    Se vuoi, puoi anche aggiungere una nota nella visualizzazione dell'istogramma che chiarisca il processo:
    
    axs[2, j].set_title(f"Histogram of Heatmap Activation Values (Raw, before Normalization) Class {condition_names[cls]}", fontsize=12)
    oppure
    axs[2, j].set_title(f"Histogram of Mean Heatmap Activation Values - Class {condition_names[cls]}", fontsize=12)
    
    è molto chiara e corretta!
    Indica perfettamente che stai visualizzando l'istogramma dei valori di attivazione medi della heatmap, 
    senza fare confusione sul fatto che si tratti di valori medi per ciascuna classe.
    
    In sintesi, questa frase comunica in modo preciso che stai mostrando la distribuzione delle attivazioni mediate dalla mappa Grad-CAM
    per una specifica classe. Quindi sì, va benissimo!
    
    '''
    
    #PER PLOT RIGA 1 
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    # Il range minimo e massimo su cui plottare entrambe le heatmap medie raw in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta raw
    
    vmin_raw = all_vals_raw.min()
    vmax_raw = all_vals_raw.max()
    
    
    # Prima riga: Visualizza l'istogramma della heatmap media rispetto alla distribuzione congiunta!
    for j, cls in enumerate([0, 1]):
        
        # Calcola l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
        axs[0, j].hist(mean_cams[cls].flatten(), bins= 'auto', color='blue', edgecolor='black')
        #axs[0, j].set_title(f"Histogram of Mean Grad-CAM values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_title(f"Histogram of Mean Heatmap Activation Values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_xlabel("Grad-CAM value", fontsize=10)
        axs[0, j].set_ylabel("Frequency", fontsize=10)
        
    
    
    # PLOT RIGA 2: Visualizzazione dei valori delle heatmap medie delle due classi
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    
    '''
    Questi valori rappresentano le heatmap medie delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta NORMALIZZATA

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Calcolo la distribuzione congiunta dei valori raw medi di ogni classe (su distribuzione congiunta!): 
        Stai visualizzando un istogramma di questi valori medi, sulla DISTRIBUZIONE CONGIUNTA, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                           
                                                           - calcolo media e deviazione standard delle distribuzione congiunta
                                                           - faccio la normalizzazione della distribuzione congiunta
                                                           
                                                           - calcolo minimo e massimo a seguito della normalizzazione e non prima
                                                             della distribuzione congiunta normalizzata
                                                           
                                                           - faccio i plot di entrambe delle heatmap medie normalizzate,
                                                             ma rispetto alla distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello NORMALIZZATO!
                                                        
    '''
    
    '''SOPRA ABBIAMO CREATO --> all_vals_raw'''
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    #all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    #Calcolo media e deviazione standard della distribuzione congiunta dei valori (raw) delle heatmap medie di entrambe le classi 
    #joint_mean = np.mean(all_vals_raw)
    #joint_std = np.std(all_vals_raw)
    
    # Normalizzazione Z-score della distribuzione congiunta
    #normalized_mean_cams = {}
    
    #for cls in [0, 1]:
        #normalized_mean_cams[cls] = (mean_cams[cls] - joint_mean) / joint_std

    # Il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione
    
    #normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    #vmin_normalized = normalized_all_vals.min()
    #vmax_normalized = normalized_all_vals.max()
    
    vmin_normalized = all_vals_raw.min()
    vmax_normalized = all_vals_raw.max()
    
    
    '''
    # Opzione: normalizzazione robusta con percentili
    vmin_normalized, vmax_normalized = np.percentile(all_vals_raw, [5, 95])
    '''

    # Seconda riga: Mean heatmap di ogni classe normalizzata a partire dalla distribuzione congiunta ( = di entrambe le classi)
    for j, cls in enumerate([0, 1]):
    
        
        im = axs[1, j].imshow(
            mean_cams[cls],
            #normalized_mean_cams[cls], #QUI LA RENDO IN 2D, NON IN 1D COME PRIMA
            #delta,
            #cmap='seismic',
            cmap='RdYlBu_r',
            vmin= vmin_normalized, vmax= vmax_normalized,
            #extent=[0, 64, 0, 45],
            extent=[0, 61, 0, 45],
            aspect='auto',
            origin='lower'
        )
    
        
        # → calcola 6 tick equi-spaziati
        ticks = np.linspace(vmin_normalized, vmax_normalized, 6)  
        
        cbar = fig.colorbar(
            im,
            ax=axs[1, j],
            orientation='horizontal',
            pad=0.12,
            ticks=ticks)
        
        cbar.ax.set_xticklabels([f"{t:.4f}" for t in ticks])

        axs[1, j].set_title(f"Mean Grad-CAM Heatmap (Raw) - Class {condition_names[cls]}", fontsize=12)
        
        '''QUESTA NON CONSENTE DEFINIZIONE ASSI!'''
        #axs[1, j].axis('off')
        
        axs[1, j].axis('on') 
        axs[1,j].set_xlabel("EEG Channels")
        axs[1,j].set_ylabel("Frequency (Hz)")
        
        # Calcola le posizioni in modo che il numero di tick corrisponda al numero di canali
        # Se sono disponibili i nomi dei canali, impostiamo le xticks:
        if channel_names is not None and len(channel_names) == extent[1]:
            num_channels = len(channel_names)
            ticks = np.linspace(0.5, extent[1] - 1, num_channels)  # crea num_channels posizioni equidistanti

            # Imposta i tick e le etichette
            axs[1, j].set_xticks(ticks)
            axs[1, j].set_xticklabels(channel_names, rotation=90, fontsize=6)
        else:
            axs[1, j].axis("off")
        
    print(f"\033[1mRange heatmap raw globale (vmin_raw, vmax_raw): {vmin_normalized}, {vmax_normalized}\033[0m")
    
    # PLOT RIGA 3: Spettrogramma medio (raw) per ciascuna classe log-scaled
    
    '''
    Spiegazione delle modifiche aggiunte:

    1) Calcolo dello spettrogramma medio raw:

    Dopo aver raccolto i campioni nel dizionario samples, viene creato il dizionario mean_raw_spectrograms.
    Per ogni classe, i tensori vengono concatenati lungo la dimensione batch e si calcola la media sul batch (dim=0).
    
    Poi, però, ogni spettogramma medio deve congiunto in una distribuzione in modo da plottare poi il valore dello spettrogramma  
    rispetto al minimo ed al massimo della distribuzione congiunta dello spettrogramma medio di entrambe le classi! 
    
    Il risultato viene convertito in un array NumPy per il plotting.

    '''
    
    # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
    #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
    
    '''SE VOLESSI RESTRINGERE TRA 5° e 95° PERCENTILE'''
    #low_raw, high_raw = np.percentile(all_vals_raw, [5, 95])
    #half_width_raw = max(abs(low_raw), abs(high_raw))   
    #vmin_raw, vmax_raw = -half_width_raw, +half_width_raw
    
    '''ALTRIMENTI, TENGO TUTTO IL RANGE, DAL MINIMO AL MASSIMO'''
    
    #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
    #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
    '''
    
    1) Qual è la differenza tra prima e ora?
    
    Prima calcolavo, dentro il for cls in [0,1], un nuovo vmin_raw_samples e vmax_raw_samples separatamente per ciascuna classe.
    Di conseguenza ogni subplot sulla riga 3 aveva la sua scala di colori, rendendo impossibile un confronto diretto visivo 
    fra le due condizioni.
    
    Ora invece calcolerai una sola volta il log-power medio di entrambe le classi, ne ricavi un unico array congiunto,
    quindi ne estrai un solo vmin e vmax. Questo ti garantisce che entrambi i subplot della riga 3 useranno la stessa scala di colori.


    Per far sì che tutte e due le condizioni usino lo stesso minimo e massimo, sposto la raccolta dei limiti fuori dal ciclo,
    usando la distribuzione congiunta dei log-power di entrambe le classi
    
    vmin_raw_samples e vmax_raw_samples li calcoli una volta sola, su tutti i valori logaritmici concatenati.
    Entrambe le mappe usano esattamente lo stesso range, così le barre dei colori saranno allineate.
    
    Con questa modifica:

    log_mean_power contiene già i valori in scala logaritmica.
    vmin_raw_samples e vmax_raw_samples sono condivisi fra entrambe le colonne.
    Ogni subplot userà la stessa “barretta” di colore, quindi potrai confrontare direttamente “deep blues” e “reds” delle due condizioni.


    '''
    
    # 1. Calcola i log-power medi per ciascuna classe
    log_mean_power = {
        cls: np.log1p(mean_raw_spectrograms[cls])
        for cls in [0,1]
    }

    # 2. Raccogli TUTTI i valori in un unico array
    all_log_vals = np.concatenate([
        log_mean_power[0].flatten(),
        log_mean_power[1].flatten()
    ])

    # 3. Estrai un unico vmin/vmax condiviso
    vmin_raw_samples = all_log_vals.min()
    vmax_raw_samples = all_log_vals.max()
    
    
    # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
    for j, cls in enumerate([0, 1]):
        
        #if mean_raw_spectrograms[cls] is not None:
        if log_mean_power[cls] is not None:    
            
            #Trasformo in scala logaritmica i miei dati EEG sulla spettro medio di ogni classe
            #mean_raw_spectrograms[cls] = np.log1p(mean_raw_spectrograms[cls])
            
            # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
            #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
            
            #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
            #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
            
            im = axs[2, j].imshow(log_mean_power[cls],
                                  #mean_raw_spectrograms[cls], 
                                  extent= extent,
                                  aspect='auto', 
                                  cmap='jet', 
                                  vmin = vmin_raw_samples, vmax = vmax_raw_samples,
                                  origin='lower')
            
            axs[2, j].set_title(f"Log-Scaled Mean Raw Spectrogram - Class {condition_names[cls]}", fontsize=12)
            axs[2, j].set_xlabel("EEG Channels", fontsize=10)
            axs[2, j].set_ylabel("Frequency (Hz)", fontsize=10)
            
        
            '''
            ATTENZIONE QUI CHE C'ERA UN GRAVE ERRORE
            
            --> fig.colorbar(im, ax=axs[3, j]) 
            
            #Qui la Color Bar Verticale sarebbe 
            #scala dello spettrogramma raw, finita per sbaglio sul Δ-GradCAM perché hai scritto ax=axs[3,j] invece di ax=axs[4,j].
            
            
            La barra VERTICALE (CHE DOVEVA STAR NELLA 5° RIGA!!!!) della color bar accanto alla heatmap ti sta mostrando
            
            i VALORI ASSOLUTI della Grad-CAM (nel tuo caso non normalizzati, quindi scala di milioni --> variabile hist_data
            ossia l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
            '''
    
            fig.colorbar(im, ax=axs[2, j])
            
            axs[2, j].axis('on')
            
            # Calcola le posizioni in modo che il numero di tick corrisponda al numero di canali
            # Se sono disponibili i nomi dei canali, impostiamo le xticks:
            if channel_names is not None and len(channel_names) == extent[1]:
                num_channels = len(channel_names)
                ticks = np.linspace(0.5, extent[1] - 1, num_channels)  # crea num_channels posizioni equidistanti

                # Imposta i tick e le etichette
                axs[2, j].set_xticks(ticks)
                axs[2, j].set_xticklabels(channel_names, rotation=90, fontsize=6)
        else:
            axs[2, j].axis("off")
            
            
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

In [ ]:
'''

                                                                        NEW VERSION 26/07/2025
                                                                        
                                                                
                                                                        
                                                                  
                                                                    VERSION FREQUENCY x CHANNELS
                                                                            
                                                                     ****** PER GRID 2D! ******
                                                                      ****** MULTI BAND******
                                                                      
                                                                      PER CONVOLUZIONE 3D (PURA)
                                                                              +
                                                                      PER CONVOLUZIONI SEPARABILI
                                                                      
                                                              
                                                                CON VALORI MEAN GRADCAM e MEAN RAW POWER
                                                                            SU STESSA SCALA
                                                                PER OGNI CLASSE E BANDA DI FREQUENZA!
                                                                
                                                                        ^^^^^SENZA COMMENTI^^^^^
                                                                        ^^^^^            ^^^^^
                                                                        
                                                                        

'''

'''
SINTESI DELLA FUNZIONE check_negative_residuals


| Blocco                          | Scopo                                                                                                                                                                                                                                                                                                                                                                                                                                         | Perché serve                                                                                                                                                                                                                                                                                                                                                                                   |
| ------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. eps / “dynamic tol”**      | Calcola una soglia *dinamica* sotto la quale i valori negativi sono quasi certamente puro rumore numerico.<br><br>`python<br>eps32 = np.finfo(np.float32).eps  # ≈ 1.19e‑7<br>dynamic_tol = -eps32 * max(arr.max(), 1.0)`                                                                                                                                                                                                                     | *machine‑epsilon* (ε) è l’errore di arrotondamento massimo relativo per `float32` vicino a 1.<br>Moltiplicandolo per il massimo **positivo** trovato nella mappa definiamo una “fascia di tolleranza” proporzionale alla scala reale del dato. Tutto ciò che cade **sotto** `dynamic_tol` è troppo piccolo perché sia fisico — è, con ottima probabilità, soltanto rumore di rappresentazione. |
| **2. Negativi “significativi”** | Conta e, se ce ne sono, stampa quante celle sono < `dynamic_tol`, quanto è il minimo osservato e la tolleranza stessa.                                                                                                                                                                                                                                                                                                                        | Ti permette di capire a colpo d’occhio se il preprocessing ha generato valori negativi (che non dovrebbero esistere in potenza) che **superano** il rumore ammesso.                                                                                                                                                                                                                            |
| **3. Log‑hint**                 | Se *non* ci sono negativi significativi, calcola il **dynamic range** della mappa (max/min **> 0**) e decide se suggerire la scala log.<br><br>`python<br>positive = arr[arr > 0]<br>if positive.size == 0:<br>    ratio = np.inf          # tutto zero<br>else:<br>    min_pos = positive.min()<br>    max_pos = positive.max()<br>    ratio   = max_pos / max(min_pos, 1e‑12)<br>note = "LOG consigliato" if ratio > 1e3 else "lineare ok"` | – Ignoriamo gli zeri: la scala log non li supporta.<br>– Aggiungiamo un “cuscinetto” di `1e‑12` per evitare div/0.<br>– Se il range è > 10³ (tre ordini di grandezza ≃ “la potenza massima è **> 1000 ×** la minima”), la lettura lineare diventa poco informativa → meglio log10.                                                                                                             |



In sintesi

Prima parte → caccia ai negativi “numeric‑noise” tramite ε.

Seconda parte → valuta solo i valori positivi e suggerisce log‑scale quando il dynamic‑range supera ~3 decadi (≈ 10³).


Quando il codice passa in log‑scale?
Raccogli tutte le mappe (di entrambe le classi e di tutte le bande)


all_mean_pow = np.concatenate([...])

Filtra i positivi e trova vmin_pow (con un 10 % di margine per non “appiattire” il minimo nella color‑bar)

positive_vals = all_mean_pow[all_mean_pow > 0]
vmin_pow = positive_vals.min() * 0.9 if positive_vals.size else 1e‑12
vmax_pow = all_mean_pow.max()

Decidi

use_log = vmax_pow / max(vmin_pow, 1e‑12) > 1e3
Se la potenza massima è > 1000 × la minima positiva, usare LogNorm.

Come si spiega “tre ordini di grandezza”?

“Il massimo è mille volte il minimo”.
“Dynamic‑range di 3 decadi”.
Oppure “max/min > 10³”.



Riassunto finale
ε: misura il rumore di quantizzazione, ti dice quando un (piccolo) negativo è solo un effetto di arrotondamento.

Dynamic‑range: se la banda ha valori reali che variano più di 10³ ×, la scala log10 rende le differenze leggibili senza “schiacciare” i dettagli bassi.

La funzione: un’unica utility per

diagnosticare residui numerici,

suggerire in automatico la rappresentazione (lineare / log) più sensata per i tuoi plot di potenza.

Così il flusso diventa:


check_negative_residuals(...)   # → log “SCALA LOG consigliata”
↓
use_log = True                  # ratio > 1e3
↓
plot con LogNorm + LogLocator   # color‑bar pulita, dettagli visibili




Perché vedi ratio = inf
* Nelle bande Beta e Gamma la tua mappa media è tutta a zero (o comunque tutti i valori ≤ ε).
* Con soli zeri il vettore positive = arr[arr > 0] è vuoto, quindi lo tratto come “dynamic‑range infinito” per evitare la divisione per 0.
* In realtà non hai un’«escursione infinita»; semplicemente non hai segnale in quelle bande → la scala log non aggiungerebbe nulla.

Se vuoi evitare quel suggerimento “falso‑positivo”, basta cambiare la logica così (l’ho già indicato ma lo riscrivo compatto):

positive = arr[arr > 0]
if positive.size < 2:          # 0 o 1 valore positivo → niente dinamica utile
    ratio = 0                  # forza il consiglio a “lineare”
else:
    ratio = positive.max() / max(positive.min(), 1e-12)
    
    
Linear vs log: quale usare davvero?
Banda	Dynamic‑range (≈ max/min)	Scala consigliabile
Delta ‑ Theta	3‑11 ×	Lineare: già leggibile.
Alpha	12 ×	Ancora lineare (o log, ma non cambia molto).
Beta – Gamma	0 (tutti zeri)	Log inutile: non c’è potenza da mostrare.

Di conseguenza:

Mantieni la scala lineare globale come nel tuo blocco finale.

Se in altri dataset vedrai rapporti > 1 000 con almeno 2 valori positivi, allora attiva la parte LogNorm.



'''

def check_negative_residuals(band_names, tensor_dict, tag, log_hint=True):
    
    """
    • band_names   : lista di stringhe  (lunghezza = n_bands)
    • tensor_dict  : {cls: [np.ndarray(H,W), … n_bands]} --> dict  {cls: [np.ndarray(H,W), … 5 bande]}
    • tag          : prefisso stampato nel log --> string visualizzato nel log
    • log_hint     : se True mostra il rapporto max/min ⇒ aiuta a decidere
                     se usare la scala log nei plot.
    """
    
    
    #https://numpy.org/doc/2.1/reference/generated/numpy.finfo.html
    eps32 = np.finfo(np.float32).eps        # 1.19e‑7
    for cls in [0, 1]:
        for b, b_name in enumerate(band_names):
            arr   = tensor_dict[cls][b]
            
            # Soglia dinamica = −eps * valore_massimo_della_mappa
            
            #dynamic_tol = -eps32 * arr.max()      # tolleranza dinamica
            
            dynamic_tol = -eps32 * max(arr.max(), 1.0)   # evita max==0
            
            neg_mask  = arr < dynamic_tol # “negativi significativi”
            
            min_v, max_v = float(arr.min()), float(arr.max())
            
            if np.any(neg_mask):
                print(f"Valori sotto la soglia per classe e banda:\n")
                
                #forma	cosa fa	quando differisce
                #np.count_nonzero(neg_mask)	converte il bool‑array in int (True→1, False→0) e somma gli 1	è sempre un intero Python
                
                #neg_mask.sum()	chiama il metodo .sum() dell’ndarray; 
                #per tipo bool fa esattamente la stessa somma di sopra	restituisce uno numpy.int_ (stesso valore, differente tipo)
                
                n_neg   = np.count_nonzero(neg_mask)
                
                min_val = arr.min()
                
                print(f"[{tag} {b_name}] class={cls}  band={b_name:<6}  "
                      f"neg={n_neg}  min={min_val:.3e}  tol={dynamic_tol:.3e}")
            else:
                print(f"Nessun valore sotto la soglia per classe {cls} e banda {b_name}\n")
                print(f"Definisco il range minimo e massimo per classe {cls} e banda{b_name} :\n")
                
                # ‑‑ opzionale: suggerimento scala log
                if log_hint:                    
                    #if max_v == 0 or min_v == 0:
                    positive = arr[arr > 0]           # considera solo i valori > 0
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    if positive.size == 0:
                        ratio = float('inf')          # tutto zero → range “infinito”
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    else:
                        min_pos = positive.min()
                        max_pos = positive.max()
                        ratio   = max_pos / max(min_pos, 1e-12)
                    note = f"\033[1mSCALA LOG per plots consigliata\033[0m" if ratio > 1e3 else f"\033[1mSCALA LINEARE per plots consigliata ok\033[0m"
                    print(f"        dynamic‑range ≈ {ratio:8.1f}  → {note}")
            print()
                
                
                
import torch.nn as nn

def model_has_cudnn_rnn(model):
    """Ritorna True se il modello usa LSTM/GRU/RNN supportati da CuDNN."""
    return any(isinstance(m, (nn.LSTM, nn.GRU, nn.RNN)) for m in model.modules())


from matplotlib.ticker import FixedLocator

from matplotlib.colors import LogNorm
from matplotlib.ticker import (LogLocator, LogFormatterMathtext,
                               ScalarFormatter)


'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''
def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, channel_names=None, debug = False):
    
    
    '''SOLO PER I MODELLI OTTIMIZZATI CON ANCHE LA LSTM'''
    
    #Solo i modelli con LSTM entrano in questo giro; gli altri non cambiano di stato.
    #Con questa sequenza:
    #non ottieni più l’errore “cudnn RNN backward…”;
    #la rete “si comporta” come in eval (Dropout off, BN congelato) mentre calcoli le CAM;
    #l’ambiente di chiamata (il tuo loop di testing) riceve il modello esattamente nello stato in cui l’aveva passato alla funzione compute_gradcam_figure
    

    ### Perché serve model.train() anche se la CAM è presa prima della LSTM
    
    #Il backward, per arrivare dal loss (o dal logit scelto) fino al tuo layer conv3, deve comunque attraversare l’LSTM che sta più avanti nella rete.
    #Le implementazioni CuDNN degli RNN (LSTM/GRU) alzano un’eccezione se provi a chiamare tensor.backward() mentre il modulo è in modalità eval().
    #RuntimeError: cudnn RNN backward can only be called in training mode
    #Quindi, anche se la CAM è calcolata su conv3, devi mettere l’intero modello in train() per il tempo del backward.
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    ### Che cos’è model.training
    
    #model.training è un semplice flag booleano (impostato da nn.Module.train() / nn.Module.eval()), ereditato da tutti i sotto‑moduli.
    #Con was_training = model.training ricordi in che stato era il modello (quasi sempre False, cioè eval, nel tuo flusso)
    #per poterlo ripristinare dopo.
    
    #Facendo così
    
    #for m in model.modules():
    #if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                      #nn.Dropout, nn.Dropout2d, nn.Dropout3d)):
        #if m.training:         # cioè erano in train
            #m.eval()
            #frozen_layers.append(m)
    
    #Li sposti in eval uno per uno, senza toccare il resto della rete che deve restare in train() per far funzionare CuDNN‑RNN.
    
    
    ### Perché, a fine blocco, servono due ripristini
    
    #1) Riattivo i BatchNorm / Dropout che avevo forzato in eval:
    
    #for m in frozen_layers:
        #m.train()              # torna come prima
    
    #2) Riporto l’intero modello nello stato in cui si trovava prima del Grad‑CAM:
    
    #model.train(was_training)  # se era eval() torna eval, altrimenti resta train
    
    #Se non facessi il punto 1, lasceresti quei moduli permanentemente in eval anche quando, più tardi, 
    #rientri in training (per esempio in un fine‑tuning).
    #Se non facessi il punto 2, lasceresti tutto il modello in train → dropout attivo, BN che accumula statistiche, ecc.

    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❶ — se serve, abilito temporaneamente la modalità train per il modello ottimizzato che aveva ANCHE la LSTM... 
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    needs_train_mode = model_has_cudnn_rnn(model)
    
    if needs_train_mode:
        was_training = model.training      # salvo lo stato
        model.train()                      # abilito backward su CuDNN‑RNN
        
        # ➊ salvo lo stato di OGNI BN/Dropout
        
        saved = [(m, m.training) for m in model.modules()
             if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                               nn.Dropout, nn.Dropout2d, nn.Dropout3d))]
        
        model.train()                              # abilita backward su CuDNN‑RNN
        
        # ➋ congelo in ogni layer della rete gli strati di BatchNorm e Dropout
        for m, _ in saved:
            m.eval()
    
    # ------------------------------------------------------------
    # ❷ — QUI sotto metti tutto il tuo codice Grad‑CAM
    #      (forward, backward, costruzione delle mappe, plot, …)
    # ------------------------------------------------------------

    # … il tuo lunghissimo corpo della funzione rimane invariato …
    # → al momento di fare backward NON avrà più l’eccezione
    #   “cudnn RNN backward can only be called in training mode”

    
    target_layer = model.conv2b #model.conv3
    gradcam = GradCAM(model, target_layer)
    
    # Determina il target layer in base al tipo di modello
    #if isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.dw_conv1  # Per il modello separabile 2D
    #else:
        #target_layer = model.conv3  # Per il modello CNN3D
        

    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    # ✅ Raccogli TUTTI i campioni per ciascuna classe
    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    
    
    #PER IL CASO CONV 3D
    
    # Ogni mio sample è 3D, perché infatti è fatto per convoluzione 3d pura o convoluzioni separabili, 
    # Quindi ha shape  (B, C, D, H, W), dove:  
    
    #B = batch (in questo caso, per ogni singolo esempio quindi sarà 1 -> singolo esempio alla volta)
    #C = feature maps/canali (numero di feature maps estratte dalla convoluzione, o meglio anche noti come  canali convoluzionali)
    #D = depth (la dimensione di profondità del mio tensore --> 5 ossia, la potenza spettrale ad ogni banda di frequenza - i.e.,  delta, theta, alfa, beta e gamma)
    #H = height (altezza, prima dimensione SPAZIALE del mio tensore i.e., altezza griglia, canali EEG) 
    #W = width (larghezza, seconda dimensione SPAZIALE del mio tensore i.e., larghezza griglia, canali EEG)
    
    
    '''SHAPE DEI DATI ORIGINALE SAREBBE (B, 9, 9, 5)'''
    samples = {0: [], 1: []}
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples[label_int].append(inputs[i].unsqueeze(0))
                
    
    '''TEST_LOADER RAW (B, 9, 9, 5)'''
    samples_raw = {0: [], 1: []}
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
                
    # ============================================================
    # Calcolo delle Grad-CAM per ogni singola banda di frequenza
    # ============================================================
    
    n_bands = 5  # numero di canali/bande di frequenza
    band_names = ['Delta (δ)', 'Theta (Θ)', 'Alpha (α)', 'Beta (β) ', 'Gamma (γ)']  
    
    
    '''STRUTTURE DATI PER CONV3D'''
    #✅ Struttura per il GradCAM 3D, ossia qui raccolgo il GradCAM 3D di ogni esempio
    # quindi la mappa di attivazione che identifica l'attivazioni più rilevanti per la classificazione di una esemplare di una certa classe
    # sia spazialmente (height and width, ossia le dimensioni spaziali del mio tensore)
    # sia frequenzialmente (depth), ossia le attivazioni più rilevanti in base alla banda di frequenza
    
    global_cams_3d = {0: [], 1: []} # shape (D, 9, 9)
    
    #Poi qui abbiamo: 

    # ✅ Struttura: classe → banda → immagini raw di input filtrate per singola banda (senza passare dal modello)
    #tutte le mappe di potenza per la classe cls nella banda b-esima
    
    #La struttura per salvare invece lo potenza spettrale media per ogni relativa banda 
    raw_power_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    #La struttura che salverà la "fetta" del gradcam3D, ossia dove plotto solo la fetta della banda a partire dal global_cam_3d
    #Ossia per ogni esempio di una specifica classe, prenderò la mappa di attivazione spazialmente più rilevante, in base alla specifica banda di frequenza indagata

    #✅ Struttura: classe → banda → lista CAM
    #cams_per_band_3d[cls][banda]: la slice D-esima (ossia la slice frequenziale) della mappa GradCAM per ogni campione di classe cls.
    
    cams_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''STRUTTURE DATI PER CONV SEPARABLE'''

    
    global_cams_2d = {0: [], 1: []} 
    
    raw_power_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    cams_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''
    
    Da qui in giù, vado a eseguire i passaggi essenziali per:
    
    1) Calcolare il GradCAM 3D per ogni esempio,
    2) Isolare le fette (slice) per banda,
    3) Raccogliere le immagini di potenza raw per ogni banda,
    4) Calcolare le medie per classe e banda.
    
    '''


    for cls in [0, 1]:
        
        for sample_input, sample_input_raw in zip(samples[cls], samples_raw[cls]):
            
            # 1) Preparo il sample per il calcolo
            sample_input = sample_input.clone().detach().requires_grad_(True)
            
            '''
            
            Devo far squeeze perché essendo prelevati dal test loader, li avevo già resi con un 1 davanti ossia qui sopra

            samples[label_int].append(inputs[i].unsqueeze(0))
            
            perché io, è vero che prendo i dati dal test_loader che son in formato batch (batch, D, H, W)
            ma siccome poi prelevo ogni singolo esempio per metterli dentro a "samples", allora quella dimensione di batch si leva nel dizionario "samples".
            
            Per cui, quando voglio prendermi il singolo esempio per salvarmelo dentro a "raw_vol" (ossia la potenza spettrale del volume proprio di ogni esempio),
            allora devo ri-assegnare ad ogni esempio la dimensione del batch (ossia 1, ossia il singolo esempio)
            quando li salvo dentro samples. 
            
            Ed infatti, quindi quell' ".unsqueeze(0)" --> samples[label_int].append(inputs[i].unsqueeze(0)) serve proprio a questo...

            Quando poi, devo prendere il singolo esempio da salvare, nei termini di potenza spettrale di ogni esempio (come volume 3D), 
            allora devo rifare .squeeze(), per togliere nuovamente la dimensione del batch (ossia 1, ossia il singolo esempio)
            perché la rappresentazione del singolo esempio è, appunto, composta da, le sole dimensioni che costituiscono proprio il singolo esempio,
            ossia 9x9x5 -->  ossia la griglia 3D !

            E quindi, per ogni esempio, mi salvo la griglia 3d, ossia
            "raw_vol = sample_input.detach().cpu().numpy().squeeze()   # → (9, 9, 5)"
            
            E poi, la divido però per banda 
            
            "for b in range(n_bands):
                raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
            "
            '''
            
            # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
            #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
            
            raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
            
            '''CASTING IN FLOAT64'''
            #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
            
            for b in range(n_bands):
                
                
                # Determina in base al tipo di modello i dati e le shape da salvare:
                
                #if isinstance(model, CNN3D_LSTM_FC):
                #if isinstance(model, SeparableCNN2D_LSTM_FC):
                
                    #target_layer = model.dw_conv1  # Per il modello separabile 2D
                #else:
                    #target_layer = model.conv3  # Per il modello CNN3D
                    
                if isinstance(model, CNN3D_LSTM_FC):
                    
                    #raw_power_per_band_3d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda

                    raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
                    
                elif isinstance(model, SeparableCNN2D_LSTM_FC):
                    raw_power_per_band_2d[cls][b].append(raw_vol[:,:,b])

            
            # 2) Esegui il forward pass
            output = model(sample_input)
            target_class = output.argmax(dim=1).item()
            
            # 3) Esegui il backward pass
            model.zero_grad()
            target = output[0, target_class]
            target.backward()
            
            # 4) Preleva attivazioni e gradienti
            activ = gradcam.activations   # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            grads = gradcam.gradients # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            
            #Nel caso 3D dovrebbe essere

            #Media dei gradienti solo su H,W → (B,C,D,1,1)
            #w3d = torch.mean(grads, dim=(3, 4))

            # b) Sommo sui canali → (B,D,H,W)
            #cam3d = F.relu(torch.sum(w3d * activ, dim=1))
            
            #e così la shape finale sarebbe 3D con (B,C,D)
            
            #✔️ w3d è correttamente calcolato per ogni (B, C, D, 1, 1)
            #✔️ La somma su dim=1 aggrega le feature maps con pesi per ogni banda
            #✔️ ReLU rimuove componenti negative
            
            '''
            Nel caso della CNN3D calcoli una mappa Grad-CAM 3D "globale" direttamente dal layer convoluzionale 3D, ottenendo attivazioni di shape 
            (B,C,D,H,W) e quindi una CAM volumetrica per ogni trial
            
            CNN3D → calcoli una CAM 3D da attivazioni (B,C,D,H,W), una volta sola per ogni esempio.

            '''
            
            if activ.ndim == 5:  # Caso per modello CNN3D pura 
                
                # 3D Volumetric Grad-CAM
                
                # a) Media dei gradienti solo su H,W → (B,C,D,1,1)
                w3d = torch.mean(grads, dim=(3, 4), keepdim=True)

                # b) Sommo sui canali (feature maps) → (B,D,H,W)
                cam3d = F.relu(torch.sum(w3d * activ, dim=1))
                
                # c) Upsample H×W, mantenendo D intatto
                B, D, H, W = cam3d.shape
                cam_flat = cam3d.view(B*D, 1, H, W)
                cam_up   = F.interpolate(cam_flat,
                                         size=(9, 9),
                                         mode='bilinear',
                                         align_corners=False)
                
                cam_vol  = cam_up.view(B, D, 9, 9).cpu().numpy()
                
                # d) Prendi ogni batch-item
                
                '''
                Quindi qui ottengo che:
            
                1) appendo a global_cams_3d che cosa qui? il gradcam 3D ossia la mappa di attivazione di volume,
                ossia OGNI esempio (volumetrico) per ogni classe (ossia 9x9x5 ancora, di OGNI esempio)
                
                Quindi semplicemente anziché rendere il dato come 'batch, D, H, W'.. siccome prendiamo ogni esempio UNO ALLA VOLTA
                è inutile mantenere la dimensione batch (che sarebbe sempre 1, perché parliamo di ogni esempio, uno alla volta)
                
                ossia anziché fare 
                
                global_cams_3d[cls].append(cam_vol)
                
                faccio
                
                global_cams_3d[cls].append(cam_vol[0])
                
                
                E quindi, mi salvo per OGNI esempio direttamente la mappa cam 3d, per ogni banda, direttamente
                ossia ogni esempio sarà costituito da 3 dimensioni (D, H, W) anziché dire 
                
                "Ogni dato (ossia ogni esempio) è composto da (batch, D, H, W) 
                se tanto il batch = 1 (perché il batch è il singolo esempio ogni volta)
                
                e quindi significherebbe aggiungere una dimensione (quella del batch) che in realtà è inutile, 
                perché si riferisce all'esempio stesso di già!
                
                Quindi:
                👉 cam_vol[0] estrae la CAM 3D senza la dimensione "batch", che è inutile in quel contesto
                👉 Serve per poter fare medie e slicing banda per banda correttamente dopo lo stack
                👉 Questo rende compatibile il risultato finale con imshow (che accetta solo 2D o 3D RGB)
       
                2) appendo anche l'esempio volumetrico a cams_per_band_3d, MA GIA' suddiviso per banda! (per cui diventa 2d là dentro! 9x9)

                '''
                
                global_cams_3d[cls].append(cam_vol[0])
                
                for b in range(n_bands):
                    cams_per_band_3d[cls][b].append(cam_vol[0,b])
            
                '''
                Nel caso della SeparableCNN2D, invece, il layer convoluzionale è 2D e riceve in input 
                (B,5,9,9), cioè con le bande di frequenza come canali (non come profondità). 

                Questo significa che non puoi ottenere direttamente una CAM 3D nello stesso modo, 
                ma si può ottenere una CAM 2D per ogni banda, 
                "mascherando" l’input attivando una banda alla volta

                SeparableCNN2D → non hai accesso diretto a una "profondità" come in GradCAM 3D, quindi:

                Simuli la profondità attivando una banda alla volta.

                Ottieni una CAM 2D per ogni slice (banda), iterando sulle bande.

                Questo approccio ti consente di costruire comunque strutture 3D:
                cams_per_band_2d[cls][b] con b = 0...4 contiene 
                le CAM 2D relative alla banda b, ricostruendo idealmente la distribuzione tridimensionale
                '''
            
            elif activ.ndim == 4: #Caso per il modello Conv Separabili 
                
                # 1) Preparo il sample per il calcolo
                sample_input = sample_input.clone().detach().requires_grad_(True) # sample_input: (1, 9, 9, 5)
                
                
                '''
                il mio sample input ora è sempre 4D, ma a differenza di prima, io sto trattando in questo caso 
                le bande come CANALI, e non come DEPTH (della convoluzione 3d pura!)
                
                Come prima, devo togliere la dimensione del batch ...
                '''
                
                
                # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
                #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
                
                raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
                
                '''CASTING IN FLOAT64'''
                #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
                
                
                '''
                cams_per_band_2d e raw_power_per_band_2d son dentro al loop di MASKING, 
                perché entrambe si riferiscono alla banda e quindi devo essere inserite dentro al loop di masking...
                
                --> raw_power_per_band_2d[cls][b] e cams_per_band_2d[cls][b] sono dentro il loop (for b in ...) 


                
                mentre global_cams_2d è FUORI da quel loop, perché raccoglie tutti gli esempi delle gradcam, 
                considerando però le mappe di attivazione di ogni banda singolarmente 
                e le aggrega per avere una visualizzazione dell'impatto complessivo di ogni singola banda sulla decisione del modello,
                facendo vedere dove son maggiormente concentrate le attivazioni a livello spaziale TRA le bande ( = considerando TUTTE le bande assieme!)
                
                --> global_cams_2d[cls] sta dopo quel for b, raccogliendo una sola mappa 2D “complessiva” per trial
                
                In questo modo:

                raw_power_per_band_2d e cams_per_band_2d catturano tutti i trial per banda.

                global_cams_2d cattura un’unica mappa per trial, che poi aggregherò in global_mean_cams_2d per ottenere la heatmap 2D “globale”
                che comprende tutte le bande insieme

                '''
                
                for b in range(n_bands):
                    
                    #raw_power_per_band_2d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda
                    raw_power_per_band_2d[cls][b].append(raw_vol[:, :, b])
                    
                    # ✅ Creo un input mascherato con **solo** la banda b attiva
                    masked = np.zeros_like(raw_vol)  # (9, 9, 5)
                    
                    masked[:, :, b] = raw_vol[:, :, b]  # attiva solo la banda b
                    
                    #Qui lo prepari in formato 4D, come vorrebbe il modello Conv Separable
                    masked_tensor = torch.tensor(masked).unsqueeze(0).to(device)  # (1, 9, 9, 5)
                    
                    # Preparo il sample
                    masked_tensor.requires_grad_(True)
                    
                    # Forward + backward
                    output = model(masked_tensor)
                    target_class = output.argmax(dim=1).item()
                    model.zero_grad()
                    
                    target = output[0, target_class]
                    target.backward()

                    activ = gradcam.activations   # (B, C, H, W)
                    grads = gradcam.gradients     # (B, C, H, W)
                    
                    # Calcolo CAM 2D (come standard GradCAM)
                    w2d = torch.mean(grads, dim=(2, 3), keepdim=True)  # (B, C, 1, 1)
                    cam = F.relu(torch.sum(w2d * activ, dim=1))        # (B, H, W) --> # (1, H, W)
                    
                    #Riporto la shape con .unsqueeze(1) a 4D per fare interpolation
                    cam_up = F.interpolate(cam.unsqueeze(1), size=(9, 9), mode='bilinear', align_corners=False)
                    
                    #Riporto la shape con .squeeze(1) a 3D per salvare i dati
                    cam_2d = cam_up.squeeze(1).cpu().numpy()  # (B, 9, 9)

                    # ✅ Aggiungo la mappa CAM alla banda corrispondente
                    cams_per_band_2d[cls][b].append(cam_2d[0])  # prende il CAM per il sample corrente (9x9)
                
                #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
                #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:
                # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
                global_cams_2d[cls].append(cam_2d[0])
            else:
                raise RuntimeError(f"activ.ndim inatteso: {activ.ndim}")
                
                
    '''
    CASO MODELLO CON 3D PURO, dovrei fare: 

    1) per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante)

    '''
    
    if isinstance(model, CNN3D_LSTM_FC):
        
        #1) global_cams_3d → media “globale” del volume 3D Grad‑CAM
        # media sul numero di esempi per ogni classe → ottieni un array (D, H, W)
        global_mean_cams_3d = {
            cls: np.mean(np.stack(global_cams_3d[cls]), axis=0)  # da [ (1,D,H,W), … ] a (D,H,W)
            for cls in [0,1]
        }


        '''
        2) poi, dentro a raw_power_per_band_3d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_3d)
        '''

        #2) raw_power_per_band_3d → media della potenza raw per banda
        #Hai già raccolto, per ogni cls e per ogni banda b, tutte le mappe 2D raw_power_per_band_3d[cls][b] (una per trial).
        #La media diventa:

        mean_raw_power_per_band_3d = {
            cls: [ np.mean(np.stack(raw_power_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }

        # risultato: mean_raw_power_per_band_3d[cls][b] è (H,W)

        '''
        3) dentro a cams_per_band_3d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_3d_per_band) 

        '''

        #3) cams_per_band_3d → media della Grad‑CAM per banda
        #Analogamente hai raccolto tutte le slice 2D di Grad‑CAM in cams_per_band_3d[cls][b].
        #La media diventa:

        mean_cams_per_band_3d = {
            cls: [ np.mean(np.stack(cams_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_3d_per_band[cls][b] ha shape (H,W)


        '''
        Con queste tre strutture (global_mean_cams_3d, mean_raw_power_per_band_3d, mean_cam_3d_per_band) puoi

        riga 1–2: istogrammi di mean_cam_3d_per_band[cls][b]

        riga 3–4: heatmap di mean_cam_3d_per_band[cls][b]

        riga 5–6: heatmap di mean_raw_power_per_band_3d[cls][b]

        riga 7–8: slice di global_mean_cams_3d[cls] per ogni b

        '''
    
    elif isinstance(model, SeparableCNN2D_LSTM_FC):
    
        '''
        CASO MODELLO CONV SEPARABLE, dovrei fare: 

        1) per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
        all'interno di TUTTE LE BANDE ASSIME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
        e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
        ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE...

        Per la SeparableCNN2D ricostruisci un Grad‑CAM “3D” artificiale facendo 5 Grad‑CAM 2D una per ogni banda
        '''

        #global_cams_2d
        #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
        #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:

        # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
        #global_cams_2d[cls].append(cam_2d[0])

        global_mean_cams_2d = {
            cls: np.mean(np.stack(global_cams_2d[cls]), axis=0)
            for cls in [0,1]
        }
        # global_mean_cams_2d[cls] shape = (9,9)

        '''
        2) poi, dentro a raw_power_per_band_2d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_2d)
        '''

        #raw_power_per_band_2d
        #Hai già in raw_power_per_band_2d[cls][b] tutte le mappe 2D di potenza (9×9) per trial, per ciascuna banda b.
        #La media finale:


        mean_raw_power_per_band_2d = {
            cls: [ np.mean(np.stack(raw_power_per_band_2d[cls][b]), axis=0)
                  for b in range(n_bands) ]
            for cls in [0,1]
        }

        # mean_raw_power_per_band_2d[cls][b] shape = (9,9)


        '''
        3) dentro a cams_per_band_2d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_2d_per_band) 

        '''

        #cams_per_band_2d
        #Durante il masking loop appendi in cams_per_band_2d[cls][b] il CAM 2D (9×9) di ogni trial.
        #La media finale:

        mean_cams_per_band_2d = {
            cls: [ np.mean(np.stack(cams_per_band_2d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_2d_per_band[cls][b] shape = (9,9)

        '''
        Con queste tre strutture —

        mean_raw_power_per_band_2d (5 mappe 9×9),

        mean_cam_2d_per_band (5 mappe 9×9),

        global_mean_cams_2d (1 mappa 9×9) —

        puoi costruire esattamente le stesse righe di plot che avevi per il caso 3D, solo che al posto di “slice” del volume userai le CAM 2D mascherate.


        '''
    
    # Preleva la struttura corretta in base al modello
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        mean_cams_per_band = mean_cams_per_band_2d
        mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
    
    else:  # Caso per modello CNN3D
        mean_cams_per_band = mean_cams_per_band_3d
        mean_raw_power_per_band = mean_raw_power_per_band_3d
        global_mean_cams = global_mean_cams_3d
        
    
    
    
    
    
    # prima di salvare la figura, solo se richiesto vedi i valori delle potenze medie per banda e condizione sperimentale...
    
    #"tag_names=[f"{model.__class__.__name__} power"]")
    
    if debug:
        if isinstance(model, CNN3D_LSTM_FC):
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_3d,
                                     model_tag)
        else:
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_2d,
                                     model_tag)
            
    
    
    
    # Crea la figura dinamicamente in base al modello
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 2 righe per 5 colonne per modello 2D
    else:
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 5 righe per 2 colonne per modello 3D

    
    
            
    title = (
        f"Grad-CAM Mapping over EEG Trials – Experimental Conditions: {exp_cond}\n\n"
        "Row 1-2: Histogram of Mean Grad-CAM raw values for each class and frequency band\n"
        "Row 3-4: Normalized Mean Grad-CAM heatmaps for Class 0 (top) and Class 1 (bottom)\n"
        "Row 5-6: Raw power maps for Class 0 (top) and Class 1 (bottom)\n"
        "Row 7-8: Global CAM per Class"
        )
    
    plt.suptitle(title, fontsize=15)

    # Spaziatura verticale per evitare sovrapposizione
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    plt.subplots_adjust(hspace = 0.7, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    
    #PER PLOT RIGA 1-2 
    
    from matplotlib.ticker import ScalarFormatter

    # Crea un formatter per notazione scientifica
    sci_formatter = ScalarFormatter(useMathText=True)
    
    #Questa chiamata serve a forzare il range entro cui usare la notazione scientifica
    #Se non metti questo limite, il comportamento può variare leggermente a seconda della scala dei dati —
    #a volte sarà decimale (0.0001), altre volte esponenziale (1e-4), e potrebbe non essere uniforme tra subplot.
    
    sci_formatter.set_powerlimits((-3, 3))  # usa 1e-xxx se valori sono piccoli

    for b, b_name in enumerate(band_names):
    
        for j, cls in enumerate([0, 1]):
            
            # Calcola l'istogramma dei valori della heatmap media
            # rispetto alle 2 classi in base alla banda di frequenza isolata
            
            ax = axs[0, b] if cls == 0 else axs[1, b]
            
            ax.hist(mean_cams_per_band[cls][b].flatten(), bins='auto', color='blue', edgecolor='black')
            ax.set_title(f"{b_name} - Class {condition_names[cls]}", fontsize=10)
            ax.set_xlabel("Grad-CAM Value")
            ax.set_ylabel("Count")
            
            # ✅ Format tick con notazione scientifica
            ax.xaxis.set_major_formatter(sci_formatter)
    
    
    
    #PER PLOT RIGA 3-4 
    
    '''
    Concateno tutte le mean-CAM (cls 0+1, tutte le bande) in un unico array
    in modo da confrontare le Gradcam tra classi e bande tra di loro! 
    '''
    
    all_mean_cams = np.concatenate([
        mean_cams_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_cams_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_cam = all_mean_cams.min()
    vmax_cam = all_mean_cams.max()
    
    
    for b, band in enumerate(band_names):                                 
        
        for j, cls in enumerate([0, 1]):
            
            ax = axs[2, b] if cls == 0 else axs[3, b]
            
            cam = mean_cams_per_band[cls][b] 
            
            # Controlla se la forma è corretta per l'input di imshow
            assert cam.ndim == 2, f"Expected 2D array, got {cam.ndim}D array"
            
            im = ax.imshow(
                cam,
                cmap = 'RdYlBu_r',
                vmin = vmin_cam, 
                vmax = vmax_cam,
                aspect = 'equal',
                origin = 'upper'
            )
            
            ticks = np.linspace(vmin_cam, vmax_cam, 6)

            cbar = fig.colorbar(
                im, ax=ax, orientation='horizontal', pad=0.12, ticks=ticks, format='%.1e')
            
            cbar.set_ticks(ticks)
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])

            ax.set_title(
                f"{band} - Class {condition_names[cls]}",
                fontsize=10
            )

            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(
                        x, y, name,
                        ha='center', va='center',
                        fontsize=6, color='black', weight='bold'
                    )
            else:
                ax.axis("off")
    
    
    
    #PER PLOT RIGA 5-6 (SCALA LINEARE)
    
    from matplotlib.ticker import ScalarFormatter

    sci = ScalarFormatter(useMathText=True)
    sci.set_powerlimits((-2, 2))          # forza 1eX fuori dall’intervallo 1e‑2 … 1e2

    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    
    ticks = np.linspace(vmin_pow, vmax_pow, 6)
    
    # Riga 3: Mappa della potenza media rispetto a distribuzione congiunta (su ciascuna banda e classe)
    for b, band in enumerate(band_names):
        
        for cls in [0, 1]:
            ax = axs[4, b] if cls == 0 else axs[5, b]
            
            power = mean_raw_power_per_band[cls][b] 
            
            im = ax.imshow(
                power, 
                cmap='jet',
                vmin= vmin_pow,
                vmax= vmax_pow,
                aspect='equal',
                origin='upper'
            )
            
            #ticks = np.linspace(vmin_pow, vmax_pow, 6)
            
            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12, ticks = ticks, format= sci)#format='%.1e')
            
            #cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            #cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            
            cbar.ax.xaxis.set_major_formatter(sci)   # <-- solo formatter
            cbar.ax.tick_params(labelsize=6)
            
            ax.set_title(f"{band} Power - Class {condition_names[cls]}", fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
    
    
    '''
    #PER PLOT RIGA 5-6 (SCALA LOGARITMICA)
        
    
    # ----- 1. calcola vmin_pow / vmax_pow -----
    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    #Filtra solo i valori strettamente > 0 (la scala log non accetta zeri o negativi).
    #Perché? Se l’intero array fosse ≤ 0 (caso patologico) avremmo positive.size == 0.
    positive = all_mean_pow[all_mean_pow > 0]
    
    
    
    #In pratica: stiamo abbassando il bordo inferiore del colormap di un 10 % rispetto al minimo positivo reale, 
    #così quei pixel non finiscono “incollati” al limite della color‑bar. Se preferisci usare un altro margine (5 %, 1 %) basta cambiare 0.9 in 0.95, 0.99, ecc.
    #Se invece vuoi proprio che vada esattamente sul minimo, puoi togliere *0.9 (ma occhio ai warning di Matplotlib)
    
    #Il resto del blocco:

    #Calcola vmax_pow dal massimo globale.
    #Decide automaticamente use_log se il dynamic‑range supera 10³.
    #Imposta una sola logica di plotting: quando use_log è True usa LogNorm, LogLocator e LogFormatterMathtext; altrimenti scala lineare + ScalarFormatter.

    #I titoli aggiungono “(log10)” solo quando serve.
    #Nota: quando use_log è True, passiamo vmin/vmax tramite LogNorm; quando è False, li passiamo direttamente a imshow con i parametri vmin=…, vmax=….
    #Così la stessa funzione disegna correttamente entrambe le situazioni senza dover duplicare codice.
    
    
    #1. Se esistono valori positivi, prende il più piccolo e lo moltiplica per 0.9 (−10 %).
    # Obiettivo: Creare un piccolo margine: il vero minimo non cade esattamente sul bordo inferiore della scala log, evitando clip / warning.
    
    #2. Se non esistono, imposta un fallback sicur0
    # Obiettivo: Garantire che vmin_pow > 0 in ogni caso (requisito di LogNorm).
    
    #vmin_pow = positive.min()*0.9 if positive.size else 1e-12
    #vmin_pow = positive.min() if positive.size else 1e-12
    
    vmin_pow = positive.min()
    
    vmax_pow = all_mean_pow.max()

    use_log  = vmax_pow / max(vmin_pow, 1e-12) > 1e3   # o il flag suggest_log

    if use_log:
        norm      = LogNorm(vmin=vmin_pow, vmax=vmax_pow)
        locator   = LogLocator(base=10.0)
        formatter = LogFormatterMathtext(base=10.0)
    else:
        norm      = None
        locator   = None
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_powerlimits((-2, 2))         # 1e‑2 – 1e2 lineare

    # ----- 2. plot -----
    for b, band in enumerate(band_names):
        for cls in (0, 1):
            ax   = axs[4, b] if cls == 0 else axs[5, b]
            pow_ = mean_raw_power_per_band[cls][b]

            im = ax.imshow(pow_, cmap='jet', norm=norm,
                           vmin=None if use_log else vmin_pow,
                           vmax=None if use_log else vmax_pow,
                           aspect='equal', origin='upper')

            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12)
            
            if locator is not None:
                cbar.locator   = locator
            cbar.formatter = formatter
            cbar.update_ticks()
            cbar.ax.tick_params(labelsize=8)

            scale = "(log10)" if use_log else ""
            ax.set_title(f"{band} Power {scale} – Class {condition_names[cls]}",
                         fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
    
    '''
    
    
    
    #PER PLOT RIGA 7-8
    
    '''
    Vorrei solo verificare allora l'ultima riga 7-8 per la differenza tra i due modelli, perchè: 
    
    
    1) Nel caso del modello 3d puro, ho ancora una fetta rappresentata, ossia

    per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante) 

    ed è quello che vorrei fare per il modello Conv3D puro...

    2) per il modello Conv Separabili invece, 

    per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
    all'interno di TUTTE LE BANDE ASSIEME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
    
    e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
    ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE ...

    devo verificare che per queste righe 7-8, a seconda del modello, il codice sia corretto, in base a come so che 

    global_mean_cams_3d e global_mean_cams_2d sono in realtà adesso ossia 
    
    global_mean_cams_3d[cls]	(5, 9, 9)	volume medio 3D
    global_mean_cams_2d[cls]	(9, 9)	heatmap 2D “globale” su tutte le bande

    '''
    
    
    '''
    Costruiamo la distribuzione congiunta della media del singolo input multi-canale per ogni classe
    '''
    
    all_global_mean_cams = np.concatenate([global_mean_cams[0].flatten(), global_mean_cams[1].flatten()])
    
    global_vmin_cam = all_global_mean_cams.min()
    global_vmax_cam = all_global_mean_cams.max()
    
    
    '''
    In sintesi:

    CNN3D: global_mean_cams_3d[cls] è già shape (5,9,9), quindi fai subito mat2d = global_mean_cams_3d[cls][b]
    SeparableCNN2D: global_mean_cams_2d[cls] è shape (9,9), e la metti in axs[6, cls]
    
    '''
    
    
    #'''Global CAM 2D: una mappa per classe, entrambe su riga 6'''
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        
        #mean_cams_per_band = mean_cams_per_band_2d
        #mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
        
        # Global 2D: una sola heatmap per classe
        for cls in [0, 1]:
            ax = axs[6, cls]
            mat2d = global_mean_cams[cls]  # (9,9)
            im = ax.imshow(mat2d,
                           cmap='RdYlBu_r',
                           vmin=global_vmin_cam,
                           vmax=global_vmax_cam,
                           aspect='equal',
                           origin='upper')
            ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
            cbar = fig.colorbar(im, ax=ax,
                                orientation='horizontal',
                                pad=0.12,
                                ticks=ticks,
                                format='%.1e')
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            ax.set_title(f"Global CAM 2D – Class {condition_names[cls]}", fontsize=10)

            if channel_names is not None:
                ax.set_xticks([]);  ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name,
                            ha='center', va='center',
                            fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")

        # spegni i subplot vuoti
        for col in range(2, n_bands):
            axs[6, col].axis("off")
        for col in range(n_bands):
            axs[7, col].axis("off")

    else:
        
        # Global 3D: una heatmap per banda e per classe
        #mean_cams_per_band = mean_cams_per_band_3d
        #mean_raw_power_per_band = mean_raw_power_per_band_3d
        global_mean_cams = global_mean_cams_3d
        
        for b, band in enumerate(band_names):
            
            #for cls in [0, 1]:
            for j, cls in enumerate([0, 1]):
                
                #ax = axs[6 + cls, b]  # cls==0→riga6, cls==1→riga7
                
                ax = axs[6, b] if cls == 0 else axs[7, b]
                
                vol3d = global_mean_cams[cls]     # (5,9,9) --> perché? 
                                                  # Perché sopra è stato fatto 'global_cams_3d[cls].append(cam_vol[0])'
                                                  # Quindi ogni dato non era più fatto da (B, D, W, H) dove B = 1 (ossia l'esempio stesso)
                                                  # Per cui dopo in global_mean_cams_3d quando ho fatto la media, ho ottenuto una rappresentazione MEDIA
                                                  # del gradcam 3D, PER OGNI BANDA. Quindi, quando prelevo la SINGOLA BANDA, basta che faccio lo 'slicing' ossia
                                                  # mat2d = vol3d[b]  --> da (5,9,9) diventa --> (9,9)
                
                mat2d = vol3d[b]                  # slice b → (9,9)
                
                im = ax.imshow(mat2d,
                               cmap='RdYlBu_r',
                               vmin=global_vmin_cam,
                               vmax=global_vmax_cam,
                               aspect='equal',
                               origin='upper')
                ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
                cbar = fig.colorbar(im, ax=ax,
                                    orientation='horizontal',
                                    pad=0.12,
                                    ticks=ticks,
                                    format='%.1e')
                cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
                cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
                ax.set_title(f"{band} Global CAM 3D – Class {condition_names[cls]}", fontsize=10)

                if channel_names is not None:
                    ax.set_xticks([]);  ax.set_yticks([])
                    for name, (y, x) in channel_names.items():
                        ax.text(x, y, name,
                                ha='center', va='center',
                                fontsize=6, color='black', weight='bold')
                else:
                    ax.axis("off")
                    
    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❸ — Ripristino allo stato precedente il modello ottimizzato trovato migliore, che aveva incluso anche layer LSTM
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    if needs_train_mode:
        # ➌ ripristino layer singoli (i.e., riporto BN/Dropout dove stavano in eval mode)
        for m, old_flag in saved:
            m.train(old_flag)
        # ➍ ripristino lo stato globale del modello (di nuovo ad .eval())
        # i.e.,  come era stato passato in input alla funzione compute_gradcam_figure a partire 'load_best_run_results'!
        
        #Così simuli l’eval (Dropout off, BN congelato) pur essendo in train() per soddisfare CuDNN‑RNN.
        model.train(was_training)
        
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
                                
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

In [66]:
'''

                                                                        NEW VERSION 17/11/2025
                                                                        
                                                                
                                                                        
                                                                  
                                                                    VERSION FREQUENCY x CHANNELS
                                                                            
                                                                     ****** PER GRID 2D! ******
                                                                      ****** MULTI BAND******
                                                                      
                                                                      PER CONVOLUZIONE 3D (PURA)
                                                                              +
                                                                      PER CONVOLUZIONI SEPARABILI
                                                                      
                                                              
                                                                CON VALORI MEAN GRADCAM e MEAN RAW POWER
                                                                            SU STESSA SCALA
                                                                PER OGNI CLASSE E BANDA DI FREQUENZA!
                                                                
                                                                        ^^^^^SENZA COMMENTI^^^^^
                                                                        ^^^^^            ^^^^^
                                                                        
                                                                    SENZA ADOZIONE DELLA MASCHERA 
                                                                PER INDICARE LE POSIZIONI DELLA GRIGLIA REALI    
                                                            
                                                            1) STIMOLA A VERIFICARE CHE LA RETE DISTINGUA TRA 
                                                        
                                                        COORDINATE ELETTRODICHE REALI VS FITTIZIE (SPAZI VUOTI GRIGLIA)
                                                            
                                                            2) CONFERMA IN MODO DATA-DRIVEN LA RILEVANZA NEUROFISIOLOGICA
                                                            DEL FENOMENO IPOTIZZATO
                                                            

OLTRETUTTO

1) senza mask vedevi hotspot che escono fuori dalla “sagoma” degli elettrodi;

2) con la mask, l’informazione si appiattisce / cambia abbastanza 
→ questo ti dice che il modello 2D sta effettivamente usando anche le celle fittizie / bordi / padding come feature.
Mascherando a posteriori, tu forzi la visualizzazione dentro la sagoma, 
ma non stai più mostrando fedelmente dove il modello guarda.


Quindi la tua intuizione:

“non imporre una maschera aiuta a capire se davvero il modello discrimina tra posizioni reali e non reali”

è giusta al 100%. La mask è solo un filtro di visualizzazione, non cambia il modello: se la applichi 
puoi rendere la mappa “più neuro-plausibile”, ma rischi di nascondere il fatto che
la Separable CNN2D sta facendo cose un po’ spurie nella parte fittizia della griglia.


'''

'''
SINTESI DELLA FUNZIONE check_negative_residuals


| Blocco                          | Scopo                                                                                                                                                                                                                                                                                                                                                                                                                                         | Perché serve                                                                                                                                                                                                                                                                                                                                                                                   |
| ------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. eps / “dynamic tol”**      | Calcola una soglia *dinamica* sotto la quale i valori negativi sono quasi certamente puro rumore numerico.<br><br>`python<br>eps32 = np.finfo(np.float32).eps  # ≈ 1.19e‑7<br>dynamic_tol = -eps32 * max(arr.max(), 1.0)`                                                                                                                                                                                                                     | *machine‑epsilon* (ε) è l’errore di arrotondamento massimo relativo per `float32` vicino a 1.<br>Moltiplicandolo per il massimo **positivo** trovato nella mappa definiamo una “fascia di tolleranza” proporzionale alla scala reale del dato. Tutto ciò che cade **sotto** `dynamic_tol` è troppo piccolo perché sia fisico — è, con ottima probabilità, soltanto rumore di rappresentazione. |
| **2. Negativi “significativi”** | Conta e, se ce ne sono, stampa quante celle sono < `dynamic_tol`, quanto è il minimo osservato e la tolleranza stessa.                                                                                                                                                                                                                                                                                                                        | Ti permette di capire a colpo d’occhio se il preprocessing ha generato valori negativi (che non dovrebbero esistere in potenza) che **superano** il rumore ammesso.                                                                                                                                                                                                                            |
| **3. Log‑hint**                 | Se *non* ci sono negativi significativi, calcola il **dynamic range** della mappa (max/min **> 0**) e decide se suggerire la scala log.<br><br>`python<br>positive = arr[arr > 0]<br>if positive.size == 0:<br>    ratio = np.inf          # tutto zero<br>else:<br>    min_pos = positive.min()<br>    max_pos = positive.max()<br>    ratio   = max_pos / max(min_pos, 1e‑12)<br>note = "LOG consigliato" if ratio > 1e3 else "lineare ok"` | – Ignoriamo gli zeri: la scala log non li supporta.<br>– Aggiungiamo un “cuscinetto” di `1e‑12` per evitare div/0.<br>– Se il range è > 10³ (tre ordini di grandezza ≃ “la potenza massima è **> 1000 ×** la minima”), la lettura lineare diventa poco informativa → meglio log10.                                                                                                             |



In sintesi

Prima parte → caccia ai negativi “numeric‑noise” tramite ε.

Seconda parte → valuta solo i valori positivi e suggerisce log‑scale quando il dynamic‑range supera ~3 decadi (≈ 10³).


Quando il codice passa in log‑scale?
Raccogli tutte le mappe (di entrambe le classi e di tutte le bande)


all_mean_pow = np.concatenate([...])

Filtra i positivi e trova vmin_pow (con un 10 % di margine per non “appiattire” il minimo nella color‑bar)

positive_vals = all_mean_pow[all_mean_pow > 0]
vmin_pow = positive_vals.min() * 0.9 if positive_vals.size else 1e‑12
vmax_pow = all_mean_pow.max()

Decidi

use_log = vmax_pow / max(vmin_pow, 1e‑12) > 1e3
Se la potenza massima è > 1000 × la minima positiva, usare LogNorm.

Come si spiega “tre ordini di grandezza”?

“Il massimo è mille volte il minimo”.
“Dynamic‑range di 3 decadi”.
Oppure “max/min > 10³”.



Riassunto finale
ε: misura il rumore di quantizzazione, ti dice quando un (piccolo) negativo è solo un effetto di arrotondamento.

Dynamic‑range: se la banda ha valori reali che variano più di 10³ ×, la scala log10 rende le differenze leggibili senza “schiacciare” i dettagli bassi.

La funzione: un’unica utility per

diagnosticare residui numerici,

suggerire in automatico la rappresentazione (lineare / log) più sensata per i tuoi plot di potenza.

Così il flusso diventa:


check_negative_residuals(...)   # → log “SCALA LOG consigliata”
↓
use_log = True                  # ratio > 1e3
↓
plot con LogNorm + LogLocator   # color‑bar pulita, dettagli visibili




Perché vedi ratio = inf
* Nelle bande Beta e Gamma la tua mappa media è tutta a zero (o comunque tutti i valori ≤ ε).
* Con soli zeri il vettore positive = arr[arr > 0] è vuoto, quindi lo tratto come “dynamic‑range infinito” per evitare la divisione per 0.
* In realtà non hai un’«escursione infinita»; semplicemente non hai segnale in quelle bande → la scala log non aggiungerebbe nulla.

Se vuoi evitare quel suggerimento “falso‑positivo”, basta cambiare la logica così (l’ho già indicato ma lo riscrivo compatto):

positive = arr[arr > 0]
if positive.size < 2:          # 0 o 1 valore positivo → niente dinamica utile
    ratio = 0                  # forza il consiglio a “lineare”
else:
    ratio = positive.max() / max(positive.min(), 1e-12)
    
    
Linear vs log: quale usare davvero?
Banda	Dynamic‑range (≈ max/min)	Scala consigliabile
Delta ‑ Theta	3‑11 ×	Lineare: già leggibile.
Alpha	12 ×	Ancora lineare (o log, ma non cambia molto).
Beta – Gamma	0 (tutti zeri)	Log inutile: non c’è potenza da mostrare.

Di conseguenza:

Mantieni la scala lineare globale come nel tuo blocco finale.

Se in altri dataset vedrai rapporti > 1 000 con almeno 2 valori positivi, allora attiva la parte LogNorm.



'''

def check_negative_residuals(band_names, tensor_dict, tag, log_hint=True):
    
    """
    • band_names   : lista di stringhe  (lunghezza = n_bands)
    • tensor_dict  : {cls: [np.ndarray(H,W), … n_bands]} --> dict  {cls: [np.ndarray(H,W), … 5 bande]}
    • tag          : prefisso stampato nel log --> string visualizzato nel log
    • log_hint     : se True mostra il rapporto max/min ⇒ aiuta a decidere
                     se usare la scala log nei plot.
    """
    
    
    #https://numpy.org/doc/2.1/reference/generated/numpy.finfo.html
    eps32 = np.finfo(np.float32).eps        # 1.19e‑7
    for cls in [0, 1]:
        for b, b_name in enumerate(band_names):
            arr   = tensor_dict[cls][b]
            
            # Soglia dinamica = −eps * valore_massimo_della_mappa
            
            #dynamic_tol = -eps32 * arr.max()      # tolleranza dinamica
            
            dynamic_tol = -eps32 * max(arr.max(), 1.0)   # evita max==0
            
            neg_mask  = arr < dynamic_tol # “negativi significativi”
            
            min_v, max_v = float(arr.min()), float(arr.max())
            
            if np.any(neg_mask):
                print(f"Valori sotto la soglia per classe e banda:\n")
                
                #forma	cosa fa	quando differisce
                #np.count_nonzero(neg_mask)	converte il bool‑array in int (True→1, False→0) e somma gli 1	è sempre un intero Python
                
                #neg_mask.sum()	chiama il metodo .sum() dell’ndarray; 
                #per tipo bool fa esattamente la stessa somma di sopra	restituisce uno numpy.int_ (stesso valore, differente tipo)
                
                n_neg   = np.count_nonzero(neg_mask)
                
                min_val = arr.min()
                
                print(f"[{tag} {b_name}] class={cls}  band={b_name:<6}  "
                      f"neg={n_neg}  min={min_val:.3e}  tol={dynamic_tol:.3e}")
            else:
                print(f"Nessun valore sotto la soglia per classe {cls} e banda {b_name}\n")
                print(f"Definisco il range minimo e massimo per classe {cls} e banda{b_name} :\n")
                
                # ‑‑ opzionale: suggerimento scala log
                if log_hint:                    
                    #if max_v == 0 or min_v == 0:
                    positive = arr[arr > 0]           # considera solo i valori > 0
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    if positive.size == 0:
                        ratio = float('inf')          # tutto zero → range “infinito”
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    else:
                        min_pos = positive.min()
                        max_pos = positive.max()
                        ratio   = max_pos / max(min_pos, 1e-12)
                    note = f"\033[1mSCALA LOG per plots consigliata\033[0m" if ratio > 1e3 else f"\033[1mSCALA LINEARE per plots consigliata ok\033[0m"
                    print(f"        dynamic‑range ≈ {ratio:8.1f}  → {note}")
            print()
                
                
                
import torch.nn as nn

def model_has_cudnn_rnn(model):
    """Ritorna True se il modello usa LSTM/GRU/RNN supportati da CuDNN."""
    return any(isinstance(m, (nn.LSTM, nn.GRU, nn.RNN)) for m in model.modules())


from matplotlib.ticker import FixedLocator

from matplotlib.colors import LogNorm
from matplotlib.ticker import (LogLocator, LogFormatterMathtext,
                               ScalarFormatter)


'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''
def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, channel_names=None, debug = False):
    
    
    '''SOLO PER I MODELLI OTTIMIZZATI CON ANCHE LA LSTM'''
    
    #Solo i modelli con LSTM entrano in questo giro; gli altri non cambiano di stato.
    #Con questa sequenza:
    #non ottieni più l’errore “cudnn RNN backward…”;
    #la rete “si comporta” come in eval (Dropout off, BN congelato) mentre calcoli le CAM;
    #l’ambiente di chiamata (il tuo loop di testing) riceve il modello esattamente nello stato in cui l’aveva passato alla funzione compute_gradcam_figure
    

    ### Perché serve model.train() anche se la CAM è presa prima della LSTM
    
    #Il backward, per arrivare dal loss (o dal logit scelto) fino al tuo layer conv3, deve comunque attraversare l’LSTM che sta più avanti nella rete.
    #Le implementazioni CuDNN degli RNN (LSTM/GRU) alzano un’eccezione se provi a chiamare tensor.backward() mentre il modulo è in modalità eval().
    #RuntimeError: cudnn RNN backward can only be called in training mode
    #Quindi, anche se la CAM è calcolata su conv3, devi mettere l’intero modello in train() per il tempo del backward.
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    ### Che cos’è model.training
    
    #model.training è un semplice flag booleano (impostato da nn.Module.train() / nn.Module.eval()), ereditato da tutti i sotto‑moduli.
    #Con was_training = model.training ricordi in che stato era il modello (quasi sempre False, cioè eval, nel tuo flusso)
    #per poterlo ripristinare dopo.
    
    #Facendo così
    
    #for m in model.modules():
    #if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                      #nn.Dropout, nn.Dropout2d, nn.Dropout3d)):
        #if m.training:         # cioè erano in train
            #m.eval()
            #frozen_layers.append(m)
    
    #Li sposti in eval uno per uno, senza toccare il resto della rete che deve restare in train() per far funzionare CuDNN‑RNN.
    
    
    ### Perché, a fine blocco, servono due ripristini
    
    #1) Riattivo i BatchNorm / Dropout che avevo forzato in eval:
    
    #for m in frozen_layers:
        #m.train()              # torna come prima
    
    #2) Riporto l’intero modello nello stato in cui si trovava prima del Grad‑CAM:
    
    #model.train(was_training)  # se era eval() torna eval, altrimenti resta train
    
    #Se non facessi il punto 1, lasceresti quei moduli permanentemente in eval anche quando, più tardi, 
    #rientri in training (per esempio in un fine‑tuning).
    #Se non facessi il punto 2, lasceresti tutto il modello in train → dropout attivo, BN che accumula statistiche, ecc.

    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❶ — se serve, abilito temporaneamente la modalità train per il modello ottimizzato che aveva ANCHE la LSTM... 
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    needs_train_mode = model_has_cudnn_rnn(model)
    
    if needs_train_mode:
        was_training = model.training      # salvo lo stato
        model.train()                      # abilito backward su CuDNN‑RNN
        
        # ➊ salvo lo stato di OGNI BN/Dropout
        
        saved = [(m, m.training) for m in model.modules()
             if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                               nn.Dropout, nn.Dropout2d, nn.Dropout3d))]
        
        model.train()                              # abilita backward su CuDNN‑RNN
        
        # ➋ congelo in ogni layer della rete gli strati di BatchNorm e Dropout
        for m, _ in saved:
            m.eval()
    
    # ------------------------------------------------------------
    # ❷ — QUI sotto metti tutto il tuo codice Grad‑CAM
    #      (forward, backward, costruzione delle mappe, plot, …)
    # ------------------------------------------------------------

    # … il tuo lunghissimo corpo della funzione rimane invariato …
    # → al momento di fare backward NON avrà più l’eccezione
    #   “cudnn RNN backward can only be called in training mode”

    
    '''SE VUOI USARE STESSO LAYER PER GRADCAM IN ENTRAMBE ARCHITETTURE'''
    target_layer = model.conv2b #model.conv3
    
    
    '''SE VUOI USARE DIVERSI LAYER PER GRADCAM NELLE 2 ARCHITETTURE'''
    #if isinstance(model, CNN3D_LSTM_FC):
        #target_layer = model.conv2b
        
    #elif isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.pw_conv1  # feature map 9x9, allineata 1:1 alla griglia
    
    gradcam = GradCAM(model, target_layer)
    
    # Determina il target layer in base al tipo di modello
    #if isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.dw_conv1  # Per il modello separabile 2D
    #else:
        #target_layer = model.conv3  # Per il modello CNN3D
        

    '''OLD APPROACH'''
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    '''NEW APPROACH'''
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    # -------------------------------
    # Mapping etichette condizioni per i TITOLI dei plot
    # -------------------------------
    
    label_map = {
        "th_resp": "obs_resp",
        "pt_resp": "rec_resp",
    }
    
    def remap_condition_label(s: str) -> str:
        for old, new in label_map.items():
            s = s.replace(old, new)
        return s
    
    # Rimappa i nomi delle condizioni usati nei titoli dei subplot
    condition_names = [remap_condition_label(x) for x in condition_names]
    
    # Rimappa anche la stringa mostrata nel titolo principale (suptitle)
    exp_cond_display = remap_condition_label(exp_cond)
    
    
    # ✅ Raccogli TUTTI i campioni per ciascuna classe
    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    
    
    #PER IL CASO CONV 3D
    
    # Ogni mio sample è 3D, perché infatti è fatto per convoluzione 3d pura o convoluzioni separabili, 
    # Quindi ha shape  (B, C, D, H, W), dove:  
    
    #B = batch (in questo caso, per ogni singolo esempio quindi sarà 1 -> singolo esempio alla volta)
    #C = feature maps/canali (numero di feature maps estratte dalla convoluzione, o meglio anche noti come  canali convoluzionali)
    #D = depth (la dimensione di profondità del mio tensore --> 5 ossia, la potenza spettrale ad ogni banda di frequenza - i.e.,  delta, theta, alfa, beta e gamma)
    #H = height (altezza, prima dimensione SPAZIALE del mio tensore i.e., altezza griglia, canali EEG) 
    #W = width (larghezza, seconda dimensione SPAZIALE del mio tensore i.e., larghezza griglia, canali EEG)
    
    
    '''SHAPE DEI DATI ORIGINALE SAREBBE (B, 9, 9, 5)'''
    samples = {0: [], 1: []}
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples[label_int].append(inputs[i].unsqueeze(0))
                
    
    '''TEST_LOADER RAW (B, 9, 9, 5)'''
    samples_raw = {0: [], 1: []}
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
                
    # ============================================================
    # Calcolo delle Grad-CAM per ogni singola banda di frequenza
    # ============================================================
    
    n_bands = 5  # numero di canali/bande di frequenza
    band_names = ['Delta (δ)', 'Theta (Θ)', 'Alpha (α)', 'Beta (β) ', 'Gamma (γ)']  
    
    
    '''STRUTTURE DATI PER CONV3D'''
    #✅ Struttura per il GradCAM 3D, ossia qui raccolgo il GradCAM 3D di ogni esempio
    # quindi la mappa di attivazione che identifica l'attivazioni più rilevanti per la classificazione di una esemplare di una certa classe
    # sia spazialmente (height and width, ossia le dimensioni spaziali del mio tensore)
    # sia frequenzialmente (depth), ossia le attivazioni più rilevanti in base alla banda di frequenza
    
    global_cams_3d = {0: [], 1: []} # shape (D, 9, 9)
    
    #Poi qui abbiamo: 

    # ✅ Struttura: classe → banda → immagini raw di input filtrate per singola banda (senza passare dal modello)
    #tutte le mappe di potenza per la classe cls nella banda b-esima
    
    #La struttura per salvare invece lo potenza spettrale media per ogni relativa banda 
    raw_power_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    #La struttura che salverà la "fetta" del gradcam3D, ossia dove plotto solo la fetta della banda a partire dal global_cam_3d
    #Ossia per ogni esempio di una specifica classe, prenderò la mappa di attivazione spazialmente più rilevante, in base alla specifica banda di frequenza indagata

    #✅ Struttura: classe → banda → lista CAM
    #cams_per_band_3d[cls][banda]: la slice D-esima (ossia la slice frequenziale) della mappa GradCAM per ogni campione di classe cls.
    
    cams_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''STRUTTURE DATI PER CONV SEPARABLE'''

    
    global_cams_2d = {0: [], 1: []} 
    
    raw_power_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    cams_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''
    
    Da qui in giù, vado a eseguire i passaggi essenziali per:
    
    1) Calcolare il GradCAM 3D per ogni esempio,
    2) Isolare le fette (slice) per banda,
    3) Raccogliere le immagini di potenza raw per ogni banda,
    4) Calcolare le medie per classe e banda.
    
    '''


    for cls in [0, 1]:
        
        for sample_input, sample_input_raw in zip(samples[cls], samples_raw[cls]):
            
            # 1) Preparo il sample per il calcolo
            sample_input = sample_input.clone().detach().requires_grad_(True)
            
            '''
            
            Devo far squeeze perché essendo prelevati dal test loader, li avevo già resi con un 1 davanti ossia qui sopra

            samples[label_int].append(inputs[i].unsqueeze(0))
            
            perché io, è vero che prendo i dati dal test_loader che son in formato batch (batch, D, H, W)
            ma siccome poi prelevo ogni singolo esempio per metterli dentro a "samples", allora quella dimensione di batch si leva nel dizionario "samples".
            
            Per cui, quando voglio prendermi il singolo esempio per salvarmelo dentro a "raw_vol" (ossia la potenza spettrale del volume proprio di ogni esempio),
            allora devo ri-assegnare ad ogni esempio la dimensione del batch (ossia 1, ossia il singolo esempio)
            quando li salvo dentro samples. 
            
            Ed infatti, quindi quell' ".unsqueeze(0)" --> samples[label_int].append(inputs[i].unsqueeze(0)) serve proprio a questo...

            Quando poi, devo prendere il singolo esempio da salvare, nei termini di potenza spettrale di ogni esempio (come volume 3D), 
            allora devo rifare .squeeze(), per togliere nuovamente la dimensione del batch (ossia 1, ossia il singolo esempio)
            perché la rappresentazione del singolo esempio è, appunto, composta da, le sole dimensioni che costituiscono proprio il singolo esempio,
            ossia 9x9x5 -->  ossia la griglia 3D !

            E quindi, per ogni esempio, mi salvo la griglia 3d, ossia
            "raw_vol = sample_input.detach().cpu().numpy().squeeze()   # → (9, 9, 5)"
            
            E poi, la divido però per banda 
            
            "for b in range(n_bands):
                raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
            "
            '''
            
            # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
            #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
            
            raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
            
            '''CASTING IN FLOAT64'''
            #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
            
            for b in range(n_bands):
                
                
                # Determina in base al tipo di modello i dati e le shape da salvare:
                
                #if isinstance(model, CNN3D_LSTM_FC):
                #if isinstance(model, SeparableCNN2D_LSTM_FC):
                
                    #target_layer = model.dw_conv1  # Per il modello separabile 2D
                #else:
                    #target_layer = model.conv3  # Per il modello CNN3D
                    
                if isinstance(model, CNN3D_LSTM_FC):
                    
                    #raw_power_per_band_3d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda

                    raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
                    
                elif isinstance(model, SeparableCNN2D_LSTM_FC):
                    raw_power_per_band_2d[cls][b].append(raw_vol[:,:,b])

            
            # 2) Esegui il forward pass
            output = model(sample_input)
            target_class = output.argmax(dim=1).item()
            
            # 3) Esegui il backward pass
            model.zero_grad()
            target = output[0, target_class]
            target.backward()
            
            # 4) Preleva attivazioni e gradienti
            activ = gradcam.activations   # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            grads = gradcam.gradients # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            
            #Nel caso 3D dovrebbe essere

            #Media dei gradienti solo su H,W → (B,C,D,1,1)
            #w3d = torch.mean(grads, dim=(3, 4))

            # b) Sommo sui canali → (B,D,H,W)
            #cam3d = F.relu(torch.sum(w3d * activ, dim=1))
            
            #e così la shape finale sarebbe 3D con (B,C,D)
            
            #✔️ w3d è correttamente calcolato per ogni (B, C, D, 1, 1)
            #✔️ La somma su dim=1 aggrega le feature maps con pesi per ogni banda
            #✔️ ReLU rimuove componenti negative
            
            '''
            Nel caso della CNN3D calcoli una mappa Grad-CAM 3D "globale" direttamente dal layer convoluzionale 3D, ottenendo attivazioni di shape 
            (B,C,D,H,W) e quindi una CAM volumetrica per ogni trial
            
            CNN3D → calcoli una CAM 3D da attivazioni (B,C,D,H,W), una volta sola per ogni esempio.

            '''
            
            if activ.ndim == 5:  # Caso per modello CNN3D pura 
                
                # 3D Volumetric Grad-CAM
                
                # a) Media dei gradienti solo su H,W → (B,C,D,1,1)
                w3d = torch.mean(grads, dim=(3, 4), keepdim=True)

                # b) Sommo sui canali (feature maps) → (B,D,H,W)
                cam3d = F.relu(torch.sum(w3d * activ, dim=1))
                
                # c) Upsample H×W, mantenendo D intatto
                B, D, H, W = cam3d.shape
                cam_flat = cam3d.view(B*D, 1, H, W)
                cam_up   = F.interpolate(cam_flat,
                                         size=(9, 9),
                                         mode='bilinear',
                                         align_corners=False)
                
                cam_vol  = cam_up.view(B, D, 9, 9).cpu().numpy()
                
                # d) Prendi ogni batch-item
                
                '''
                Quindi qui ottengo che:
            
                1) appendo a global_cams_3d che cosa qui? il gradcam 3D ossia la mappa di attivazione di volume,
                ossia OGNI esempio (volumetrico) per ogni classe (ossia 9x9x5 ancora, di OGNI esempio)
                
                Quindi semplicemente anziché rendere il dato come 'batch, D, H, W'.. siccome prendiamo ogni esempio UNO ALLA VOLTA
                è inutile mantenere la dimensione batch (che sarebbe sempre 1, perché parliamo di ogni esempio, uno alla volta)
                
                ossia anziché fare 
                
                global_cams_3d[cls].append(cam_vol)
                
                faccio
                
                global_cams_3d[cls].append(cam_vol[0])
                
                
                E quindi, mi salvo per OGNI esempio direttamente la mappa cam 3d, per ogni banda, direttamente
                ossia ogni esempio sarà costituito da 3 dimensioni (D, H, W) anziché dire 
                
                "Ogni dato (ossia ogni esempio) è composto da (batch, D, H, W) 
                se tanto il batch = 1 (perché il batch è il singolo esempio ogni volta)
                
                e quindi significherebbe aggiungere una dimensione (quella del batch) che in realtà è inutile, 
                perché si riferisce all'esempio stesso di già!
                
                Quindi:
                👉 cam_vol[0] estrae la CAM 3D senza la dimensione "batch", che è inutile in quel contesto
                👉 Serve per poter fare medie e slicing banda per banda correttamente dopo lo stack
                👉 Questo rende compatibile il risultato finale con imshow (che accetta solo 2D o 3D RGB)
       
                2) appendo anche l'esempio volumetrico a cams_per_band_3d, MA GIA' suddiviso per banda! (per cui diventa 2d là dentro! 9x9)

                '''
                
                global_cams_3d[cls].append(cam_vol[0])
                
                for b in range(n_bands):
                    cams_per_band_3d[cls][b].append(cam_vol[0,b])
            
                '''
                Nel caso della SeparableCNN2D, invece, il layer convoluzionale è 2D e riceve in input 
                (B,5,9,9), cioè con le bande di frequenza come canali (non come profondità). 

                Questo significa che non puoi ottenere direttamente una CAM 3D nello stesso modo, 
                ma si può ottenere una CAM 2D per ogni banda, 
                "mascherando" l’input attivando una banda alla volta

                SeparableCNN2D → non hai accesso diretto a una "profondità" come in GradCAM 3D, quindi:

                Simuli la profondità attivando una banda alla volta.

                Ottieni una CAM 2D per ogni slice (banda), iterando sulle bande.

                Questo approccio ti consente di costruire comunque strutture 3D:
                cams_per_band_2d[cls][b] con b = 0...4 contiene 
                le CAM 2D relative alla banda b, ricostruendo idealmente la distribuzione tridimensionale
                '''
            
            elif activ.ndim == 4: #Caso per il modello Conv Separabili 
                
                # 1) Preparo il sample per il calcolo
                sample_input = sample_input.clone().detach().requires_grad_(True) # sample_input: (1, 9, 9, 5)
                
                
                '''
                il mio sample input ora è sempre 4D, ma a differenza di prima, io sto trattando in questo caso 
                le bande come CANALI, e non come DEPTH (della convoluzione 3d pura!)
                
                Come prima, devo togliere la dimensione del batch ...
                '''
                
                
                # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
                #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
                
                raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
                
                '''CASTING IN FLOAT64'''
                #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
                
                
                '''
                cams_per_band_2d e raw_power_per_band_2d son dentro al loop di MASKING, 
                perché entrambe si riferiscono alla banda e quindi devo essere inserite dentro al loop di masking...
                
                --> raw_power_per_band_2d[cls][b] e cams_per_band_2d[cls][b] sono dentro il loop (for b in ...) 


                
                mentre global_cams_2d è FUORI da quel loop, perché raccoglie tutti gli esempi delle gradcam, 
                considerando però le mappe di attivazione di ogni banda singolarmente 
                e le aggrega per avere una visualizzazione dell'impatto complessivo di ogni singola banda sulla decisione del modello,
                facendo vedere dove son maggiormente concentrate le attivazioni a livello spaziale TRA le bande ( = considerando TUTTE le bande assieme!)
                
                --> global_cams_2d[cls] sta dopo quel for b, raccogliendo una sola mappa 2D “complessiva” per trial
                
                In questo modo:

                raw_power_per_band_2d e cams_per_band_2d catturano tutti i trial per banda.

                global_cams_2d cattura un’unica mappa per trial, che poi aggregherò in global_mean_cams_2d per ottenere la heatmap 2D “globale”
                che comprende tutte le bande insieme

                '''
                
                
                per_band_cams = []  # 👈 CAM di questo trial, una per banda
                
                
                for b in range(n_bands):
                    
                    #raw_power_per_band_2d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda
                    raw_power_per_band_2d[cls][b].append(raw_vol[:, :, b])
                    
                    # ✅ Creo un input mascherato con **solo** la banda b attiva
                    masked = np.zeros_like(raw_vol)  # (9, 9, 5)
                    
                    masked[:, :, b] = raw_vol[:, :, b]  # attiva solo la banda b
                    
                    #Qui lo prepari in formato 4D, come vorrebbe il modello Conv Separable
                    masked_tensor = torch.tensor(masked).unsqueeze(0).to(device)  # (1, 9, 9, 5)
                    
                    # Preparo il sample
                    masked_tensor.requires_grad_(True)
                    
                    # Forward + backward
                    output = model(masked_tensor)
                    target_class = output.argmax(dim=1).item()
                    model.zero_grad()
                    
                    target = output[0, target_class]
                    target.backward()

                    activ = gradcam.activations   # (B, C, H, W)
                    grads = gradcam.gradients     # (B, C, H, W)
                    
                    # Calcolo CAM 2D (come standard GradCAM)
                    w2d = torch.mean(grads, dim=(2, 3), keepdim=True)  # (B, C, 1, 1)
                    cam = F.relu(torch.sum(w2d * activ, dim=1))        # (B, H, W) --> # (1, H, W)
                    
                    # ---- stesso nome in entrambi i casi ----
                    
                    B, H, W = cam.shape
                    
                    '''
                    NEL CASO LAYER SIA DIVERSO DA pw_conv1, ALLORA FACCIO UPSAMPLING
                    Cosa cambia in pratica:

                    Se il target_layer è pw_conv1 (feature map 32×9×9):

                    H, W = 9, 9 → salta l’interpolate, quindi nessuna distorsione spaziale.

                    Se un domani cambi target_layer a un layer dopo un pool (es. conv2b con mappe 4×4):

                    H, W != 9 → scatta l’upsampling e tutto continua a funzionare come prima.

                    Il ramo 3D (activ.ndim == 5) non lo tocchi, lì l’upsampling è ancora necessario (4×4 → 9×9).

                    Con questa modifica il caso SeparableCNN2D è “pulito” e allineato alla griglia elettrodica 1:1
                    '''
                    
                    if (H, W) != (9, 9):
                    
                        #Riporto la shape con .unsqueeze(1) a 4D per fare interpolation e alla fine di nuovo in 3d
                        cam = F.interpolate(cam.unsqueeze(1), size=(9, 9), mode='bilinear', align_corners=False).squeeze(1)
                    
                    
                    # caso pw_conv1: la mappa è già 9x9 → nessun upsampling
                    
                    #Riporto la shape con .squeeze(1) a 3D per salvare i dati
                    #cam_2d = cam_up.squeeze(1).cpu().numpy()  # (B, 9, 9)
                    
                    cam_2d = cam.detach().cpu().numpy()      # (B,9,9)

                    # ✅ Aggiungo la mappa CAM alla banda corrispondente
                    cams_per_band_2d[cls][b].append(cam_2d[0])  # prende il CAM per il sample corrente (9x9)
                    
                    # Conteggio della CAM corrente, per questo trial
                    per_band_cams.append(cam_2d[0])                   
                    
                #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
                #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:
                # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
                
                # 👇 qui, fuori dal for b
                
                all_cams_2d_per_band = np.mean(np.stack(per_band_cams), axis=0)  # (9,9)
                
                global_cams_2d[cls].append(all_cams_2d_per_band)
            else:
                raise RuntimeError(f"activ.ndim inatteso: {activ.ndim}")
                
                
    '''
    CASO MODELLO CON 3D PURO, dovrei fare: 

    1) per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante)

    '''
    
    if isinstance(model, CNN3D_LSTM_FC):
        
        #1) global_cams_3d → media “globale” del volume 3D Grad‑CAM
        # media sul numero di esempi per ogni classe → ottieni un array (D, H, W)
        global_mean_cams_3d = {
            cls: np.mean(np.stack(global_cams_3d[cls]), axis=0)  # da [ (1,D,H,W), … ] a (D,H,W)
            for cls in [0,1]
        }
        
        # ➜ CAM 2D globale per classe (media su TUTTE le bande dal VOLUME 3D)
        global_2d_from3d = {
            cls: np.mean(global_mean_cams_3d[cls], axis=0)  # (9,9)
            for cls in [0, 1]
        }
        
        


        '''
        2) poi, dentro a raw_power_per_band_3d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_3d)
        '''

        #2) raw_power_per_band_3d → media della potenza raw per banda
        #Hai già raccolto, per ogni cls e per ogni banda b, tutte le mappe 2D raw_power_per_band_3d[cls][b] (una per trial).
        #La media diventa:

        mean_raw_power_per_band_3d = {
            cls: [ np.mean(np.stack(raw_power_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }

        # risultato: mean_raw_power_per_band_3d[cls][b] è (H,W)

        '''
        3) dentro a cams_per_band_3d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_3d_per_band) 

        '''

        #3) cams_per_band_3d → media della Grad‑CAM per banda
        #Analogamente hai raccolto tutte le slice 2D di Grad‑CAM in cams_per_band_3d[cls][b].
        #La media diventa:

        mean_cams_per_band_3d = {
            cls: [ np.mean(np.stack(cams_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_3d_per_band[cls][b] ha shape (H,W)


        '''
        Con queste tre strutture (global_mean_cams_3d, mean_raw_power_per_band_3d, mean_cam_3d_per_band) puoi

        riga 1–2: istogrammi di mean_cam_3d_per_band[cls][b]

        riga 3–4: heatmap di mean_cam_3d_per_band[cls][b]

        riga 5–6: heatmap di mean_raw_power_per_band_3d[cls][b]

        riga 7–8: slice di global_mean_cams_3d[cls] per ogni b

        '''
    
    elif isinstance(model, SeparableCNN2D_LSTM_FC):
    
        '''
        CASO MODELLO CONV SEPARABLE, dovrei fare: 

        1) per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
        all'interno di TUTTE LE BANDE ASSIME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
        e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
        ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE...

        Per la SeparableCNN2D ricostruisci un Grad‑CAM “3D” artificiale facendo 5 Grad‑CAM 2D una per ogni banda
        '''

        #global_cams_2d
        #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
        #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:

        # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
        #global_cams_2d[cls].append(cam_2d[0])

        global_mean_cams_2d = {
            cls: np.mean(np.stack(global_cams_2d[cls]), axis=0)
            for cls in [0,1]
        }
        # global_mean_cams_2d[cls] shape = (9,9)

        '''
        2) poi, dentro a raw_power_per_band_2d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_2d)
        '''

        #raw_power_per_band_2d
        #Hai già in raw_power_per_band_2d[cls][b] tutte le mappe 2D di potenza (9×9) per trial, per ciascuna banda b.
        #La media finale:


        mean_raw_power_per_band_2d = {
            cls: [ np.mean(np.stack(raw_power_per_band_2d[cls][b]), axis=0)
                  for b in range(n_bands) ]
            for cls in [0,1]
        }

        # mean_raw_power_per_band_2d[cls][b] shape = (9,9)


        '''
        3) dentro a cams_per_band_2d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_2d_per_band) 

        '''

        #cams_per_band_2d
        #Durante il masking loop appendi in cams_per_band_2d[cls][b] il CAM 2D (9×9) di ogni trial.
        #La media finale:

        mean_cams_per_band_2d = {
            cls: [ np.mean(np.stack(cams_per_band_2d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_2d_per_band[cls][b] shape = (9,9)

        '''
        Con queste tre strutture —

        mean_raw_power_per_band_2d (5 mappe 9×9),

        mean_cam_2d_per_band (5 mappe 9×9),

        global_mean_cams_2d (1 mappa 9×9) —

        puoi costruire esattamente le stesse righe di plot che avevi per il caso 3D, solo che al posto di “slice” del volume userai le CAM 2D mascherate.


        '''
    
    # Preleva la struttura corretta in base al modello
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        mean_cams_per_band = mean_cams_per_band_2d
        mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
    
    else:  # Caso per modello CNN3D
        mean_cams_per_band = mean_cams_per_band_3d
        mean_raw_power_per_band = mean_raw_power_per_band_3d
        
        #global_mean_cams = global_mean_cams_3d
        
        
        
        '''NUOVA MODIFICA''' 
        global_mean_cams = global_2d_from3d
        
    
    
    
    
    
    # prima di salvare la figura, solo se richiesto vedi i valori delle potenze medie per banda e condizione sperimentale...
    
    #"tag_names=[f"{model.__class__.__name__} power"]")
    
    if debug:
        if isinstance(model, CNN3D_LSTM_FC):
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_3d,
                                     model_tag)
        else:
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_2d,
                                     model_tag)
            
    
    
    
    # Crea la figura dinamicamente in base al modello
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 2 righe per 5 colonne per modello 2D
    else:
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 5 righe per 2 colonne per modello 3D

    
    
            
    title = (
        f"Grad-CAM Mapping over EEG Trials – Experimental Conditions: {exp_cond_display}\n\n"
        "Row 1-2: Histogram of Mean Grad-CAM Raw Values for Each Class and Frequency Band\n"
        "Row 3-4: Normalized Mean Grad-CAM Heatmaps for Class 0 (top) and Class 1 (bottom)\n"
        #"Row 5-6: Log-Scaled Mean Raw Spectrogram for Class 0 (top) and Class 1 (bottom)\n"
        "Row 5-6: Log-Scaled Mean Raw Power Maps for Class 0 (top) and Class 1 (bottom)\n"
        #"Row 7-8: Global CAM per Class"
        "Row 7: Global CAM per Class 0 (left side) and Class 1 (right side)"
        )
    
    plt.suptitle(title, fontsize=15)

    # Spaziatura verticale per evitare sovrapposizione
    #plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    plt.subplots_adjust(hspace = 0.7, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    
    #PER PLOT RIGA 1-2 
    
    from matplotlib.ticker import ScalarFormatter, MaxNLocator

    # Crea un formatter per notazione scientifica
    sci_formatter = ScalarFormatter(useMathText=True)
    
    #Questa chiamata serve a forzare il range entro cui usare la notazione scientifica
    #Se non metti questo limite, il comportamento può variare leggermente a seconda della scala dei dati —
    #a volte sarà decimale (0.0001), altre volte esponenziale (1e-4), e potrebbe non essere uniforme tra subplot.
    
    sci_formatter.set_powerlimits((-3, 3))  # usa 1e-xxx se valori sono piccoli

    for b, b_name in enumerate(band_names):
    
        for j, cls in enumerate([0, 1]):
            
            # Calcola l'istogramma dei valori della heatmap media
            # rispetto alle 2 classi in base alla banda di frequenza isolata
            
            ax = axs[0, b] if cls == 0 else axs[1, b]
            
            '''NEW UPDATED
            Histograms are shown with per-band x-limits [0, max] to avoid binning artifacts when distributions are near-zero.
            '''
            vals = mean_cams_per_band[cls][b].ravel()
            vals = vals[np.isfinite(vals)]  # safety
            
            if vals.size == 0:
                ax.text(0.5, 0.5, "No finite values", ha="center", va="center")
                ax.axis("off")
                continue
            
            #vmax_h = vals.max() if vals.size else 0.0
            #vmax_h = max(vmax_h, 1e-12)  # evita range=(0,0)
            
            '''
            2) Range robusto (upper bound): percentile invece del max
            Cosa fa np.percentile(vals, 99.5)?

            Prende un valore vmax_h tale che:

            il 99.5% dei valori in vals è ≤ vmax_h
            e solo lo 0.5% più grande sta sopra (outlier estremi)

            Quindi usi come limite destro dell’istogramma non il massimo assoluto, ma un massimo “robusto”.

            Perché aiuta?

            Nel tuo errore succedeva che:
            
            max(vals) era grande (magari per pochi pixel),
            ma il resto dei valori era piccolissimo / quasi tutto zero,
            e con bins="auto" Numpy provava a mettere troppi bin per “risolvere” la parte piccola → esplodeva.

            Con vmax_h robusto:

            “tagli” gli outlier (solo per l’istogramma),
            quindi la scala x è più “umana” e non induce binning patologico.
            max(vmax_h, 1e-12) a cosa serve?

            Evita il caso in cui vmax_h venga 0 o quasi 0 (es. valori tutti 0):

            se vmax_h=0, il range (0, vmax_h) diventa (0,0) e Matplotlib/Numpy si lamentano o producono bin degeneri.
            1e-12 è un “minimo tecnico” solo per non rompere il plot.
            
            
            '''
            
            # ✅ invece di usare il max (outlier), usa un upper bound robusto
            vmax_h = np.percentile(vals, 99.5)
            vmax_h = max(vmax_h, 1e-12)  # evita range=(0,0)
            
            
            '''
            
            3) Bins adattivi + cap
            
            sqrt(vals.size) cosa significa?

            È una regola classica per istogrammi:
            più campioni ⇒ più bin (ma lentamente)
            è molto stabile e non esplode
            Esempio: se vals.size = 81 ⇒ sqrt=9 (ma poi lo porti almeno a 20 col cap minimo).
            
            Perché il cap 20–120?

            È un “freno di sicurezza”:
            min 20: evita istogrammi troppo “a blocchi” e poco informativi
            max 120: evita istogrammi con centinaia/migliaia di bin che:

            pesano tanto in rendering
            e possono comunque diventare instabili in casi strani

            Quindi ottieni un numero di bin “ragionevole” sempre.

            '''
            # ✅ bins adattivi + cap (niente "auto")
            bins_h = int(np.sqrt(vals.size))
            bins_h = max(20, min(bins_h, 120))
            
            '''OLD VERSION'''
            #ax.hist(mean_cams_per_band[cls][b].flatten(), bins='auto', color='blue', edgecolor='black')
            
            '''NEW UPDATED'''
            #ax.hist(vals, bins="auto", range=(0, vmax_h), color="blue", edgecolor="black")
            
            '''
            4) Plot dell’istogramma dentro quel range e con quei bin

            Cosa fa in pratica?

            Considera i valori vals
            costruisce un istogramma in bins_h bin
            solo nel range [0, vmax_h]
            Se ci sono valori > vmax_h (quello 0.5% di outlier), vengono ignorati dall’istogramma (o meglio: “cadono fuori range” e non vengono conteggiati nei bin).
            Questo è voluto: ti interessa rappresentare la massa principale.
            
            SINTESI:

            Per questa banda e classe, fammi un istogramma che rappresenti bene il 99.5% dei valori, con un numero di bin che cresce con la quantità di dati
            ma non diventa mai folle. Se non ho dati validi, non fare nulla.”

            '''
            ax.hist(vals, bins=bins_h, range=(0, vmax_h), color="blue", edgecolor="black")
            ax.set_xlim(0, vmax_h)
            
            
            ax.set_title(f"{b_name} - Class {condition_names[cls]}", fontsize=10)
            ax.set_xlabel("Grad-CAM Value")
            ax.set_ylabel("Count")
            
            # ✅ Format tick con notazione scientifica
            ax.xaxis.set_major_formatter(sci_formatter)
            ax.xaxis.set_major_locator(MaxNLocator(4))      # max ~4 tick
            
            #ax.tick_params(axis='x', labelrotation=45, labelsize=6)
    
    
    
    #PER PLOT RIGA 3-4 
    
    '''
    Concateno tutte le mean-CAM (cls 0+1, tutte le bande) in un unico array
    in modo da confrontare le Gradcam tra classi e bande tra di loro! 
    '''
    
    all_mean_cams = np.concatenate([
        mean_cams_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_cams_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_cam = all_mean_cams.min()
    vmax_cam = all_mean_cams.max()
    
    
    for b, band in enumerate(band_names):                                 
        
        for j, cls in enumerate([0, 1]):
            
            ax = axs[2, b] if cls == 0 else axs[3, b]
            
            cam = mean_cams_per_band[cls][b] 
            
            # Controlla se la forma è corretta per l'input di imshow
            assert cam.ndim == 2, f"Expected 2D array, got {cam.ndim}D array"
            
            im = ax.imshow(
                cam,
                cmap = 'RdYlBu_r',
                vmin = vmin_cam, 
                vmax = vmax_cam,
                aspect = 'equal',
                origin = 'upper'
            )
            
            ticks = np.linspace(vmin_cam, vmax_cam, 6)

            cbar = fig.colorbar(
                im, ax=ax, orientation='horizontal', pad=0.12, ticks=ticks, format='%.1e')
            
            cbar.set_ticks(ticks)
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])

            ax.set_title(
                f"{band} - Class {condition_names[cls]}",
                fontsize=10
            )

            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(
                        x, y, name,
                        ha='center', va='center',
                        fontsize=6, color='black', weight='bold'
                    )
            else:
                ax.axis("off")
    
    
    
    #PER PLOT RIGA 5-6 
    
    '''
    Concateno tutte lo spettogrammam medio logaritmico
    da confrontare tra le classi per ogni banda tra di loro! 
    '''
    
    from matplotlib import colors

    
    # Log-trasform delle mappe di potenza per banda
    log_mean_power_per_band = {
        cls: [np.log1p(mean_raw_power_per_band[cls][b])
              for b in range(n_bands)]
        for cls in [0, 1]
    }
    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le log mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        log_mean_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        log_mean_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    
    '''NEW UPDATE'''
    gamma = 2.5
    # NEW: power-law normalization globale (scegli gamma)
    norm_pow = colors.PowerNorm(gamma = gamma , vmin=vmin_pow, vmax=vmax_pow)  # prova 1.5–3.0
    
    '''NEW UPDATE'''
    # ✅ (consigliato) colormap percettiva
    cmap_pow = "magma"  # oppure "viridis" / "cividis"

    
    #ticks = np.linspace(vmin_pow, vmax_pow, 6)
    
    '''NEW UPDATE'''
    n_ticks = 6   # <-- metti 3 oppure 4
    '''NEW UPDATE'''
    u = np.linspace(0, 1, n_ticks)  # posizioni uniformi nello spazio colore
    
    '''NEW UPDATE'''
    # se la tua versione di matplotlib supporta inverse():
    try:
        tick_vals = norm_pow.inverse(u)
    except Exception:
        # inversione manuale di PowerNorm
        tick_vals = vmin_pow + (vmax_pow - vmin_pow) * (u ** (1.0 / gamma))
        
    
    from matplotlib.ticker import ScalarFormatter

    sci = ScalarFormatter(useMathText=True)
    sci.set_powerlimits((-2, 2))          # forza 1eX fuori dall’intervallo 1e‑2 … 1e2

    for b, band in enumerate(band_names):
        
        for cls in [0, 1]:
            ax = axs[4, b] if cls == 0 else axs[5, b]

            log_power = log_mean_power_per_band[cls][b]

            im = ax.imshow(
                log_power,
                
                #cmap='jet',
                #vmin=vmin_pow,
                #vmax=vmax_pow,
                
                cmap=cmap_pow,
                norm=norm_pow,      # ✅ NEW (invece di vmin/vmax)
                
                aspect='equal',
                origin='upper'
            )
            
            #cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12, ticks = ticks, format= sci)#format='%.1e')
            
            '''NEW UPDATE'''
            cbar = fig.colorbar(im, ax=ax, orientation="horizontal", pad=0.12, ticks = tick_vals, format= sci)

            cbar.ax.xaxis.set_major_formatter(sci) # <-- solo formatter
            cbar.ax.tick_params(labelsize=6)
            
            # (opzionale ma utile se cambi ticks a posteriori)
            cbar.update_ticks()

            ax.set_title(f"{band} Log Mean Power - Class {condition_names[cls]}", fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
                
                
    '''
    OLD PLOTS SPETTROGRAMMA RAW 
    
    
    #****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******
     
    #PER PLOT RIGA 5-6 (SCALA LINEARE)
    
    from matplotlib.ticker import ScalarFormatter

    sci = ScalarFormatter(useMathText=True)
    sci.set_powerlimits((-2, 2))          # forza 1eX fuori dall’intervallo 1e‑2 … 1e2

    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    
    ticks = np.linspace(vmin_pow, vmax_pow, 6)
    
    # Riga 3: Mappa della potenza media rispetto a distribuzione congiunta (su ciascuna banda e classe)
    for b, band in enumerate(band_names):
        
        for cls in [0, 1]:
            ax = axs[4, b] if cls == 0 else axs[5, b]
            
            power = mean_raw_power_per_band[cls][b] 
            
            im = ax.imshow(
                power, 
                cmap='jet',
                vmin= vmin_pow,
                vmax= vmax_pow,
                aspect='equal',
                origin='upper'
            )
            
            #ticks = np.linspace(vmin_pow, vmax_pow, 6)
            
            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12, ticks = ticks, format= sci)#format='%.1e')
            
            #cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            #cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            
            cbar.ax.xaxis.set_major_formatter(sci)   # <-- solo formatter
            cbar.ax.tick_params(labelsize=6)
            
            ax.set_title(f"{band} Power - Class {condition_names[cls]}", fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
                
                
    
    #****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******
    
    #PER PLOT RIGA 5-6 (SCALA LOGARITMICA)
        
    
    # ----- 1. calcola vmin_pow / vmax_pow -----
    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    #Filtra solo i valori strettamente > 0 (la scala log non accetta zeri o negativi).
    #Perché? Se l’intero array fosse ≤ 0 (caso patologico) avremmo positive.size == 0.
    positive = all_mean_pow[all_mean_pow > 0]
    
    
    
    #In pratica: stiamo abbassando il bordo inferiore del colormap di un 10 % rispetto al minimo positivo reale, 
    #così quei pixel non finiscono “incollati” al limite della color‑bar. Se preferisci usare un altro margine (5 %, 1 %) basta cambiare 0.9 in 0.95, 0.99, ecc.
    #Se invece vuoi proprio che vada esattamente sul minimo, puoi togliere *0.9 (ma occhio ai warning di Matplotlib)
    
    #Il resto del blocco:

    #Calcola vmax_pow dal massimo globale.
    #Decide automaticamente use_log se il dynamic‑range supera 10³.
    #Imposta una sola logica di plotting: quando use_log è True usa LogNorm, LogLocator e LogFormatterMathtext; altrimenti scala lineare + ScalarFormatter.

    #I titoli aggiungono “(log10)” solo quando serve.
    #Nota: quando use_log è True, passiamo vmin/vmax tramite LogNorm; quando è False, li passiamo direttamente a imshow con i parametri vmin=…, vmax=….
    #Così la stessa funzione disegna correttamente entrambe le situazioni senza dover duplicare codice.
    
    
    #1. Se esistono valori positivi, prende il più piccolo e lo moltiplica per 0.9 (−10 %).
    # Obiettivo: Creare un piccolo margine: il vero minimo non cade esattamente sul bordo inferiore della scala log, evitando clip / warning.
    
    #2. Se non esistono, imposta un fallback sicur0
    # Obiettivo: Garantire che vmin_pow > 0 in ogni caso (requisito di LogNorm).
    
    #vmin_pow = positive.min()*0.9 if positive.size else 1e-12
    #vmin_pow = positive.min() if positive.size else 1e-12
    
    vmin_pow = positive.min()
    
    vmax_pow = all_mean_pow.max()

    use_log  = vmax_pow / max(vmin_pow, 1e-12) > 1e3   # o il flag suggest_log

    if use_log:
        norm      = LogNorm(vmin=vmin_pow, vmax=vmax_pow)
        locator   = LogLocator(base=10.0)
        formatter = LogFormatterMathtext(base=10.0)
    else:
        norm      = None
        locator   = None
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_powerlimits((-2, 2))         # 1e‑2 – 1e2 lineare

    # ----- 2. plot -----
    for b, band in enumerate(band_names):
        for cls in (0, 1):
            ax   = axs[4, b] if cls == 0 else axs[5, b]
            pow_ = mean_raw_power_per_band[cls][b]

            im = ax.imshow(pow_, cmap='jet', norm=norm,
                           vmin=None if use_log else vmin_pow,
                           vmax=None if use_log else vmax_pow,
                           aspect='equal', origin='upper')

            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12)
            
            if locator is not None:
                cbar.locator   = locator
            cbar.formatter = formatter
            cbar.update_ticks()
            cbar.ax.tick_params(labelsize=8)

            scale = "(log10)" if use_log else ""
            ax.set_title(f"{band} Power {scale} – Class {condition_names[cls]}",
                         fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
    
    '''
    
    
    
    #PER PLOT RIGA 7-8
    
    '''
    Vorrei solo verificare allora l'ultima riga 7-8 per la differenza tra i due modelli, perchè: 
    
    
    1) Nel caso del modello 3d puro, ho ancora una fetta rappresentata, ossia

    per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante) 

    ed è quello che vorrei fare per il modello Conv3D puro...

    2) per il modello Conv Separabili invece, 

    per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
    all'interno di TUTTE LE BANDE ASSIEME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
    
    e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
    ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE ...

    devo verificare che per queste righe 7-8, a seconda del modello, il codice sia corretto, in base a come so che 

    global_mean_cams_3d e global_mean_cams_2d sono in realtà adesso ossia 
    
    global_mean_cams_3d[cls]	(5, 9, 9)	volume medio 3D
    global_mean_cams_2d[cls]	(9, 9)	heatmap 2D “globale” su tutte le bande

    '''
    
    
    '''
    Costruiamo la distribuzione congiunta della media del singolo input multi-canale per ogni classe
    '''
    
    all_global_mean_cams = np.concatenate([global_mean_cams[0].flatten(), global_mean_cams[1].flatten()])
    
    global_vmin_cam = all_global_mean_cams.min()
    global_vmax_cam = all_global_mean_cams.max()
    
    
    '''
    In sintesi:

    CNN3D: global_mean_cams_3d[cls] è già shape (5,9,9), quindi fai subito mat2d = global_mean_cams_3d[cls][b]
    SeparableCNN2D: global_mean_cams_2d[cls] è shape (9,9), e la metti in axs[6, cls]
    
    '''
    
    
    #'''Global CAM 2D: una mappa per classe, entrambe su riga 6'''
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        
        #mean_cams_per_band = mean_cams_per_band_2d
        #mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
        
        # Global 2D: una sola heatmap per classe
        for cls in [0, 1]:
            ax = axs[6, cls]
            mat2d = global_mean_cams[cls]  # (9,9)
            im = ax.imshow(mat2d,
                           cmap='RdYlBu_r',
                           vmin=global_vmin_cam,
                           vmax=global_vmax_cam,
                           aspect='equal',
                           origin='upper')
            ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
            cbar = fig.colorbar(im, ax=ax,
                                orientation='horizontal',
                                pad=0.12,
                                ticks=ticks,
                                format='%.1e')
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            ax.set_title(f"Global CAM 2D (across bands) – Class {condition_names[cls]}", fontsize=10)

            if channel_names is not None:
                ax.set_xticks([]);  ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name,
                            ha='center', va='center',
                            fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")

        # spegni i subplot vuoti
        for col in range(2, n_bands):
            axs[6, col].axis("off")
        for col in range(n_bands):
            axs[7, col].axis("off")

    else:
        
        '''
        # Global 3D: una heatmap per banda e per classe
        #mean_cams_per_band = mean_cams_per_band_3d
        #mean_raw_power_per_band = mean_raw_power_per_band_3d
        
        global_mean_cams = global_mean_cams_3d
        
        for b, band in enumerate(band_names):
            
            #for cls in [0, 1]:
            for j, cls in enumerate([0, 1]):
                
                #ax = axs[6 + cls, b]  # cls==0→riga6, cls==1→riga7
                
                ax = axs[6, b] if cls == 0 else axs[7, b]
                
                vol3d = global_mean_cams[cls]     # (5,9,9) --> perché? 
                                                  # Perché sopra è stato fatto 'global_cams_3d[cls].append(cam_vol[0])'
                                                  # Quindi ogni dato non era più fatto da (B, D, W, H) dove B = 1 (ossia l'esempio stesso)
                                                  # Per cui dopo in global_mean_cams_3d quando ho fatto la media, ho ottenuto una rappresentazione MEDIA
                                                  # del gradcam 3D, PER OGNI BANDA. Quindi, quando prelevo la SINGOLA BANDA, basta che faccio lo 'slicing' ossia
                                                  # mat2d = vol3d[b]  --> da (5,9,9) diventa --> (9,9)
                
                mat2d = vol3d[b]                  # slice b → (9,9)
                
                im = ax.imshow(mat2d,
                               cmap='RdYlBu_r',
                               vmin=global_vmin_cam,
                               vmax=global_vmax_cam,
                               aspect='equal',
                               origin='upper')
                ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
                cbar = fig.colorbar(im, ax=ax,
                                    orientation='horizontal',
                                    pad=0.12,
                                    ticks=ticks,
                                    format='%.1e')
                cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
                cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
                ax.set_title(f"{band} Global CAM 3D – Class {condition_names[cls]}", fontsize=10)

                if channel_names is not None:
                    ax.set_xticks([]);  ax.set_yticks([])
                    for name, (y, x) in channel_names.items():
                        ax.text(x, y, name,
                                ha='center', va='center',
                                fontsize=6, color='black', weight='bold')
                else:
                    ax.axis("off")
        '''
        
        
        '''
        # Global 3D: una heatmap per classe TRA le bande
        '''
        
        # ➜ CNN3D: una sola mappa 2D globale per classe (media sulle bande)
        #   global_2d_from3d[cls] ha shape (9,9)
        
        global_mean_cams = global_2d_from3d
        
        for cls in [0, 1]:
            ax = axs[6, cls]   # uso riga 7, colonne 0 e 1
            mat2d = global_2d_from3d[cls]  # (9,9)

            im = ax.imshow(
                mat2d,
                cmap='RdYlBu_r',
                vmin=global_vmin_cam,
                vmax=global_vmax_cam,
                aspect='equal',
                origin='upper'
            )
            ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
            cbar = fig.colorbar(
                im, ax=ax,
                orientation='horizontal',
                pad=0.12,
                ticks=ticks,
                format='%.1e'
            )
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            ax.set_title(f"Global CAM 3D (across bands) – Class {condition_names[cls]}", fontsize=10)

            if channel_names is not None:
                ax.set_xticks([]); ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name,
                            ha='center', va='center',
                            fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")

        # spegni gli altri subplot della riga 7–8
        for col in range(2, n_bands):
            axs[6, col].axis("off")
        for col in range(n_bands):
            axs[7, col].axis("off")
    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❸ — Ripristino allo stato precedente il modello ottimizzato trovato migliore, che aveva incluso anche layer LSTM
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    if needs_train_mode:
        # ➌ ripristino layer singoli (i.e., riporto BN/Dropout dove stavano in eval mode)
        for m, old_flag in saved:
            m.train(old_flag)
        # ➍ ripristino lo stato globale del modello (di nuovo ad .eval())
        # i.e.,  come era stato passato in input alla funzione compute_gradcam_figure a partire 'load_best_run_results'!
        
        #Così simuli l’eval (Dropout off, BN congelato) pur essendo in train() per soddisfare CuDNN‑RNN.
        model.train(was_training)
        
    
    #plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # ---- layout finale (dopo aver aggiunto TUTTO) ----
    print(">>> layout", flush=True)
    try:
        fig.tight_layout(rect=[0, 0, 1, 0.95])   # <-- usa fig, non plt
    except Exception as e:
        print("tight_layout skipped:", e, flush=True)
        fig.subplots_adjust(left=0.04, right=0.98, bottom=0.05, top=0.92,
                            wspace=0.4, hspace=0.7)
                                
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------
    
    print(">>> savefig", flush=True)

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    print(">>> done", flush=True)
    return fig_image

##### **NUOVO LOOP PER DATI NON HYPER SU CNN2D, BiLSTM e Transformer**

In [50]:
import os
import re

import random
#perché è importante numpy.random.seed()?
#https://www.analyticsvidhya.com/blog/2021/12/what-does-numpy-random-seed-do/#:~:text=The%20numpy%20random%20seed%20is,displays%20the%20same%20random%20numbers.
from numpy.random import seed

import numpy as np
import copy as cp

from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

#importing librerie pytorch
import torch 
import torch.nn as nn #neural network module
import torch.optim as optim #ottimizzatore
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset

#from sklearn.model_selection import KFold

#importing librerie numpy, pandas, scikit-learn e matplotlib
import numpy as np


import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from tqdm import tqdm



In [51]:
data_dict = {}

experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

for condition in experimental_conditions:
    
    # inizializza livello condition
    data_dict.setdefault(condition, {})
    
    for data_type in ["spectrograms"]:
        
        # inizializza livello data_type
        data_dict[condition].setdefault(data_type, {})
        
        for category in ["familiar", "unfamiliar"]:
            
            for subject_type in ["th", "pt"]:
                
                # Caricamento dati
                if data_type == "wavelet":
                    X, y = load_data(data_type, category, subject_type, wavelet_level="delta")
                else:
                    X, y = load_data(data_type, category, subject_type)

                # chiave del terzo livello (come si aspetta la funzione di conversione)
                category_subject = f"{category}_{subject_type}"   # es. "familiar_th"

                data_dict[condition][data_type][category_subject] = (X, y)

                # stampa di controllo (solo per debug)
                flat_key = f"{condition}_{data_type}_{category}_{subject_type}"
                print(f"Dataset caricato: \033[1m{flat_key}\033[0m - Forma X: {X.shape}, Lunghezza y: {len(y)}")


Dataset caricato: th_resp_vs_pt_resp_spectrograms_familiar_th - Forma X: (1586, 45, 61), Lunghezza y: 1586
Dataset caricato: th_resp_vs_pt_resp_spectrograms_familiar_pt - Forma X: (1580, 45, 61), Lunghezza y: 1580
Dataset caricato: th_resp_vs_pt_resp_spectrograms_unfamiliar_th - Forma X: (1667, 45, 61), Lunghezza y: 1667
Dataset caricato: th_resp_vs_pt_resp_spectrograms_unfamiliar_pt - Forma X: (1667, 45, 61), Lunghezza y: 1667
Dataset caricato: th_resp_vs_shared_resp_spectrograms_familiar_th - Forma X: (1586, 45, 61), Lunghezza y: 1586
Dataset caricato: th_resp_vs_shared_resp_spectrograms_familiar_pt - Forma X: (1580, 45, 61), Lunghezza y: 1580
Dataset caricato: th_resp_vs_shared_resp_spectrograms_unfamiliar_th - Forma X: (1667, 45, 61), Lunghezza y: 1667
Dataset caricato: th_resp_vs_shared_resp_spectrograms_unfamiliar_pt - Forma X: (1667, 45, 61), Lunghezza y: 1667
Dataset caricato: pt_resp_vs_shared_resp_spectrograms_familiar_th - Forma X: (1586, 45, 61), Lunghezza y: 1586
Dataset c

#### **Creazione Griglia 2D per Interrogait - EEG Spectrograms - Electrodes x Frequencies**

In [52]:
import pandas as pd

path = '/home/stefano/Interrogait/all_datas/'

with open(f"{path}EEG_channels_names.pkl", "rb") as f:
    EEG_channels_names = pickle.load(f)
    
# Caricare file xlsx con pickle
path_xlsx = f'{path}EEG_grid_interrogait.xlsx'

# Caricamento del file in un DataFrame
EEG_file_interrogait = pd.read_excel(path_xlsx)

In [53]:
EEG_file_interrogait

,Electrode,grid_x,grid_y
0,EMPTY,0.000,0.0
1,EMPTY,0.125,0.0
2,EMPTY,0.250,0.0
3,Fp1,0.375,0.0
4,Fpz,0.500,0.0
...,...,...,...
76,Oz,0.500,1.0
77,O2,0.625,1.0
78,EMPTY,0.750,1.0
79,EMPTY,0.875,1.0


In [54]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List

def _build_grid_maps(
    eeg_grid_df: pd.DataFrame,
    eeg_channels_names: List[str],
    grid_shape: Tuple[int, int] = (9, 9),
):
    """
    Crea:
      - label_grid: matrice (9x9) di etichette (elettrodi o 'EMPTY')
      - electrode_grid_map: dict {elettrodo -> (y, x)} (solo elettrodi reali)
      - placement_idx: matrice (9x9) di indici canale (>=0) o -1 per EMPTY/non presenti
    """
    df = eeg_grid_df.copy()
    df["Electrode"] = df["Electrode"].astype(str).str.strip()

    H, W = grid_shape
    label_grid = np.full((H, W), "", dtype=object)
    electrode_grid_map = {}

    # Mappa canale -> indice colonna in X_data
    ch_to_idx = {ch: i for i, ch in enumerate(eeg_channels_names)}

    # Matrice con indici canale (per riempimento veloce delle griglie)
    placement_idx = np.full((H, W), -1, dtype=int)

    for _, row in df.iterrows():
        elec = row["Electrode"]
        x = int(round(row["grid_x"] * (W - 1)))
        y = int(round(row["grid_y"] * (H - 1)))

        label_grid[y, x] = "" if elec == "EMPTY" else elec

        if elec != "EMPTY":
            electrode_grid_map[elec] = (y, x)
            if elec in ch_to_idx:
                placement_idx[y, x] = ch_to_idx[elec]
            # se l'elettrodo non è nella lista canali, placement resta -1 (verrà messo 0 in griglia)

    # Controllo elettrodi presenti nell'Excel ma non nei dati
    excel_elec = set(df["Electrode"].unique()) - {"EMPTY"}
    missing = sorted(elec for elec in excel_elec if elec not in ch_to_idx)
    if missing:
        print("⚠️ Elettrodi nel file Excel ma non presenti in EEG_channels_names:", missing)

    return label_grid, electrode_grid_map, placement_idx


def convert_fft_images_to_2d_grids_all_freqs_interrogait(
    data_dict: Dict[str, Dict[str, Dict[str, Tuple[np.ndarray, np.ndarray]]]],
    eeg_grid_df: pd.DataFrame,
    eeg_channels_names: List[str],
    grid_shape: Tuple[int, int] = (9, 9),
    fs: int = 250,
    n_fft_points: int = 250,
    bands: Dict[str, Tuple[float, float]] = None,
    verbose: bool = True,
) -> Tuple[Dict, np.ndarray, Dict]:
    """
    Converte OGNI X_data nella struttura annidata di `data_dict`:
      (B, n_freqs, n_channels)  →  (B, 9, 9, 5)
    sommando la potenza sulle frequenze per ciascuna banda EEG e mappando
    i canali nelle posizioni (y,x) definite dal file Excel della griglia.

    Struttura in input (immutata nelle chiavi):
      data_dict[condition][data_type][category_subject] = (X_data, y_data)

    In output mantiene la stessa struttura ma con X_data trasformato:
      X_grid: (B, 9, 9, 5),  y invariato.

    Ritorna:
      - new_data_dict: stessa struttura annidata con X trasformati
      - label_grid: matrice (9x9) con le etichette
      - electrode_grid_map: dict {elettrodo -> (y, x)}
    """
    if bands is None:
        # Ordine fisso (profondità = 5)
        bands = {
            "delta": (1, 4),
            "theta": (4, 8),
            "alpha": (8, 13),
            "beta":  (13, 30),
            "gamma": (30, 45),
        }
    band_order = ["delta", "theta", "alpha", "beta", "gamma"]

    # Precostruisco mappe della griglia
    label_grid, electrode_grid_map, placement_idx = _build_grid_maps(
        eeg_grid_df=eeg_grid_df,
        eeg_channels_names=eeg_channels_names,
        grid_shape=grid_shape,
    )

    H, W = grid_shape

    # Trovo un esempio per determinare n_freqs effettivi (bins) e costruire le maschere
    example_found = False
    n_freqs_example = None
    n_channels_example = None

    for condition, data_types in data_dict.items():
        for data_type, categories in data_types.items():
            for category_subject, (X_data, y_data) in categories.items():
                if X_data is not None and len(X_data) > 0:
                    n_freqs_example = X_data.shape[1]
                    n_channels_example = X_data.shape[2]
                    example_found = True
                    break
            if example_found:
                break
        if example_found:
            break

    if not example_found:
        raise ValueError("Impossibile determinare n_freqs/n_channels: data_dict è vuoto?")

    # Frequenze in Hz per i bins RFFT (tronco ai primi n_freqs effettivi)
    all_freqs_full = np.fft.rfftfreq(n_fft_points, d=1.0 / fs)
    all_freqs = all_freqs_full[:n_freqs_example]

    # Maschere per ciascuna banda sull'asse delle frequenze
    band_masks = {
        b: (all_freqs >= fmin) & (all_freqs <= fmax) for b, (fmin, fmax) in bands.items()
    }

    # Avvisi utili
    if verbose:
        print(f"fs={fs} Hz, n_fft_points={n_fft_points}")
        print(f"n_freqs in X_data = {n_freqs_example} (verranno usati i primi {n_freqs_example} bins di rfftfreq)")
        print("Bande usate:", {b: bands[b] for b in band_order})

    # Trasformazione
    new_data_dict = {}
    for condition, data_types in data_dict.items():
        new_data_dict.setdefault(condition, {})
        for data_type, categories in data_types.items():
            new_data_dict[condition].setdefault(data_type, {})

            for category_subject, (X_data, y_data) in categories.items():
                
                # X_data: (B, n_freqs, n_channels)
                if X_data is None or X_data.size == 0:
                    new_data_dict[condition][data_type][category_subject] = (X_data, y_data)
                    continue

                B, n_freqs, n_channels = X_data.shape
                if n_freqs != n_freqs_example:
                    # Le maschere sono state costruite su n_freqs_example; se differisce, le rigenero on-the-fly
                    all_freqs_local = np.fft.rfftfreq(n_fft_points, d=1.0 / fs)[:n_freqs]
                    band_masks_local = {
                        b: (all_freqs_local >= bands[b][0]) & (all_freqs_local <= bands[b][1])
                        for b in band_order
                    }
                else:
                    band_masks_local = band_masks

                if n_channels != len(eeg_channels_names):
                    print(
                        f"⚠️ Attenzione: n_channels={n_channels} "
                        f"diverso da len(EEG_channels_names)={len(eeg_channels_names)} "
                        f"per {condition} / {data_type} / {category_subject}. "
                        f"Userò SOLO i canali presenti in placement_idx (gli altri verranno ignorati)."
                    )

                # Output per questo blocco: (B, H, W, 5)
                X_out = np.zeros((B, H, W, len(band_order)), dtype=X_data.dtype)

                # Precompute posizione valide nella griglia (non EMPTY)
                valid_pos = placement_idx >= 0
                idx_lin = placement_idx[valid_pos]  # indici canale per le posizioni valide
                yy, xx = np.where(valid_pos)       # coordinate y,x da riempire

                for b in range(B):
                    # Per ciascuna banda: somma lungo le frequenze → vettore (n_channels,)
                    per_band_grids = []
                    sample = X_data[b]  # (n_freqs, n_channels)

                    for bi, band_name in enumerate(band_order):
                        mask = band_masks_local[band_name]
                        if not np.any(mask):
                            # nessun bin in banda → griglia a zero
                            continue

                        # potenza totale per canale nella banda
                        band_power_per_ch = sample[mask, :].sum(axis=0)  # (n_channels,)

                        # riempi griglia rapidamente con indicizzazione
                        grid = np.zeros((H, W), dtype=sample.dtype)
                        # assegna solo posizioni con elettrodi mappati (placement_idx >= 0)
                        grid[yy, xx] = band_power_per_ch[idx_lin]

                        X_out[b, :, :, bi] = grid

                new_data_dict[condition][data_type][category_subject] = (X_out, y_data)

                if verbose:
                    print(
                        f"[OK] {condition} / {data_type} / {category_subject} : "
                        f"{X_data.shape}  →  {X_out.shape}"
                    )

    return new_data_dict, label_grid, electrode_grid_map


In [55]:
# 3) converti TUTTI i blocchi del tuo data_dict
data_dict, label_grid, electrode_grid_map = convert_fft_images_to_2d_grids_all_freqs_interrogait(
    data_dict,
    eeg_grid_df=EEG_file_interrogait,
    eeg_channels_names=EEG_channels_names,
    grid_shape=(9, 9),
    fs=250,
    n_fft_points=250,
    verbose=True
)

fs=250 Hz, n_fft_points=250
n_freqs in X_data = 45 (verranno usati i primi 45 bins di rfftfreq)
Bande usate: {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 45)}
[OK] th_resp_vs_pt_resp / spectrograms / familiar_th : (1586, 45, 61)  →  (1586, 9, 9, 5)
[OK] th_resp_vs_pt_resp / spectrograms / familiar_pt : (1580, 45, 61)  →  (1580, 9, 9, 5)
[OK] th_resp_vs_pt_resp / spectrograms / unfamiliar_th : (1667, 45, 61)  →  (1667, 9, 9, 5)
[OK] th_resp_vs_pt_resp / spectrograms / unfamiliar_pt : (1667, 45, 61)  →  (1667, 9, 9, 5)
[OK] th_resp_vs_shared_resp / spectrograms / familiar_th : (1586, 45, 61)  →  (1586, 9, 9, 5)
[OK] th_resp_vs_shared_resp / spectrograms / familiar_pt : (1580, 45, 61)  →  (1580, 9, 9, 5)
[OK] th_resp_vs_shared_resp / spectrograms / unfamiliar_th : (1667, 45, 61)  →  (1667, 9, 9, 5)
[OK] th_resp_vs_shared_resp / spectrograms / unfamiliar_pt : (1667, 45, 61)  →  (1667, 9, 9, 5)
[OK] pt_resp_vs_shared_resp / spectrograms / familiar_th 

#### **Implementazione: Carico Dati** 

In [56]:
# 4) FLAT: torniamo al formato { key: (X, y) } come prima
data_dict_flat = {}

In [57]:
for condition, data_types in data_dict.items():          # es. "th_resp_vs_pt_resp"
    for data_type, categories in data_types.items():            # es. "spectrograms"
        for category_subject, (X_grid, y_data) in categories.items():  # es. "familiar_th"

            # Ricostruiamo la chiave piatta come la usavi prima
            key = f"{condition}_{data_type}_{category_subject}"
            # es: "th_resp_vs_pt_resp_spectrograms_familiar_th"

            data_dict_flat[key] = (X_grid, y_data)

            # opzionale: print di debug
            print(
                f"[FLAT] {key} -> X: {X_grid.shape}, y: {y_data.shape}"
            )

# Sovrascrivi data_dict così il resto del codice resta IDENTICO
data_dict = data_dict_flat

[FLAT] th_resp_vs_pt_resp_spectrograms_familiar_th -> X: (1586, 9, 9, 5), y: (1586,)
[FLAT] th_resp_vs_pt_resp_spectrograms_familiar_pt -> X: (1580, 9, 9, 5), y: (1580,)
[FLAT] th_resp_vs_pt_resp_spectrograms_unfamiliar_th -> X: (1667, 9, 9, 5), y: (1667,)
[FLAT] th_resp_vs_pt_resp_spectrograms_unfamiliar_pt -> X: (1667, 9, 9, 5), y: (1667,)
[FLAT] th_resp_vs_shared_resp_spectrograms_familiar_th -> X: (1586, 9, 9, 5), y: (1586,)
[FLAT] th_resp_vs_shared_resp_spectrograms_familiar_pt -> X: (1580, 9, 9, 5), y: (1580,)
[FLAT] th_resp_vs_shared_resp_spectrograms_unfamiliar_th -> X: (1667, 9, 9, 5), y: (1667,)
[FLAT] th_resp_vs_shared_resp_spectrograms_unfamiliar_pt -> X: (1667, 9, 9, 5), y: (1667,)
[FLAT] pt_resp_vs_shared_resp_spectrograms_familiar_th -> X: (1586, 9, 9, 5), y: (1586,)
[FLAT] pt_resp_vs_shared_resp_spectrograms_familiar_pt -> X: (1580, 9, 9, 5), y: (1580,)
[FLAT] pt_resp_vs_shared_resp_spectrograms_unfamiliar_th -> X: (1667, 9, 9, 5), y: (1667,)
[FLAT] pt_resp_vs_shared_re

In [58]:
data_dict.keys()

dict_keys(['th_resp_vs_pt_resp_spectrograms_familiar_th', 'th_resp_vs_pt_resp_spectrograms_familiar_pt', 'th_resp_vs_pt_resp_spectrograms_unfamiliar_th', 'th_resp_vs_pt_resp_spectrograms_unfamiliar_pt', 'th_resp_vs_shared_resp_spectrograms_familiar_th', 'th_resp_vs_shared_resp_spectrograms_familiar_pt', 'th_resp_vs_shared_resp_spectrograms_unfamiliar_th', 'th_resp_vs_shared_resp_spectrograms_unfamiliar_pt', 'pt_resp_vs_shared_resp_spectrograms_familiar_th', 'pt_resp_vs_shared_resp_spectrograms_familiar_pt', 'pt_resp_vs_shared_resp_spectrograms_unfamiliar_th', 'pt_resp_vs_shared_resp_spectrograms_unfamiliar_pt'])

In [59]:
'''
perfetto ora, siccome ho creato data_dict nel modo di cui sopra, 
ora dentro ogni chiave, 
ci sono già tutte le chiavi associate correttamente, per estrarmi i dati e labels corrispondenti di quella combinazione di fattori lì.

infatti dentro ogni chiave c'è una tupla, con 2 elementi, il primo è l'array dei dati, il secondo è l'array delle labels
'''

data_dict['th_resp_vs_pt_resp_spectrograms_familiar_th'][0].shape

(1586, 9, 9, 5)

#### **Implementazione: Richiamo Reti Ottimizzate dopo W&B** 

In [60]:
'''NEW VERSION'''

# Percorso base per il salvataggio
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_post_WB"

#_params_hyperparams
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_hyperparams_post_WB"
                                        #spectrograms_best_models_channels_frequencies_params_hyperparams_post_WB

    
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_hyperparams_post_WB_GradCAM_Checks"    

base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks"    

os.makedirs(base_folder, exist_ok=True)

# Condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

# Creazione della struttura delle cartelle
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)
            
            print(f"Cartella creata: \033[1m{path}\033[0m")

Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_pt_resp/spectrograms/th_fam
Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_pt_resp/spectrograms/th_unfam
Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_pt_resp/spectrograms/pt_fam
Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_pt_resp/spectrograms/pt_unfam
Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_shared_resp/spectrograms/th_fam
Cartella creata: /home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks/th_resp_vs_shared_resp/spectrograms/th_unfam
Cartella creata: /home/stefano/Interro

In [61]:
path = '/home/stefano/Interrogait/all_datas/'

#with open(f"{path}EEG_channels_names.pkl", "rb") as f:
    #EEG_channels_names = pickle.load(f)
    
with open(f"{path}electrode_grid_map_interrogait.pkl", "rb") as f:
    EEG_channels_names = pickle.load(f)

In [62]:
'''VERSIONE NUOVA UFFICIALE


Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, 
lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.


'''

from wandb import Api
import torch
import numpy as np

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import io
import matplotlib.pyplot as plt
from PIL import Image


import re


    
'''
1) questa serve per plottare le metriche di loss e accuracy in ogni modello e condizione sperimentale
per salvarla dentro al dizionario 'training_plot' come buffer di memoria
'''


def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    '''
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    #Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    
    #plt.close(fig)
    
    '''
    # Salvare il plot in un buffer di memoria
    '''
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()
    
    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data


'''
2) questa serve per estrarmi le stringhe per ricostruire il nome del progetto su W&B per 
poi estrarmi le metriche ottenute sul training e validation 
da salvare sempre dentro al dizionario 'training_plot' 
'''

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "spectrograms" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        


'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
#combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
#condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

#print("Condizione:", condition_experiment)
#print("Data Type:", data_type)
#print("Soggetto:", subject_key)



'''

Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.

'''


'''
3) Dopodiché, comincia la funzione di load_best_run_results che, 
per ogni progetto e sweep del relativo modello,

si va ad estrarre le metriche del train (corregge il calcolo del train_auc)
e si calcola anche per il validation phase la confusion matrix e classification report


4) dopodichè dovrebbe richiamare la funzione di 
"plot_training_results" in modo che poi si salvi i plot di training e validation (sia loss che accuracy)
in modo che si salvi tutto in una immagine come buffer che viene spuntato fuori da quella funzione 

e poi inserito come valore dentro al dizionario training_results che sarà l'output di "load_best_run_results" 


quindi qui sotto mi manca richiamare la funzione "plot_training_results" con una variabile tipo training_plot = plot_training_results che avrà come argomenti

queste liste qua salvate come colonne del df creato dentro a 'load_best_run_results!'


loss_train_history     = df["train_loss"].tolist()
loss_val_history       = df["val_loss"].tolist()
accuracy_train_history = df["train_accuracy"].tolist()
accuracy_val_history   = df["val_accuracy"].tolist()


5) dopodiché mi serve caricare tutte queste info dentro al dizionario train_results, che sarà l'output di load_best_run_results... 
e su questo ho dei dubbi su quali chiavi del dizionario tenere separate oppure se "unirne" qualcuna, aggregando tutte le info del sweep_config assieme, 
sia che siano veri iper-parametri (learning rate etc) o parametri architetturali della rete (anche se avevano valori fissi) il più delle volte se vedi



sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        # --- setup generale ---
        "model_name":   {"value": "CNN2D"},
        "n_epochs":     {"value": 100},
        "patience":     {"value": 12},
        "batch_size":   {"values": [32, 48, 64, 96]},
        "standardization": {"value": True},   # fisso a True

        # --- ottimizzatore ---
        "lr":           {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "beta1":        {"values": [0.9, 0.95]},
        "beta2":        {"values": [0.99, 0.995]},
        "eps":          {"values": [1e-8, 1e-7]},

        # --- iperparametri architettura CNN2D (fissi) ---
        "conv_out_channels": {"value": 16},

        "conv_k1_h": {"value": 3}, "conv_k1_w": {"value": 5},
        "conv_k2_h": {"value": 3}, "conv_k2_w": {"value": 5},
        "conv_k3_h": {"value": 3}, "conv_k3_w": {"value": 5},

        "conv_s1_h": {"value": 1}, "conv_s1_w": {"value": 2},
        "conv_s2_h": {"value": 1}, "conv_s2_w": {"value": 2},
        "conv_s3_h": {"value": 1}, "conv_s3_w": {"value": 2},

        "pool_p1_h": {"value": 1}, "pool_p1_w": {"value": 2},
        "pool_p2_h": {"value": 1}, "pool_p2_w": {"value": 2},
        "pool_p3_h": {"value": 1}, "pool_p3_w": {"value": 1},

        "pool_type":  {"value": "max", "avg"},     # se vuoi fissarlo; se vuoi provarlo, usa {"values":["max","avg"]}
        "fc1_units":  {"value": 12},
        "cnn_act1":   {"value": "relu"},
        "cnn_act2":   {"value": "relu"},
        "cnn_act3":   {"value": "relu"},
        "dropout":    {"value": 0.5}
    }
}



'''

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def load_best_run_results(
    key, # es. "rest_vs_left_fist_spectrograms_familiar_th"
    model, # # <-- istanza PyTorch già caricata con i pesi best es. "CNN3D_LSTM_FC"
    
    sweep_config,      # <— qui richiamo lo sweep config del modello corrispondente
    
    data_loaders, # dict con DataLoader per "train" e "val"
    entity= "my_wb_entity"): # entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
    
    
    # --- 1) Parse key e ricava project name ---
    exp_cond, data_type, category_subject = parse_combination_key(key)
    
    
    '''CAMBIATA PER DATI INTERROGAIT IN RAPPRESENTAZIONE TIME DOMAIN 1D'''
    #project = f"{exp_cond}_{data_type}_channels_freqs_new_3d_grid_multiband"
    #project = f"{exp_cond}_{data_type}_time_freqs_new_imagery_3d_grid_multiband"
    
    project = f"{exp_cond}_{data_type}_channels_freqs_{category_subject}"
    
    model_name = type(model).__name__
    

    '''SE ESTRAGGO SWEEP ID A POSTERIORI DAL PROGETTO

    1) Prendo tutte le run del progetto e modello corrispondente
    2) Filtro solo quelle con config["model_name"] == model_name.
    3) Controllo che ce ne sia almeno una (altrimenti errore).
    4) Costruisce un set di tutti gli r.sweep e verifica che sia esattamente uno (altrimenti errore).
    5) Estrae quello unico (.pop()) e lo stampa insieme al numero di run.
    6) Infine, seleziona la singola best_run sulla base di val_accuracy.

    '''
    
    # 2) Recupero tutte le run del progetto
    api  = Api()
    runs = api.runs(f"{entity}/{project}")

    # 3) filtro solo quelle del modello giusto
    runs_filtered = [r for r in runs if r.config.get("model_name", "") == model_name]
    n_runs = len(runs_filtered)

    if n_runs == 0:
        raise RuntimeError(f"Nessuna run trovata per progetto `{project}` e modello `{model_name}`")

    # 4) controllo che le run filtrate appartengano tutte allo stesso sweep
    unique_sweeps = {r.sweep for r in runs_filtered}
    if len(unique_sweeps) != 1:
        raise RuntimeError(
            f"Trovati più sweep per progetto `{project}` e modello `{model_name}`: {unique_sweeps}"
        )

    # 5) estraggo lo sweep_id
    sweep_id_unico = unique_sweeps.pop()
    #print(f"✓ Trovate \033[1m{n_runs}\033[0m runs in progetto `{project}` e modello `{model_name}`, sweep: `{sweep_id_unico}`")
    print(f"✓ Trovate \033[1m{n_runs}\033[0m runs\n")
    print(f"✓ Progetto \033[1m`{project}`\033[0m\n")
    print(f"✓ Modello \033[1m`{model_name}`\033[0m\n")
    print(f"✓ Sweep \033[1m`{sweep_id_unico}`\033[0m\n\n")

    # 6) scelgo la run con val_accuracy massima
    best_run = max(runs_filtered, key=lambda r: r.summary.get("val_accuracy", 0.0))

    # --- 7) Estraggo tutta la history (compresi i train_auc sbagliati) ---
    df = best_run.history(
        keys=[
          "train_loss","train_accuracy","train_precision",
          "train_recall","train_f1","train_auc",
          "val_loss","val_accuracy"
        ],
        pandas=True
    )
    # converto in liste
    loss_train_history     = df["train_loss"].tolist()
    loss_val_history       = df["val_loss"].tolist()
    accuracy_train_history = df["train_accuracy"].tolist()
    accuracy_val_history   = df["val_accuracy"].tolist()
    precision_train_history= df["train_precision"].tolist()
    recall_train_history   = df["train_recall"].tolist()
    f1_train_history       = df["train_f1"].tolist()
    auc_train_history      = df["train_auc"].tolist()

    # best_epoch (su val_accuracy)
    best_epoch = int(df["val_accuracy"].idxmax())

    # --- 8) Prendo il modello ottimizzato .pkl corrispondente passato in input ---
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device).eval()

    # --- 9) Ricalcolo vero train AUC–ROC sul train_loader ---
    y_t_train, y_s_train = [], []
    with torch.no_grad():
        for x,y in data_loaders["train"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            y_s_train.extend(probs)
            y_t_train.extend(y.numpy())
            
    true_auc_train = roc_auc_score(np.array(y_t_train), np.array(y_s_train))

    # Sovrascrivo il vecchio valore sbagliato
    auc_train_history[best_epoch] = true_auc_train

    # Ricostruisco best_metrics
    best_metrics = {
      "train_loss":       [round(loss_train_history[best_epoch],4)],
      "train_accuracy":   [round(accuracy_train_history[best_epoch],4)],
      "train_precision":  [round(precision_train_history[best_epoch],4)],
      "train_recall":     [round(recall_train_history[best_epoch],4)],
      "train_f1_score":   [round(f1_train_history[best_epoch],4)],
      "train_auc":        [round(true_auc_train,4)]
    }

    #Solo una nota: qui non serve per training che l'auc abbia l'average='weighted' 
    #perché è binario e stai usando score continui.
    #anche se sopra lo avevi messo in "training_sweep".
    
    #Per le altre metriche (precision, recall, f1_score invece) l'average andava bene!
    #Anche in binario: average='weighted' = fai la media pesata per supporto delle metriche per ciascuna classe (0 e 1). 
    #È sensato se hai sbilanciamento e vuoi che le metriche riflettano anche quanto è frequente ciascuna classe. 
    
    #L’unica cosa da essere consapevoli è che non stai riportando “F1 della classe positiva”, 
    #ma una F1 complessiva pesata sulle due classi. 
    #Ma va bene, basta essere coerenti e chiari nel testo della tesi/paper."
    

    # --- 10) Ricreo confusion matrix e classification report su val set ---
    
    #Per il validation set, invece, rifai il calcolo:
    
    #y_t_val = true labels (0/1).
    #y_p_val = predizioni binarie (0/1), usate per accuracy / precision / recall / f1.
    #y_s_val = score continui (probabilità o logit della classe 1), usati per il calcolo dell'AUC-ROC:
    
    #Quindi diventerà ---> val_auc = roc_auc_score(y_t_val, y_s_val)
    #Quindi qui "y_s_val" è semplicemente la lista di p(y=1) per ogni campione di validation.
    
    y_t_val, y_p_val, y_s_val = [], [], []
    with torch.no_grad():
        for x,y in data_loaders["val"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds  = (probs >= 0.5).astype(int)
            
            
            y_p_val.extend(preds) # predizioni 0/1
            y_s_val.extend(probs) # score continui per AUC
            y_t_val.extend(y.numpy()) 
            
    confusion_matrix_val = confusion_matrix(y_t_val, y_p_val)
    classification_report_val = classification_report(y_t_val, y_p_val, output_dict=False)
    
    # Metriche Validation
    #val_accuracy = accuracy_score(y_t_val, y_p_val)
    val_precision = precision_score(y_t_val, y_p_val, average='weighted')
    val_recall    = recall_score(y_t_val, y_p_val, average='weighted')
    val_f1        = f1_score(y_t_val, y_p_val, average='weighted')
    
    try:
        val_auc = roc_auc_score(y_t_val, y_s_val)   # <-- NOTHING average=... qui
    except ValueError:
        print("⚠️ AUC non calcolabile: nel val set c'è una sola classe.")
        val_auc = np.nan
    
    # Val performances alla best_epoch
    '''
    Qui la cosa importante è: il modello con cui stai facendo il forward su data_loaders["val"] 
    è il best model, cioè quello che hai caricato da .pkl e che dovrebbe corrispondere esattamente 
    ai pesi di best_epoch per quella run.
    
    Per cui, val_loss e val_accuracy che salvi nel dict sono proprio quelli loggati all’epoca best_epoch durante il training.
    Questi sono coerenti con “la migliore epoca secondo val_accuracy”.
    
    Mentre le altre metriche (precision, recall, f1_score, son ricalcolate in base al best model che aveva ottenuto
    a quella epoca specifica la migliore val_accuracy!
    
    '''
    
    validation_performances = {
        # dalla history di W&B (loss/acc per quella epoch)
        "val_loss":       [round(loss_val_history[best_epoch],4)],
        "val_accuracy":   [round(accuracy_val_history[best_epoch],4)],
        
        # dalle metriche ricalcolate con il best_model
        "val_precision":  [round(val_precision,4)],
        "val_recall":     [round(val_recall,4)],
        "val_f1_score":   [round(val_f1,4)],
        "val_auc":        [round(val_auc,4)],
    }
    
        
    # --- 10) Plot delle curve loss/accuracy tra train e test ---
    training_plot = plot_training_results(
        loss_train_history,
        loss_val_history,
        accuracy_train_history,
        accuracy_val_history
    )

    # --- 11) Composizione del dict finale identico a `training()` ---
    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        
        "best_model": model,
        
        # VALIDATION
        "validation_performances": validation_performances,
        
        "confusion_matrix": confusion_matrix_val,
        "classification_report": classification_report_val,
    
        "hyperparams" : {k: best_run.config[k] for k in best_run.config.keys() if k in sweep_config["parameters"]},
            
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }
    
    '''
    Ho questo errore "Errore “cudnn RNN backward can only be called in training mode”" solo con i dati di 
    left_fist_vs_right_fist, per il modello SeparableCNN2D_LSTM_FC, 
    mentre con i dati delle altre condizioni sperimentali, ossia:
    
    rest_vs_left_fist o rest_vs_right_fist, sempre per il modello SeparableCNN2D_LSTM_FC,non succede ... come mai solo con l'ultimo succede? 
    
    cioè dove dovrei aver lasciato il modello caricato in eval.() ?
    
    probabilmente qui nella funzione load_best_train_results!?
    
    quindi qui poi alla fine dovrei rimettere il modello in un'altra modalità alla fine della funzione? 
    perché in sostanza, dovrebbe succedere che in sostanza... non succede nulla per lo stesso modello per  gli altri dati, 
    perché ogni volta che ne prendo uno lo porto in eval e vabbè.. ma poi il problema succede solo per l'ultimo caso solo, 
    perché forse l'ultimo proprio, ossia solo SeparableCNN2D_LSTM_FC usa proprio il layer LSTM e quindi da errore là,
    perché dentro a load_best_train_results è rimasto in .eval() ed ha il layer LSTM e quindi dà errore?
    
    
    
    Perché l’errore appare “solo” con l’ultima combinazione

    1. load_best_run_results() termina con:

    model.to(device).eval()   # ← il modello rimane in eval()
    
    2. In compute_gradcam_figure() tu usi il best model che hai messo in train_results["best_model"] (quello appena impostato in eval()), poi esegui:

   
    output = model(sample_input)
    ...
    target.backward()         # <-- gradiente attraverso l’LSTM
    3. Il kernel CuDNN per gli RNN (LSTM/GRU) rifiuta il backward quando il modulo è in modalità inference (eval()), e solleva:

    
    RuntimeError: cudnn RNN backward can only be called in training mode
    
    4. Per le combinazioni precedenti con lo stesso modello “SeparableCNN2D_LSTM_FC” non è esploso perché, con ogni probabilità, 
    use_lstm=False nelle relative run migliori (quindi l’LSTM non c’è e CuDNN non interviene).
    
    Nell’ultima combinazione invece la best‑run ha use_lstm=True, quindi compare l’LSTM e l’errore salta fuori.
    
    '''

    return train_results

In [ ]:
'''
sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        # --- setup generale ---
        "model_name":   {"value": "CNN2D"},
        "n_epochs":     {"value": 100},
        "patience":     {"value": 12},
        "batch_size":   {"values": [32, 48, 64, 96]},
        "standardization": {"value": True},   # fisso a True

        # --- ottimizzatore ---
        "lr":           {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "beta1":        {"values": [0.9, 0.95]},
        "beta2":        {"values": [0.99, 0.995]},
        "eps":          {"values": [1e-8, 1e-7]},

        # --- iperparametri architettura CNN2D (fissi) ---
        "conv_out_channels": {"value": 16},

        "conv_k1_h": {"value": 3}, "conv_k1_w": {"value": 5},
        "conv_k2_h": {"value": 3}, "conv_k2_w": {"value": 5},
        "conv_k3_h": {"value": 3}, "conv_k3_w": {"value": 5},

        "conv_s1_h": {"value": 1}, "conv_s1_w": {"value": 2},
        "conv_s2_h": {"value": 1}, "conv_s2_w": {"value": 2},
        "conv_s3_h": {"value": 1}, "conv_s3_w": {"value": 2},

        "pool_p1_h": {"value": 1}, "pool_p1_w": {"value": 2},
        "pool_p2_h": {"value": 1}, "pool_p2_w": {"value": 2},
        "pool_p3_h": {"value": 1}, "pool_p3_w": {"value": 1},

        "pool_type":  {"value": "max", "avg"},     # se vuoi fissarlo; se vuoi provarlo, usa {"values":["max","avg"]}
        "fc1_units":  {"value": 12},
        "cnn_act1":   {"value": "relu"},
        "cnn_act2":   {"value": "relu"},
        "cnn_act3":   {"value": "relu"},
        "dropout":    {"value": 0.5}
    }
}
'''

In [ ]:
'''
class CNN2D(nn.Module):
    def __init__(
        self,
        input_channels: int,
        num_classes: int,

        # da sweep: numero di feature map di base
        conv_out_channels: int,

        # da sweep: kernel size H×W per i 3 blocchi
        conv_k1_h: int, conv_k1_w: int,
        conv_k2_h: int, conv_k2_w: int,
        conv_k3_h: int, conv_k3_w: int,

        # da sweep: stride H×W per i 3 blocchi
        conv_s1_h: int, conv_s1_w: int,
        conv_s2_h: int, conv_s2_w: int,
        conv_s3_h: int, conv_s3_w: int,

        # da sweep: pool kernel H×W per i 3 blocchi
        pool_p1_h: int, pool_p1_w: int,
        pool_p2_h: int, pool_p2_w: int,
        pool_p3_h: int, pool_p3_w: int,

        # da sweep: tipo di pooling
        pool_type: str,  # "max" o "avg"

        # fully‑connected
        fc1_units: int,
        dropout: float,

        # attivazioni per i 3 blocchi
        cnn_act1: str,
        cnn_act2: str,
        cnn_act3: str,
    ):
        super().__init__()
        mapping = {'relu': F.relu, 'selu': F.selu, 'elu': F.elu}
        self.act_fns = [
            mapping[cnn_act1],
            mapping[cnn_act2],
            mapping[cnn_act3],
        ]
        
        # calcolo padding “quasi‐same” per ciascun blocco
        p1_h = (conv_k1_h - 1) // 2
        p1_w = (conv_k1_w - 1) // 2
        p2_h = (conv_k2_h - 1) // 2
        p2_w = (conv_k2_w - 1) // 2
        p3_h = (conv_k3_h - 1) // 2
        p3_w = (conv_k3_w - 1) // 2
        
        # Primo blocco
        self.conv1 = nn.Conv2d(
            input_channels, conv_out_channels,
            kernel_size = (conv_k1_h, conv_k1_w),
            stride = (conv_s1_h, conv_s1_w),
            #padding='same'
            padding = (p1_h, p1_w)
        )
        self.bn1   = nn.BatchNorm2d(conv_out_channels)
        self.pool1 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p1_h, pool_p1_w))

        # Secondo blocco (×2 feature map)
        self.conv2 = nn.Conv2d(
            conv_out_channels, conv_out_channels*2,
            kernel_size=(conv_k2_h, conv_k2_w),
            stride=(conv_s2_h, conv_s2_w),
            #padding='same'
            padding = (p2_h, p2_w) 
        )
        self.bn2   = nn.BatchNorm2d(conv_out_channels*2)
        self.pool2 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p2_h, pool_p2_w))

        # Terzo blocco (×3 feature map)
        self.conv3 = nn.Conv2d(
            conv_out_channels*2, conv_out_channels*3,
            kernel_size=(conv_k3_h, conv_k3_w),
            stride=(conv_s3_h, conv_s3_w),
            #padding='same'
            padding = (p3_h, p3_w)
        )
        self.bn3   = nn.BatchNorm2d(conv_out_channels*3)
        self.pool3 = (nn.MaxPool2d if pool_type=='max' else nn.AvgPool2d)((pool_p3_h, pool_p3_w))

        # FC finale
        self.fc1     = nn.LazyLinear(fc1_units)
        self.dropout = nn.Dropout(dropout)
        self.fc2     = nn.LazyLinear(num_classes)
    
    
    def forward(self, x):
        
        # Input Iniziale
        #x: (batch, frequenze, canali)
        
        #🔁 Prima:
        
        # Sappaimo che x abbia forma (batch_size, 45, 61)
        # Se i 61 sono i canali, allora occorre trasporre le dimensioni:
        
        # Permutiamo per ottenere (batch, canali, frequenze)
        #x = x.permute(0, 2, 1)  # Ora ha forma (batch_size, 61, 45)
        
        # Aggiungiamo una dimensione extra per adattarlo alla convoluzione 2D
        #x = x.unsqueeze(3)  # Ora ha forma (batch_size, 61, 45, 1)
        
        #✅ Ora:
        #Siccome i dati arrivano come (B, 45, 61) — cioè frequenze × canali, non serve permutare. Ti basta:
        
        # Aggiungiamo una dimensione per il canale "immagine"
        x = x.unsqueeze(1)  # → (B, 1, 45, 61)
            
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = self.act_fns[0](x)
        
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = self.act_fns[1](x)
        
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = self.act_fns[2](x)
       
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.relu(x)
       
        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x
       
       
'''

In [ ]:
'''


        "conv_k1_h": {"value": 3}, "conv_k1_w": {"value": 5},
        "conv_k2_h": {"value": 3}, "conv_k2_w": {"value": 5},
        "conv_k3_h": {"value": 3}, "conv_k3_w": {"value": 5},

        "conv_s1_h": {"value": 1}, "conv_s1_w": {"value": 2},
        "conv_s2_h": {"value": 1}, "conv_s2_w": {"value": 2},
        "conv_s3_h": {"value": 1}, "conv_s3_w": {"value": 2},

        "pool_p1_h": {"value": 1}, "pool_p1_w": {"value": 2},
        "pool_p2_h": {"value": 1}, "pool_p2_w": {"value": 2},
        "pool_p3_h": {"value": 1}, "pool_p3_w": {"value": 1},

        "pool_type":  {"values": ["max", "avg"]},     # se vuoi fissarlo; se vuoi provarlo, usa {"values":["max","avg"]}
        "fc1_units":  {"value": 12},
        "cnn_act1":   {"value": "relu"},
        "cnn_act2":   {"value": "relu"},
        "cnn_act3":   {"value": "relu"},
        "dropout":    {"value": 0.5}
        }
        
        
        
    
        model = CNN2D(
                input_channels   = 1,
                num_classes      = num_classes,
                conv_out_channels= config.conv_out_channels,

                conv_k1_h = config.conv_k1_h, 
                conv_k1_w = config.conv_k1_w,

                conv_k2_h = config.conv_k2_h, 
                conv_k2_w = config.conv_k2_w,

                conv_k3_h = config.conv_k3_h,
                conv_k3_w = config.conv_k3_w,

                conv_s1_h = config.conv_s1_h, 
                conv_s1_w = config.conv_s1_w,

                conv_s2_h = config.conv_s2_h,
                conv_s2_w = config.conv_s2_w,

                conv_s3_h = config.conv_s3_h,
                conv_s3_w = config.conv_s3_w,

                pool_p1_h = config.pool_p1_h,
                pool_p1_w = config.pool_p1_w,

                pool_p2_h = config.pool_p2_h,
                pool_p2_w = config.pool_p2_w,

                pool_p3_h = config.pool_p3_h,
                pool_p3_w = config.pool_p3_w,

                pool_type = config.pool_type,

                fc1_units = config.fc1_units,
                dropout   = config.dropout,

                cnn_act1  = config.cnn_act1,
                cnn_act2  = config.cnn_act2,
                cnn_act3  = config.cnn_act3,
            )
            
            
'''

In [63]:
# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}

In [ ]:
'''

PER CNN2D
'''


# Imposta il seme per la riproducibilità

#Imposta il seme per i generatori casuali di PyTorch (per operazioni sui tensori e inizializzazione dei pesi dei modelli).
#Importante se vuoi garantire che l'addestramento del modello produca gli stessi risultati in diverse esecuzioni.
torch.manual_seed(32)

#Imposta il seme per NumPy, utile se NumPy viene usato per operazioni casuali (ad es. shuffling dei dati, inizializzazione di matrici, ecc.).
#Importante se usi NumPy per il preprocessing dei dati e vuoi riproducibilità.

np.random.seed(32)

#mposta il seme per il modulo random di Python (utile se si usano funzioni di randomizzazione di Python puro).
#Importante solo se usi random per operazioni come mescolamento di liste.
random.seed(32)

#Imposta il seme per i generatori casuali su GPU, se disponibile.
#Utile se stai eseguendo il codice su una GPU per garantire riproducibilità anche in quel contesto.

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(32)

       
'''

In questo caso, 

il set processed_datasets traccia i dataset già elaborati, 
e il set processed_models tiene traccia delle combinazioni già effettuate (modello + dataset). 

In questo modo, puoi escludere un dataset dal training se è già stato utilizzato in precedenza, 
anche se usato con un modello differente.
'''


# Dizionario per tracciare la standardizzazione usata per ogni combinazione d
# Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)
models_info = {}

EEG_channels = EEG_channels_names 

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Set per tenere traccia delle combinazioni già elaborate
processed_models = set()


# Path delle performance dei modelli ottimizzati con weight and biases
# Path per trovare le best performances di ogni modello per ogni combinazione dei dati
base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies"
                                        #WB_spectrograms_best_results_channels_frequencies_params_hyperparams/

# Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder
#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_post_WB"

#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_hyperparams_post_WB"
save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks"



# --- LOOP PRINCIPALE (con minime modifiche) ---
for key, (X_data, y_data) in data_dict.items():
    
    print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")
    
    if key in processed_datasets:
        print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
        continue
        
    processed_datasets.add(key)
    
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
    print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")
    
    
    '''
    CREO COPIA TEST_LOADER_RAW PER I PLOT DEL POWER RAW PER BANDA E CLASSE
    '''
    # 1) salva una copia RAW dei soli dati di test PRIMA di standardizzare
    X_test_raw = X_test.copy()
    y_test_raw = y_test.copy()
    
    # 2) tensori
    X_raw_tensor = torch.tensor(X_test_raw, dtype=torch.float32)
    y_raw_tensor = torch.tensor(y_test_raw, dtype=torch.long)
    
    
    
    #for model_name in ["CNN2D", "BiLSTM", "Transformer"]:
    
    '''ATTENZIONE MODIFICA QUI'''
    
    #for model_name in ["CNN1D", "BiLSTM", "Transformer"]:
    
    for model_name in ["CNN2D"]:

        model_key = f"{model_name}_{key}"
        if model_key in processed_models:
            print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
            continue
        processed_models.add(model_key)
        
        print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")
        
        # Prova a caricare la configurazione e i pesi ottimali dal file .pkl
        
        '''
        load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
        parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate
        
        exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
        pesi del modello e i suoi iper-parametri
        
        Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
        '''
        
        config, best_weights = load_config_if_available(key, model_name, base_folder)
        
        if config is None:
            raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")
        
        '''
        Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
        MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
        '''
        
        # Parsifica la chiave una volta sola e memorizza i valori
        exp_cond, data_type, category_subject = parse_combination_key(key)
        
        '''
        Dpodiché, 
        
        1) si carica i vari valori degli iper-parametri,
        2) si esegue la standardizzazione se servisse,
        3) prepara il modello per la divisione in train_loader etc.,
        4) si carica la configurazione dei pesi del modello, 
        5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 
        
        6) esegue il training e il test e poi
        
        7) si salva il tutto nella path corrispondente...
        
        
        
        
        Ricordati di aggiungere le varie variabili associate alla definizione dinamica dei valori dei parametri model-specific qui!
    
        
        
        '''
        
        '''
        PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
        DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''
        
        
        model_lr = config["lr"]
        model_weight_decay = config["weight_decay"]
        model_n_epochs = config["n_epochs"]
        model_patience = config["patience"]
        
        
        model_batch_size = config["batch_size"]
        model_standardization = config["standardization"]
        
        #model_n_epochs = config["n_epochs"]
        #model_patience = config["patience"]
        
        #model_lr = config["lr"]
        
        '''NUOVE MODIFICHE'''
        model_beta1 =  config["beta1"]
        model_beta2 =  config["beta2"]
        model_eps = config["eps"]
        
        
        
        #model_weight_decay = config["weight_decay"]
        #model_standardization = config["standardization"]
        
        #print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, model_beta1= \033[1m{model_beta1}\033[0m,  model_beta2= \033[1m{model_beta2}\033[0m,  model_eps= \033[1m{model_eps}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        
        # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
        models_info[model_key] = {"standardization": model_standardization}
        
        
        # 3) dataset & loader per test set (per plots power raw) –‑  IMPORTANTISSIMO: shuffle=False
        raw_dataset = TensorDataset(X_raw_tensor, y_raw_tensor)
        test_loader_raw = DataLoader(raw_dataset,
                             batch_size=model_batch_size,
                             shuffle=False)
        
        
        
        '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
        IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
        '''

        if model_standardization:
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
        else:
            print(f"\033[1mNO Standardizzazione Dati!\033[0m")
        
        # Sposta il modello sulla GPU (se disponibile)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
        # Preparazione dei dataloaders
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)
        

        '''
        # ====== MODELLO CNN2D FREQUENCY x CHANNELS 2D ======
        
        PRENDO LA SHAPE DEI DATI PER FORNIRE VALORI GIUSTI PER OGNI INPUt DI CIASCUNA RETE
    
        # Appena caricato X_train, X_val, X_test, etc.
        # X_train.shape == (N, freq_bins, channels)

        _, freq_bins, channels = X_train.shape

        #NEW VERSION
        if config.model_name == "CNN2D":

            #model = CNN2D(
                #input_channels   = 1,
                #num_classes      = 2,
                #conv_out_channels= config.conv_out_channels,
                #conv2d_kernel_size = tuple(config.conv2d_kernel_size),
                #conv2d_stride      = tuple(config.conv2d_stride),
                #pool_type        = config.pool_type,
                #pool2d_kernel_size = tuple(config.pool2d_kernel_size),
                #fc1_units        = config.fc1_units,
                #dropout          = config.dropout,
                #activations      = tuple(config.activations)
            #)
            #print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
        
    
        model = CNN2D(
                input_channels   = 1,
                num_classes      = num_classes,
                conv_out_channels= config.conv_out_channels,

                conv_k1_h = config.conv_k1_h, 
                conv_k1_w = config.conv_k1_w,

                conv_k2_h = config.conv_k2_h, 
                conv_k2_w = config.conv_k2_w,

                conv_k3_h = config.conv_k3_h,
                conv_k3_w = config.conv_k3_w,

                conv_s1_h = config.conv_s1_h, 
                conv_s1_w = config.conv_s1_w,

                conv_s2_h = config.conv_s2_h,
                conv_s2_w = config.conv_s2_w,

                conv_s3_h = config.conv_s3_h,
                conv_s3_w = config.conv_s3_w,

                pool_p1_h = config.pool_p1_h,
                pool_p1_w = config.pool_p1_w,

                pool_p2_h = config.pool_p2_h,
                pool_p2_w = config.pool_p2_w,

                pool_p3_h = config.pool_p3_h,
                pool_p3_w = config.pool_p3_w,

                pool_type = config.pool_type,

                fc1_units = config.fc1_units,
                dropout   = config.dropout,

                cnn_act1  = config.cnn_act1,
                cnn_act2  = config.cnn_act2,
                cnn_act3  = config.cnn_act3,
            )
        
        
        ...
    
        
        
        '''
        
        
        '''PARAMETRI MODEL-SPECIFIC DI CNN2D, ma richiamati al momenti della inizializzazione dei relativi sweep_config! '''
        
        #model_dropout = config['dropout']
        #model_bidirectional = config['bidirectional']
        #model_d_model = config['d_model']
        #model_num_heads = config['num_heads']
        #model_num_layers  = config['num_layers']
        
        
        # Per caricare la shape dei dati 2D da X_train, X_val, X_test, etc.
        
        #Prelevo le dimensioni di frequencies e channels dei miei dati 
        
        _, freq_bins, channels = X_train.shape
        
        # Nome dei tre sweep config usati
        
        # sweep_config_cnn2d

        if model_name == "CNN2D":
            
            #Sweep Config CNN2D (sweep_config_cnn2d)
            
            sweep_config = sweep_config_cnn2d
        
            #Numero di classi da riconoscere (es. binaria)
            num_classes = 2
            
            model_conv_out_channels = config["conv_out_channels"]
            
            model_conv_k1_h =config["conv_k1_h"]
            model_conv_k1_w =config["conv_k1_w"]
            
            model_conv_k2_h =config["conv_k2_h"]
            model_conv_k2_w =config["conv_k2_w"]
            
            model_conv_k3_h =config["conv_k3_h"]
            model_conv_k3_w =config["conv_k3_w"]
            
            model_conv_s1_h =config["conv_s1_h"]
            model_conv_s1_w =config["conv_s1_w"]
            
            model_conv_s2_h =config["conv_s2_h"]
            model_conv_s2_w =config["conv_s2_w"]
            
            model_conv_s3_h =config["conv_s3_h"]
            model_conv_s3_w =config["conv_s3_w"]
            
            model_pool_p1_h = config["pool_p1_h"]
            model_pool_p1_w = config["pool_p1_w"]
            
            model_pool_p2_h = config["pool_p2_h"]
            model_pool_p2_w = config["pool_p2_w"]
            
            model_pool_p3_h = config["pool_p3_h"]
            model_pool_p3_w = config["pool_p3_w"]
            
            
            model_pool_type =config["pool_type"]
            
            
            model_fc1_units = config["fc1_units"]
            
            model_dropout = config["dropout"]
            
            model_cnn_act1 = config["cnn_act1"]
            model_cnn_act2 = config["cnn_act2"]
            model_cnn_act3 = config["cnn_act3"]
            
        
            model = CNN2D(
                input_channels   = 1,
                num_classes      = num_classes,
                conv_out_channels= model_conv_out_channels, 
                
                conv_k1_h        = model_conv_k1_h,
                conv_k1_w        = model_conv_k1_w,
                
                conv_k2_h        = model_conv_k2_h,
                conv_k2_w        = model_conv_k2_w,
                
                conv_k3_h        = model_conv_k3_h,
                conv_k3_w        = model_conv_k3_w,
                
                
                conv_s1_h        = model_conv_s1_h,
                conv_s1_w        = model_conv_s1_w,
                
                conv_s2_h        = model_conv_s2_h,
                conv_s2_w        = model_conv_s2_w,
                
                conv_s3_h        = model_conv_s3_h,
                conv_s3_w        = model_conv_s3_w,
                
            
                pool_p1_h        = model_pool_p1_h,
                pool_p1_w        = model_pool_p1_w,
                
                pool_p2_h        = model_pool_p2_h,
                pool_p2_w        = model_pool_p2_w,
                
                pool_p3_h        = model_pool_p3_h,
                pool_p3_w        = model_pool_p3_w,
                
                
                pool_type        = model_pool_type,
                
                
                fc1_units        = model_fc1_units,
                dropout          = model_dropout,
                
                cnn_act1         = model_cnn_act1,
                cnn_act2         = model_cnn_act2,
                cnn_act3         = model_cnn_act3,
            )
    
        
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
        
        # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
        if best_weights is not None:
            try:
                model.load_state_dict(best_weights)
                print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
            except Exception as e:
                print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                continue
        
        
        '''NUOVE MODIFICHE'''
        # Definizione del criterio di perdita
        criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
        
        '''OLD VERSION'''
        # Definizione dell'ottimizzatore con i parametri aggiornati
        #optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)
        
        '''NUOVE MODIFICHE'''
        
         # 10) ottimizzatore + scheduler + early stopping
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr    = model_lr,
            betas = (model_beta1, model_beta2),
            eps   = model_eps,
            weight_decay = model_weight_decay
            
        )
            
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode     = "min",   # monitoriamo val_loss
            factor   = 0.1,
            patience = 8,
            verbose  = True
        )
        early_stopping = EarlyStopping(patience=model_patience, mode="min")
        
        
        #criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
        
        
        '''OLD VERSION'''
        #print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)
    
        #print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_test_results = testing(my_train_results, test_loader, criterion)
        
        '''NEW VERSION'''
        # --- dopo model.load_state_dict(best_weights) e criterion = nn.CrossEntropyLoss() ---

        # 1) prepara i data_loaders per train/val
        data_loaders = {
            "train": train_loader,
            "val":   val_loader
        }
        
        print(f"🏋️‍♂️Salvo le metriche del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m a seguito della ottimizzazione su W&B ...")
        # 2) richiama la funzione che pesca da W&B la best‐run e corregge la train AUC
        
        #ATTENZIONE al potenziale problema di stringa, non di API: 

        #i due esempi che hai postato in realtà usano diversi caratteri “‑” (uno è il classico ASCII U+002D, l’altro è un non‑breaking hyphen U+2011 o simili), quindi quando chiami
        
        #entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
        #stai passando un nome che W&B non riconosce (e quindi api.projects(entity=…) torna vuoto), mentre con

        #entity = "stefano-bargione-universit-di-roma-tor-vergata"
        #funziona perché lì usi i semplici - ASCII.

        my_train_results = load_best_run_results(
            key=key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity = "stefano-bargione-universit-di-roma-tor-vergata"
        )
        
        
        '''
        L’entity che passi a Api().runs(f"{entity}/{project}") è semplicemente il tuo account (o l’organizzazione) su W&B,
        cioè la parte che compare subito prima del nome del progetto nell’URL.

        Per esempio, se quando apri il tuo progetto su W&B vedi un indirizzo del tipo
        
        -> https://wandb.ai/steclab/some_project_name, allora entity = "steclab".
        
        Se invece lavori sotto un’organizzazione 
        
        -> “cool‑team”, e l’URL è https://wandb.ai/cool-team/some_project_name, allora userai entity = "cool-team".

        Puoi verificarlo:

        Accedi a wandb.ai e vai sul progetto.
        Leggi la prima parte dell’URL (tra wandb.ai/ e il /project_name).
        Copiala esattamente come stringa in entity.

        Così il tuo Api().runs(f"{entity}/{project}") andrà a pescare proprio le run che hai lanciato tu.

        my_train_results = load_best_run_results(
            key= key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity= "mio-entity"
        )
        
        '''
        
        print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        # 3) usa il best_model caricato dentro `train_results` e chiama il testing
        my_test_results = testing(my_train_results, test_loader, criterion)
        
                        
        #print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #save_performance_results(model_name,
                                 #my_train_results,
                                 #my_test_results,
                                 #key,
                                 #exp_cond,
                                 #model_standardization,
                                 #base_folder = save_path_folder)
        
        
        
        
        
        #++++++++++++++++++++++++ ++++++++++++ ++++++++++++ ++++++++++++ ++++++++++++ ++++++++++++ ++++++++++++
        
        '''
        GRADCAM COMPUTATION PER IL MODELLO CNN3D e ConvSep
        
        La funzione compute_gradcam_figure estrae i campioni (per ogni classe) e crea una figura con le due righe richieste.
        
        Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
        'save_performance_results', in modo da essere salvato nella path corretta. 
        
        La funzione 'save_performance_results' è stata modificata 
        per gestire ANCHE questo nuovo input dell'immagine 
        
        (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
        seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:
        
        - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
        - tipologia di dato EEG prelevato (i.e., spectrograms) 
        - provenienza del dato stesso (i.e., familiar_th)
        )
        
        Spiegazione:
        
        La funzione compute_gradcam_figure eseguire il calcolo di GradCAM (vedi dettagli nella sua funzione)
        e alla fine ritornerà in output una variabile 
        
        'fig_image' che sarà poi assegnata alla variabile 'gradcam_image',
        che è un oggetto buffer, che contiene i dati binari dell'immagine in formato PNG
        (poiché abbiamo usato plt.savefig con format='png'). 
        
        Quindi, quando passi gradcam_image (cioè fig_image) alla funzione 'save_performance_results',
        viene scritto direttamente su disco come file PNG.
        
        Non c'è bisogno di ri-aprire o convertire ulteriormente, a meno che tu non voglia manipolare l'immagine in seguito.
        Quindi, la soluzione è corretta così com'è:
        il buffer viene salvato come file PNG nella directory specificata, 
        e successivamente potrai aprirlo con una libreria come cv2 o PIL se necessario.        
        
        Quindi, gradcam_image (i.e., fig_image) viene quindi passato correttamente dentro al loop di training e test, 
        tramite 'save_performance_results', come input, 
        che salverà quindi poi l'immagine nella path corrispondente 

        '''
        
        # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
        gradcam_image = None
        
        #if model_name == "CNN2D":
        
        '''ATTENZIONE MODIFICA QUI'''
        
        #if model_name in ("CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"):
        if model_name == "CNN2D":
            
            #def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, channel_names = None):
            
            '''ATTENZIONE HO AGGIUNTO IL TEST LOADER RAW PER VISUALIZZAZIONE SPETTROGRAMMI GREZZI (test_loader_raw)'''
            gradcam_image = compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, EEG_channels_names)
            if gradcam_image is not None:
                print(f"Creazione di \033[1mGradCAM Image\033[0m per il modello \033[1m{model_name}\033[0m.")
                
        print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        save_performance_results(model_name,
                                 my_train_results,
                                 my_test_results,
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder = save_path_folder,
                                 gradcam_image = gradcam_image)
        
        
        '''
        N.B
        
        gradcam_image = None avverrà solo all'inizio cioè per il primo modello, che verrà testato con una certa combinazione di dati.. 
        ma servirebbe tracciare in qualche modo 

        1) o che la gradcam_image di ogni combinazione venga ri-azzerata alla fine loop
        2) o che venga monitorato che gradcam_image di una combinazione di dati già analizzata venga esclusa poi
        (o messa in un set) in modo che rivenga per errore sovrascritta più volte.. 
        
        Forse la strada più veloce potrebbe essere la soluzione 1)
        
        La soluzione più veloce e semplice è re-impostare la variabile gradcam_image a None alla fine dell'iterazione per ogni combinazione di dati
        (cioè, all'interno del ciclo esterno che itera su key). 
         
        In questo modo, per ogni nuovo dataset la variabile viene "azzera" e viene calcolata l'immagine GradCAM solo per quella combinazione, 
        evitando di sovrascrivere accidentalmente i risultati già calcolati per combinazioni precedenti.
         
        Un'altra possibilità sarebbe tenere traccia delle chiavi (o combinazioni) per cui hai già calcolato la GradCAM,
        ad esempio usando un set, e saltare il calcolo se la combinazione è già presente. 
        
        Tuttavia, se ogni combinazione deve avere la sua immagine, 
        la soluzione più semplice è quella di reimpostare gradcam_image = None alla fine dell'iterazione.
        
        Quindi, per esempio, alla fine del ciclo per ogni dataset (key) potresti fare:
        (VEDI SOTTO)
        
        In questo modo, ti assicuri che per ogni nuova combinazione la variabile sia pulita e pronta per essere ricalcolata, 
        senza rischio di sovrascrivere o confondere i risultati
        '''
        
        # Reimposta gradcam_image a None per la prossima combinazione di dati
        gradcam_image = None

In [ ]:
'''
PER CNN3D PURA o CNN2D CONV SEP
'''

# Imposta il seme per la riproducibilità

#Imposta il seme per i generatori casuali di PyTorch (per operazioni sui tensori e inizializzazione dei pesi dei modelli).
#Importante se vuoi garantire che l'addestramento del modello produca gli stessi risultati in diverse esecuzioni.
torch.manual_seed(32)

#Imposta il seme per NumPy, utile se NumPy viene usato per operazioni casuali (ad es. shuffling dei dati, inizializzazione di matrici, ecc.).
#Importante se usi NumPy per il preprocessing dei dati e vuoi riproducibilità.

np.random.seed(32)

#mposta il seme per il modulo random di Python (utile se si usano funzioni di randomizzazione di Python puro).
#Importante solo se usi random per operazioni come mescolamento di liste.
random.seed(32)

#Imposta il seme per i generatori casuali su GPU, se disponibile.
#Utile se stai eseguendo il codice su una GPU per garantire riproducibilità anche in quel contesto.

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(32)

       
'''

In questo caso, 

il set processed_datasets traccia i dataset già elaborati, 
e il set processed_models tiene traccia delle combinazioni già effettuate (modello + dataset). 

In questo modo, puoi escludere un dataset dal training se è già stato utilizzato in precedenza, 
anche se usato con un modello differente.
'''


# Dizionario per tracciare la standardizzazione usata per ogni combinazione d
# Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)
models_info = {}

EEG_channels = EEG_channels_names 

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Set per tenere traccia delle combinazioni già elaborate
processed_models = set()


# Path delle performance dei modelli ottimizzati con weight and biases
# Path per trovare le best performances di ogni modello per ogni combinazione dei dati
base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies"
                                        #WB_spectrograms_best_results_channels_frequencies_params_hyperparams/

# Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder
#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_post_WB"

#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_hyperparams_post_WB"
save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks"

# --- LOOP PRINCIPALE (con minime modifiche) ---
for key, (X_data, y_data) in data_dict.items():
    
    print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")
    
    if key in processed_datasets:
        print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
        continue
        
    processed_datasets.add(key)
    
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
    print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")
    
    
    '''
    CREO COPIA TEST_LOADER_RAW PER I PLOT DEL POWER RAW PER BANDA E CLASSE
    '''
    # 1) salva una copia RAW dei soli dati di test PRIMA di standardizzare
    X_test_raw = X_test.copy()
    y_test_raw = y_test.copy()
    
    # 2) tensori
    X_raw_tensor = torch.tensor(X_test_raw, dtype=torch.float32)
    y_raw_tensor = torch.tensor(y_test_raw, dtype=torch.long)
    
    
    #for model_name in ["CNN2D", "BiLSTM", "Transformer"]:
    
    '''ATTENZIONE MODIFICA QUI'''
    
    #for model_name in ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]:
    for model_name in ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]:

        model_key = f"{model_name}_{key}"
        if model_key in processed_models:
            print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
            continue
        processed_models.add(model_key)
        
        print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")
        
        # Prova a caricare la configurazione e i pesi ottimali dal file .pkl
        
        '''
        load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
        parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate
        
        exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
        pesi del modello e i suoi iper-parametri
        
        Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
        '''
        
        config, best_weights = load_config_if_available(key, model_name, base_folder)
        
        if config is None:
            raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")
        
        '''
        Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
        MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
        '''
        
        # Parsifica la chiave una volta sola e memorizza i valori
        exp_cond, data_type, category_subject = parse_combination_key(key)
        
        '''
        Dpodiché, 
        
        1) si carica i vari valori degli iper-parametri,
        2) si esegue la standardizzazione se servisse,
        3) prepara il modello per la divisione in train_loader etc.,
        4) si carica la configurazione dei pesi del modello, 
        5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 
        
        6) esegue il training e il test e poi
        
        7) si salva il tutto nella path corrispondente...
        
        '''
        
        '''
        PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
        DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''
        
        
        #"lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        #"weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        #"n_epochs": {"value": 100},
        #"patience": {"value": 12},
        
        #"model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        #"batch_size": {"values": [32, 48, 64, 96]},
        #"standardization": {"values": [True, False]},
        #"beta1": {"values": [0.9, 0.95]},
        #"beta2": {"values": [0.99, 0.995]},
        #"eps": {"values": [1e-8, 1e-7]},
        
        #"use_lstm": {"values": [True, False]},
        #"lstm_hidden": {"values": [32]},
        #"dropout": {"values": [0.5]},
        
        
        model_lr = config["lr"]
        model_weight_decay = config["weight_decay"]
        model_n_epochs = config["n_epochs"]
        model_patience = config["patience"]
        
        
        model_batch_size = config["batch_size"]
        model_standardization = config["standardization"]
    
        
        '''NUOVE MODIFICHE'''
        model_beta1 =  config["beta1"]
        model_beta2 =  config["beta2"]
        model_eps = config["eps"]
        

        '''Per CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC'''
        model_use_lstm      = config["use_lstm"]
        model_lstm_hidden   = config["lstm_hidden"]
        model_dropout       = config["dropout"]
    
            
        #print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, model_beta1= \033[1m{model_beta1}\033[0m,  model_beta2= \033[1m{model_beta2}\033[0m,  model_eps= \033[1m{model_eps}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        
        # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
        models_info[model_key] = {"standardization": model_standardization}
        
        
        
        # 3) dataset & loader per test set (per plots power raw) –‑  IMPORTANTISSIMO: shuffle=False
        raw_dataset = TensorDataset(X_raw_tensor, y_raw_tensor)
        test_loader_raw = DataLoader(raw_dataset,
                             batch_size=model_batch_size,
                             shuffle=False)
        
        '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
        IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
        '''

        if model_standardization:
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
        else:
            print(f"\033[1mNO Standardizzazione Dati!\033[0m")
        
        # Sposta il modello sulla GPU (se disponibile)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
        # Preparazione dei dataloaders
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)
        

        # Inizializzazione del modello
        #if model_name == "CNN2D":
        #    model = CNN2D(input_channels=64, num_classes=2)
        
        #if model_name == "CNN2D_LSTM_FC":
            
            #model = CNN2D_LSTM_FC(n_freq = 45, input_channels=64, num_classes=2, dropout = 0.2)
            #model = CNN2D_LSTM_FC(input_channels = 5, num_classes=2, dropout=0.2)
            
        #elif model_name == "BiLSTM":
        #    model = ReadMEndYou(input_size= 64 * 81, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #elif model_name == "Transformer":
        #    model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=81)
        
        #elif model_name == "TopomapNet":
            #model = TopomapNet(
                #input_channels=5,
                #num_classes=2,
                #base_channels=model_base_channels,
                #use_lstm=model_use_lstm,
                #lstm_hidden=model_lstm_hidden,
                #dropout=model_dropout
            #)
        
        '''OCCHIO QUI CAMBIATO PER GRIGLIA 3D'''
        if model_name == "CNN3D_LSTM_FC":
            
            sweep_config = sweep_config_cnn3d
            
            model = CNN3D_LSTM_FC(
                num_classes=2,
                dropout=model_dropout,
                hidden_size=model_lstm_hidden,
                use_lstm=model_use_lstm)

            print(f"\nInizializzazione Modello \033[1mCNN3D_LSTM_FC\033[0m")
        
        elif model_name == "SeparableCNN2D_LSTM_FC":
            
            sweep_config = sweep_config_cnn_sep
            
            model = SeparableCNN2D_LSTM_FC(
                num_classes=2,
                dropout=model_dropout,
                hidden_size=model_lstm_hidden,
                use_lstm=model_use_lstm
            )
            print(f"\nInizializzazione Modello \033[1mSeparableCNN2D_LSTM_FC\033[0m")
            
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
        
        
        # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
        if best_weights is not None:
            try:
                model.load_state_dict(best_weights)
                print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
            except Exception as e:
                print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                continue
        
        '''OLD VERSION'''
        # Definizione del criterio di perdita
        #criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
        
        '''NUOVE MODIFICHE'''
        criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
        
        '''OLD VERSION'''
        # Definizione dell'ottimizzatore con i parametri aggiornati
        #optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)
        
        '''NUOVE MODIFICHE'''
        
         # 10) ottimizzatore + scheduler + early stopping
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr    = model_lr,
            betas = (model_beta1, model_beta2),
            eps   = model_eps, 
            weight_decay = model_weight_decay
        )
            
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode     = "min",   # monitoriamo val_loss
            factor   = 0.1,
            patience = 8,
            verbose  = True
        )
        
        
        #early_stopping = EarlyStopping(patience=model_patience, mode="min")
        
        '''OLD VERSION'''
        #print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)
        
        '''NEW VERSION'''
        # --- dopo model.load_state_dict(best_weights) e criterion = nn.CrossEntropyLoss() ---

        # 1) prepara i data_loaders per train/val
        data_loaders = {
            "train": train_loader,
            "val":   val_loader
        }
        
        print(f"🏋️‍♂️Salvo le metriche del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m a seguito della ottimizzazione su W&B ...")
        # 2) richiama la funzione che pesca da W&B la best‐run e corregge la train AUC
        
        #ATTENZIONE al potenziale problema di stringa, non di API: 

        #i due esempi che hai postato in realtà usano diversi caratteri “‑” (uno è il classico ASCII U+002D, l’altro è un non‑breaking hyphen U+2011 o simili), quindi quando chiami
        
        #entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
        #stai passando un nome che W&B non riconosce (e quindi api.projects(entity=…) torna vuoto), mentre con

        #entity = "stefano-bargione-universit-di-roma-tor-vergata"
        #funziona perché lì usi i semplici - ASCII.

        my_train_results = load_best_run_results(
            key=key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity = "stefano-bargione-universit-di-roma-tor-vergata"
        )
        
        
        
        '''
        L’entity che passi a Api().runs(f"{entity}/{project}") è semplicemente il tuo account (o l’organizzazione) su W&B,
        cioè la parte che compare subito prima del nome del progetto nell’URL.

        Per esempio, se quando apri il tuo progetto su W&B vedi un indirizzo del tipo
        
        -> https://wandb.ai/steclab/some_project_name, allora entity = "steclab".
        
        Se invece lavori sotto un’organizzazione 
        
        -> “cool‑team”, e l’URL è https://wandb.ai/cool-team/some_project_name, allora userai entity = "cool-team".

        Puoi verificarlo:

        Accedi a wandb.ai e vai sul progetto.
        Leggi la prima parte dell’URL (tra wandb.ai/ e il /project_name).
        Copiala esattamente come stringa in entity.

        Così il tuo Api().runs(f"{entity}/{project}") andrà a pescare proprio le run che hai lanciato tu.

        my_train_results = load_best_run_results(
            key= key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity= "mio-entity"
        )
        
        '''
        
        print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        # 3) usa il best_model caricato dentro `train_results` e chiama il testing
        my_test_results = testing(my_train_results, test_loader, criterion)
        
        '''
        GRADCAM COMPUTATION PER IL MODELLO CNN3D e ConvSep
        
        La funzione compute_gradcam_figure estrae i campioni (per ogni classe) e crea una figura con le due righe richieste.
        
        Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
        'save_performance_results', in modo da essere salvato nella path corretta. 
        
        La funzione 'save_performance_results' è stata modificata 
        per gestire ANCHE questo nuovo input dell'immagine 
        
        (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
        seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:
        
        - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
        - tipologia di dato EEG prelevato (i.e., spectrograms) 
        - provenienza del dato stesso (i.e., familiar_th)
        )
        
        Spiegazione:
        
        La funzione compute_gradcam_figure eseguire il calcolo di GradCAM (vedi dettagli nella sua funzione)
        e alla fine ritornerà in output una variabile 
        
        'fig_image' che sarà poi assegnata alla variabile 'gradcam_image',
        che è un oggetto buffer, che contiene i dati binari dell'immagine in formato PNG
        (poiché abbiamo usato plt.savefig con format='png'). 
        
        Quindi, quando passi gradcam_image (cioè fig_image) alla funzione 'save_performance_results',
        viene scritto direttamente su disco come file PNG.
        
        Non c'è bisogno di ri-aprire o convertire ulteriormente, a meno che tu non voglia manipolare l'immagine in seguito.
        Quindi, la soluzione è corretta così com'è:
        il buffer viene salvato come file PNG nella directory specificata, 
        e successivamente potrai aprirlo con una libreria come cv2 o PIL se necessario.        
        
        Quindi, gradcam_image (i.e., fig_image) viene quindi passato correttamente dentro al loop di training e test, 
        tramite 'save_performance_results', come input, 
        che salverà quindi poi l'immagine nella path corrispondente 

        '''
        
        # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
        gradcam_image = None
        
        #if model_name == "CNN2D":
        
        '''ATTENZIONE MODIFICA QUI'''
        
        #if model_name == "CNN2D_LSTM_FC":
        
        if model_name in ("CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"):
            
            gradcam_image = compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, EEG_channels_names, debug = False)
            if gradcam_image is not None:
                print(f"Creazione di \033[1mGradCAM Image\033[0m per il modello \033[1m{model_name}\033[0m.")
                
        print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        save_performance_results(model_name,
                                 my_train_results,
                                 my_test_results,
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder = save_path_folder,
                                 gradcam_image = gradcam_image)
        
        
        '''
        N.B
        
        gradcam_image = None avverrà solo all'inizio cioè per il primo modello, che verrà testato con una certa combinazione di dati.. 
        ma servirebbe tracciare in qualche modo 

        1) o che la gradcam_image di ogni combinazione venga ri-azzerata alla fine loop
        2) o che venga monitorato che gradcam_image di una combinazione di dati già analizzata venga esclusa poi
        (o messa in un set) in modo che rivenga per errore sovrascritta più volte.. 
        
        Forse la strada più veloce potrebbe essere la soluzione 1)
        
        La soluzione più veloce e semplice è re-impostare la variabile gradcam_image a None alla fine dell'iterazione per ogni combinazione di dati
        (cioè, all'interno del ciclo esterno che itera su key). 
         
        In questo modo, per ogni nuovo dataset la variabile viene "azzera" e viene calcolata l'immagine GradCAM solo per quella combinazione, 
        evitando di sovrascrivere accidentalmente i risultati già calcolati per combinazioni precedenti.
         
        Un'altra possibilità sarebbe tenere traccia delle chiavi (o combinazioni) per cui hai già calcolato la GradCAM,
        ad esempio usando un set, e saltare il calcolo se la combinazione è già presente. 
        
        Tuttavia, se ogni combinazione deve avere la sua immagine, 
        la soluzione più semplice è quella di reimpostare gradcam_image = None alla fine dell'iterazione.
        
        Quindi, per esempio, alla fine del ciclo per ogni dataset (key) potresti fare:
        (VEDI SOTTO)
        
        In questo modo, ti assicuri che per ogni nuova combinazione la variabile sia pulita e pronta per essere ricalcolata, 
        senza rischio di sovrascrivere o confondere i risultati
        '''
        
        # Reimposta gradcam_image a None per la prossima combinazione di dati
        gradcam_image = None
    



Estrazione Dati per il dataset: th_resp_vs_pt_resp_spectrograms_familiar_th, 	Shape X: (1586, 9, 9, 5), Shape y: (1586,)
Dataset Splitting: Train: (1014, 9, 9, 5), Val: (254, 9, 9, 5), Test: (318, 9, 9, 5)

Preparazione dati per il dataset th_resp_vs_pt_resp_spectrograms_familiar_th e il modello CNN3D_LSTM_FC...
🕵️‍♂️🔍Caricamento file .pkl: /home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies/th_resp_vs_pt_resp/spectrograms/familiar_th/CNN3D_LSTM_FC_th_resp_vs_pt_resp_spectrograms_familiar_th.pkl
✅ File .pkl trovato per CNN3D_LSTM_FC su th_resp_vs_pt_resp_spectrograms_familiar_th
Parametri per CNN3D_LSTM_FC: batch_size= 64, n_epochs= 100, patience= 12, lr= 0.005, model_beta1= 0.9,  model_beta2= 0.995,  model_eps= 1e-07, standardization= True
SÌ Standardizzazione Dati!

Inizializzazione Modello CNN3D_LSTM_FC
📊 Modello CNN3D_LSTM_FC inizializzato con  pesi ottimizzati tramite hyper-parameter tuning su Weight & Biases
🏋️‍♂️Salvo le metriche del training per CNN3D_

/home/stefano/anaconda3/envs/new_pd_env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✓ Trovate 203 runs

✓ Progetto `th_resp_vs_pt_resp_spectrograms_channels_freqs_familiar_th`

✓ Modello `CNN3D_LSTM_FC`

✓ Sweep `<Sweep stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp_spectrograms_channels_freqs_familiar_th/fzz3drlj (RUNNING)>`


Avvio del testing per CNN3D_LSTM_FC sul dataset th_resp_vs_pt_resp_spectrograms_familiar_th...


Loss: 0.8696: 100%|██████████████████████████████| 5/5 [00:00<00:00, 265.77it/s]


Test Accuracy: 0.5377

Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.74      0.63       168
           1       0.52      0.31      0.38       150

    accuracy                           0.54       318
   macro avg       0.53      0.53      0.51       318
weighted avg       0.53      0.54      0.51       318



...

✓ Trovate 202 runs

✓ Progetto `pt_resp_vs_shared_resp_spectrograms_channels_freqs_unfamiliar_pt`

✓ Modello `SeparableCNN2D_LSTM_FC`

✓ Sweep `<Sweep stefano-bargione-universit-di-roma-tor-vergata/pt_resp_vs_shared_resp_spectrograms_channels_freqs_unfamiliar_pt/25e9pu8h (RUNNING)>`


Avvio del testing per SeparableCNN2D_LSTM_FC sul dataset pt_resp_vs_shared_resp_spectrograms_unfamiliar_pt...


Loss: 0.7283: 100%|██████████████████████████████| 7/7 [00:00<00:00, 316.53it/s]


Test Accuracy: 0.4251

Classification Report:
               precision    recall  f1-score   support

           0       0.46      0.34      0.39       181
           1       0.40      0.53      0.46       153

    accuracy                           0.43       334
   macro avg       0.43      0.43      0.42       334
weighted avg       0.43      0.43      0.42       334




/home/stefano/anaconda3/envs/new_pd_env/lib/python3.9/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


>>> layout


In [ ]:
import os
from ipykernel.connect import get_connection_file

print("PID:", os.getpid())
print("Conn file:", get_connection_file())

In [ ]:
print("finito")

In [ ]:
print(models_info.keys())

In [ ]:
#with open('/home/stefano/Interrogait/models_info_spectrograms_EEG_GradCAM_Checks.pkl', 'wb') as f:
#    pickle.dump(models_info, f)

with open('/home/stefano/Interrogait/spectrograms_EEG_channels_freqs_params_GradCAM_Checks.pkl', 'wb') as f:
    pickle.dump(models_info, f)

##### **CREAZIONE DELLE TABLES CON INTEGRAZIONE DELLE PERFORMANCE TRAINING & TEST DEI MODELLI DENTRO DATAFRAME**

#### Integrazioni Performance Training e Test del Modello dentro DataFrame - OLD APPROACH

##### **OLD BEST APPROACH**

In [ ]:
import os
import pickle

# Definiamo le path
paths = {
    "TH_FAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/TH_FAM_UNSCALED/",
    "PT_FAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/PT_FAM_UNSCALED/",
    "TH_UNFAM": "/home/stefano/Interrogait/PRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/TH_UNFAM_UNSCALED/",
    "PT_UNFAM": "/home/stefano/Interrogait/PPRE_WB_OPTIMIZATION_MODELS_RESULTS _TIME_DOMAIN/Model_Results/PT_UNFAM_UNSCALED/"
}


# Identificatori delle triplette
identifiers = ["1_20", "1_45", "wavelet_delta"]

# Dizionario per salvare i risultati
all_models_dict = {}

# Iteriamo su ogni path
for condition, path in paths.items():
    models_dict = {identifier: {} for identifier in identifiers}  # Dizionario per i modelli della path corrente
    
    # Controlliamo che la directory esista
    if not os.path.exists(path):
        print(f"Directory non trovata: {path}")
        continue
    
    # Otteniamo la lista di file nella directory
    files = os.listdir(path)
    
    # Filtriamo e carichiamo i file per ciascun identificatore
    for identifier in identifiers:
        for file in files:
            if file.endswith(f"{identifier}.pkl"):  # Controlliamo se il file termina con l'identificatore
                file_path = os.path.join(path, file)
                try:
                    with open(file_path, "rb") as f:
                        models_dict[identifier][file] = pickle.load(f)
                except Exception as e:
                    print(f"Errore nel caricamento di {file}: {e}")
    
    # Salviamo il dizionario della path corrente nel dizionario principale
    all_models_dict[condition] = models_dict


In [ ]:
# Ora all_models_dict contiene i dati strutturati per ogni path e identificatore
# Stampa i tipi di ogni sotto-dizionario
for path_key, identifier_dict in all_models_dict.items():
    print(f"Path: {path_key} - Tipo: {type(identifier_dict)}")
    for identifier, model_dict in identifier_dict.items():
        print(f"  Identifier: {identifier} - Tipo: {type(model_dict)}")
        for model, data in model_dict.items():
            print(f"    Model: {model} - Tipo: {type(data)}")

In [ ]:
all_models_dict.keys()

In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Definiamo le path
paths = {
    "TH_FAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/TH_FAM_UNSCALED/",
    "PT_FAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/PT_FAM_UNSCALED/",
    "TH_UNFAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/TH_UNFAM_UNSCALED/",
    "PT_UNFAM_UNSCALED": "/home/stefano/Interrogait/Model_Results/PT_UNFAM_UNSCALED/"
}

# Identificatori delle triplette
identifiers = ["1_20", "1_45", "wavelet_delta"]

# Iteriamo su ogni path
for condition, path in paths.items():
    
    # Dizionario per i modelli della path corrente
    models_dict = {identifier: {} for identifier in identifiers}
    
    # Controlliamo che la directory esista
    if not os.path.exists(path):
        print(f"Directory non trovata: {path}")
        continue
    
    # Otteniamo la lista di file nella directory
    files = os.listdir(path)
    
    # Filtriamo e carichiamo i file per ciascun identificatore
    for identifier in identifiers:
        for file in files:
            if file.endswith(f"{identifier}.pkl"):  # Controlliamo se il file termina con l'identificatore
                file_path = os.path.join(path, file)
                try:
                    with open(file_path, "rb") as f:
                        models_dict[identifier][file] = pickle.load(f)
                except Exception as e:
                    print(f"Errore nel caricamento di {file}: {e}")

    # Ora creiamo un file separato per ogni identificatore
    for identifier in identifiers:
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}
        
        print(f"\nProcessing condition: {condition}, identifier: {identifier}\n")

        # Iteriamo sui modelli relativi a questo identificatore
        for model_name, model_data in models_dict[identifier].items():
            name_model = model_name.split("_")[0]  # Prende solo la parte prima del primo '_'
            print(f"    Processing model: {name_model}")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}

                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]

            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame per l'identificatore specifico
        df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis('off')

        # Usa pandas per creare una tabella nel grafico
        tabla = table(ax, df_performances, loc='center', colWidths=[0.2]*len(df_performances.columns))

        # Personalizza la tabella
        tabla.auto_set_font_size(True)
        tabla.set_fontsize(10)
        tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        for key, cell in tabla.get_celld().items():
            if key[0] == 0:  # Se la riga è la prima (intestazioni delle colonne)
                cell.set_text_props(weight='bold')  # Grassetto

        # Creazione della directory se non esiste
        output_dir = paths[condition]
        file_name = f"{condition}_{identifier}_models.png"
        img_file_path = os.path.join(output_dir, file_name)

        # Salva l'immagine della tabella
        fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        plt.close(fig)  # Chiudi la figura per liberare memoria

        print(f"Tabella salvata in: {img_file_path}")


#### Integrazioni Performance Training e Test del Modello dentro DataFrame - NEW APPROACH

#### Spiegazione

Ok in questo modo, model_standardization_dict dovrebbe andare a salvarsi se, i dati per quella combinazione di fattori, rispetto ad uno specifico modello, siano stati standardizzati o meno.

Di conseguenza, dentro questo loop

    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    from pandas.plotting import table

    # Base folder
    base_folder = "/home/stefano/Interrogait/time_domain_best_models_post_WB"

    # Condizioni sperimentali
    experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

    # Tipologie di dati
    data_types = ["1_20", "1_45", "wavelet_delta"]

    # Subfolders per tipologia di soggetto
    subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

    # Dizionario per salvare tutti i modelli
    all_models = {}

    # Caricamento dei modelli
    for condition in experimental_conditions:
        for data_type in data_types:
            for subfolder in subfolders:

                path = os.path.join(base_folder, condition, data_type, subfolder)

                if not os.path.exists(path):
                    print(f"Directory non trovata: {path}")
                    continue

                # Creiamo la chiave per questa combinazione
                key = f"{condition}_{data_type}_{subfolder}"
                all_models[key] = {}

                # Otteniamo la lista di file nella directory
                files = os.listdir(path)

                # Filtriamo e carichiamo i file .pkl
                for file in files:
                    if file.endswith(".pkl"):  # Controlliamo se è un file modello
                        file_path = os.path.join(path, file)
                        try:
                            with open(file_path, "rb") as f:
                                all_models[key][file] = pickle.load(f)
                        except Exception as e:
                            print(f"Errore nel caricamento di {file}: {e}")

    # Creazione delle tabelle di performance
    for key, models_dict in all_models.items():

        # Otteniamo le informazioni dalla chiave
        #condition, data_type, subfolder = key.split("_", 2)
        condition, data_type, subfolder = parse_combination_models_keys(key)

        print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")

        # Creazione della tabella
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

        # Iteriamo sui modelli caricati
        for model_name, model_data in models_dict.items():
            name_model = model_name.split("_")[0]  # Nome modello
            print(f"    Processing model: \033[1m{name_model}\033[0m")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}


                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]


            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame
        #df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        #fig, ax = plt.subplots(figsize=(10, 6))
        #ax.axis('off')
        #tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
        #tabla.auto_set_font_size(True)
        #tabla.set_fontsize(10)
        #tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        #for key, cell in tabla.get_celld().items():
        #    if key[0] == 0:
        #        cell.set_text_props(weight='bold')

        # Salva l'immagine della tabella
        path = os.path.join(base_folder, condition, data_type, subfolder)
        file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
        img_file_path = os.path.join(path, file_name)
        #fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        #plt.close(fig)

        print(f"\nTabella dei dati di \033[1m{key}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")


vorrei provare ad iterare con "zip", sia all_models che su model_standardization_dict ...? (che forse dovrebbero avere la stessa struttura, che renderebbe possibile questa cosa...?)

E, nel momento in cui si aggiungono le metriche del training e test del relativo modello, controllare rispetto a model_standardization_dict (di cui si ha la chiave per accedere all' informazione su se quel modello, per quella combinazioni di fattori che compongono quel dato) se il dato sia stato standardizzato... 

Se questo è VERO, allora nella colonna del dataframe che si riferisce al modello... vorrei che ci mettessi accanto, alla stringa che si riferisce al nome del modello (name_model) un asterisco, SOLO SE, per quel modello, allenato con quella combinazioni di fattori che compongono quel dato, i dati siano stati standardizzati...

chiaro?

#### Implementazione 

In [ ]:
import pickle 
path = '/home/stefano/Interrogait/'

with open(f"{path}spectrograms_EEG_channels_freqs_params_GradCAM_Checks.pkl", "rb") as f:
    models_info = pickle.load(f)

In [ ]:
'''
In questo codice:

model_info.get('standardization', False) cerca la chiave 'standardization' all'interno di ogni sottodizionario. 
Se non esiste, restituirà False come valore di default.
Se standardization è True, stampa la chiave associata.
'''

# Ciclo attraverso le chiavi di 'models_info'
for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiave



In [ ]:
models_info

In [ ]:
#for key, model_info in all_models.items():
#    print(key)

In [ ]:
'''
Siccome la stringa associata alla category subject è diversa tra i due.. 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt  da un lato (models_info)
th_fam, pt_fam, th_unfam, pt_unfam  dall'altro (all_models)

la corrispondenza non avverrà mai... per cui, si deve fare il mapping corrispondente tra 
le stringhe di uno e dell'altro, in modo che models_info cambi come parte della stringa della sua chiave da queste 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt
a queste
th_fam, pt_fam, th_unfam, pt_unfam 

'''

mapping_subject = {
    "familiar_th": "th_fam",
    "familiar_pt": "pt_fam",
    "unfamiliar_th": "th_unfam",
    "unfamiliar_pt": "pt_unfam"
}

# Creiamo un nuovo dizionario con le chiavi corrette
updated_models_info = {}

for key, value in models_info.items():
    for old_suffix, new_suffix in mapping_subject.items():
        if key.endswith(old_suffix):
            new_key = key.replace(old_suffix, new_suffix)
            updated_models_info[new_key] = value
            break  # Evita sostituzioni multiple se una è già stata fatta
    else:
        # Se nessuna sostituzione è stata fatta, mantieni la chiave originale
        updated_models_info[key] = value

# Sostituisci il vecchio dizionario con quello aggiornato
models_info = updated_models_info


In [ ]:
models_info.keys()

In [ ]:
''' Ciclo attraverso le chiavi di 'models_info' AGGIORNATO!'''

for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiavi

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''

# Funzione per parsare la chiave
def parse_combination_models_keys(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    
    Il formato atteso PRIMA è:
    
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    
    Il formato atteso ORA è:
    
     "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "th_fam|th_unfam|pt_fam|pt_unfam"
     
    """
    
    match = re.match(
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$", 
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
    return exp_cond, data_type, category_subject

In [ ]:
'''
NEW APPROACH 

Adesso replichiamo l'approccio usato prima, ma stavolta integrado tutte le combinazioni di dati. 
Andiamo a

1) iterare sulla struttura delle directory a partire da base_folder, 
2) caricare i modelli .pkl per ogni combinazione di fattori che compongono i dati
3) creare un DataFrame che raccolga le metriche di tutti i modelli relativi alla stessa combinazione di dati. 

Infine, salviamo questa tabella come immagine all'interno della cartella corrispondente
'''


import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table


# Base folder
base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks"


# Condizioni sperimentali
experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

# Dizionario per salvare tutti i modelli
all_models = {}

# Caricamento dei modelli
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue
            
            # Creiamo la chiave per questa combinazione
            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            # Otteniamo la lista di file nella directory
            files = os.listdir(path)
            
            # Filtriamo e carichiamo i file .pkl
            for file in files:
                if file.endswith(".pkl"):  # Controlliamo se è un file modello
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")

# Creazione delle tabelle di performance
for key, models_dict in all_models.items():
    
    # Otteniamo le informazioni dalla chiave
    condition, data_type, subfolder = parse_combination_models_keys(key)
    
    print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")
    
    # Creazione della tabella
    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    # Iteriamo sui modelli caricati
    for model_name, model_data in models_dict.items():
        
        # Estrai il nome del modello dal file (ad esempio, "CNN1D" da "CNN1D_performances_...pkl")
        name_model = model_name.split("_")[0]
        
        print(f"    Processing model: \033[1m{name_model}\033[0m")
        
        # Costruisci la chiave utilizzata nel dizionario models_info
        
        '''
        Nota: occorrerà che il formato della chiave sia consistente tra i due loop.
        
        Ad esempio, se nel primo loop era f"{key}_{model_name}", qui potresti dover fare:
        model_key = f"{key}_{name_model}"
        
        Oppure, se nel primo loop era f"{model_name}_{key}", qui potresti dover fare:
        model_key = f"{name_model}_{key}"
        
        '''
        model_key = f"{name_model}_{key}"
        
        # Controlla se i dati sono stati standardizzati per questo modello
        standardization_flag = models_info.get(model_key, {}).get("standardization", False)
        
        if standardization_flag:
            suffix = "*" 
        else:
            suffix = "" 
        
        try:
            # Recupera i risultati di training e testing
            train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
            test_scores = model_data.get('my_test_results', {}).get('test_performances', {})
            
            # Converti i valori in float
            train_scores = {key: float(value[0]) for key, value in train_scores.items()}
            test_scores = {key: float(value[0]) for key, value in test_scores.items()}
            
            
            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_train = f"{name_model} (Training){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_train}"] = [
                train_scores["train_accuracy"],
                train_scores["train_loss"],
                train_scores["train_precision"],
                train_scores["train_recall"],
                train_scores["train_f1_score"],
                train_scores["train_auc"],
            ]

            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_test = f"{name_model} (Test){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_test}"] = [
                test_scores["test_accuracy"],
                test_scores["test_loss"],
                test_scores["test_precision"],
                test_scores["test_recall"],
                test_scores["test_f1_score"],
                test_scores["test_auc"],
            ]
        
        except Exception as e:
            print(f"    Errore nell'elaborazione di {model_name}: {e}")

    # Creazione del DataFrame
    df_performances = pd.DataFrame(df_data)

    # Crea un'immagine della tabella
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    
    # Aggiunta del titolo
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}, Subject: {subfolder}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    # Evidenzia i nomi delle colonne
    for key, cell in tabla.get_celld().items():
        if key[0] == 0:
            cell.set_text_props(weight='bold')

    # Salva l'immagine della tabella
    path = os.path.join(base_folder, condition, data_type, subfolder)
    file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    img_file_path = os.path.join(path, file_name)
    fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
    plt.close(fig)

    print(f"\nTabella dei dati di \033[1m{condition}_{data_type}_{subfolder}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")

#### **Integrazioni in Tabelle AGGREGATE delle Performance Training e Test del Modello dentro DataFrame - NEW APPROACH**

In [ ]:
'''
perfetto ora va. ma io vorrei anche rendere le tabelle ancora più informative.. ossia

vorrei ricreare lo stesso codice ma questa volta anziché avere una tabella specifica SOLO
per un certo tipo di condizione sperimentale, tipo di dato e soggetto...

io vorrei provare quanto meno ad 'allargare' le tabelle, nel senso di mettere nella stessa tabella
la stessa condizione sperimentale e tipo di dato, per tutti e 3 i modelli, 

ma confrontando però la performance dello STESSO MODELLO per gli STESSI TIPI DI CONDIZIONE SPERIMENTALE, TIPO DI DATO e TIPI DI SOGGETTI (ossia RUOLO nel task)
... ossia ad esempio


A) Ossia.. quindi, farei prima i RUOLI di th_fam e th_unfam ...ossia

per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_45

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 




poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_20

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 




poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati delta_wavelet

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per th_fam 
2) sia per th_unfam.. 

in modo da avere un confronto diretto visivo per la stessa condizione sperimentale, stesso tipo di feature dei dati EEG usata, 
rispetto allo stesso modello, ma confrontando però la performance tra i due soggetti che hanno fatto lo STESSO RUOLO nei 2 gruppi (controllo e sperimentale).




B) Allo stesso modo.. quindi, farei la STESSA COSA anche per i RUOLI di pt_fam e pt_unfam...ossia

 
per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_45

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 



poi per 'th_resp_vs_pt_resp', (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!), per i dati 1_20

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test'
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 



poi per 'th_resp_vs_pt_resp' (e così come poi per 'th_resp_vs_shared_resp' e 'pt_resp_vs_shared_resp'!!!!),, per i dati delta_wavelet

io vorrei che nella stessa tabella, ci fossero le performance di tutti e 3 i modelli sia in fase di training che di test' 
(CNN1D, poi BiLSTM ed infine per Transformer...) ma

1) sia per pt_fam 
2) sia per pt_unfam.. 
  
magari, nella prima riga metto le performance di training e test dei modelli che son con "_fam" 
e invece sotto le stesse performance dello stesso modello, condizione e tipo di dato, per chi è "_unfam", 


in modo da distinguire in base alla riga quali sono le performance di uno rispetto a quelle dell'altro soggetto, 
che avrà svolto lo stesso ruolo ma nel gruppo o di controllo o sperimentale...



In [ ]:
'''
Yes! idea chiarissima. Senza stravolgere il tuo codice, 
aggiungi un secondo pass che costruisce (e salva) le tabelle aggregate per ruolo per ogni (condizione, data_type). 

Le colonne restano i 3 modelli × (Training/Test), le righe diventano le metriche replicate per i due soggetti del ruolo (fam / unfam). 

Il simbolo * per la standardizzazione lo mettiamo dentro la cella (così può cambiare tra fam e unfam).

Incolla questo blocco dopo aver popolato all_models (puoi tenere anche le tabelle “singole” che già fai):



# ===== TABELLE AGGREGATE PER RUOLO (th_fam vs th_unfam e pt_fam vs pt_unfam) =====

MODEL_ORDER = ["CNN1D", "BiLSTM", "Transformer"]
METRICS = [
    ("Accuracy",  "train_accuracy", "test_accuracy"),
    ("Loss",      "train_loss",     "test_loss"),
    ("Precision", "train_precision","test_precision"),
    ("Recall",    "train_recall",   "test_recall"),
    ("F1-Score",  "train_f1_score", "test_f1_score"),
    ("AUC-ROC",   "train_auc",      "test_auc"),
]

def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None)."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        # match robusto: inizia con "<MODEL>_"
        if fname.startswith(model_prefix + "_"):
            return blob
    return None

def fmt(v, star=False):
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("*" if star else "")

# Gruppi di ruolo
ROLE_GROUPS = {
    "THroles": ["th_fam", "th_unfam"],
    "PTroles": ["pt_fam", "pt_unfam"],
}

for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            # Costruisci righe: una sezione per sub='..._fam' e una per '..._unfam'
            rows = []
            # Colonne: 3 modelli × (Training/Test)
            columns = ["Metriche"]
            for m in MODEL_ORDER:
                columns.append(f"{m} (Training)")
                columns.append(f"{m} (Test)")

            df_data = {c: [] for c in columns}

            for subfolder in subs:
                # intestazione “visiva” delle righe: preferisci th_fam/th_unfam ecc.
                for label, tr_key, te_key in METRICS:
                    df_data["Metriche"].append(f"{subfolder} — {label}")

                    for m in MODEL_ORDER:
                        # recupero blob salvato per quel subfolder/modello
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        # standardization flag (per cella)
                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            # niente file -> celle vuote
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            # training_performances/test_performances hanno valori come liste [val]
                            tr_val = tr.get(tr_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))
                        except Exception:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")

            # DataFrame e salvataggio
            df_performances = pd.DataFrame(df_data)

            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis('off')

            title = f"DL Models performances — {condition} — EEG: {data_type} — {role_label}"
            ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

            tabla = table(ax, df_performances, loc='center',
                          colWidths=[0.25] + [0.12]*(len(df_performances.columns)-1))
            tabla.auto_set_font_size(True)
            tabla.set_fontsize(9)
            tabla.scale(1.2, 1.2)

            for k, cell in tabla.get_celld().items():
                if k[0] == 0:
                    cell.set_text_props(weight='bold')

            out_dir = os.path.join(base_folder, condition, data_type)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
            out_path = os.path.join(out_dir, out_name)
            fig.savefig(out_path, bbox_inches='tight', dpi=300)
            plt.close(fig)

            print(f"Tabella aggregata salvata: {out_path}")

In [ ]:
'''
perfetto — qui sotto trovi il tuo script “chiavi in mano” con il secondo pass che genera anche le tabelle aggregate per ruolo 

(THroles = th_fam/th_unfam, PTroles = pt_fam/pt_unfam) per ogni coppia (condizione, data_type).

Ho aggiunto:

parse_combination_models_keys() con wavelet_delta nel regex.

Caricamento “robusto” di models_info (se non esiste, procede senza *).

Funzioni di supporto find_model_blob() e fmt().

Secondo pass che salva i PNG ..._{condition}_{data_type}_{THroles|PTroles}.png nella cartella di quella coppia.


Se vuoi nascondere completamente il vecchio primo pass e tenere solo i comparativi per ruolo, basta commentare/bloccare la sezione “Pass 1”.




Puoi commentare tutto il “Pass 1” senza problemi: il “Pass 2” non dipende da quello.
Il “Pass 2” usa solo:

all_models (riempito nel blocco di caricamento iniziale, prima del Pass 1),

models_info (per mettere l’asterisco * se standardizzato),

le funzioni helper (find_model_blob, fmt) e le costanti (MODEL_ORDER, ROLE_GROUPS, ecc.).
Quindi, finché lasci il caricamento di all_models e gli helper, funziona da solo.

Sì, le tabelle aggregate del Pass 2 vengono salvate esattamente nel percorso costruito da queste righe:

out_dir = os.path.join(base_folder, condition, data_type)
os.makedirs(out_dir, exist_ok=True)
out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"


quindi avrai file tipo:

/home/stefano/Interrogait/time_domain_1D_best_models_post_WB/th_resp_vs_pt_resp/1_45/models_performances_th_resp_vs_pt_resp_1_45_THroles.png

/home/stefano/Interrogait/time_domain_1D_best_models_post_WB/pt_resp_vs_shared_resp/wavelet_delta/models_performances_pt_resp_vs_shared_resp_wavelet_delta_PTroles.png

Se preferisci tenerle in una sottocartella tipo aggregated/, cambia così:

out_dir = os.path.join(base_folder, condition, data_type, "aggregated")


'''


import os
import re
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# ---------- Helpers ----------
def parse_combination_models_keys(combination_key: str):
    """
    Ritorna (exp_cond, data_type, category_subject) da chiavi tipo:
    th_resp_vs_pt_resp_1_45_th_fam
    """
    match = re.match(
        #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(1_20|1_45|wavelet_delta)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        combination_key
    )
    if match:
        return match.groups()
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

# Carica models_info (flag standardization per cella). Se non c'è, prosegue senza '*'
try:
    with open("/home/stefano/Interrogait/spectrograms_EEG_channels_freqs_params_GradCAM_Checks.pkl", "rb") as f:
        models_info = pickle.load(f)
except Exception:
    print("⚠️  models_info non trovato/caricabile: le tabelle verranno create senza indicatore * di standardizzazione.")
    models_info = {}

def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None) cercando per filename prefix."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        if fname.startswith(model_prefix + "_"):
            return blob
    return None

def fmt(v, star=False):
    """Formatta un valore numerico a 3 decimali e aggiunge '*' se standardizzato."""
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("*" if star else "")

# ---------- Config ----------
base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks" 

experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]
#data_types = ["1_20", "1_45", "wavelet_delta"]
data_types = ["spectrograms"]
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

#MODEL_ORDER = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]

MODEL_ORDER = ["CNN2D"]

METRICS = [
    ("Accuracy",  "train_accuracy", "test_accuracy"),
    ("Loss",      "train_loss",     "test_loss"),
    ("Precision", "train_precision","test_precision"),
    ("Recall",    "train_recall",   "test_recall"),
    ("F1-Score",  "train_f1_score", "test_f1_score"),
    ("AUC-ROC",   "train_auc",      "test_auc"),
]

ROLE_GROUPS = {
    "Observer_Role": ["th_fam", "th_unfam"],
    "Receiver_Role": ["pt_fam", "pt_unfam"],
}


# --- aggiungi in alto (vicino a ROLE_GROUPS / MODEL_ORDER) ---
DISPLAY_LABELS = {
    "th_fam":   "observer_fam",
    "th_unfam": "observer_unfam",
    "pt_fam":   "receiver_fam",
    "pt_unfam": "receiver_unfam",
}



# ---------- Caricamento modelli ----------
all_models = {}

for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            path = os.path.join(base_folder, condition, data_type, subfolder)
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue

            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            for file in os.listdir(path):
                if file.endswith(".pkl"):
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")

# ---------- Pass 1: tabella per singola combinazione (come avevi) ----------

'''
for key, models_dict in all_models.items():
    condition, data_type, subfolder = parse_combination_models_keys(key)
    print(f"\nProcessing: {condition} - {data_type} - {subfolder}\n")

    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    for filename, model_data in models_dict.items():
        name_model = filename.split("_")[0]  # CNN1D / BiLSTM / Transformer

        # Standardization: usa models_info[colonna] a livello di modello-subfolder
        model_key = f"{name_model}_{key}"
        standardization_flag = bool(models_info.get(model_key, {}).get("standardization", False))
        suffix = "*" if standardization_flag else ""

        try:
            train_scores = model_data.get("my_train_results", {}).get("training_performances", {})
            test_scores  = model_data.get("my_test_results", {}).get("test_performances", {})

            # convert list -> float
            train_scores = {k: float(v[0]) for k, v in train_scores.items()}
            test_scores  = {k: float(v[0]) for k, v in test_scores.items()}

            col_train = f"{name_model} (Training){suffix}"
            col_test  = f"{name_model} (Test){suffix}"

            df_data[col_train] = [
                train_scores.get("train_accuracy", float("nan")),
                train_scores.get("train_loss", float("nan")),
                train_scores.get("train_precision", float("nan")),
                train_scores.get("train_recall", float("nan")),
                train_scores.get("train_f1_score", float("nan")),
                train_scores.get("train_auc", float("nan")),
            ]
            df_data[col_test] = [
                test_scores.get("test_accuracy", float("nan")),
                test_scores.get("test_loss", float("nan")),
                test_scores.get("test_precision", float("nan")),
                test_scores.get("test_recall", float("nan")),
                test_scores.get("test_f1_score", float("nan")),
                test_scores.get("test_auc", float("nan")),
            ]
        except Exception as e:
            print(f"    Errore nell'elaborazione di {filename}: {e}")

    df_performances = pd.DataFrame(df_data)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis("off")
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc="center", colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    for kcell, cell in tabla.get_celld().items():
        if kcell[0] == 0:
            cell.set_text_props(weight="bold")

    out_dir = os.path.join(base_folder, condition, data_type, subfolder)
    os.makedirs(out_dir, exist_ok=True)
    out_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    out_path = os.path.join(out_dir, out_name)
    fig.savefig(out_path, bbox_inches="tight", dpi=300)
    plt.close(fig)

    print(f"Tabella singola salvata: {out_path}")
'''


# ---------- Pass 2: tabelle aggregate per ruolo ----------
for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            columns = ["Metriche"]
            for m in MODEL_ORDER:
                columns.append(f"{m} (Training)")
                columns.append(f"{m} (Test)")

            df_data = {c: [] for c in columns}

            for idx_sub, subfolder in enumerate(subs):
                
                '''CONVERSIONE LABELS DEL RUOLO --> da th_fam a observer_fam etc'''
                # solo per la visualizzazione converto th_fam->observer_fam, ecc.
                subfolder_disp = DISPLAY_LABELS.get(subfolder, subfolder)
                
                # per ciascun subfolder (fam / unfam) aggiungo le 6 metriche
                for label, tr_key, te_key in METRICS:
                    
                    #df_data["Metriche"].append(f"{subfolder} — {label}")
                    
                    # usa il suffisso "display" SOLO per la label della riga
                    df_data["Metriche"].append(f"{subfolder_disp} — {label}")
                    

                    for m in MODEL_ORDER:
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            tr_val = tr.get(tr_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))
                        except Exception:
                            df_data[f"{m} (Training)"].append("-")
                            df_data[f"{m} (Test)"].append("-")

                # riga separatrice tra fam e unfam (opzionale ma utile visivamente)
                if idx_sub == 0:
                    df_data["Metriche"].append("")  # riga vuota
                    for m in MODEL_ORDER:
                        df_data[f"{m} (Training)"].append("")
                        df_data[f"{m} (Test)"].append("")

            df_performances = pd.DataFrame(df_data)
            
            
            
            SHOW_ONLY = False  # <- True per visualizzare, False per salvare
            
            
            
            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis("off")

            title = f"DL Models performances — {condition} — EEG feature: {data_type} — {role_label}"
            ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

            tabla = table(
                ax,
                df_performances,
                loc="center",
                colWidths=[0.25] + [0.12] * (len(df_performances.columns) - 1),
            )
            tabla.auto_set_font_size(True)
            tabla.set_fontsize(9)
            tabla.scale(1.2, 1.2)

            for kcell, cell in tabla.get_celld().items():
                if kcell[0] == 0:
                    cell.set_text_props(weight="bold")
            
            
            '''Con "aggregated", io aggiungo una sotto-cartella ancora alla path di salvataggio delle tabelle'''
            
            if SHOW_ONLY:
                plt.show()
                print(f"Tabella aggregata di: models_performances_{condition}_{data_type}_{role_label}.png")
            else:
                
                out_dir = os.path.join(base_folder, condition, data_type, "aggregated")
                os.makedirs(out_dir, exist_ok=True)
                out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                out_path = os.path.join(out_dir, out_name)
                fig.savefig(out_path, bbox_inches="tight", dpi=300)
                plt.close(fig)

                print(f"Tabella aggregata salvata: {out_path}")

#### Implementazione : Versione dal 24 novembre 2025 - Versione Aggregata

In [ ]:
'''METRICHE PRIMA DI TUTTI I MODELLI SUL TRAIN ... POI SUL VALIDATION ...  E POI SUL TEST SET'''

import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




# ---------- Helpers ----------
def find_model_blob(all_models, condition, data_type, subfolder, model_prefix):
    """Ritorna il dict salvato a disco per il modello richiesto (o None) cercando per filename prefix."""
    key = f"{condition}_{data_type}_{subfolder}"
    if key not in all_models:
        return None
    for fname, blob in all_models[key].items():
        if fname.startswith(model_prefix + "_"):
            return blob
    return None


def fmt(v, star=False):
    """Formatta un valore numerico a 3 decimali e aggiunge '*' se standardizzato (se vuoi)."""
    try:
        val = float(v)
        s = f"{val:.3f}"
    except Exception:
        s = "-"
    return s + ("" if star else "")


def format_col_label(label: str) -> str:
    """
    Converte 'CNN1D (Training)' -> 'CNN1D\n(Training)' per header a due righe.
    Lascia 'Metrics' invariato.
    """
    if label == "Metrics":
        return label
    if "(" in label and label.endswith(")"):
        model, phase = label.split("(", 1)
        model = model.strip()
        phase = "(" + phase.strip()
        return f"{model}\n{phase}"
    return label


def pretty_condition_name(cond: str) -> str:
    """
    th_resp_vs_pt_resp        -> 'observer resp vs receiver resp'
    th_resp_vs_shared_resp    -> 'observer resp vs shared resp'
    pt_resp_vs_shared_resp    -> 'receiver resp vs shared resp'
    """
    token_map = {
        "th_resp": "observer resp",
        "pt_resp": "receiver resp",
        "shared_resp": "shared resp",
    }
    parts = cond.split("_vs_")
    pretty_parts = [token_map.get(p, p.replace("_", " ")) for p in parts]
    return " vs ".join(pretty_parts)


# ---------- Config ----------
base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks" 

experimental_conditions = [
    "th_resp_vs_pt_resp",
    "th_resp_vs_shared_resp",
    "pt_resp_vs_shared_resp",
]

#data_types = ["1_20", "1_45", "wavelet"]
data_types = ["spectrograms"]
subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

#MODEL_ORDER = ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]

MODEL_ORDER = ["CNN3D_LSTM", "SeparableCNN2D_LSTM"]

PHASES = ["Training", "Validation", "Test"]   # <--- aggiunto per chiarezza

# (label, train_key, val_key, test_key)
METRICS = [
    ("Accuracy",  "train_accuracy",  "val_accuracy",  "test_accuracy"),
    ("Loss",      "train_loss",      "val_loss",      "test_loss"),
    ("Precision", "train_precision", "val_precision", "test_precision"),
    ("Recall",    "train_recall",    "val_recall",    "test_recall"),
    ("F1-Score",  "train_f1_score",  "val_f1_score",  "test_f1_score"),
    ("AUC-ROC",   "train_auc",       "val_auc",       "test_auc"),
]

ROLE_GROUPS = {
    "Observer_Role": ["th_fam", "th_unfam"],
    "Receiver_Role": ["pt_fam", "pt_unfam"],
}

# Etichette che appariranno nella colonna "Metrics"
DISPLAY_LABELS = {
    "th_fam":   "observers familiar group",
    "th_unfam": "observers unfamiliar group",
    "pt_fam":   "receivers familiar group",
    "pt_unfam": "receivers unfamiliar group",
}

# Etichette per il tipo di dato nel titolo

DATA_LABELS = {
    "spectrograms": "Electrodes x Frequency"
}
SHOW_ONLY = False  # cambia in False per salvare i PNG
#SHOW_ONLY = True  # cambia in False per salvare i PNG

# ---------- Carica models_info se esiste ----------
try:
    with open("/home/stefano/Interrogait/spectrograms_EEG_channels_freqs_params_GradCAM_Checks.pkl", "rb") as f:
        models_info = pickle.load(f)
except Exception:
    print("⚠️  models_info non trovato/caricabile: nessun indicatore di standardizzazione (*).")
    models_info = {}

# ---------- Caricamento modelli ----------
all_models = {}

for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            path = os.path.join(base_folder, condition, data_type, subfolder)
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue

            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            for file in os.listdir(path):
                if file.endswith(".pkl"):
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")


# =========================================================
#  PASS 2: tabelle aggregate per ruolo (Observer / Receiver)
# =========================================================
for condition in experimental_conditions:
    for data_type in data_types:
        for role_label, subs in ROLE_GROUPS.items():

            print(f"\nProcessing aggregate table: {condition} - {data_type} - {role_label}")

            # ---------- COSTRUZIONE DF ----------
            # ORA: prima tutte le colonne Train (tutti i modelli),
            #      poi tutte le colonne Validation, poi Test.
            columns = ["Metrics"]
            for phase in PHASES:                      # <- loop su Training / Validation / Test
                for m in MODEL_ORDER:                 #    e dentro sui modelli
                    columns.append(f"{m} ({phase})")

            df_data = {c: [] for c in columns}

            for idx_sub, subfolder in enumerate(subs):

                subfolder_disp = DISPLAY_LABELS.get(subfolder, subfolder)

                for label, tr_key, val_key, te_key in METRICS:

                    df_data["Metrics"].append(f"{subfolder_disp} — {label}")

                    for m in MODEL_ORDER:
                        blob = find_model_blob(all_models, condition, data_type, subfolder, m)

                        mi_key = f"{m}_{condition}_{data_type}_{subfolder}"
                        std_flag = bool(models_info.get(mi_key, {}).get("standardization", False))

                        if blob is None:
                            for phase in PHASES:
                                df_data[f"{m} ({phase})"].append("-")
                            continue

                        try:
                            tr = blob.get("my_train_results", {}).get("training_performances", {})
                            va = blob.get("my_train_results", {}).get("validation_performances", {})
                            te = blob.get("my_test_results", {}).get("test_performances", {})

                            tr_val = tr.get(tr_key,  [None])[0]
                            va_val = va.get(val_key, [None])[0]
                            te_val = te.get(te_key, [None])[0]

                            df_data[f"{m} (Training)"].append(fmt(tr_val, star=std_flag))
                            df_data[f"{m} (Validation)"].append(fmt(va_val, star=std_flag))
                            df_data[f"{m} (Test)"].append(fmt(te_val, star=std_flag))

                        except Exception:
                            for phase in PHASES:
                                df_data[f"{m} ({phase})"].append("-")

                # riga vuota di separazione (fam / unfam)
                if idx_sub == 0:
                    df_data["Metrics"].append("")
                    for phase in PHASES:
                        for m in MODEL_ORDER:
                            df_data[f"{m} ({phase})"].append("")

            df_performances = pd.DataFrame(df_data)

            # =========================
            #  PREPARAZIONE PER PLOT
            # =========================
            df_display = df_performances.copy()

            col_weights = []
            for col in df_display.columns:
                header_len = len(str(col))
                body_max = df_display[col].astype(str).map(len).max()
                col_weights.append(max(header_len, body_max))

            col_weights = np.array(col_weights, dtype=float)
            col_weights[0] *= 1.4  # "Metrics" più larga
            col_widths = (col_weights / col_weights.sum()) * 0.98

            # ---------- FIGURA & AX ----------
            fig, ax = plt.subplots(figsize=(14, 8))
            ax.axis("off")

            cond_pretty = pretty_condition_name(condition)
            data_pretty = DATA_LABELS.get(data_type, data_type)
            role_pretty = role_label.replace("_", " ")

            line1 = "Deep Learning Models performances for Brain Decoding of Sense of Responsibility"
            line2 = f"Experimental Conditions: {cond_pretty} — EEG Spectrograms: {data_pretty} — Subject Cohort: {role_pretty}"

            ax.set_title(
                f"{line1}\n{line2}",
                fontsize=11,
                pad=6,
            )

            col_labels = [format_col_label(c) for c in df_display.columns]

            tabla = ax.table(
                cellText=df_display.values,
                colLabels=col_labels,
                loc="upper center",
                cellLoc="center",
                colWidths=col_widths.tolist(),
            )

            tabla.auto_set_font_size(False)
            base_fontsize = 6
            header_fontsize = 6

            tabla.set_fontsize(base_fontsize)
            tabla.scale(1.1, 1.1)

            for (row, col), cell in tabla.get_celld().items():
                if row == 0:
                    cell.set_text_props(weight="bold", fontsize=header_fontsize)

            if SHOW_ONLY:
                plt.show()
                plt.close(fig)
                #print(f"Tabella aggregata mostrata: {condition} - {data_type} - {role_label}")
                #out_dir = os.path.join(base_folder, condition, data_type)
                #os.makedirs(out_dir, exist_ok=True)
                #out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                #out_path = os.path.join(out_dir, out_name)
                #print(f'out_path: {out_path}') 
            else:
                out_dir = os.path.join(base_folder, condition, data_type, "aggregated")
                os.makedirs(out_dir, exist_ok=True)
                out_name = f"models_performances_{condition}_{data_type}_{role_label}.png"
                out_path = os.path.join(out_dir, out_name)
                print(f'out_path: \033[1m{out_path}\033[0m') 

                fig.savefig(out_path, bbox_inches="tight", dpi=300)
                plt.close(fig)

                print(f"Tabella aggregata salvata: {out_path}")


In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE B
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE B (SEMPLIFICATA!)




                                                                    POST-AGGIORNAMENTO 
                                                                    
Conclusioni
Le funzioni di versioning (get_model_config_key e update_model_version) 
sembrano corrette e funzionano come previsto per generare 
una rappresentazione univoca della configurazione interna e assegnare versioni progressive.

L'integrazione nel loop di training è quasi completa; ho segnalato alcuni dettagli (come la virgola mancante) 
e suggerito di verificare che tutte le variabili (ad esempio, cp, wandb, standardize_data, prepare_data_for_model, EarlyStopping, ecc.) 
siano definite o importate correttamente.

Il meccanismo di versioning ti consentirà di tenere traccia delle diverse configurazioni (versioni) per ogni combinazione di dati,
aggiornando il salvataggio del modello se la validation accuracy migliora.

'''

#                                                                      IMPORTANTE
#Questa struttura garantisce che ogni sweep abbia una gestione separata delle versioni del modello, senza conflitti tra sweep differenti
#e ogni esecuzione di training_sweep può aggiornare e usare correttamente il dizionario model_versions per tracciare le versioni dei modelli.

import time  # Importa il modulo time

# Crea un dizionario per tenere traccia delle versioni del modello per questo sweep
model_versions = {}
                  
# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                sweep_id, combination_key = sweep_tuple
                
                # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione
                
                # Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili
                def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_versions):
                    def train_wrapper():
                        
                        # Qui chiamiamo la funzione di training con i parametri appropriati
                        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
                        
                        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
                        training_sweep(
                            data_dict_preprocessed, 
                            sweep_config,
                            sweep_ids,
                            sweep_id,
                            sweep_tuple,
                            best_models,  # Best models viene aggiornato all'interno della funzione,
                            model_versions # Passa model_versions come argomento
                        )
                    return train_wrapper
                
                # Crea la funzione wrapper per l'agent
                agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_versions)
                
                # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                   ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''
                
                print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")
                wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_params_hyperparams", count=15)
                
                    
                print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

In [ ]:
#model_versions['CNN2D_th_resp_vs_pt_resp_spectrograms_familiar_th']

In [ ]:
print('Finito Training su W&B !')

In [ ]:
# Stampa il numero totale di sweeps
#print(f"Numero totale di sweeps che verranno eseguiti: {total_sweeps}")

In [ ]:
#sweep_ids.keys()